In [1]:
import os
import json
import PromptManager

def create_prompt_manager(filepath: str, initial_data: list = None):
    """
    JsonManager에서 사용할 JSON 파일을 생성합니다.
    
    :param filepath: 생성할 JSON 파일의 경로.
    :param initial_data: 초기 데이터 (리스트 형태). 기본값은 두 개의 샘플 프롬프트 항목이 들어있는 리스트입니다.
    """
    
    # 중복된 파일 경로가 존재하면 예외 발생
    if os.path.exists(filepath):
        raise FileExistsError(f"파일 '{filepath}'이(가) 이미 존재합니다.")
    
    # 초기 데이터를 지정하지 않은 경우 기본 샘플 데이터를 사용합니다.
    if initial_data is None:
        initial_data = [
            {
                "subject": "init",
                "text": "프롬프트 내용",
                "type": "프롬프트 종류(sys, user)",
                "author": "작성자(선택)",
                "version": 1
            }
        ]
    
    with open(filepath, "w", encoding="utf8") as f:
        json.dump(initial_data, f, ensure_ascii=False, indent=4)
    print(f"파일 '{filepath}'이(가) 생성되었습니다.")
    
class PromptManager:
    def __init__(self, filename):
        """
        생성자: 파일명을 받아 JSON 파일을 로드합니다.
        만약 파일이 없거나 JSON 형식에 문제가 있을 경우 빈 리스트로 초기화합니다.
        """
        self.filename = filename
        self.data = self.load_json()

    def load_json(self):
        """JSON 파일을 읽어 파이썬 데이터(리스트)로 변환합니다."""
        try:
            with open(self.filename, 'r', encoding='utf8') as f:
                data = json.load(f)
            # 데이터가 리스트 형태가 아니라면 빈 리스트로 재설정합니다.
            if not isinstance(data, list):
                data = []
            return data
        except (FileNotFoundError, json.JSONDecodeError):
            return []

    def save_json(self):
        """현재 데이터를 JSON 파일에 저장합니다."""
        with open(self.filename, 'w', encoding='utf8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=4)

    def update_entry(self, subject, new_values):
        """
        주어진 subject에 해당하는 항목을 업데이트합니다.
        :param subject: 업데이트 대상 항목의 subject 값
        :param new_values: 업데이트할 내용 (딕셔너리 형태)
        :return: 업데이트 성공 시 True, 실패 시 False
        :raises ValueError: 업데이트하려는 text나 version 값이 기존 엔트리와 동일한 경우 예외 발생.
        """
        for entry in self.data:
            if entry.get("subject") == subject:
                # text 업데이트 시 중복 체크
                if "text" in new_values and new_values["text"] == entry.get("text"):
                    raise ValueError("업데이트하려는 text 값이 기존 엔트리와 동일합니다.")
                
                # version 업데이트 시 중복 체크
                if "version" in new_values and new_values["version"] == entry.get("version"):
                    raise ValueError("업데이트하려는 version 값이 기존 엔트리와 동일합니다.")
                
                # 예외 없이 업데이트 수행
                entry.update(new_values)
                self.save_json()  # 업데이트 후 변경 내용 저장
                return True
        return False


    def delete_entry(self, subject):
        """
        주어진 subject에 해당하는 항목을 삭제합니다.
        :param subject: 삭제 대상 항목의 subject 값
        :return: 삭제 성공 시 True, 실패 시 False
        """
        for i, entry in enumerate(self.data):
            if entry.get("subject") == subject:
                del self.data[i]
                self.save_json()  # 삭제 후 변경 내용 저장
                return True
        return False

    def create_entry(self, subject: str, text: str, type: str, author: str = None, version: int = None):
        """
        새 프롬프트 항목을 생성하여 데이터에 추가하고 파일에 저장합니다.
        subject는 50자 이내여야 합니다.
        
        :param subject: 프롬프트의 제목. 50자 이내여야 합니다.
        :param text: 프롬프트 내용.
        :param type: 프롬프트 종류.
        :param author: 작성자 (선택사항).
        :param version: 버전 정보 (선택사항).
        :raises ValueError: subject 길이가 50자를 초과하면 예외 발생.
        """
        if len(subject) > 50:
            raise ValueError("subject는 50자 이내로 입력되어야 합니다.")
                # 중복 체크: 기존 엔트리 중 text가 동일한 항목이 있으면 생성하지 않습니다.
        for entry in self.data:
            if entry.get("subject") == subject:
                raise ValueError("동일한 subject를 가진 엔트리가 이미 존재합니다.")
        # 중복 체크: 기존 엔트리 중 text가 동일한 항목이 있으면 생성하지 않습니다.
        for entry in self.data:
            if entry.get("text") == text:
                raise ValueError("동일한 text를 가진 엔트리가 이미 존재합니다.")
        
        prompt_entry = {
            "subject": subject,
            "text": text,
            "type": type,
            "author": author,
            "version": version
        }
        # 기존 데이터에 새 항목을 추가한 후 파일에 저장합니다.
        self.data.append(prompt_entry)
        self.save_json()
        print(f'{subject}가 저장되었습니다.')
        
    def listup_subjects(self):
        """
        각 딕셔너리에서 'subject' 키와 해당 값을 출력합니다.
        """
        for entry in self.data:
            subject = entry.get("subject")
            print(f"subject: {subject}")
            
            
    def get_text(self, subject: str):
        """
        주어진 subject에 해당하는 엔트리의 text를 문자열로 출력하고 반환합니다.
        
        :param subject: 조회할 엔트리의 subject 값.
        :return: 해당 엔트리의 text 문자열, 엔트리를 찾지 못하면 None 반환.
        """
        for entry in self.data:
            if entry.get("subject") == subject:
                text = entry.get("text")
                return text
        print(f"'{subject}'에 해당하는 엔트리를 찾을 수 없습니다.")
        return None


def create_prompt_manager(filepath: str, initial_data: list = None):
    """
    JsonManager에서 사용할 JSON 파일을 생성합니다.
    
    :param filepath: 생성할 JSON 파일의 경로.
    :param initial_data: 초기 데이터 (리스트 형태). 기본값은 두 개의 샘플 프롬프트 항목이 들어있는 리스트입니다.
    """
    
    # 중복된 파일 경로가 존재하면 예외 발생
    if os.path.exists(filepath):
        raise FileExistsError(f"파일 '{filepath}'이(가) 이미 존재합니다.")
    
    # 초기 데이터를 지정하지 않은 경우 기본 샘플 데이터를 사용합니다.
    if initial_data is None:
        initial_data = [
            {
                "subject": "init",
                "text": "프롬프트 내용",
                "type": "프롬프트 종류(sys, user)",
                "author": "작성자(선택)",
                "version": 1
            }
        ]
    
    with open(filepath, "w", encoding="utf8") as f:
        json.dump(initial_data, f, ensure_ascii=False, indent=4)
    print(f"파일 '{filepath}'이(가) 생성되었습니다.")

In [2]:
# create_prompt_manager('fupsheet_prompt.json')

In [2]:
manager = PromptManager('./fupsheet_prompt.json')
manager.listup_subjects()

subject: sys_prompt
subject: machinetype_prompt
subject: description_prompt
subject: instruction_prompt
subject: switchgeartype_prompt
subject: description_prompt_A_H
subject: format_prompt
subject: description_prompt_I_U


In [6]:
manager.delete_entry("description_prompt_I_S")

True

In [5]:
print(manager.get_text(subject='description_prompt'))


You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)

# A-1
# General Conditions
# Applicable International Standards
# "Specifies whether the high-voltage switchgear must comply with IEC or IEEE standards, which govern its design, testing, and performance requirements. This determines which standard group’s definitions, design principles, test protocols, and performance criteria must be followed. Only standards applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic:
Step 1: Extract only one primary applicable standard—either IEC Standards or IEEE Standards—applying to the high-voltage switchgear explicitly required in the project specification.
Step 2: If both standards are mentioned, AI must determine whether one gov

## CREATE

In [4]:
# description_prompt_I_U
manager.create_entry(subject="description_prompt_I_U",text='''
You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)


# I-1
# Voltage Transformer
# Type
# "Specifies the construction and sensing technology of the voltage transformer (VT) used in high-voltage switchgear. Accepted types include Inductive VT, Capacitive VT (CVT), Optical VT, and Low-Power Instrument Transformer (LPIT), Non-Conventional Instrument Transformer (NCIT)."
# AI Extraction Logic
- Identify any explicit mention of VT type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - Inductive VT → `Inductive`
  - Capacitive VT or CVT → `Capacitive`
  - Optical VT → `Optical`
  - 'Resistive Voltage Divider'
  - 'Electro-Optic Voltage Sensor'
  - Low-Power Instrument Transformer → `LPIT`
  - Non-Conventional Instrument Transformer → `NCIT`
- If VT is specified as mounted inside GIS (e.g., “GIS-integrated,” “SF₆ gas insulated,” “in GIS enclosure”), add flag: `GIS-integrated` after the type.
  - Example outputs:
    - `Inductive; GIS-integrated`
    - `Inductive / Capacitive; GIS-integrated`
- If multiple types are allowed, output as: `Inductive / Capacitive`
- If one type is preferred and another conditionally allowed, output as: `LPIT (preferred); Inductive allowed upon approval`
- If not specified, leave the field blank.

# I-2
# Voltage Transformer
# Phase Configuration
# "Specifies whether the project requires Voltage Transformers (VTs or PTs) to be single-phase, three-phase, or permits both configurations. This distinction affects enclosure design, wiring, metering accuracy, and redundancy strategy in high-voltage switchgear systems."
# AI Extraction Logic
- Identify any explicit mention of VT phase configuration in the project-binding specification applicable to the validated SwitchgearType.
- Accepted configurations:
  - `Single-phase`
  - `Three-phase`
  - If both are allowed: `Single-phase / Three-phase`
- If one configuration is preferred and another conditionally allowed, output as:
  - `Single-phase (preferred); Three-phase allowed upon approval`
- Ensure the statement refers specifically to Voltage Transformers (VTs/PTs), not CTs or general phase references.
- If not specified, leave the field blank.

# I-3
# Voltage Transformer
# Rated Voltage Factor
# "Specifies the ratio between the maximum permissible operating voltage and the rated voltage of the voltage transformer (VT). This factor determines the VT’s capability to withstand temporary overvoltages without exceeding thermal or dielectric limits. Typical values include 1.2 (continuous), 1.5 (short-duration), and 1.9 (short-duration)."
# AI Extraction Logic
- Identify any explicit mention of Rated Voltage Factor in the project-binding specification applicable to the validated SwitchgearType.
- Accepted formats:
  - Continuous operation: 1.2
  - Short-duration operation: 1.5, 1.9 (include duration if stated, e.g., 1.5 for 30 s)
  - Long-duration operation: 1.9 (include duration if stated, e.g., 1.5 for 8 h)
- If multiple values are provided (continuous and short-duration), output as:
  - Continuous: 1.2; Short-duration: 1.9 for 30 s
  - If only one value is stated, extract that value.
- If not specified, leave the field blank.

# I-4
# Voltage Transformer
# Anti Ferro-Resonance Requirement
# "Specifies whether the voltage transformer (VT or PT) must include design measures to suppress ferroresonance phenomena, which can occur due to non-linear behavior under low-load or no-load conditions in high-voltage systems. Ferroresonance can lead to sustained overvoltages and equipment damage; therefore, some specifications explicitly require anti-ferroresonance features such as suppression resistors, ferroresonance suppressors, or design-based mitigation."
# AI Extraction Logic
- Identify any explicit mention of anti-ferroresonance requirement in the project-binding specification applicable to the validated SwitchgearType.
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`
- If specific features are mentioned, append detail:
  - `Required; ferroresonance suppressor`
  - `Required; suppression resistor`
- If not specified, leave the field blank.

# I-5
# Voltage Transformer
# Isolating Device Requirement
# "Specifies whether the specification requires a Voltage Transformer (VT or PT) to be equipped with an isolating device or disconnecting switch to facilitate HV system isolation, dielectric testing, or safe maintenance. Such isolating devices may be either integrated (e.g., integral isolating switch operated from GIS exterior) or separate (e.g., dedicated disconnector) and typically include features like positive position indication and locking mechanism."**
# AI Extraction Logic
- Identify any explicit mention of isolating device or disconnecting switch for VT/PT in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords: `isolating device`, `disconnecting switch`, `separate disconnector`, `means to isolate`, `facility for dielectric test`, `positive position indication`.
- If isolating device is required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required for dielectric testing`
  - `Optional; required for maintenance`
- If type is specified, append detail:
  - `Required; Integral isolating device`
  - `Required; Separate disconnector`
- If not specified, leave the field blank.

# I-6
# Voltage Transformer
# Manufacturer / Country of Origin
# "Specifies the manufacturer (vendor) and the country of origin of the voltage transformer applied in high-voltage switchgear. This information is typically stated to ensure supplier qualification, regulatory compliance, or manufacturing traceability. Only manufacturer names and countries explicitly listed in the technical specification or data schedule shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of VT manufacturer and country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `Siemens`, `Hitachi`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `Siemens / Nissin`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `Siemens (preferred); Nissin allowed upon approval`
- If not specified, leave the field blank.

# J-1
# Current Transformer
# Type
# "Specifies the type of current transformer (CT) used in high-voltage switchgear. The CT type determines measurement accuracy, insulation characteristics, and interface compatibility with protection and metering systems. Only CT types explicitly stated or clearly implied through standard references or construction terminology shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of CT type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - `Ring Core` (or Toroidal Core)
  - `Optical CT` (or Fiber Optic Current Sensor, FOCS)
  - `Rogowski Coil`
  - `Low Power CT` (or LPCT)
  - `Digital CT` (only if clearly linked to optical or LPCT structure)
- If multiple types are allowed, output as:
  - `Ring Core / Optical CT`
- If one type is preferred and another conditionally allowed, output as:
  - `Ring Core (preferred); Optical CT allowed upon approval`
- If not specified, leave the field blank.

# J-2
# Current Transformer
# CT Location Relative to Enclosure
# "Specifies whether Current Transformers (CTs) are installed outside the GIS/HGIS enclosure or outside the insulating medium zone for high-voltage switchgear. Extract only explicit external-location statements. Do not infer or emit “Internal”; when no explicit external statement exists, leave the field blank. Ignore electrical sizing and secondary wiring/termination content."
# AI Extraction Logic
- Identify any explicit statement that CTs are installed outside the GIS/HGIS enclosure or outside the insulating medium zone.
- Accepted keywords:
  - `installed outside enclosure`
  - `mounted external to GIS enclosure`
  - `located outside SF₆ zone`
- If external installation is explicitly prohibited, output as: `External prohibited`.
- If external installation is allowed or conditional, output as:
  - `External allowed; [verbatim qualifier]`
- If external installation is required, output as: `External to enclosure`.
- Do **not** infer location from:
  - Electrical sizing (ratio, accuracy, burden, knee point)
  - Wiring or termination details
  - Bushing-related phrases (e.g., “through bushing”)
- If not specified, leave the field blank.

# J-3
# Current Transformer
# Insulation Class
# "Specifies the insulation class of the Current Transformer (CT) used in high-voltage switchgear, including GIS, HGIS, and DTB systems. Insulation class (e.g., Class E, Class F) defines the CT’s thermal endurance level and determines material specifications based on the ambient and internal temperature rise. If different insulation classes are specified based on CT location (e.g., inside or outside SF₆ gas zone, or based on GIS/HGIS distinction), these shall be noted accordingly."
# AI Extraction Logic
- Identify any explicit mention of CT insulation class in the project-binding specification applicable to the validated SwitchgearType.
- Accepted values:
  - `Class E`
  - `Class F`
  - Other IEC/IEEE recognized classes (e.g., Class B, Class H)
- If different classes are specified for GIS vs. HGIS or based on CT location, output both:
  - Example: `Class E for GIS; Class F for HGIS`
- If not specified, leave the field blank.

# J-4
# Current Transformer
# Continuous Thermal Current Rating Factor
# "Specifies the multiplier applied to the CT’s rated current to define its continuous thermal capability under normal operating conditions. Typical values include 1.0, 1.2, and 1.5."
# AI Extraction Logic
- Identify any explicit mention of continuous thermal current rating factor in the project-binding specification applicable to the validated SwitchgearType.
- Accepted formats:
  - `1.0`
  - `1.2`
  - `1.5`
- If multiple factors are stated for different CT locations or configurations, output both:
  - Example: `1.2 for GIS; 1.5 for HGIS`
- If not specified, leave the field blank.

# J-5
# Current Transformer
# Current Transformer Protection Unit
# "Indicates whether the Current Transformer Protection Unit (CTPU) is required for high-voltage switchgear applications. The CTPU provides open-circuit protection for CT secondary circuits. Only requirements explicitly stated in the specification or clearly implied through mandatory inclusion shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of CTPU or CT open-circuit protection in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `CTPU`
  - `CT open circuit protection`
  - `CT protection unit`
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`
- If not specified, leave the field blank.

# J-6
# Current Transformer
# Manufacturer / Country of Origin
# "Specifies the designated manufacturer and/or country of origin for Current Transformers (CTs) used in high-voltage switchgear rated at nominal system voltages of 66 kV and above. This information may reflect customer preferences, approved vendor lists, or procurement restrictions aimed at ensuring product quality, local content compliance, or geopolitical risk mitigation."
# AI Extraction Logic
- Identify any explicit mention of CT manufacturer and/or country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `GE`, `Trench`, `Ritz`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `GE / Trench`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `GE (preferred); Trench allowed upon approval`
- If country restrictions or preferences are stated, append detail:
  - `GE / Germany`
  - `Trench / Any country except China`
- If not specified, leave the field blank.

# K-1
# Surge Arrester
# Type
# "Specifies the required type of surge arrester for high-voltage switchgear. This includes the arrester's configuration (e.g., free-standing, GIS-housed) and its internal design characteristics (e.g., metal-oxide without gaps). Only explicitly stated types or clearly referenced standards (e.g., IEC 60099-4, IEEE C62.11) shall be considered."
# AI Extraction Logic
- Identify any explicit mention of surge arrester type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - `Metal-oxide surge arrester (gapless)`
  - `Free-standing outdoor surge arrester`
  - `GIS-housed surge arrester`
- Normalize variations such as `gap-less`, `gapless`, `without gap` → `gapless`.
- If the specification references standards that define arrester types:
  - IEC 60099-4 → `Metal-oxide surge arrester (gapless) (as per IEC 60099-4)`
  - IEEE C62.11 → `Metal-oxide surge arrester (gapless) (as per IEEE C62.11)`
- If not specified, leave the field blank.

# K-2
# Surge Arrester
# Designation
# "Specifies the Line Discharge Class of the Surge Arrester applied to high-voltage switchgear (GIS, DTB, HGIS, or SF₆ Gas-Free systems). Line Discharge Class defines the energy absorption capability of the arrester and is typically designated by IEC 60099-4 as SL (low), SM (medium), or SH (high). IEEE-based specifications may instead describe discharge current values or energy ratings without using SL/SM/SH terms."
# AI Extraction Logic
- Identify any explicit mention of Line Discharge Class or equivalent designation in the project-binding specification applicable to the validated SwitchgearType.
- If IEC 60099-4 is referenced, accepted values:
  - `SL` (Station Low)
  - `SM` (Station Medium)
  - `SH` (Station High)
- If IEEE C62.11 is referenced, accepted values:
  - 'Station'
  - 'Intermediate'
- If not specified, leave the field blank.

# K-3
# Surge Arrester
# Rated Voltage
# "Specifies the rated voltage of the surge arrester, defined as the maximum RMS power frequency voltage the arrester can withstand for a limited time without failure. In IEC 60099-4, this is denoted as 'Rated Voltage' and typically expressed in kV RMS. In IEEE C62.11, it is also referred to as 'Arrester Rated Voltage' and has the same definition. This value is used to coordinate the surge arrester’s operation with the insulation level and system temporary overvoltage performance."
# AI Extraction Logic
- Identify any explicit mention of surge arrester rated voltage in the project-binding specification applicable to the validated SwitchgearType.
- Extract the numerical value and unit (e.g., `96 kV`, `144 kV`), ensuring it refers specifically to the surge arrester.
- If not specified, leave the field blank.

# K-4
# Surge Arrester
# Continuous Operating Voltage
# "Specifies the maximum RMS voltage that may be continuously applied across the terminals of the surge arrester without degrading its performance. This value is commonly referred to as the Maximum Continuous Operating Voltage (MCOV) in IEEE C62.11 and as Continuous Operating Voltage (Uc) in IEC 60099-4. The value is essential for ensuring coordination with system voltage levels and arresters' temporary overvoltage withstand capability."
# AI Extraction Logic
- Identify any explicit mention of Continuous Operating Voltage (IEC term: `Uc`) or Maximum Continuous Operating Voltage (IEEE term: `MCOV`) in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Continuous Operating Voltage`
  - `Uc`
  - `MCOV`
  - `Maximum Continuous Operating Voltage`
- Extract the numerical value and unit (e.g., `10.2 kV`, `144 kV`), ensuring it refers specifically to the surge arrester.
- If not specified, leave the field blank.

# K-5
# Surge Arrester
# Nominal Discharge Current
# "Specifies the nominal discharge current (In) of the surge arrester, which is the peak value of lightning impulse current (8/20 μs waveform) used to classify the arrester’s energy-handling capability. According to IEC 60099-4, this value is typically 5 kA or 10 kA for distribution and transmission class arresters, and may reach 20 kA for high-energy designs. IEEE C62.11 also defines nominal discharge current for station- and intermediate-class arresters, commonly 10 kA or 5 kA respectively."
# AI Extraction Logic
- Identify any explicit mention of nominal discharge current (In) in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Nominal discharge current`
  - `In`
  - `Discharge current rating`
- Extract the numerical value and unit (e.g., `5 kA`, `10 kA`, `20 kA`), ensuring it refers to lightning impulse current (8/20 μs waveform).
- If not specified, leave the field blank.

# K-6
# Surge Arrester
# Isolating Device Requirement
# "Specifies whether the surge arrester requires an isolating device or disconnecting feature to allow safe disconnection during system dielectric tests. This may include plug-in construction, disconnecting link types, or other means that enable mechanical isolation of the arrester from the system. The requirement is typically stated for SF₆ gas-insulated arresters but may also apply to other types depending on installation conditions."
# AI Extraction Logic
- Identify any explicit mention of isolating or disconnecting device for surge arresters in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `plug-in construction`
  - `disconnecting link`
  - `arrester isolation`
  - `removal for dielectric test`
  - `test disconnection device`
- If isolating device is required, output as: `Required`.
- If multiple isolating device options are allowed, output all using slash format:
  - Example: `Plug-in construction / Disconnecting link`
- If one option is preferred and another conditionally allowed, output as:
  - `Plug-in construction (preferred); Disconnecting link allowed upon approval`
- If not specified, leave the field blank.

# K-7
# Surge Arrester
# Leakage Current Monitor Requirement
# "Specifies whether the surge arrester shall be equipped with a leakage current monitor (LCM) for condition monitoring purposes. The LCM typically measures the resistive or total leakage current to ground and may be specified with a particular measurement range (e.g., 0–50 mA), sensitivity, or display format (e.g., analog, digital)."
# AI Extraction Logic
- Identify any explicit mention of leakage current monitor (LCM) or leakage current measurement for surge arresters in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `leakage current monitor`
  - `LCM`
  - `leakage current measurement`
  - `leakage detection device`
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`
- If not specified, leave the field blank.

# K-8
# Surge Arrester
# Manufacturer / Country of Origin
# "Specifies the required manufacturer and/or country of origin for Surge Arresters used in high-voltage switchgear. Such requirements may reflect customer preferences, quality assurance policies, supply chain localization goals, or trade restrictions."
# AI Extraction Logic
- Identify any explicit mention of surge arrester manufacturer and/or country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `Siemens`, `Hitachi`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `Siemens / Hitachi`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `Siemens (preferred); Hitachi allowed upon approval`
- If country restrictions or preferences are stated, append detail:
  - `Siemens / Germany`
  - `Hitachi / Any country except China`
- If not specified, leave the field blank.

# L-1
# Air Bushing
# Insulating Bushing – Type
# "Specifies the required external insulation type of the gas-to-air bushing used in high-voltage switchgear. Extract only if explicitly stated for the validated SwitchgearType. Common types include Porcelain, Composite (e.g., silicone rubber), Polymer, and RIP (Resin-Impregnated Paper). If exclusions are mentioned (e.g., 'Porcelain not acceptable'), capture them as well."
# AI Extraction Logic
- Identify any explicit mention of air bushing insulation type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Porcelain`
  - `Composite`
  - `Polymer`
  - `Silicone rubber`
  - `Resin-Impregnated Paper (RIP)`
- Output rules:
  - If multiple types are allowed → "Porcelain / Composite"
  - If one option is preferred and another conditionally allowed → "Composite (preferred); Porcelain allowed upon approval"
  - If specific type is not allowed → "Prohibited: Porcelain"
  - If location-specific (e.g., transformer-side, cable-side), append context:
    - Example: "Composite (transformer-side)"
  - If not specified, leave the field blank.

# L-2
# Air Bushing
# Insulating Bushing – Creepage
# "Specifies the required minimum creepage distance of the gas-to-air insulating bushing used in high-voltage switchgear. Creepage distance is the shortest path along the surface of the insulating material between two conductive parts, typically between live conductor and ground. It is often specified in mm or mm/kV and may reflect environmental conditions such as pollution level, altitude, or installation type. Extract only if explicitly stated for the validated SwitchgearType or calculable from provided pollution severity or insulation class."
# AI Extraction Logic
- Identify any explicit mention of creepage distance in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Creepage distance`
  - `Minimum creepage`
  - `Surface distance`
  - `mm/kV`
  - `mm per kV`
- Output rules:
  - If specific value given → "2750 mm" or "31 mm/kV"
  - If linked to pollution severity → "Based on Heavy Pollution – 31 mm/kV"
  - If exclusion or special condition → "Not less than 31 mm/kV"
  - If location-specific (e.g., transformer-side, cable-side), append context:
    - Example: "2750 mm (transformer-side)"
  - If not specified, leave the field blank.

# L-3
# Air Bushing
# Insulating Bushing - Color
# # "Specifies the required external color of the gas-to-air insulating bushing used in high-voltage switchgear. The color is typically specified for standardization, visual inspection, UV resistance, or coordination with other substation equipment. Common options include brown, light gray, and dark gray. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of bushing color in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Bushing color`
  - `Insulator color`
  - `Porcelain color`
  - `Silicone rubber color`
  - `Housing color`
  - Color codes (e.g., `RAL 7035`)
- Output rules:
  - If specific color stated → "Brown", "Light gray", "Dark gray", or "RAL 7035"
  - If multiple colors allowed → "Brown / Light gray"
  - If one color preferred and another conditionally allowed → "Light gray (preferred); Brown allowed upon approval"
  - If exclusion → "Prohibited: Black"
  - If location-specific → append context:
    - Example: "Light gray (transformer-side)"
  - If not specified, leave the field blank.

# L-4
# Air Bushing
# Insulating Bushing – Terminal Hole Requirement
# "Specifies any customer-specified requirements for the terminal hole or terminal pad of the gas-to-air insulating bushing in high-voltage switchgear. These may include standard flange specifications such as NEMA 2-hole or 4-hole pad, or specific bolt dimensions and drilling patterns. Such specifications are intended to ensure mechanical compatibility with site equipment. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of terminal hole or pad requirements in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Terminal hole`
  - `Bolt hole`
  - `Pad type`
  - `Drilling pattern`
  - `NEMA 2-hole`
  - `NEMA 4-hole`
  - `IEC flange`
  - `M16 bolt spacing`
- Output rules:
  - If standard flange type stated → "NEMA 2-hole" or "NEMA 4-hole"
  - If described by dimensions → "4-hole terminal pad" or "2 x M12 bolt hole"
  - If multiple options allowed → "NEMA 2-hole / NEMA 4-hole"
  - If exclusion → "Prohibited: NEMA 2-hole"
  - If location-specific → append context:
    - Example: "NEMA 4-hole (transformer-side)"
  - If not specified, leave the field blank.

# M-1
# Cable Connection
# Cable Termination – Type
# "Specifies the required type of cable termination for high-voltage switchgear, typically classified by its insulation and sealing structure. Common types include dry type, fluid-filled (oil or compound), and plug-in or separable connector types. The requirement is often based on installation conditions such as indoor/outdoor use, fire protection, or compatibility with specific cable designs. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of cable termination type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Cable termination type`
  - `Termination method`
  - `Terminating system`
  - `Dry type`
  - `Fluid-filled`
  - `Oil-filled`
  - `Plug-in termination`
  - `Separable connector`
  - `Stress cone`
  - `Heat-shrink termination`
- Output rules:
  - If only one type stated → "Dry type" or "Plug-in type"
  - If multiple types allowed → "Dry type / Plug-in type"
  - If one type preferred and another conditionally allowed → "Dry type (preferred); Plug-in allowed upon approval"
  - If exclusion → "Prohibited: Fluid-filled"
  - If location-specific → append context:
    - Example: "Dry type (indoor)"
  - If not specified, leave the field blank.

# M-2
# Cable Connection
# Cable Termination – DC Withstand Test Requirement
# "Specifies whether the customer requires a DC dielectric (withstand) test to be performed on the cable termination or cable interface of the high-voltage switchgear. This test is typically required for verifying insulation integrity after installation or as part of routine field commissioning. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
Identify any explicit mention of DC dielectric (withstand) test requirement in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `DC test`
  - `DC withstand test`
  - `Direct current voltage test`
- Output rules:
  - If required → "DC test required for cable termination"
  - If specific details provided → append context:
    - Example: "DC test required for cable termination (80 kV for 15 min)"
  - If not specified, leave the field blank.

# M-3
# Cable Connection
# Cable Termination – Socket Supply Responsibility
# "Specifies whether the customer requires the high-voltage switchgear supplier to provide insular collar (socket-type) components for cable termination. This includes plug-in or separable connector collars that form part of the cable interface system. Extract if the specification explicitly assigns supply responsibility to the high-voltage switchgear manufacturer or if FAT conditions imply high-voltage switchgear must deliver with collar assembled."
# AI Extraction Logic
- Identify any explicit mention that insular collar (socket-type termination components) shall be supplied by the high-voltage switchgear supplier in the project-binding specification applicable to the validated SwitchgearType.
- Also check for indirect requirements via FAT conditions (e.g., cable head conductor assembled during GIS routine test).
- Accepted keywords:
  - Insular collar
  - Socket to be supplied
  - Socket-type termination to be provided
  - GIS supplier shall supply socket
  - Socket included in GIS scope
  - Cable head assembled during GIS factory test
- Output rules:
  - If high-voltage switchgear supplier required to supply → "Supplier to supply insular collar"
  - If optional or conditionally required → "Optional; Supplier to supply upon request"
  - If implied by FAT → "Implied: Collar assembled for FAT"`
  - If not specified, leave the field blank.

# M-4
# Cable Connection
# Cable Termination – Dummy Plug Supply Responsibility
# "Specifies whether the customer explicitly requires unused cable termination ports or bushings on the high-voltage switchgear to be sealed using dummy plugs, blind flanges, or equivalent specific sealing methods, rather than simply requiring sealing for gas containment. Extract only if the use of dummy plugs or similar hardware is clearly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of sealing unused or spare cable terminations using specific hardware in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Dummy plug`
  - `Dummy bushing`
  - `Blind plug`
  - `Blanking flange`
  - `Spare termination shall be plugged`
- Output rules:
  - If dummy plug required → "Dummy plug required"
  - If blind flange required → "Blind flange required"
  - If multiple options allowed → "Dummy plug / Blind flange"
  - If not specified, leave the field blank.

# M-5
# Cable Connection
# Cable Termination – Non-Linear Resistor Supply Responsibility
# "Specifies whether the customer requires the high-voltage switchgear supplier to provide non-linear resistors (NLR) as part of the cable termination assembly. NLRs are typically used for grading or controlling transient voltages at the cable-GIS interface. Extract if the specification explicitly assigns supply responsibility to the high-voltage switchgear manufacturer or if interface compliance (IEC 62271-209) implies inclusion in high-voltage switchgear scope."
# AI Extraction Logic
- Identify any explicit mention that non-linear resistors shall be supplied by the high-voltage switchgear supplier in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Non-linear resistor
  - NLR to be supplied
  - GIS supplier shall supply NLR
  - NLR included in GIS scope
  - Grading resistor
  - Voltage grading device
- Output rules:
  - If high-voltage switchgear supplier required → "Supplier to supply NLR"
  - If optional or conditionally required → "Optional; Supplier to supply upon request"
  - If not specified → leave the field blank.

# N-1
# Direct Connection
# Insulated Junction Requirement
# "Specifies whether the customer requires an insulated junction (also called insulated joint or insulated spacer) when making a direct cable-to-GIS connection without a conventional cable termination. This component ensures electrical insulation and mechanical integrity at the interface. Extract only if the specification explicitly mandates the use of an insulated junction for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of insulated junction/joint requirement in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Insulated junction
  - Insulated joint
  - Direct connection with insulated joint
  - GIS direct cable connection requires insulated junction
- Output rules:
  - If required → "Insulated junction required for direct connection"
  - If optional or conditionally required → "Optional; insulated junction upon request"
  - If not specified → leave the field blank.

# N-2
# Direct Connection
# Non-Linear Resistor Requirement
# "Specifies whether the customer requires non-linear resistors (NLR) to be installed when making a direct cable-to-GIS connection without a conventional cable termination. NLRs are typically used for voltage grading and transient control at the direct interface. Extract only if the specification explicitly mandates NLR for direct connection or if compliance with IEC 62271-209 implies its inclusion."
# AI Extraction Logic
- Identify any explicit mention of non-linear resistor requirement for direct cable-to-GIS connection in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Non-linear resistor
  - NLR required for direct connection
  - Voltage grading resistor
  - Grading device for direct interface
  - GIS direct cable connection requires NLR
- Output rules:
  - If required → "NLR required for direct connection"
  - If optional or conditionally required → "Optional; NLR upon request"
  - If implied by standard → "Implied: NLR included for interface compliance"
  - If not specified → leave the field blank.

# O-1
# Installation Requirements
# Embedded Materials for Foundations — Supply Responsibility & Scope
# "Specifies whether the high-voltage switchgear supplier is responsible for supplying embedded materials required for foundations and defines the scope of such materials. Embedded materials typically include items such as embedded plates, anchor bolts, inserts, and other foundation-related components necessary for installation of GIS, GIB, or associated equipment."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of supply responsibility for embedded materials in sections related to site installation, foundation preparation, or civil interface requirements.
- Accepted keywords:
  - embedded plate
  - anchor bolt
  - foundation insert
  - embedded materials
  - civil interface
  - supply responsibility
- Output Rules
  - If high-voltage switchgear supplier must supply all embedded materials → "Supplier responsible for embedded plates and anchor bolts"
  - If high-voltage switchgear supplier supplies only specific items → List items explicitly, e.g., "Supplier responsible for anchor bolts only"
  - If responsibility lies with customer or civil contractor → "Not supplier's scope"
  - If not specified → leave blank

# O-2
# Installation Requirements
# Cable Duct Type Requirement
# "Specifies whether the customer requires or prefers a specific cable duct type integrated with GIS or GIB bodies for routing outgoing control cables (e.g., CT, VT, LCC cables). Common descriptions include 'perforated duct', 'solid duct', or 'mesh duct'."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of cable duct type in sections related to GIS body accessories, cable routing, or installation details.
- Accepted keywords:
  - cable duct
  - duct type
  - perforated duct
  - solid duct
  - mesh duct
- Output rules:
  - If only one type allowed → "Perforated Duct" (or the exact type mentioned).
  - If multiple types acceptable → List all allowed types, e.g., "Perforated / Solid / Mesh Duct".
  - If preferred type and acceptable types are mentioned → "Perforated Duct (Solid acceptable)".
  - If not specified → leave blank.

# O-3
# Installation Requirements
# Gas Piping Allowance for Gas Monitoring
# "Specifies whether the customer requires or prohibits external gas piping connections for GIS or GIB."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of gas piping allowance or prohibition for monitoring systems in sections related to gas monitoring, accessories, or interface provisions.
- Accepted keywords:
  - gas piping
  - external piping
  - monitoring pipe
  - no external piping
  - eye level, remote readable, accessible for operator
- Output rules:
  - If explicitly allowed → "Allowed".
  - If explicitly required (e.g., “pressure gauge/density monitor shall be at eye level” or “remote readable”) → "Required".
  - If prohibited (including phrases like “direct mounting required” or “no external piping”) → "Not Allowed".
  - If not specified → leave blank.

# O-4
# Installation Requirements
# Platform Type Requirement
# "Specifies whether the customer requires or prefers a specific type of access platform for maintenance or operation of GIS or GIB equipment. Access platforms may include fixed galleries, stairways, walkways, or movable platforms. Extract only if explicitly mentioned as required, preferred, or allowed in the specification."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of access platform type in sections related to GIS maintenance, operation, or installation.
- Accepted keywords:
  - fixed platform
  - movable platform
  - gallery
  - stairway
  - walkway
  - removable platform
  - transportable platform
- Output Rules
  - If only fixed type allowed → "Fixed type only"
  - If only movable type allowed → "Movable type only"
  - If both acceptable → "Fixed or movable type acceptable"
  - If mentioned but no specific type → leave blank

# O-5
# Installation Requirements
# Sun Shade Requirement
# "Specifies whether the customer requires a sun shade (protective roof or cover) to be installed over outdoor high-voltage switchgear or gas-insulated bus ducts (GIB). Sun shades are typically intended to reduce thermal stress, protect personnel from heat, and extend equipment life. The requirement may apply to GIS, DTB, HGIS, GIB, or outdoor-mounted control panels. Extract only if explicitly required, conditionally specified, or optional in the specification."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of sun shade or protective cover in sections related to outdoor GIS, GIB, DTB, HGIS, or local control panels.
- Accepted keywords:
  - Sun shade
  - Protective cover
  - Roof cover
  - UV shielding
  - Solar protection
  - Canopy over GIS
  - Heat shield over switchgear
- Output Rules
  - If explicitly required → "Required"
  - If requirement is conditional (e.g., based on ambient temperature or solar exposure) → "Conditionally required"
  - If mentioned but not mandatory → "Optional"
  - If not specified → leave blank

# P-1
# Surface Treatment & Painting
# Enclosure Painting Policy
# "Specifies whether the specification prohibits painting of the internal or external surface of the enclosure or metallic compartment of high-voltage switchgear. This typically applies to gas compartments or metallic inner walls (internal) and external surfaces where painting may be restricted by project requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of painting prohibition for internal or external surfaces in sections related to surface treatment, painting, or gas compartment requirements.
- Accepted keywords:
  - internal surface shall not be painted
  - no painting on the inside of the enclosure
  - painting inside gas compartments is not allowed
  - do not paint internal metallic surfaces
  - prohibit internal painting
  - painting to be limited to external surfaces only
  - external surface shall not be painted
  - no painting on the outside of the enclosure
- Output Rules:
  - If internal painting is prohibited → "Internal painting not allowed"
  - If external painting is prohibited → "External painting not allowed"
  - If both prohibited → "Internal and external painting not allowed"
  - If not specified → leave blank

# P-2
# Surface Treatment & Painting
# Enclosure Color
# "Specifies the required color for the exterior surface of the high-voltage switchgear enclosure. The color may be defined by a specific code (e.g., RAL 7032, ANSI 70 Gray), customer-defined naming (e.g., “light gray”, “desert tan”), or referred to project standard specifications."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of enclosure color in sections related to painting, finishing, or surface treatment of HV switchgear.
- Accepted keywords:
  - color of enclosure
  - paint color shall be
  - RAL
  - ANSI color code
  - finish: [color]
  - shade: light gray / grey / desert tan / beige / olive green
- Output Rules
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7032").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light gray").
  - If not specified → leave blank

# P-3
# Surface Treatment & Painting
# Color of Partition
# "Specifies whether the customer requires or prefers a specific paint color for partition components used in high-voltage switchgear, such as gas-tight support insulators or gas barrier insulators. These components are part of internal compartments in GIS or hybrid switchgear and are often referred to as gas barrier insulators, partition insulators, or sectionalizing bushings."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of color requirement for partition components in sections related to internal GIS compartments or partitioning insulators.
- Accepted keywords:
  - partition color
  - gas barrier insulator color
  - sectionalizing bushing color
  - RAL
  - light grey
  - neutral tone
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light grey").
  - If not specified → leave blank.

# P-4
# Surface Treatment & Painting
# Mechanism Color
# "Specifies the required color for the exterior surface of the high-voltage circuit breaker mechanism housing (e.g., operating mechanism box). The color may be defined by a standard code (e.g., RAL, ANSI), a descriptive name (e.g., “dark gray”, “silver gray”)."
# AI Extraction Logic:
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of mechanism housing color in sections related to painting, finishing, or surface treatment of operating mechanism boxes.
- Accepted keywords:
  - mechanism box color
  - operating mechanism enclosure finish/color
  - paint color for mechanism
  - mechanism housing shall be painted
  - CB mechanism color
  - RAL
  - ANSI color code
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Silver gray").
  - If not specified → leave blank.

# P-5
# Surface Treatment & Painting
# Paint Thickness of Enclosure
# "Specifies whether the customer defines the required total paint thickness for the main enclosure of high-voltage switchgear. The enclosure refers to the external housing of GIS, DTB, or HGIS. The paint system may be described as a total dry film thickness (DFT) or as individual layers (e.g., primer + top coat). The total thickness is often specified in micrometers (μm) and may vary depending on installation area (e.g., outdoor, coastal)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness for the main enclosure in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - paint thickness
  - DFT
  - dry film thickness
  - primer + top coat
  - minimum microns
  - μm
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "100 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 40 μm + Top coat 60 μm" → "100 μm").
  - If a range is specified → extract as range (e.g., "80–120 μm").
  - If not specified → leave blank.

# P-6
# Surface Treatment & Painting
# Paint Thickness of Mechanism Box
# "Specifies whether the customer defines the required paint thickness or alternative surface treatment for the mechanism box of high-voltage switchgear. The mechanism box houses operating components such as spring or hydraulic mechanisms and is typically mounted outside the main GIS, DTB, or HGIS enclosure. The specification may define a total dry film thickness (DFT), a layer-wise coating system, or an alternative treatment (e.g., zinc plating, galvanization) instead of painting."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness or surface treatment for the mechanism box (or operating mechanism housing) in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - mechanism box
  - operating mechanism housing
  - control box (if contextually linked to mechanical parts)
  - paint thickness
  - DFT
  - dry film thickness
  - primer + top coat
  - minimum microns
  - μm
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "80 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 30 μm + Top coat 50 μm" → "80 μm").
  - If a range is specified → extract as range (e.g., "70–100 μm").
  - If not specified → leave blank.

# P-7
# Surface Treatment & Painting
# Surface Preparation Standard
# "Specifies whether the customer defines the required surface preparation standard before painting (e.g., SSPC-SP 5, SSPC-SP 10, SSPC-SP 8). This refers to the cleaning and preparation of steel surfaces prior to coating, which affects corrosion protection and paint adhesion."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of surface preparation standards in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - SSPC-SP 5
  - SSPC-SP 10
  - SSPC-SP 8
  - White Metal Blast Cleaning
  - Near White Blast Cleaning
  - Pickling
- Output Rules:
  - If a single standard is specified → extract as text (e.g., "SSPC-SP 5 White Metal Blast Cleaning").
  - If multiple standards are mentioned → list all in order (e.g., "SSPC-SP 5, SSPC-SP 10").
  - If not specified → leave blank.

# P-8
# Surface Treatment & Painting
# Surface Treatment and Thickness of Support Structure
# "Specifies whether the customer defines the required surface treatment and/or paint thickness for the support structure of high-voltage switchgear. The support structure refers to steel frames or stands that hold GIS, DTB, or HGIS equipment. The specification may require hot-dip galvanizing, painting with a defined dry film thickness (DFT), or a duplex system (galvanizing + painting) for corrosion protection."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of:
  - Hot-dip galvanizing (including zinc coating thickness)
  - Painting system and DFT
  - Duplex system (galvanizing + painting)
- Accepted keywords:
  - support structure
  - steel frame
  - hot-dip galvanizing
  - zinc coating
  - duplex system
  - paint thickness
  - DFT
  - μm
- Output Rules:
  - If unconditional → extract treatment and thickness (e.g., "Hot-dip galvanizing + 100 μm painting").
  - If conditional → extract treatment and thickness with condition in parentheses:
    - Example: Hot-dip galvanizing (Duplex system required if within 25 km from coast: + 80 μm painting)
  - If not specified → leave blank.

# P-9
# Surface Treatment & Painting
# Duplex System Requirement
# "Specifies whether the customer requires a duplex system (hot-dip galvanizing + painting) for corrosion protection and under what conditions. Duplex systems are typically applied in highly corrosive environments such as coastal areas, industrial zones, or as per ISO 12944 corrosion category (e.g., C5-M)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the specification.
- Identify any condition or environment that triggers duplex system requirement:
  - Coastal installation
  - High salinity or marine environment
  - ISO 12944 category (C4, C5-M)
- Accepted keywords:
  - duplex system
  - galvanizing + painting
  - coastal
  - marine
  - ISO 12944
  - corrosion category
- Output Rules:
  - If duplex system is required → extract as text (e.g., "Duplex system required").
  - If condition is specified → include condition (e.g., "Required for coastal installation (within 25 km)").
  - If not specified → leave blank.

# Q-1
# Local Control Cubicle
# Installation Area
# "Specifies where the Local Control Cubicle (LCC) for high-voltage switchgear shall be installed according to the customer specification. This includes whether the LCC is installed outdoors, inside an outdoor container, in a GIS room, or near outdoor GIS equipment. The requirement may also define environmental conditions or physical constraints for installation."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Installation location type:
    - Outdoor (e.g., “LCC shall be installed outdoors”)
    - Outdoor container (e.g., “LCC inside container”)
    - GIS room (e.g., “LCC in GIS building”)
    - Near outdoor GIS (e.g., “LCC adjacent to GIS yard”)
- Accepted keywords:
  - outdoor installation
  - container
  - GIS room
  - GIS building
  - near GIS
- Output Rules:
  - If location clearly specified → 
    - "Installation area: Outdoor",
    - "Installation area: GIS room",
    - "Installation area: Outdoor container"
  - If multiple options allowed → list all:
    - "Installation area: Outdoor or GIS room"
  - If not specified → leave blank.

# Q-2
# Local Control Cubicle
# Installation Type
# "Specifies whether the customer defines the installation type of the Local Control Cubicle (LCC) for high-voltage switchgear. The LCC may be freestanding (separate from the GIS) or integrated (attached or built into the GIS structure). The specification may state one option explicitly or allow both. This requirement affects panel design, wiring layout, and enclosure mounting."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of installation type for LCC in sections related to control panels or cubicles.
- Accepted keywords:
  - Local Control Cubicle
  - LCC
  - Local Control Panel
  - freestanding
  - integrated
  - attached to GIS
  - separate panel
- Output Rules:
  - If only freestanding is mentioned → extract: Freestanding
  - If only integrated is mentioned → extract: Integrated
  - If both are allowed → extract: Freestanding or Integrated
  - If not specified → leave blank.

# Q-3
# Local Control Cubicle
# Protection Degree
# "Specifies whether the customer defines the required protection degree (e.g., IP rating) for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. The protection degree may be defined based on IEC 60529 (e.g., IP41, IP54) and may vary depending on the switchgear type or installation location, such as indoor GIS or outdoor HGIS. The specification may assign fixed ratings or define minimum values per installation condition."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of protection degree for LCC in sections related to enclosure protection or environmental requirements.
- Accepted keywords:
  - IP rating
  - protection degree
  - IEC 60529
  - IP41, IP54, IP55, etc.
- Output Rules:
  - If one rating applies to all conditions → extract as single value (e.g., "IP54").
  - If different ratings apply for GIS vs HGIS → extract both with conditions:
    - Example: IP41 (GIS), IP54 (HGIS)
  - If minimum rating is specified → extract as "Minimum IP54".
  - If not specified → leave blank.

# Q-4
# Local Control Cubicle
# Enclosure Material
# "Specifies whether the customer defines the required enclosure material for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. The enclosure material may be mild steel, stainless steel, or aluminum, and the choice often depends on environmental conditions (e.g., indoor vs outdoor, corrosive areas) and protection requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of enclosure material for LCC in sections related to cabinet construction or corrosion protection.
- Accepted keywords:
  - enclosure material
  - mild steel
  - stainless steel
  - aluminum
  - galvanized steel
  - non-ferrous material
- Output Rules:
  - If one material is specified → extract as text (e.g., "Stainless steel").
  - If multiple options are allowed → extract all (e.g., "Mild steel or stainless steel").
  - If not specified → leave blank.

# Q-5
# Local Control Cubicle
# Duplex System Requirement
# "Specifies whether the customer requires a duplex system (hot-dip galvanizing + painting) for the Local Control Cubicle (LCC) and under what conditions. Duplex systems are typically applied in highly corrosive environments such as coastal areas, industrial zones, or as per ISO 12944 corrosion category (e.g., C5-M)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of duplex system requirement for LCC and any associated condition:
  - Coastal installation
  - High salinity or marine environment
  - ISO 12944 category (C4, C5-M)
- Accepted keywords:
  - duplex system
  - galvanizing + painting
  - coastal
  - marine
  - ISO 12944
  - corrosion category
- Output Rules:
  - If duplex system is required unconditionally → extract as text (e.g., "Duplex system required").
  - If condition is specified → include condition in parentheses:
    - Example: Duplex system required (for coastal installation within 25 km)
  - If not specified → leave blank.

# Q-6
# Local Control Cubicle
# External Color
# "Specifies the required external color of the Local Control Cubicle (LCC) used with high-voltage switchgear (66kV and above), including any RAL code, ANSI designation, or general color description such as light grey or beige. Only requirements clearly associated with LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of external color for Local Control Cubicle (LCC), Local Control Cabinet, Local Control Panel, or High-voltage Control Cabinet.
- Accepted keywords:
  - Local Control Cubicle color
  - Local Control Cabinet color
  - Local Control Panel color
  - High-voltage Control Cabinet color
  - RAL
  - ANSI color code
  - light grey, beige
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light grey").
  - If not specified → leave blank.

# Q-7
# Local Control Cubicle
# Internal Color
# "Specifies the required internal color of the Local Control Cubicle (LCC) used with high-voltage switchgear (66 kV and above), including any RAL code, ANSI designation, or general color description such as white or light grey. Only requirements clearly associated with the interior of LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of internal color for Local Control Cubicle (LCC), Local Control Cabinet, Local Control Panel, or High-voltage Control Cabinet.
- Accepted keywords:
  - internal color of LCC
  - inside of local control cabinet
  - interior shall be painted
  - RAL
  - white, light grey
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "White").
  - If not specified → leave blank.

# Q-8
# Local Control Cubicle
# Paint Thickness
# "Specifies whether the customer defines the required paint thickness for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. LCC refers to the external panel or cabinet that contains protection and control devices. The paint system may be defined by a total dry film thickness (DFT), a layer-wise structure, or a specific coating method (e.g., powder coating, epoxy paint). In some cases, the specification may refer to manufacturer’s standard or require matching the main switchgear enclosure."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness or coating method for LCC in sections related to painting or finishing.
- Accepted keywords:
  - paint thickness
  - DFT
  - dry film thickness
  - powder coating
  - epoxy paint
  - μm
  - same as enclosure
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "80 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 30 μm + Top coat 50 μm" → "80 μm").
  - If only coating method is specified → extract method (e.g., "Powder coating").
  - If reference to another item (e.g., "same as enclosure") → extract as "Same as enclosure".
  - If not specified → leave blank.

# Q-9
# Local Control Cubicle
# Inclusion of BCU / IED in LCC
# "Specifies whether the customer requires the Bay Control Unit (BCU) or Intelligent Electronic Device (IED) to be installed inside the Local Control Cubicle (LCC) of high-voltage switchgear. The requirement may be stated directly (e.g., BCU shall be mounted in LCC) or indirectly (e.g., LCC to include IEC 61850-compliant IEDs). Inclusion of BCU or IED affects wiring, communication interface, and functional integration of the switchgear control system."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of BCU or IED inclusion in LCC in sections related to control system or cubicle configuration.
- Accepted keywords:
  - BCU
  - Bay Control Unit
  - IED
  - Intelligent Electronic Device
  - IEC 61850
  - installed in LCC
  - housed inside control cabinet
- Output Rules:
  - If explicitly included → extract: Included
  - If explicitly excluded (e.g., “by others” or “to be installed elsewhere”) → extract: Not included
  - If ambiguous but implied (e.g., “BCU to be connected to LCC terminal blocks”) → extract: Possibly included
  - If not specified → leave blank.

# Q-10
# Local Control Cubicle
# Protection Devices in LCC
# "Specifies whether the customer requires protection-related devices to be installed inside the Local Control Cubicle (LCC) of high-voltage switchgear. These devices may include Pole Discrepancy Switch, Trip Coil Supervision Relay, Lockout Relay, Protection Unit, or other auxiliary protection components. Inclusion of these devices affects wiring, space allocation, and functional integration of the control system."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of protection devices in LCC in sections related to protection, control, or auxiliary equipment.
- Accepted keywords:
  - Pole Discrepancy Switch
  - Trip Coil Supervision Relay
  - Lockout Relay
  - Bay Protection Unit / BPU
  - Fuse Failure Relay
  - Auxiliary protection device
- Output Rules:
  - If one or more devices are specified → extract all as a comma-separated list (e.g., "Pole Discrepancy Switch, Trip Coil Supervision Relay").
  - If not specified → leave blank.

# Q-11
# Local Control Cubicle
# Control Cable Connection Type
# "Specifies whether the customer defines the required method for connecting control cables to the Local Control Cubicle (LCC) of high-voltage switchgear. The requirement may include the cable entry direction (bottom, top, side) and the connection method (gland type or plug-in type). This affects cubicle design, gland plate arrangement, sealing, and cable routing."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of:
  - Cable entry direction (bottom, top, side)
  - Connection method (gland type, plug-in type, jack type)
- Accepted keywords:
  - bottom entry
  - top entry
  - side entry
  - cable gland
  - gland plate
  - plug-in connection
  - pre-fabricated cable
  - connector type
  - jack type (rare, but include for completeness)
- Output Rules:
  - If only entry type is specified → extract as text (e.g., "Bottom entry").
  - If only connection method is specified → extract as text (e.g., "Gland type" or "Plug-in type").
  - If both are specified → extract combined (e.g., "Bottom entry, Gland type").
  - If not specified → leave blank.

# Q-12
# Local Control Cubicle
# Voltage Detector Requirement
# "Specifies whether the customer requires a voltage detector to be installed in the Local Control Cubicle (LCC) of high-voltage switchgear. A voltage detector provides indication of live status for safety and maintenance purposes. The specification may require a specific type (e.g., capacitive voltage indicator, neon lamp type) or compliance with standards (e.g., IEC 61243)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of voltage detector requirement in sections related to safety devices or LCC equipment.
- Accepted keywords:
  - voltage detector
  - voltage indicator
  - capacitive voltage indicator
  - neon lamp type
  - IEC 61243
- Output Rules:
  - If voltage detector is required → extract as "Voltage detector required" (include type if specified, e.g., "Capacitive voltage indicator").
  - If explicitly excluded → extract "Not required".
  - If not specified → leave blank.

# Q-13
# Local Control Cubicle
# Key Interlock System Requirement
# "Specifies whether the customer requires a key interlock system to be provided in the Local Control Cubicle (LCC) for high-voltage switchgear, and identifies the interlock logic and associated devices. A key interlock system ensures operational safety by mechanically or electrically preventing incorrect switching sequences. The specification may define the number of keys, interlock logic, and the devices involved (e.g., Line Earthing Switch, Disconnector, Circuit Breaker, Capacitor Bank)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for key interlock system
  - Devices involved in interlock (e.g., ES, DS, CB, Capacitor Bank)
  - Logic or sequence (e.g., key released only when ES closed)
- Accepted keywords:
  - key interlock
  - mechanical interlock
  - electrical interlock
  - interlock between ES and DS
  - interlock between CB and Capacitor Bank
  - interlock logic
  - sequence
- Output Rules:
  - If interlock required → extract "Key interlock system required"
  - If devices and logic specified → extract full detail:
    - Key interlock system required (between Line ES and DS; logic: key released when ES closed)
  - If multiple interlocks → list all in parentheses:
    - Key interlock system required (ES–DS; CB–Capacitor Bank)
  - If not specified → leave blank.

# Q-14
# Local Control Cubicle
# Manufacturer / Country of Origin
# "Specifies whether the customer requires a specific manufacturer or country of origin for the Local Control Cubicle (LCC) used in high-voltage switchgear. This may include explicit naming of a vendor, restriction to domestic or international suppliers, or requirements for factory origin or sourcing. The specification may also indicate whether the LCC shall be supplied by the switchgear manufacturer or by a designated third-party vendor."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer name (e.g., ABB, Siemens, Hyundai Electric)
  - Country of origin (e.g., Korea, Japan, EU)
  - Nationality requirement (e.g., local vendor only, domestic manufacturing)
  - Supply condition (e.g., LCC shall be supplied by switchgear manufacturer)
- Accepted keywords:
  - manufacturer
  - vendor
  - supplier
  - country of origin
  - domestic
  - local vendor
  - approved manufacturer
  - supplied by switchgear manufacturer
- Output Rules:
  - If manufacturer specified → extract as:
    - "Manufacturer: [Name]"
    - (e.g., Manufacturer: Hyundai Electric)
  - If country of origin specified → extract as:
    - "Country of origin: [Country]"
    - (e.g., Country of origin: Korea)
  - If requirement is local/domestic only → extract as:
    - "Local vendor only"
  - If not specified → leave blank.

# R-1
# Control Cable
# Insulation Type
# "Specifies whether the customer requires a specific insulation material for control or power cables used in high-voltage switchgear. Valid insulation materials include PVC (polyvinyl chloride), XLPE (cross-linked polyethylene), and EPR (ethylene propylene rubber), or may be referenced via standard cable codes (e.g., CVV, CCV)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Insulation material (e.g., PVC, XLPE, EPR)
  - Cable type codes (e.g., CVV, CCV, CVVS) and map to insulation type if possible
- Accepted keywords:
  - PVC
  - XLPE
  - EPR
  - CVV
  - CCV
  - CVVS
- Output Rules:
  - If one or more insulation materials specified → extract as list:
    - "Insulation type: PVC, XLPE"
  - If only cable codes provided → extract codes and map if possible:
    - "Insulation type: CVV (PVC), CCV (XLPE)"
  - If not specified → leave blank.

# R-2
# Control Cable
# Steel Armored Shield Requirement
# "Specifies whether the customer requires control cables used in high-voltage switchgear to include a steel armored shield. This may be specified to enhance mechanical protection, improve EMI shielding, or comply with installation conditions such as underground routing or outdoor exposure. The requirement may apply to all control cables, or only to specific portions (e.g., external, underground, or field-installed cables)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for steel armor (e.g., steel wire armored, SWA)
  - Conditional requirement (e.g., only for underground or outdoor cables)
- Accepted keywords:
  - steel armored
  - steel wire armored (SWA)
  - armored control cable
  - metallic sheath
  - mechanical protection
- Output Rules:
  - If steel armor required → "Steel armored shield: Required"
  - If conditionally required → "Steel armored shield: Conditionally required (e.g., underground cables)"
  - If not specified → leave blank.

# R-3
# Control Cable
# Jacket Material
# "Specifies whether the customer requires a specific jacket (outer sheath) material for control or power cables used in high-voltage switchgear systems. The jacket provides mechanical protection, environmental resistance, and may contribute to flame retardancy or chemical resistance. Common jacket materials include PVC, PE, neoprene, LSZH, and hypalon."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Jacket material (e.g., PVC, PE, neoprene, LSZH, hypalon)
  - Multiple options if listed
- Accepted keywords:
  - PVC
  - PE
  - neoprene
  - LSZH
  - hypalon
  - outer sheath
  - cable jacket
- Output Rules:
  - If one or more jacket materials specified → extract as list:
    - "Jacket material: PVC, LSZH"
  - If multiple options allowed → list all:
    - "Jacket material: PVC or neoprene"
  - If not specified → leave blank.

# R-4
# Control Cable
# Halogen-Free Requirement
# "Specifies whether the customer requires control or power cables used in high-voltage switchgear to be halogen-free (also referred to as LSZH: Low Smoke Zero Halogen). This requirement is often applied to reduce toxic gas emission in case of fire and to comply with environmental or safety standards. It may apply to all cable types or only to specific installations (e.g., indoor, tunnel, control room)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for halogen-free or LSZH cables
  - Conditional requirement (e.g., only for indoor or tunnel installations)
- Accepted keywords:
  - halogen-free
  - LSZH
  - low smoke zero halogen
  - zero halogen material
  - non-toxic under fire
- Output Rules:
  - If halogen-free required → "Halogen-free: Required"
  - If conditionally required → "Halogen-free: Conditionally required (e.g., indoor cables only)"
  - If not specified → leave blank.

# R-5
# Control Cable
# Anti-Termite Requirement
# "Specifies whether the customer requires control or power cables used in high-voltage switchgear systems to be resistant to termites or other insects. Anti-termite protection is typically applied through the use of treated PVC or special sheath materials and is often required for underground or outdoor installations in regions with termite risk."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for anti-termite protection
  - Conditional requirement (e.g., only for underground or outdoor cables)
- Accepted keywords:
  - anti-termite
  - termite-proof cable
  - termite resistant
  - PVC treated against insects
- Output Rules:
  - If anti-termite required → "Anti-termite: Required"
  - If conditionally required → "Anti-termite: Conditionally required (e.g., underground cables)"
  - If not specified → leave blank.

# S-1
# Monitoring System
# Manufacturer-Specified Monitoring System
# "Specifies whether the customer requires or permits a specific manufacturer’s monitoring system to be applied to the high-voltage switchgear. It includes cases where the name of a vendor, brand, or model of the monitoring system is explicitly stated or referenced. The requirement may appear in the main technical specification, appendix, or in approved vendor list (AVL)-related documents. Only extract the manufacturer name and system type if clearly mentioned."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer or brand name (e.g., Qualitrol, Omicron, GE, ABB, Siemens)
  - Monitoring system type (e.g., PDMS, gas monitoring, CB monitoring)
- Accepted keywords:
  - monitoring system
  - vendor name
  - approved vendor
  - preferred monitoring system
  - brand-specific monitoring
- Output Rules:
  - If clearly required → "Required: [Manufacturer Name] ([System Type])"
    (e.g., Required: Qualitrol (Gas Monitoring))
  - If only permitted or preferred → "Preferred: [Manufacturer Name] ([System Type])"
  - If not specified → leave blank.

# S-2
# Monitoring System
# Partial Discharge (PD) Sensor Provision
# "Specifies whether the customer requires PD sensors to be installed on the high-voltage switchgear or prepared for future connection. The requirement may refer to UHF sensors, couplers, or test ports for PD monitoring. The condition may be stated as mandatory, optional, or not required."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Quantity or scope:
    - Every compartment
    - Each bay
    - Specific number per bay (e.g., 1 per bay)
    - Specific section (e.g., cable termination)
- Accepted keywords:
    - every compartment
    - each gas section
    - each bay
    - PD sensor per [unit]
    - minimum [X] sensors per bay
- Output Rules:
    - If requirement is general → "PD sensor placement: Every compartment"
    - If per bay → "PD sensor placement: Each bay"
    - If numeric → "PD sensor placement: [X] sensors per bay"
    - If specific section → "PD sensor placement: Cable termination section"
    - If not specified → leave blank.

# S-3
# Monitoring System
# PD Sensor Quantity / Placement
# "Specifies whether the customer defines the required quantity or installation scope of PD sensors for high-voltage switchgear. This includes requirements such as sensors in every gas compartment, each bay, or specific sections like cable terminations."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Quantity or scope:
    - Every compartment
    - Each bay
    - Specific number per bay (e.g., 1 per bay)
    - Specific section (e.g., cable termination)
- Accepted keywords:
  - every compartment
  - each gas section
  - each bay
  - PD sensor per [unit]
  - minimum [X] sensors per bay
- Output Rules:
  - If requirement is general → **"PD sensor placement: Every compartment"**
  - If per bay → **"PD sensor placement: Each bay"**
  - If numeric → **"PD sensor placement: [X] sensors per bay"**
  - If specific section → **"PD sensor placement: Cable termination section"**
  - If not specified → leave blank.

# S-4
# Monitoring System
# Online PD Monitoring System
# "Specifies whether the customer requires an online Partial Discharge (PD) monitoring system to be installed on the high-voltage switchgear. This includes continuous monitoring solutions integrated into GIS or AIS equipment, typically involving UHF sensors, couplers, and real-time diagnostic software. The requirement may be mandatory, optional, or not required."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “online PD monitoring system shall be provided”)
    - Optional / provision only (e.g., “interface for online PD monitoring”)
    - Not required (e.g., “online PD monitoring not required”)
- Accepted keywords:
  - online PD monitoring
  - continuous PD monitoring
  - PDMS (Partial Discharge Monitoring System)
  - UHF monitoring system
  - real-time PD monitoring
- Output Rules:
  - If required → "Online PD monitoring: Required"
  - If provision only → "Online PD monitoring: Provision only (interface or preparation)"
  - If explicitly excluded → "Online PD monitoring: Not required"
  - If not specified → leave blank.

# S-5
# Monitoring System
# Portable PD Monitoring Device
# "Specifies whether the customer requires a portable Partial Discharge (PD) monitoring device to be supplied for use with high-voltage switchgear. This includes handheld or portable PD analyzers for offline or on-site diagnostics. The requirement may be mandatory, optional, or not required, and may also specify the type or brand of the device."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “portable PD device shall be supplied”)
    - Optional (e.g., “may be provided upon request”)
  - Device details if specified (e.g., brand, model)
- Accepted keywords:
  - portable PD monitoring
  - handheld PD analyzer
  - PD test kit
  - offline PD measurement device
- Output Rules:
  - If required → "Portable PD device: Required"
  - If optional → "Portable PD device: Optional"
  - If explicitly excluded → "Portable PD device: Not required"
  - If brand/model mentioned → append in parentheses (e.g., "Portable PD device: Required (Omicron MPD600)")
  - If not specified → leave blank.

# S-6
# Monitoring System
# VFTO Capacitive Sensor Provision
# "Specifies whether the customer requires VFTO (Very Fast Transient Overvoltage) capacitive sensors to be installed or provisioned on the high-voltage switchgear for monitoring purposes. VFTO sensors are typically used in GIS applications to detect transient overvoltages during switching operations. The requirement may be mandatory, optional, or not required, and may also specify the installation scope (e.g., per bay, per compartment)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “VFTO sensor shall be provided”)
    - Provision only (e.g., “interface for VFTO sensor”)
  - Scope if specified (e.g., per bay, per compartment)
- Accepted keywords:
  - VFTO sensor
  - VFTO capacitive sensor
  - VFTO monitoring
  - VFTO detection
  - transient overvoltage sensor
- Output Rules:
  - If required → "VFTO sensor: Required"
  - If provision only → "VFTO sensor: Provision only (interface or preparation)"
  - If scope mentioned → append in parentheses (e.g., "VFTO sensor: Required (per bay)")
  - If not specified → leave blank.

# S-7
# Monitoring System
# Circuit Breaker Condition Monitoring System
# "Specifies whether the customer requires a circuit breaker (CB) condition monitoring system for high-voltage switchgear. The system continuously monitors breaker performance indicators such as operation count, operating time, contact wear, travel curve profile, stored energy mechanism status, spring charging motor behavior, or insulation condition. It is typically used for predictive maintenance and lifetime assessment."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “CB condition monitoring system shall be provided”)
    - Provision only (e.g., “interface for CB diagnostics shall be included”)
Ignore:
  - Basic breaker operations (open/close) without diagnostics
  - Protection relays or trip signals unless explicitly linked to condition monitoring
- Accepted keywords:
  - CB condition monitoring
  - breaker diagnostics
  - CB health monitoring
  - lifetime monitoring
  - mechanical wear monitoring
  - travel time analysis
  - contact wear detection
- Output Rules:
  - If required → "CB condition monitoring: Required"
  - If provision only → "CB condition monitoring: Provision only"
  - If not specified → leave blank.

# S-8
# Monitoring System
# Arc Detection System
# "Specifies whether the customer requires an arc detection system to be installed in the high-voltage switchgear. This includes optical arc detection using light sensors or other technologies capable of detecting internal arc faults and initiating fast breaker operation or protection signaling. Valid implementations may include fiber-optic sensors, light-based detectors, or any system explicitly linked to fault trip logic."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Arc detection system shall be provided”)
    - Provision only (e.g., “Provision for future arc detection system shall be included”)
- Accepted keywords:
  - arc detection system
  - internal arc detection
  - optical arc sensor
  - light-based arc detection
  - fiber optic arc sensor
  - arc flash sensor
  - light-triggered trip
- Output Rules:
  - If required → "Arc detection system: Required"
  - If provision only → "Arc detection system: Provision only"
  - If not specified → leave blank.

# S-9
# Monitoring System
# Camera-Based Position Verification System for DS/ES
# "Specifies whether the customer requires a camera-based system or other advanced visual verification method to monitor the open/close position of disconnector (DS) or earthing switch (ES). Standard auxiliary switches or mechanical indicator windows shall NOT be considered valid monitoring systems for this item."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (camera-based or sensor-based)
    - Provision only (interface or preparation)
  - Monitoring method:
    - Camera-based
    - Sensor-based (e.g., proximity sensor, optical sensor)
- Ignore:
  - Any mention of auxiliary contacts or mechanical indicator windows (basic functionality)
- Accepted keywords:
  - camera monitoring for DS/ES
  - optical position sensor
  - proximity sensor for DS/ES
  - position verification system
  - DS/ES position feedback to SCADA (only if camera/sensor-based)
- Output Rules:
  - If required → "DS/ES position monitoring: Required (camera-based)" or "Required (sensor-based)"
  - If provision only → "DS/ES position monitoring: Provision only"
  - If not specified → leave blank.

# S-10
# Monitoring System
# Surge Arrester Leakage Current Monitoring System
# "Specifies whether the customer requires a monitoring system for the leakage current of surge arresters (lightning arresters) in high-voltage switchgear. Acceptable systems may include analog gauges, local indicators, or digital sensors. Only extract a requirement if it is explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (continuous monitoring with SCADA or trending analysis)
    - Required (periodic/manual monitoring via analog gauge or local indicator)
    - Provision only (interface or preparation for future monitoring)
- Accepted keywords:
  - leakage current monitoring
  - surge arrester monitoring
  - lightning arrester current detection
  - LA leakage current sensor
  - online leakage current monitor
  - resistive current component monitoring
- Output Rules:
  - If continuous monitoring required → "Surge arrester monitoring: Required (continuous)"
  - If periodic/manual monitoring required → "Surge arrester monitoring: Required (manual)"
  - If provision only → "Surge arrester monitoring: Provision only"
  - If not specified → leave blank.

# S-11
# Monitoring System
# Online Gas Monitoring System
# "Specifies whether the customer requires an online gas monitoring system for high-voltage switchgear. Online systems continuously monitor and transmit gas-related parameters such as SF₆ pressure, temperature, density, humidity, or decomposition byproducts in real-time using digital communication protocols. Mechanical devices like local gauges or two-stage relays are NOT considered online monitoring unless explicitly stated as remote or SCADA-integrated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (real-time or SCADA-integrated system)
    - Provision only (interface or preparation for future system)
  - Parameters if specified (e.g., pressure, temperature, density, humidity, dew point, decomposition gases)
- Ignore:
  - Local-only gauges or mechanical relays without remote access
  - Mentions of “continuous monitoring” without SCADA or remote communication
- Accepted keywords:
  - online gas monitoring
  - real-time SF₆ monitoring
  - SCADA-integrated gas monitoring
  - gas analyzer
  - dew point sensor
  - decomposition gas detection
- Output Rules:
  - If required → "Online gas monitoring: Required - [parameters]"
    - (e.g., Online gas monitoring: Required - pressure, temperature, humidity)
  - If provision only → "Online gas monitoring: Provision only - [parameters]"
  - If not specified → leave blank.

# S-12
# Monitoring System
# Gas Density Transmitter
# "Specifies whether the customer requires a gas density transmitter (digital type) for high-voltage switchgear. Unlike mechanical gas density monitors or relays, transmitters provide real-time measurement and remote communication of SF₆ gas parameters (pressure, temperature, density) to SCADA or monitoring systems. Only extract if explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Gas density transmitter shall be provided”)
    - Provision only (e.g., “Interface for transmitter shall be included”)
  - Parameters if specified (e.g., pressure, temperature, density)
- Ignore:
  - References to mechanical gas density monitors or relays without remote communication
- Accepted keywords:
  - gas density transmitter
  - SF₆ transmitter
  - digital gas monitoring
  - remote gas monitoring
  - SCADA-integrated gas transmitter
- Output Rules:
  - If required → "Gas density transmitter: Required"
    - (append parameters if mentioned, e.g., "Gas density transmitter: Required (pressure, temperature)")
  - If provision only → "Gas density transmitter: Provision only"
  - If not specified → leave blank.

# S-13
# Monitoring System
# Gas Density Monitor - Manufacturer
# "Specifies whether the customer requires a specific manufacturer or brand for the Gas Density Monitor used in high-voltage switchgear. Commonly specified brands include Trafag, WIKA, or other well-known pressure switch providers."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer name (e.g., Trafag, WIKA)
  - Multiple acceptable brands if listed
- Ignore:
  - Mentions like “any equivalent,” “supplier standard,” or “no preference”
- Accepted keywords:
  - gas density monitor
  - SF₆ density relay
  - pressure switch
  - Trafag
  - WIKA
  - specified manufacturer
- Output Rules:
  - If manufacturer specified → "Gas density monitor manufacturer: [Brand]"
    - (e.g., Gas density monitor manufacturer: WIKA)
  - If multiple brands → list all (e.g., Gas density monitor manufacturer: Trafag, WIKA)
  - If not specified → leave blank.

# S-14
# Monitoring System
# Gas Density Monitor – Number of Alarm Stages
# "Specifies whether the customer defines the number of alarm stages for the gas density monitor in high-voltage switchgear. Alarm stages typically correspond to different SF₆ gas pressure or density thresholds, such as ‘Alarm’ and ‘Lockout.’ Common configurations include two-stage (Alarm + Lockout) or three-stage (Alarm 1, Alarm 2, Lockout). Requirements may vary by circuit breaker, bay, or section. Extract all distinct requirements with their applicable scope."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Number of alarm stages (e.g., 2-stage, 3-stage)
  - Names or functions of stages if specified (e.g., Alarm, Lockout)
  - Scope: For circuit breaker, for bay, for bus section, etc.
- Ignore:
  - General mentions of gas density monitoring without stage details
- Accepted keywords:
  - alarm stage
  - two-stage alarm
  - three-stage alarm
  - lockout stage
  - SF₆ alarm levels
  - per breaker / per bay / per section
- Output Rules:
  - If specified → list all requirements with scope:
    - Gas density monitor alarm stages:
      - [Number] ([Stage details]) – [Scope]
  - Examples:
    - Gas density monitor alarm stages:
      - 2 (Alarm, Lockout) – For Circuit Breaker
      - 3 (Alarm 1, Alarm 2, Lockout) – For Bus Section
  - If multiple options allowed → include all
  - If not specified → leave blank.

# S-15
# Monitoring System
# Gas Density Monitor - Contact Types by Equipment
# "Specifies whether the customer defines the type and number of electrical contacts (e.g., NO, NC) for gas density monitors used in high-voltage switchgear, and whether these requirements vary by equipment (such as circuit breaker, bus section, disconnector). Contacts are typically used for alarm and lockout signaling. Extract all distinct requirements with their applicable scope."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Contact type(s): Normally Open (NO), Normally Closed (NC)
  - Number of contacts per function (e.g., Alarm: 2 NO + 1 NC)
  - Scope: For circuit breaker, for bay, for bus section, etc.
- Ignore:
  - General mentions without specifying contact type or quantity
- Accepted keywords:
  - contact type
  - NO / NC contacts
  - alarm contact
  - lockout contact
  - per breaker / per bay / per section
- Output Rules
  - If specified → list all requirements with scope:
    - Gas density monitor contacts:
      - [Function]: [Quantity and Type] – [Scope]
  - Examples:
    - Gas density monitor contacts:
      - Alarm: 2 NO + 1 NC – For Circuit Breaker
      - Lockout: 1 NO – For Bus Section
  - If multiple options allowed → include all
  - If not specified → leave blank.

# S-16
# Monitoring System
# Gas Density Monitor - High Alarm Setting
# "Specifies whether the customer explicitly requires a high-pressure alarm setting (high alarm) for gas density monitoring, particularly for compartments where higher internal pressure may exist (e.g., circuit breaker compartments). While most specifications focus on gas leakage (low alarm), high alarm may be required in CB-related compartments to detect abnormal overpressure. Extract the requirement and the specified threshold if stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether a high alarm is required (e.g., “High alarm shall be provided”)
  - Target compartment(s) (e.g., “Circuit breaker compartment,” “CB left/right side”)
  - Alarm threshold value if specified (e.g., “set at 0.70 MPa”)
- Ignore:
  - General mentions of low alarm or leakage alarm without high alarm
- Accepted keywords:
  - high alarm
  - overpressure alarm
  - excessive gas pressure
  - high-pressure detection
  - high-pressure setting
  - CB compartment high alarm
- Output Rules:
  - If specified →
    - Gas density monitor high alarm:
      - Required – [Scope] (Threshold: [Value])
  - Examples:
    - Gas density monitor high alarm:
      - Required – For Circuit Breaker Compartment (Threshold: 0.70 MPa)
  - If multiple compartments → list all
  - If not specified → leave blank.

# T-1
# Service Continuity Requirement
# MRE Requirement
# "Specifies whether the customer provides an MRE code (Maintenance, Repair, Extension) according to IEC 62271-203. The MRE code defines the number of busbar sections and feeders that can remain in service during maintenance, repair, extension, or on-site dielectric tests. Extract the code if explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - MRE code (e.g., “MRE 1.2”)
  - If multiple codes for different bays → list all
- Accepted keywords:
  - MRE
  - Maintenance, Repair, Extension code
- Output Rules:
  - If specified →  
    - MRE Requirement: [Code]
  - Example:
    - MRE Requirement: MRE 1.2
  - If not specified → leave blank.

# T-2
# Service Continuity Requirement
# Additional Buffer Bus Requirement
# "Specifies whether the customer requires an additional buffer bus to maintain service continuity during maintenance or extension, and identifies the scope (e.g., main bus, specific bay, section). Extract only if explicitly stated."
# AI Extraction Logic 
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (Required / Not required)
  - Scope: Where the buffer bus is required (e.g., main bus, CB bay, section)
- Accepted keywords:
  - buffer bus
  - additional bus for continuity
  - auxiliary bus for maintenance
- Output Rules
  - If specified →
    - Additional buffer bus: Required – [Scope]
  - Examples:
    - Additional buffer bus: Required – For Main Bus Section
    - Additional buffer bus: Required – Between CB and Main bus DS
  - If not specified → leave blank.

# T-3
# Service Continuity Requirement
# Service Continuity During Bus DS/ES Maintenance
# "Specifies whether the customer requires service continuity to be maintained during maintenance of the bus disconnector switch (DS) or earthing switch (ES). This includes planned maintenance activities that may affect the busbar section or connected feeders. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during DS/ES maintenance. General statements about equipment-level isolation or mechanical independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
  - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - DS maintenance
  - ES maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during Bus DS/ES maintenance: [Requirement] – [Scope]
  - Examples:
    - Service continuity during Bus DS/ES maintenance: All other feeders must remain in service
    - Service continuity during Bus DS/ES maintenance: Minimum one line and one transformer feeder to remain in service
    - Service continuity during Bus DS/ES maintenance: Partial outage permitted during DS/ES maintenance
  - If not specified → leave blank.

# T-4
# Service Continuity Requirement
# Service Continuity During CB Maintenance
# "Specifies whether the customer requires service continuity to be maintained during circuit breaker (CB) maintenance. This includes planned maintenance activities that may affect feeders or transformers connected to the CB. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during CB maintenance. General statements about mechanical isolation or equipment-level independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
  - Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
  - Identify:
    - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
    - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - CB maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during CB maintenance: [Requirement] – [Scope]
  - Examples:
    - Service continuity during CB maintenance: All other feeders must remain in service
    - Service continuity during CB maintenance: Minimum one line and one transformer feeder to remain in service
    - Service continuity during CB maintenance: Partial outage permitted during CB maintenance
  - If not specified → leave blank.

# T-5
# Service Continuity Requirement
# Service Continuity During Main Bus Maintenance
# "Specifies whether the customer requires service continuity to be maintained during main bus maintenance. This includes planned maintenance activities that may affect the entire busbar section or connected feeders. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during main bus maintenance. General statements about mechanical isolation or equipment-level independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
  - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - main bus maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during Main Bus maintenance: [Requirement] – [Scope]
  - Examples:
    - Service continuity during Main Bus maintenance: All other feeders must remain in service
    - Service continuity during Main Bus maintenance: Minimum one line and one transformer feeder to remain in service
    - Service continuity during Main Bus maintenance: Partial outage permitted during Main Bus maintenance
  - If not specified → leave blank.

# T-6
# Service Continuity Requirement
# Service Continuity During Extension
# "Specifies whether the customer requires service continuity to be maintained during extension of the switchgear. This includes installation of new feeders, bays, or sections added to an existing switchgear lineup. Requirements must explicitly state that the existing system shall remain energized or that no feeder outage is allowed during extension work. General statements about future extensibility or modularity shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Full service continuity required during extension
    - Partial outage allowed during extension
    - Extension requires full shutdown
  - Scope: Which feeders or sections are affected
- Ignore:
  - General references to “future extension,” “modular layout,” or “expandability” without stating continuity during work
- Accepted keywords:
  - switchgear extension
  - expansion
  - addition of feeders/bays
  - no outage allowed during extension
  - feeders remain energized during extension
- Output Rules
  - If specified →
    - Service continuity during extension: [Requirement] – [Scope]
  - Examples:
    - Service continuity during extension: Full service continuity required during extension
    - Service continuity during extension: Partial outage allowed during extension
    - Service continuity during extension: Extension requires full shutdown
  - If not specified → leave blank.

# U-1
# Customer-Specific Requirements
# Manufacturing Country Restriction
# "Specifies whether the customer requires certain equipment (e.g., switchgear, local control panels) to be manufactured in a specific country or prohibits manufacturing in certain countries. Such restrictions typically aim to ensure quality control, compliance with national standards, or strategic sourcing preferences. These requirements may apply to main equipment or subcomponents, such as control cabinets, and are usually enforced through expressions like 'shall be manufactured in [Country]' or 'not permitted to be manufactured in [Country]'."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Country restriction type:
    - Mandatory manufacturing in a specific country
    - Prohibition of manufacturing in certain countries
    - Conditional approval for external manufacturing
  - Scope: Which equipment or component is affected (e.g., switchgear, control cabinet)
- Ignore:
  - Any inference based on supplier location or origin unless explicitly stated
- Accepted keywords:
  - shall be manufactured in [Country]
  - must not be manufactured in [Country]
  - manufacturing outside [Country] not permitted without approval
- Output Rules
  - If specified →
    - Manufacturing Country Requirement: [Country] ([mandatory / preferred / conditional]) – [Scope]
  - Examples:
    - Manufacturing Country Requirement: Spain (preferred) – For Local Control Cabinet
    - Manufacturing Country Requirement: Korea (mandatory) – For Switchgear
    - Manufacturing Country Requirement: China (prohibited) – For Switchgear, LCC
    - Manufacturing Country Requirement: External manufacturing requires RE approval – For Control Panels
  - If not specified → leave blank.

# U-2
# Customer-Specific Requirements
# Customer Audit / Surveillance during Manufacturing
# "Specifies whether the customer explicitly requires the right to perform audits, surveillance inspections, or quality reviews during the manufacturing process of high-voltage switchgear (66kV and above). This includes any access to workshops, in-process inspections, welding inspections, or review of quality records prior to Factory Acceptance Test (FAT). This requirement is distinct from routine FAT attendance and implies additional oversight during fabrication or assembly. Any such manufacturing-stage audit requirement must be identified and cited."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Buyer may audit manufacturing process”)
    - Conditionally Required (e.g., “may audit upon notice”)
    - Not Required (explicitly restricted to FAT only)
  - Scope: Which activities or areas are included (e.g., welding inspection, workshop access)
- Ignore:
  - General mentions of FAT attendance or inspection before shipment
  - Any inference based on common practice or QA statements unless explicitly stated
- Accepted keywords:
  - audit during manufacturing
  - surveillance inspection during fabrication
  - access to production area
  - in-process inspection
- Output Rules
  - If specified →
    - Customer audit during manufacturing: [Required / Conditionally Required / Not Required] – [Scope]
  - Examples:
    - Customer audit during manufacturing: Required – Access to workshop and welding inspection
    - Customer audit during manufacturing: Conditionally Required – Upon prior notice
    - Customer audit during manufacturing: Not Required – Limited to FAT attendance only
  - If not specified → leave blank.

# U-3
# Customer-Specific Requirements
# Nondestructive Test Requirement
# "Specifies whether the project specification explicitly requires any nondestructive testing (NDT) methods to be applied to high-voltage switchgear components such as welded parts, structural frames, pressure vessels, or bushings. NDT methods may include but are not limited to Radiographic Testing (RT), Ultrasonic Testing (UT), Magnetic Particle Testing (MT), or Dye Penetrant Testing (PT). This item ensures compliance with applicable manufacturing standards and quality assurance criteria."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - NDT method(s) required (e.g., RT, UT, MT, PT)
  - If multiple methods are listed → extract all in the order stated
  - If requirement is conditional → mark as “Conditionally Required” and include source phrase
  - If requirement is based on a referenced standard (e.g., ISO 17636 → RT) → extract method and cite standard
- Ignore:
  - Any inference based on typical QA practice or product type unless explicitly stated
- Accepted keywords:
  - Radiographic Testing (RT)
  - Ultrasonic Testing (UT)
  - Magnetic Particle Testing (MT)
  - Dye Penetrant Testing (PT)
  - NDT as per [Standard]
- Output Rules
  - If specified →
    - Nondestructive Test Requirement: [Method(s)] – [Scope or Standard]
  - Examples:
    - Nondestructive Test Requirement: RT – For all weld joints
    - Nondestructive Test Requirement: RT / UT / PT – As per ISO 17636
    - Nondestructive Test Requirement: RT (Conditionally Required) – As per ASME Sec. VIII
  - If not specified → leave blank.

# U-4
# Customer-Specific Requirements
# Impact Recorder Requirement during Transportation
# "Specifies whether the customer requires an Impact Recorder for high-voltage switchgear. The recorder must be a digital or mechanical device that logs acceleration, shock, or vibration during transportation, handling, or installation. Do not extract if the requirement applies only to transformers, medium-voltage equipment, or passive indicators such as shock/tilt labels."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Impact Recorder required
    - Optional or conditional requirement (e.g., “may be required upon request”)
  - Scope: Confirm that it applies to high-voltage switchgear (≥ 66 kV nominal)
- Ignore:
  - Requirements for transformers, MV equipment, or passive devices (shock/tilt labels, shock indicators)
- Accepted keywords:
  - impact recorder
  - shock recorder
  - vibration logger
  - acceleration recorder
- Output Rules
  - If specified →
    - Impact Recorder Requirement: [Required / Conditional] – [Scope]
  - Examples:
    - Impact Recorder Requirement: Required – For high-voltage switchgear during transportation
    - Impact Recorder Requirement: Conditional – May be required upon request
  - If not specified → leave blank.

# U-5
# Customer-Specific Requirements
# Special Conditions for FAT / SAT
# "Specifies whether the customer specifies special conditions or additional requirements for FAT (Factory Acceptance Test) or SAT (Site Acceptance Test) beyond standard procedures. These may include dedicated time for Employer testing, SCADA signal verification, GOOSE simulation, full switchgear control access, or integrated FAT (iFAT), which involves multi-vendor system-level testing. Only extract if the condition is explicitly stated as a requirement."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Special conditions for FAT or SAT such as:
    - Dedicated time for Employer testing (e.g., “minimum 5 working days”)
    - Control access requirements (e.g., “Employer must have full control access during FAT”)
    - Communication tests (e.g., SCADA signal verification, IEC 61850 GOOSE simulation)
    - Integrated FAT (iFAT) or multi-vendor testing requirements
    - Restrictions on supplier activities during Employer testing
- Ignore:
  - General mentions of FAT or SAT without additional conditions
- Accepted keywords:
  - FAT special conditions
  - SAT special conditions
  - iFAT
  - SCADA verification
  - GOOSE simulation
- Output Rules
  - If specified →
    - Special Conditions for FAT / SAT: [Summary of conditions]
  - Examples:
    - Special Conditions for FAT / SAT: Minimum 5 working days for Employer testing; Full control access during FAT; SCADA and GOOSE signal verification required
    - Special Conditions for FAT / SAT: Integrated FAT (iFAT) including multi-vendor system-level testing
  - If not specified → leave blank.

# U-6
# Customer-Specific Requirements
# Seismic Qualification Requirement
# "Specifies whether the customer requires seismic qualification for high-voltage switchgear and whether compliance must be demonstrated through a certified test report, analysis, or both. This includes identifying if the requirement applies to the entire switchgear assembly, specific components (e.g., circuit breaker, LCC), or auxiliary equipment. Extract the method of qualification (test vs. analysis), applicable standard (e.g., IEEE 693, IEC 62271-207), and any performance level (e.g., High, Moderate)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether seismic qualification is required
  - Method of compliance:
    - Test report required
    - Analysis accepted
    - Both required
  - Applicable standard (e.g., IEEE 693, IEC 62271-207)
  - Performance level (e.g., High, Moderate)
  - Scope: Entire switchgear or specific components
- Ignore:
  - General mentions of “robust design” or “earthquake resistance” without specifying qualification method or standard
- Accepted keywords:
  - seismic qualification
  - earthquake test
  - seismic analysis
  - IEEE 693
  - IEC 62271-207
- Output Rules
  - If specified →
    - Seismic Qualification Requirement: [Required] – [Method: Test / Analysis / Both] – [Standard] – [Performance Level] – [Scope]
  - Examples:
    - Seismic Qualification Requirement: Required – Method: Test – Standard: IEEE 693 – Level: High – Scope: Entire Switchgear
    - Seismic Qualification Requirement: Required – Method: Analysis accepted – Standard: IEC 62271-207 – Level: Moderate – Scope: Circuit Breaker only
  - If not specified → leave blank.

# U-7
# Customer-Specific Requirements
# Indoor Installation Clearances (Front)
# "Specifies the minimum clearance required at the front of the high-voltage switchgear for installation and maintenance purposes. This typically refers to the working space between the front of the equipment and any wall or obstruction."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Ignore:
  - Requirements for MV equipment or unrelated components
- Accepted keywords:
  - front clearance
  - working space front
  - installation clearance front
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Front): [Value + Unit] – [Condition if any]
  - Examples:
    - Indoor Installation Clearance (Front): 1500 mm – Minimum working space required
    - Indoor Installation Clearance (Front): 1800 mm – For maintenance access
  - If not specified → leave blank.

# U-8
# Customer-Specific Requirements
# Indoor Installation Clearances (Rear)
# "Specifies the minimum clearance required at the rear of the high-voltage switchgear for installation and maintenance purposes."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Ignore:
  - Requirements for MV equipment or unrelated components
- Accepted keywords:
  - rear clearance
  - working space rear
  - installation clearance rear
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Rear): [Value + Unit] – [Condition if any]
  - Examples:
    - Indoor Installation Clearance (Rear): 1200 mm – For maintenance access
    - Indoor Installation Clearance (Rear): 1000 mm – Minimum clearance behind GIS
  - If not specified → leave blank.

# U-9
# Customer-Specific Requirements
# Indoor Installation Clearances (Side)
# "Specifies the minimum clearance required between the side of the high-voltage switchgear and adjacent walls, doors, or structural elements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Accepted keywords:
  - side clearance
  - lateral clearance
  - clearance to wall/door
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Side): [Value + Unit] – [Condition if any]
  - Examples:
    - Indoor Installation Clearance (Side): 1000 mm – For safe passage
    - Indoor Installation Clearance (Side): 800 mm – Minimum clearance to wall
  - If not specified → leave blank.

# U-10
# Customer-Specific Requirements
# Indoor Installation Clearances (Between Equipment)
# "Specifies the minimum clearance required between adjacent equipment, including: Between GIS bays, Between GIS and operating mechanisms, Between GIS and GIB (Gas-Insulated Bus) sections or other associated components."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType (≥ 66 kV nominal).
- Identify:
  - Clearance value and unit (e.g., mm)
  - Scope: Bay-to-bay, GIS-to-mechanism, GIS-to-GIB
  - Any conditions (e.g., “for maintenance access”)
- Accepted keywords:
  - clearance between bays
  - clearance between equipment
  - spacing between GIS and mechanism
  - GIB clearance
  - bus duct spacing
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Between Equipment): [Value + Unit] – [Scope]
  - Examples:
    - Indoor Installation Clearance (Between Equipment): 2000 mm – Between adjacent GIS bays
    - Indoor Installation Clearance (Between Equipment): 1500 mm – Between GIS and operating mechanism
    - Indoor Installation Clearance (Between Equipment): 1000 mm – Between GIS and GIB section
  - If not specified → leave blank.

# U-11
# Customer-Specific Requirements
# Tank Heater Requirement
# "Specifies whether the customer requires a tank heater for high-voltage switchgear. The tank heater is typically used to prevent condensation or maintain temperature inside the equipment tank during cold conditions."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Tank heater required
    - Optional or conditional requirement (e.g., “may be required upon request”)
  - Scope: Confirm that it applies to high-voltage switchgear (≥ 66 kV nominal)
- Ignore:
  - Requirements for transformers, MV equipment, or unrelated auxiliary heaters
- Accepted keywords:
  - tank heater
  - anti-condensation heater
  - heater for GIS tank
  - heating device for circuit breaker tank
- Output Rules
  - If specified →
    - Tank Heater Requirement: [Required / Conditional] – [Scope]
  - Examples:
    - Tank Heater Requirement: Required – For GIS tank to prevent condensation
    - Tank Heater Requirement: Conditional – May be required in cold climate conditions
  - If not specified → leave blank.

# U-12
# Customer-Specific Requirements
# Outdoor Equipment - Supply Responsibility
# " Specifies whether the customer requires the high-voltage switchgear supplier to supply outdoor free-standing equipment such as:
  - Current Transformers (CT)
  - Capacitive Voltage Transformers (CVT)
  - Surge Arresters
  - Other outdoor auxiliary components associated with the switchyard
This requirement determines the scope of supply responsibility for outdoor equipment beyond the GIS or switchgear itself."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether outdoor equipment supply is required by the switchgear supplier
  - Scope: Which outdoor items (CT, CVT, surge arrester, etc.)
  - Any conditions (e.g., “only if specified by Employer”)
- Ignore:
  - General mentions of outdoor installation without specifying supply responsibility
- Accepted keywords:
  - outdoor equipment supply
  - free-standing CT/CVT
  - surge arrester supply
  - outdoor auxiliary equipment responsibility
- Output Rules
  - If specified →
    - Outdoor Equipment Supply Responsibility: [Required / Conditional] – [Scope]
  - Examples:
    - Outdoor Equipment Supply Responsibility: Required – Supplier to provide outdoor CT, CVT, and surge arresters
    - Outdoor Equipment Supply Responsibility: Conditional – May be required upon Employer request
  - If not specified → leave blank.

# U-13
# Customer-Specific Requirements
# Customer-Specified Spare Part Requirement
# "Specifies whether the customer requires specific spare parts for high-voltage switchgear beyond the standard manufacturer recommendation. Only extract if the specification explicitly lists the spare parts to be supplied (e.g., interrupters, trip coils, CTs, VTs, gaskets)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Exact spare parts requested by the customer (e.g., interrupters, trip coils, CTs, VTs, gaskets)
  - Quantity if provided
- Ignore:
  - General statements like “as recommended by manufacturer” without listing specific items
- Accepted keywords:
  - spare parts
  - mandatory spares
  - spare interrupter
  - spare trip coil
  - spare CT / VT
  - gasket spares
- Output Rules
  - If specified →
    - Customer-Specified Spare Part Requirement: [List of items with quantities if available]
  - Examples:
    - Customer-Specified Spare Part Requirement: 3 interrupters; 2 trip coils; 2 CTs; 2 pressure gauges
    - Customer-Specified Spare Part Requirement: 1 set of gaskets; 1 spare CVT
  - If not specified → leave blank.

# U-14
# Customer-Specific Requirements
# Customer-Specified Special Tool Requirement
# "Specifies whether the customer explicitly requires particular special tools to be supplied for installation, operation, or maintenance of high-voltage switchgear. Only extract tools that are clearly listed or mandated by the customer. Do not extract if the specification only refers to tools to be used by the supplier or states that tools shall be proposed by the manufacturer."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Exact special tools requested by the customer (e.g., disassembly jig, SF₆ filling adapter, operating mechanism tool kit)
  - Quantity if provided
- Ignore:
  - General statements like “special tools to be used by manufacturer” or “tools to be proposed by supplier”
- Accepted keywords:
  - special tools
  - mandatory tools
  - disassembly jig
  - SF₆ filling adapter
  - operating mechanism tool kit
  - lifting device
  - alignment tool
- Output Rules
  - If specified →
    - Customer-Specified Special Tool Requirement: [List of tools with quantities if available]
  - Examples:
    - Customer-Specified Special Tool Requirement: 1 disassembly jig; 1 SF₆ filling adapter
    - Customer-Specified Special Tool Requirement: 1 operating mechanism tool kit; 2 alignment tools
  - If not specified → leave blank.

# U-15
# Customer-Specific Requirements
# 3D / BIM Model Deliverable Requirement
# "Specifies whether the customer requires 3D models or BIM (Building Information Modeling) deliverables for high-voltage switchgear. These models are typically used for integration into the overall substation design, clash detection, and construction planning."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - 3D model required
    - BIM model required
  - Specific format (e.g., IFC, Revit, STEP)
  - LOD (Level of Development) if specified (e.g., LOD 100, 200, 300, 350, 400, 500)
  - Scope (e.g., entire GIS assembly, LCC only)
- Ignore:
  - General mentions of drawings or 2D layouts without specifying 3D/BIM deliverables
- Accepted keywords:
  - 3D model
  - BIM model
  - IFC format
  - Revit model
  - STEP file
  - LOD 300 / LOD 400 / LOD 500
  - CAD 3D deliverable
- Output Rules
  - If specified →
    - 3D / BIM Model Deliverable Requirement: [Required / Conditional] – [Format if specified] – [LOD Level if specified] – [Scope]
  - Examples:
    - 3D / BIM Model Deliverable Requirement: Required – Format: Revit – LOD 400 – Scope: Entire GIS assembly
    - 3D / BIM Model Deliverable Requirement: Conditional – May be required upon Employer request – Format: STEP – Scope: Entire GIS assembly
  - If not specified → leave blank.
''',
type="user",
author="탁근주",
version=251118)

description_prompt_I_U가 저장되었습니다.


In [7]:
# description_prompt_A_H
manager.create_entry(subject="description_prompt_A_H",text='''
You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)

## General Extraction Rules:
- Only consider requirements applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above.
- Only consider the following switchgear families: GIS, HGIS, DTB, SF₆ Gas-Free GIS.
- Use the SwitchgearType string exactly as provided (e.g., "380 kV GIS") without recomposing it from subfields.
- Do not infer values, constraints, or conditions unless explicitly stated.
- Extract only the controlling requirement per item; avoid listing multiple options or references.
- If multiple clauses exist, select the one explicitly governing the high-voltage switchgear. When thresholds differ, choose the strictest applicable requirement.

# A-1  
# General Conditions  
# Applicable International Standards  
# "Specifies whether the high-voltage switchgear must comply with IEC or IEEE standards, which govern its design, testing, and performance requirements."  
# AI Extraction Logic  
Step 1: Extract only one primary applicable standard—either IEC Standards or IEEE Standards—explicitly governing the high-voltage switchgear in the project specification.  
Step 2: If both are mentioned, determine which one governs based on direct reference, section title, or priority statement. If unclear, select the one predominantly cited for switchgear-related clauses.  
Step 3: Do not list all referenced standards; identify only the controlling standard group.

# A-2  
# General Conditions  
# Supply Record Requirement (Operational Track Record)  
# "Specifies whether the project requires a minimum operational track record for high-voltage switchgear and what explicit thresholds must be extracted (e.g., years of successful field performance, number of in-service projects or bays, market scope, environmental qualifiers)."  
# AI Extraction Logic  
Step 1: Locate clauses defining operational track record for the manufacturer or equipment (e.g., “operational track record,” “successful field performance,” “supply record,” “references in service,” “international market”) applicable to switchgear.  
Step 2: Confirm the clause covers the high-voltage switchgear type (GIS/HGIS/DTB/SF₆-free); umbrella terms that include switchgear or circuit breaker are acceptable.  
Step 3: Extract only explicitly stated thresholds. Recommended keys include:  
- Min_years_field_performance  
- Min_projects  
- Min_bays  
- Market_scope  
- environment_qualifier  
Step 4: Keep ambiguous terms (e.g., “similar”) as written if not defined.  
Step 5: If no explicit requirement is present, leave the field blank.

# A-3  
# General Conditions  
# Manufacturing Experience Requirement  
# "Specifies whether the project requires a minimum period of manufacturing experience for the manufacturer of high-voltage switchgear and what numeric threshold in years must be extracted."  
# AI Extraction Logic  
Step 1: Locate statements such as “manufacturing experience,” “years of manufacturing,” or equivalent that apply to the switchgear category.  
Step 2: Extract the numeric threshold in years only. Do not infer ratings, conditions, or apply inequality symbols or units.  
Step 3: If no explicit requirement is present, leave the field blank.

# A-4  
# General Conditions  
# Type Test Validity Period  
# "Specifies whether the project requires a time validity or recency limit for type-test evidence of high-voltage switchgear and what the explicit threshold and anchor (reference date/event) are."  
# AI Extraction Logic  
Step 1: Identify clauses that set a time limit or recency requirement for type tests (e.g., “type test report validity,” “not older than,” “conducted within,” “after [date]”) and confirm they apply to high-voltage switchgear.  
Step 2: Determine the threshold form and extract it exactly as stated:  
- Relative period → `Period_years: X`  
- Absolute cut-off date → `Cutoff_date: YYYY-MM-DD`  
- Unconditional validity tied to design sameness → `No_limit_if_design_unchanged: True`  
Step 3: Extract the anchor event when stated (e.g., `anchor: bid_closing`, `contract_award`, `FAT`, `delivery`). If no anchor is stated, do not infer one.  
Step 4: Output as a normalized key–value structure using only present fields. Do not infer or reformat values.  
Step 5: Ignore statements about routine/special tests, accreditation, or standard references unless explicitly tied to type test validity.  
Step 6: If no explicit validity or recency requirement is present, leave the field blank.

# C-1
# Electrical Ratings
# Nominal System Voltage
# "Specifies the nominal system voltage required for the project’s high-voltage switchgear (≥ 66 kV). “Nominal Voltage” and “System Voltage” are treated as the same concept."
# AI Extraction Logic
Step 1: Extract the nominal system voltage contained in switchgeartype(e.g., 380 kV if the switchgeartype is "380 kV GIS").

# C-2
# Electrical Ratings
# Rated Voltage
# ""Specifies the phase-to-phase RMS voltage corresponding to the maximum system voltage for which the high-voltage switchgear is designed. In IEC-based specifications, this is referred to as 'Rated Voltage'. In IEEE-based specifications, the equivalent term is 'Rated Maximum Voltage'. Extract the value as 'Rated Voltage' regardless of terminology used.""
# AI Extraction Logic
Step 1: Extract the Rated Voltage for the validated SwitchgearType (e.g., "380 kV GIS") as defined in switchgeartype.
Step 2: If Rated Voltage is not explicitly stated in the project specification, consult the General Specification only if it is explicitly binding for this project.
Step 3: If multiple Rated Voltage values exist for different switchgear types, extract only the one aligned with the validated SwitchgearType.
Step 4: Do not infer values or apply standard mappings unless explicitly stated.
Step 5: Normalize the extracted value under the field name "Rated Voltage" regardless of whether the source uses IEC or IEEE terminology.

## Common Extraction Rules for C-3 ~ C-5:
- Only extract values explicitly labeled as short-duration power-frequency withstand voltage.
- Exclude values for other equipment (e.g., transformers, bushings, cables) and other test types (e.g., switching impulse, lightning impulse, chopped-wave).
- Only extract values that apply to the validated SwitchgearType (e.g., "380 kV GIS", "380 kV HGIS", "132 kV GIS").
- If codes such as G, T, O, B appear: apply the legend when provided; if no legend exists, return:
  - Value: `"G:value; T:value; ..."`
  - Unit: `"kV"`
  - Additional flag: `needs_code_mapping: true`
- If both environment (outdoors/indoors) and condition (dry/wet) variants exist and project context is unknown, return:
  - Value: `"Outdoors_Dry:value; Outdoors_Wet:value; Indoors_Dry:value; Indoors_Wet:value"` (omit keys not present)
  - Unit: `"kV"`
- If only one axis is unknown, return variants for that axis only (e.g., `"Outdoors:value; Indoors:value"` or `"Dry:value; Wet:value"`)
- Output must be split into:
  - Value: numeric string or variant string (e.g., `"230"` or `"Outdoors:230; Indoors:200"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-3
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Phase-to-Earth)  
# "Specifies the short-duration power-frequency withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear at nominal system voltages of 66 kV and above."  
# AI Extraction Logic  
Step 1: Locate statements or table rows that define power-frequency withstand for the phase-to-earth path and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If both environment (outdoors/indoors) and condition (dry/wet) variants exist and project context is unknown, return a flattened matrix:  
`Variants: Outdoors_Dry: value;  Outdoors_Wet: value;  Indoors_Dry: value;  Indoors_Wet: value` (omit keys not present).  
Step 4: If only one axis is unknown, return variants for that axis only (e.g., `variants: outdoors:value; indoors:value` or `variants: dry:value; wet:value`).

# C-4
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Across Open Switching Device)  
# "Specifies the short-duration power-frequency withstand voltage across the open switching device (open contacts) of the high-voltage switchgear at nominal system voltages of 66 kV and above."  
# AI Extraction Logic  
Step 1: Locate statements or table rows that define power-frequency withstand across open switching device/contacts and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If both environment (outdoors/indoors) and condition (dry/wet) variants exist and project context is unknown, return a flattened matrix:  
`Variants: Outdoors_Dry: value;  Outdoors_Wet: value;  Indoors_Dry: value;  Indoors_Wet: value` (omit keys not present).  
Step 4: If only one axis is unknown, return variants for that axis only (e.g., `variants: outdoors:value; indoors:value` or `variants: dry:value; wet:value`).

# C-5
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Across the Isolating Distance)  
# "Specifies the short-duration power-frequency withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear at nominal system voltages of 66 kV and above."  
# AI Extraction Logic  
Step 1: Locate statements or table rows that define power-frequency withstand across the isolating distance and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).  
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

## Common Extraction Rules for C-6 ~ C-8:
- Only extract values explicitly labeled as switching impulse withstand voltage.
- Exclude values for other equipment (e.g., transformers, bushings, cables) and other test types (e.g., power-frequency, lightning impulse, chopped-wave).
- Only extract values that apply to the validated SwitchgearType (e.g., "380 kV GIS", "380 kV HGIS", "132 kV GIS").
- If codes such as G, T, O, B appear: apply the legend when provided; if no legend exists, return:
  - Value: `"G:value; T:value; ..."`
  - Unit: `"kV"`
  - Additional flag: `needs_code_mapping: true`
- If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:
  - Value: `"Outdoors:value; Indoors:value"` (omit keys not present)
  - Unit: `"kV"`
- Output must be split into:
  - Value: numeric string or variant string (e.g., `"950"` or `"Outdoors:950; Indoors:850"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-6
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Phase-to-Earth)
# "Specifies the switching impulse withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear at nominal system voltages of 66 kV and above. Extract only values explicitly labeled as switching impulse; do not substitute lightning impulse, chopped-wave, or power-frequency withstand values."
# AI Extraction Logic
Step 1: Locate statements or table rows that define switching impulse withstand for the phase-to-earth path and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-7
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Across Open Switching Device)
# "Specifies the switching impulse withstand voltage across the open switching device (open contacts) of the high-voltage switchgear at nominal system voltages of 66 kV and above."
# AI Extraction Logic
Step 1: Locate statements or table rows that define switching impulse withstand across open switching device/contacts and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-8
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Across the Isolating Distance)
# "Specifies the switching impulse withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear at nominal system voltages of 66 kV and above."
# AI Extraction Logic
Step 1: Locate statements or table rows that define switching impulse withstand across the isolating distance (e.g., "across the isolating distance," "across open disconnector," "across isolation distance") and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

## Common Extraction Rules for C-9 ~ C-12:
- Only extract values explicitly labeled as lightning impulse withstand voltage (or chopped-wave lightning impulse withstand voltage for C-12).
- Exclude values for other equipment (e.g., transformers, bushings, cables) and other test types (e.g., power-frequency, switching impulse, chopped-wave, protective levels).
- Only extract values that apply to the validated SwitchgearType (e.g., "380 kV GIS", "380 kV HGIS", "132 kV GIS").
- If codes such as G, T, O, B appear: apply the legend when provided; if no legend exists, return:
  - Value: `"G:value; T:value; ..."`
  - Unit: `"kV"`
  - Additional flag: `needs_code_mapping: true`
- If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:
  - Value: `"Outdoors:value; Indoors:value"` (omit keys not present)
  - Unit: `"kV"`
- Output must be split into:
  - Value: numeric string or variant string (e.g., `"1050"` or `"Outdoors:1050; Indoors:950"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-9
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Phase-to-Earth)
# "Specifies the lightning impulse withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear at nominal system voltages of 66 kV and above. Extract only values explicitly labeled as lightning impulse; do not substitute switching impulse, chopped-wave, or power-frequency withstand values."
# AI Extraction Logic
Step 1: Locate statements or table rows that define lightning impulse withstand for the phase-to-earth path (e.g., "lightning impulse withstand," "LIWV," "LI," "BIL," "1.2/50 us") and synonyms for the path such as "to earth" or "to ground" and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-10
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Across Open Switching Distance)
# "Specifies the lightning impulse withstand voltage across the open switching distance (open switching device/open contacts) of the high-voltage switchgear at nominal system voltages of 66 kV and above."
# AI Extraction Logic
Step 1: Locate statements or table rows that define lightning impulse withstand for the path across the open switching distance, including phrases such as "across open contacts," "across open pole," or "interrupter gap," and lightning-specific terms ("lightning impulse," "LI," "BIL," "1.2/50 us") and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-11
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Across the Isolating Distance)
# "Specifies the lightning impulse withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear at nominal system voltages of 66 kV and above."
# AI Extraction Logic
Step 1: Locate statements or table rows that define lightning impulse withstand for the path across the isolating distance, including phrases such as "across the isolating distance," "across open disconnector," or "across isolation distance," and lightning-specific terms ("lightning impulse," "LI," "BIL," "1.2/50 us") and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-12
# Electrical Ratings
# Chopped Wave Lightning Impulse Withstand Voltage
# "Specifies the chopped-wave lightning impulse withstand voltage in the high-voltage switchgear at nominal system voltages 66 kV and above. Extract only values explicitly labeled as chopped-wave lightning impulse; do not substitute full-wave lightning impulse, switching impulse, or power-frequency withstand values."
# AI Extraction Logic
Step 1: Locate statements or table rows that define chopped-wave lightning impulse withstand and that explicitly apply to the validated SwitchgearType.
Step 2: Apply common extraction rules.
Step 3: If OUTDOORS and INDOORS variants exist and the installation area is unknown, return:  
`Variants: Outdoors: value;  Indoors: value` (omit keys not present).
Step 4: If codes such as G, T, O, B appear: apply legend if provided; otherwise return `unmapped_codes: ...; needs_code_mapping: true`.

# C-13
# Electrical Ratings
# Rated Frequency
# "Specifies the nominal operating frequency of the high-voltage switchgear, which affects its electrical performance, insulation coordination, and compatibility with the power grid. The rated frequency is typically 50 Hz or 60 Hz, depending on the regional grid standard."
# AI Extraction Logic
Step 1: Locate statements or table rows that explicitly define the rated frequency for the validated SwitchgearType (e.g., "380 kV GIS").
Step 2: If no rated frequency is explicitly stated for the SwitchgearType, do not infer default values (e.g., 50 Hz or 60 Hz) and leave the field blank. However, if a general statement applies to all equipment in the document (e.g., "The nominal frequency for National Grid, Saudi Arabia is 60 Hz"), it may be used.

# C-14
# Electrical Ratings
# Rated Continuous Current
# "Specifies the RMS value of current that the high-voltage switchgear can carry continuously under its defined service conditions, without exceeding allowable temperature rise or mechanical limits of current-carrying components. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Locate statements or table rows that explicitly define the rated continuous current for the validated SwitchgearType.
Step 2: Exclude values for other equipment (e.g., transformers, cables) and unrelated ratings (e.g., overload, peak current).
Step 3: If multiple values exist for different SwitchgearTypes, extract each separately and link to its SwitchgearType.
Step 4: Do not infer or estimate values. If not stated, leave the field blank.

# C-15
# Electrical Ratings
# Rated Short-Time Withstand Current
# "Specifies the RMS value of the short-circuit current that the high-voltage switchgear can withstand in the closed position for a specified short duration (typically 1s or 3s), without mechanical deformation, internal damage, or insulation failure. This rating validates the switchgear’s thermal and electrodynamic withstand capability under fault conditions until the protection system isolates the fault. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Locate statements or table rows that explicitly define the rated short-time withstand current for the validated SwitchgearType.
Step 2: Exclude values for other equipment and unrelated ratings (e.g., peak current, momentary current).
Step 3: If multiple values exist for different SwitchgearTypes, extract each separately and link to its SwitchgearType.
Step 4: Do not infer or apply typical values. If not stated, leave the field blank.

# C-16
# Electrical Ratings
# Rated Peak Withstand Current
# "Specifies the peak value of the first asymmetrical short-circuit current loop that the high-voltage switchgear must withstand without mechanical failure. This rating evaluates the equipment’s ability to endure electrodynamic forces caused by the highest momentary current. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Locate statements or table rows that explicitly define the Rated Peak Withstand Current for the validated SwitchgearType (e.g., "380 kV GIS"). Extract the value (e.g., "100 kA peak") if stated in the project specification, technical datasheet, or Customer’s General Standard.
Step 2: If the value is not directly provided for the validated SwitchgearType, but both the Rated Short-Time Withstand Current (Iₜₕ) and a Peak Factor (e.g., 2.5, 2.7) are explicitly stated in the same context, calculate the peak current using:  
Rated Peak Withstand Current = Iₜₕ × Peak Factor
Step 3: Output the calculated peak current as a numeric string (e.g., "100") in the Tender Requirement field, and cite the source of the Peak Factor (e.g., "Customer Spec Section 3.2") in the Source field.
Step 4: If no numerical value or valid Peak Factor is available for the validated SwitchgearType, leave the field blank.
Step 5: Do not assume or apply any default or commonly used Peak Factor unless explicitly stated in the specification or General Standard.

# C-17
# Electrical Ratings
# Rated Duration of Short-Circuit
# "Specifies the time interval (typically 1s or 3s) during which the high-voltage switchgear must withstand the rated short-time current without mechanical or thermal failure while in the closed position. This parameter defines the thermal stress duration that the equipment can safely endure under fault conditions before current interruption. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the duration value (e.g., 1s, 3s) explicitly stated in the project specification or Customer’s General Standard.
Step 2: If different durations are specified for different product groups (e.g., GIS vs. DTB), extract and record each value separately.
Step 3: Do not assume a default duration based on switchgear type or voltage level. If no value or reference is stated, leave the field blank.

# B-1
# Environmental Conditions
# Installation Area
# "Specifies whether the high-voltage switchgear is designed for indoor, outdoor, or mixed installation, depending on the environmental and structural conditions defined in the project specification. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the installation area defined in the project-specific documents shall take precedence. If different components (e.g., Gas-Insulated Bus, Hybrid Insulated Switchgear, Dead Tank Breakers) require different installation areas, these must be recorded separately."
# AI Extraction Logic
Step 1: Search the project-specific specification for explicit installation area statements that apply to the switchgear (e.g., "switchgear shall be installed indoors," "outdoor weatherproof enclosure"). Consider a general specification only if it clearly states applicability to the project’s switchgear.
Step 2: Apply source priority: project-specific clause for the pair > pair-specific section > general specification explicitly scoped to the project switchgear. Ignore statements that apply to other equipment.
Step 3: Resolve outcomes without assumptions.
- If a single area applies to the pair, output one of: indoors, outdoors, mixed.
- If different areas are stated for distinct components of the pair (e.g., GIS indoors, GIB outdoors), output components: <label1>:indoors; <label2>:outdoors, using component labels normalized to lower_snake_case as written in the document.
- If the spec lists multiple allowed areas (e.g., “prefer indoors; outdoors acceptable with enclosure”) and project context is unknown, output variants: indoors:true; outdoors:true (conditions may be noted outside the value field if needed).
Step 4: Output rules
- Return a single line, ASCII only.
- Single value → indoors / outdoors / mixed.
- Component split → components: <label1>:indoors; <label2>:outdoors (pairs separated by ; , no trailing semicolon).
- Multiple allowed options (no component split) → variants: indoors:true; outdoors:true.
- If no explicit installation area for the pair is present, leave the field blank.

# B-2
# Environmental Conditions
# Coastal Area Classification
# "Specifies whether the installation site is explicitly classified as a coastal area, where high humidity, salt, and airborne contaminants significantly increase the risk of corrosion, surface degradation, and long-term material wear. Only classifications applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the project-specific coastal classification shall take precedence. Corrosion protection measures (e.g., duplex coating) shall not be used as indirect evidence of coastal classification."
# AI Extraction Logic
Step 1: Search the project-specific specification for explicit coastal classification statements that apply to the installation/site (e.g., “the site is a coastal area,” “non-coastal area”), or for an explicit rule that defines coastal status by distance-to-coast.
Step 2: If absent in the project-specific spec, consult a general specification only when it explicitly applies to the project’s switchgear scope and provides either a direct classification or a distance-based rule.
Step 3: Exclusions (no inference):
- Do not classify based on protective measures (e.g., duplex coating, corrosion category), enclosure/IP ratings, or phrases like “humid,” “salty air,” or geographic names (e.g., Red Sea) unless the document also provides an explicit classification or a rule mapping those references to a classification.
- Distance statements alone (e.g., “within X km of coastline”) are not a classification unless the document defines the threshold rule; if a rule exists, apply it exactly.
Step 4: Output rules
- Return a single line, ASCII only.
- If explicitly classified coastal → coastal; if explicitly classified not coastal → non_coastal.
- If multiple facilities/sites have different classifications and the pair’s facility is clearly identified, return the classification for that facility; if the pair’s facility is unknown or conflicting at equal precedence, return None.
- If no explicit classification (or applicable rule) exists for the pair, leave the field blank.

# B-3
# Environmental Conditions
# Ambient Air Temperature (Indoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated minimum indoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "indoor," AI must verify the surrounding context. It should be extracted only if clearly associated with indoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the indoor minimum temperature specific to high-voltage switchgear.
Step 4: Ignore general climate data, monthly/annual averages, or values unrelated to equipment installation.
Step 5: If no valid value is found, leave the field blank.

# B-4
# Environmental Conditions
# Ambient Air Temperature (Indoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated maximum indoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "indoor," AI must verify the surrounding context. It may be extracted only if clearly associated with indoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the indoor maximum temperature applicable to high-voltage switchgear.
Step 4: AI must ignore general climate conditions, monthly/annual temperature data, or values not specifically tied to equipment installation.
Step 5: If no valid value is found, leave the field blank.

# B-5
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated minimum outdoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "outdoor," AI must verify context and extract it only if clearly associated with outdoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the applicable outdoor minimum temperature.
Step 4: AI must ignore general climatic or geographical information (e.g., regional climate data, annual weather summaries) unless directly tied to switchgear installation requirements.
Step 5: If no valid value is found, leave the field blank.

# B-6
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated maximum outdoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "outdoor," AI must verify the surrounding context. Extract the value only if it is clearly associated with outdoor high-voltage switchgear.
Step 3: If the project specification does not provide a valid value, AI must refer to the Customer’s General Standard for an outdoor maximum temperature relevant to high-voltage switchgear.
Step 4: Ignore general climate data (e.g., hottest month average, yearly highs) unless explicitly tied to equipment installation.
Step 5: If no valid temperature is found, leave the field blank.

# B-7
# Environmental Conditions
# Altitude
# "Specifies the installation altitude applicable to high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the project-specific altitude shall take precedence. The extracted altitude must represent the actual site condition, not a design range or generic classification."
# AI Extraction Logic
Step 1: Search the project-specific specification for altitude-related values using keywords such as “Altitude above sea level,” “Site elevation,” or “Installation height,” and accept values expressed in formats such as “xxx m,” “between 1000–1200 m,” or “MSL: 950 m.”
Step 2: If the project-specific specification does not provide an altitude value, refer to the general specification only if it presents a clearly applicable fixed altitude relevant to the specific project.
Step 3: If only a range is given without a specific selection for the project, such as “0–1000 m or 1001–3000 m,” do not extract any value.
Step 4: If there are multiple conflicting altitude values in different sections of the documents, or if the project-specific value contradicts the general specification, flag the item as “Manual review required.”
Step 5: Do not infer altitude based on standard defaults such as 1000 m from IEC 62271-1 or other references unless this value is explicitly stated in the specification.
Step 6: If a valid fixed value is found, extract the numeric value with the unit such as “1200 m”; if only a range is stated without project-specific selection, extract “Not specified”; and if values are unclear or conflicting, leave the field blank.

# B-8
# Environmental Conditions
# Site Pollution Severity
# "Specifies the pollution severity level at the installation site, which affects the external insulation performance, creepage distance, and material durability of high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the pollution classification or contamination data from the project-specific documents shall take precedence."
# AI Extraction Logic
Step 1: Search the project-specific specification for explicit pollution severity classifications such as “Light,” “Medium,” “Heavy,” or “Very Heavy,” and also accept regional mappings that equate site types with severity classes, such as “coastal area shall be considered ‘Very Heavy’ pollution.”
Step 2: If the project-specific specification does not provide a pollution severity classification, refer to the general specification only if it contains a clearly applicable severity level for the actual installation site.
Step 3: If the specification provides Equivalent Salt Deposit Density (ESDD) and Non-Soluble Deposit Density (NSDD) values, extract them without converting into a severity classification unless the document explicitly maps them to an IEC or IEEE category.
Step 4: If the specification references IEC TS 60815-1 and provides corresponding thresholds, then extract the severity level only if a direct mapping is included in the document.
Step 5: If both a classification and ESDD/NSDD values are present, extract both types of information separately and record them in their respective fields.
Step 6: Do not infer or derive pollution severity based on geographic location, coastal proximity, or numeric contamination values unless the document explicitly defines the mapping or provides a credible reference.
Step 7: If there is conflicting information between documents, extract the classification from the project-specific specification and flag the entry for review if the inconsistency may affect equipment design.
Step 8: If a valid classification is found, extract it as “Medium,” “Very Heavy,” etc.; if raw data is provided, extract it as “ESDD = 0.15 mg/cm²,” “NSDD = 1.2 mg/cm²”; and if the information is unclear, missing, or conditional, leave the field blank.

# B-9
# Environmental Conditions
# Seismic Classification
# "Specifies the seismic performance requirements for the installation site of high-voltage switchgear, based on expected earthquake intensity, ground motion, or seismic zoning. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the project-specific seismic classification or data shall take precedence. No inferred classification or mapping shall be made unless explicitly stated."
# AI Extraction Logic
Step 1: Search the project-specific specification for seismic performance requirements, including classification terms such as “Low Seismic,” “IEEE 693 High Level,” or “Zone IIB,” and numerical indicators such as “PGA = 0.3g” or “Response spectrum curve per IEC 62271-207.”
Step 2: If the project-specific specification does not provide seismic requirements, refer to the general specification only if it contains a clearly applicable classification or site-specific seismic data relevant to the actual installation site.
Step 3: If both a classification and raw seismic data are present in the specification, extract both separately and do not attempt to convert between data and classification unless an explicit mapping is provided.
Step 4: If conflicting seismic values are identified across different sections or documents, extract “Manual review required” to flag the inconsistency.
Step 5: If the only information available is a vague reference to compliance with seismic standards or a regional seismic map without explicit classification, extract “Not specified.”
Step 6: If a valid classification is found, extract it as “IEEE 693 High Level,” “Zone IIB,” or equivalent; if numerical data is provided, extract it as “PGA = 0.3g” or “Response Spectrum (curve reference)”; and if the information is unclear, missing, or inconsistent, leave the field blank.

# D-2
# Design Parameters
# High-Voltage Switchgear Type
# "Specifies the high-voltage switchgear type(s) required for the project. Scope is limited to equipment associated with nominal system voltages ≥ 66 kV. Individual primary components (e.g., DS/ES/CVT/CT/VT/SA, busbars, terminations) are excluded."
# AI Extraction Logic
Step 1: Extract the high-voltage SwitchgearType(s) contained switchgeartype(e.g., "GIS" if the switchgeartype is "380 kV GIS").

# D-3
# Design Parameters
# Insulation Medium Requirement
# "Check whether the customer specifies a particular insulation medium to be used in the high-voltage switchgear. The requirement may include allowance or prohibition of SF₆ (sulfur hexafluoride), approval for SF₆-free alternatives (e.g., C4-FN, dry air, vacuum), or statements about environmental compliance. The insulation medium may apply to GIS, DTB, GIB, or other major gas-insulated components."
# AI Extraction Logic
Step 1: Identify references to insulation medium or insulating gas — Used in GIS, DTB, HGIS, GIB, or other HV switchgear types.
Step 2: Detect expressions — Such as: "SF6 shall be used as the insulation medium", "Only SF6-free solution shall be accepted", "Insulation gas must comply with F-gas regulation", "C4-FN or dry air shall be applied", "No fluorinated gas allowed".
Step 3: Extract the insulation medium type(s) mentioned — And note whether it is: Required (e.g., "SF6-free only"); Allowed (e.g., "SF6 permitted"); Prohibited (e.g., "SF6 not allowed"). If multiple options are listed (e.g., SF6 or dry air), extract all.

# D-4
# Design Parameters
# Enclosure Type
# "Specifies whether the high-voltage switchgear enclosure is designed as a single-phase or three-phase configuration. Only switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the enclosure type (Single-Phase or Three-Phase) explicitly stated in the project specification.
Step 2: If different enclosure types are defined for different voltage levels or product groups, extract them separately and link to the appropriate group.
Step 3: If both types are listed as acceptable (e.g., "either type is acceptable"), record them in the format: Single-Phase / Three-Phase.
Step 4: If one type is preferred and another conditionally allowed, record as: Single-Phase (preferred) or Single-Phase; Three-Phase allowed upon approval.
Step 5: Do not infer enclosure type from voltage level, product design, or general practice; only extract explicitly defined values.
Step 6: If enclosure type is not stated, leave the field blank.

# D-5
# Design Parameters
# Enclosure Material
# "Specifies the primary metallic material used for the enclosure (housing) of high-voltage switchgear, which impacts corrosion resistance, mechanical strength, and environmental durability. Only enclosures for equipment rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated enclosure material from the project specification (e.g., Aluminum, Stainless Steel, Galvanized Steel, Al-Zn Coated Steel).
Step 2: If different materials are defined for separate voltage levels or product groups, extract and record them separately per group.
Step 3: If multiple materials are permitted (e.g., "Aluminum or Stainless Steel"), record them in the format: Aluminum / Stainless Steel.
Step 4: If one material is preferred and another is conditionally allowed, record as: Aluminum (preferred) or Aluminum; Stainless Steel allowed upon approval.
Step 5: Do not infer enclosure material from product family, insulation type, or coating reference unless explicitly stated.
Step 6: If no valid material is provided, leave the field blank.

# D-6
# Design Parameters
# Conductor Material
# "Specifies the material used for the main current-carrying conductors (e.g., busbars, terminals) inside the high-voltage switchgear. The material affects electrical conductivity, current capacity, thermal performance, and mechanical stability. Only conductors used in switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated conductor material — (e.g., Copper, Aluminum) from the project specification.
Step 2: If both materials are allowed — Record in the format: Copper / Aluminum.
Step 3: If one is preferred and another allowed with conditions — Record as: Copper (preferred) or Copper; Aluminum allowed upon approval.
Step 4: Do not infer conductor material — Based on system voltage, current rating, or performance class.
Step 5: If not specified, leave the field blank.

D-7
Design Parameters
Partial Discharge Level
"Specifies the maximum permissible partial discharge (PD) intensity for internal components of high-voltage switchgear, including busbars, support insulators, bushings, and other insulation elements.
This requirement ensures compliance with insulation quality criteria under type test and routine test conditions.
PD limits are typically governed by international standards such as IEC 62271-203 or IEEE Std C37.122.
Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract any explicit numerical value for PD level (e.g., less than5pC) stated in the project specification or customer’s general standard.
Step 2: If only a reference to a standard is provided (e.g., "PD level as per IEC 62271-203" or "as per IEEE Std C37.122"), record the Tender Requirement as "As per [standard name]" and cite it in the Source field.
Step 3: If both a numerical value and a standard are given, record the numerical value as the Tender Requirement and the standard as the Source.
Step 4: AI must not infer or estimate PD levels based on equipment type or nominal voltage. If no clear information is available, leave the field blank.

# D-8
# Design Parameters
# Maximum Relative Gas Leakage Rate
# "Specifies the maximum allowable annual gas leakage rate, expressed as a percentage of the total gas quantity within each gas-filled compartment of high-voltage switchgear. This requirement ensures long-term gas retention integrity, pressure stability, and compliance with environmental regulations for equipment using SF₆, SF₆ mixtures, or alternative insulating gases. Leakage limits are typically defined in international standards such as IEC 62271-203 or functionally equivalent clauses in IEEE guidelines, depending on the applicable standard. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract any explicit numerical value for the annual relative leakage rate per gas compartment (e.g., less than0.5% per year) from the project specification or customer’s general standard.
Step 2: If only a reference to a standard is provided (e.g., "as per IEC 62271-203" or "as per IEEE C37.x"), extract "As per [standard name]" as the Tender Requirement, and cite the reference in the Source field.
Step 3: If both a numerical value and a standard reference are provided, record the numerical value as the Tender Requirement and the referenced standard as the Source.
Step 4: AI must not infer default values based on voltage, product type, or insulation gas type. If no clear value or reference is provided, leave the field blank.

# D-9
# Design Parameters
# Internal Fault - Short-Circuit Current
# "Specifies the RMS value of short-circuit current the high-voltage switchgear must withstand during an internal arc fault. If the internal arc withstand current is not explicitly defined in the specification, it shall be assumed equal to the Rated Short-Time Withstand Current (as per IEC 62271-203 and industry practice)."
# AI Extraction Logic
Step 1: If the internal arc fault withstand current is explicitly stated, extract that value.
Step 2: If not stated, extract the Rated Short-Time Withstand Current and apply it as the internal fault current.
Step 3: Cite the source appropriately—either direct internal arc requirement or reference to Rated Short-Time Withstand Current.
Step 4: Do not infer any alternate value based on voltage or switchgear type.

# D-10
# Design Parameters
# Internal Fault - Duration of Current
# "Specifies the minimum time duration that the high-voltage switchgear enclosure must withstand the effects of an internal arc fault without loss of containment or unsafe failure, under the specified internal fault current level. This duration (commonly expressed as 300ms, 500ms, or 1s) represents the burn-through resistance of the enclosure before fragmentation or external effects occur. IEC 62271-203 and IEEE C37.122 provide standard classifications and test protocols for burn-through time. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the internal arc duration (e.g., 300ms, 0.5s, 1s) if explicitly stated in the project specification or customer’s general standard.
Step 2: Accept terminology such as "burn-through time," "internal arc duration," or "arc withstand time", if clearly linked to GIS, HGIS, or DTB arc fault containment.
Step 3: If different durations apply to different product groups or voltage levels, extract and record them separately.
Step 4: If no value is provided, leave the field blank or flag for manual review.
Step 5: If the internal arc duration is not explicitly defined but performance criteria (e.g., "must withstand arc without external effect for 0.3s") are provided, extract the time value only if unambiguous.
Note: Do not infer this value from general short-circuit duration unless the specification clearly states it applies to internal fault conditions.

# D-11
# Design Parameters
# Maximum Length of Gas Compartment
# "Specifies the maximum allowable physical length of an individual gas-tight compartment within high-voltage switchgear (e.g., GIS or HGIS), as constrained by insulation reliability, gas leakage risk, pressure stability, and maintenance considerations. In some customer standards, limits are defined either by maximum physical length (e.g., 18m to 22.5m) or by maximum total gas volume per compartment (e.g., 4m³ of SF₆). Extensions beyond this length may be permitted with prior customer approval, provided that the total gas volume in the compartment does not exceed 4m³. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the maximum allowable length value (e.g., 22.5m) explicitly stated in the project specification or customer’s general standard.
Step 2: If the specification includes a conditional exception (e.g., longer length allowed if gas volume is 4m³ or less), extract both the base value and the exception, and record the condition in the Source field.
Step 3: If both length and volume-based conditions are given, extract the primary length limit in the Tender Requirement field and reference the volume condition in the Source field.
Step 4: If different length limits apply to different product groups, extract and record each separately.
Step 5: If no value or conditional rule is stated, leave the field blank.

# D-14
# Design Parameters
# Sealing Redundancy (Single / Double)
# "Specifies whether pressurized gas joints of the high-voltage switchgear require single or double sealing. Apply to the provided (Nominal System Voltage, High-Voltage Switchgear Type) pair. Accept synonyms such as double sealing, duplicated sealing system, dual sealing, twin seal, two seals. Do not infer from unrelated phrases unless they clearly apply to pressurized gas joints."
# AI Extraction Logic
Step 1: Locate explicit requirements for sealing redundancy at pressurized joints (e.g., flanges, compartment interfaces, feedthroughs) with terms such as "double sealing," "duplicated sealing system," "dual sealing," "two seals," "double O-ring," "double gasket."
Step 2: Scope and mapping.
- If a uniform redundancy applies to all pressurized joints, treat as a single value.
- If redundancy differs by component/joint (e.g., circuit_breaker_compartment vs busbar_compartment), output component-tagged pairs using lower_snake_case labels from the document.
- If environment options exist (e.g., outdoors = double, indoors = single) and the project environment is unknown, return variants.
Step 3: Output rules (categorical, standard).
- Single line, ASCII only.
- Single uniform value → single or double.
- Scope-bound / component split → <scope1>=<value>; <scope2>=<value>; ....
- Environment variants (when environment unknown) → indoors=<value>; outdoors=<value>
- If no explicit redundancy requirement for the pair is present, leave the field blank.

# E-2
# Circuit Breaker
# Operating Mechanism Type
# "Specifies the type of mechanism used to operate the circuit breaker in high-voltage switchgear rated at nominal system voltages of 66kV and above."
# AI Extraction Logic
Step 1: AI must extract the mechanism type explicitly stated in the project specification (e.g., Spring-Operated, Hydraulic-Operated).
Step 2: If multiple types are allowed, AI must record them using the format: Spring / Hydraulic / Pneumatic.
Step 3: If a preferred type is mentioned and others are conditionally accepted, use format: Spring (preferred); Hydraulic allowed upon approval.
Step 4: AI must not infer mechanism type based on voltage class, vendor trend, or typical configurations.
Step 5: If not specified, leave the field blank or flag for manual review.

# E-3
# Circuit Breaker
# Rated First-Pole-to-Clear Factor
# "Specifies the factor used in insulation coordination and TRV assessment to account for the recovery voltage across the first pole to clear in a three-phase circuit breaker. It represents the maximum overvoltage stress that may occur on the first pole when the other two phases are still conducting. The factor is defined in IEC 62271-100 and IEEE C37.06 and is dependent on system grounding and voltage class."
# AI Extraction Logic
Step 1: Extract the numerical value of the Rated First-Pole-to-Clear Factor (e.g., 1.3, 1.5) explicitly stated in the project specification.
Step 2: If multiple values are described based on system grounding (e.g., grounded vs. ungrounded), extract both with clear conditional labeling.
Step 3: Do not record general references like "as per IEC 62271-100" unless accompanied by an actual numerical value.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not assume default values. If no value is found, leave the field blank or flag for manual review.

# E-4
# Circuit Breaker
# Rated Operating Sequence
# "Specifies the rated operating sequence of the circuit breaker, which defines the standard sequence of open and close operations the breaker is designed to perform under defined conditions. This value is used to evaluate the breaker's mechanical and thermal performance during normal and fault-handling scenarios. The requirement applies to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the rated operating sequence exactly as stated in the customer specification (e.g., "O - CO - CO").
Step 2: If multiple sequences are specified for different applications or breaker types, extract and record each separately.
Step 3: Do not infer or generate default sequences based on breaker type or system voltage.
Step 4: If the operating sequence is only referenced via a standard (e.g., "as per IEC 62271-100"), record: Tender Requirement → "As per IEC 62271-100", Source → Cite the standard and clause if given.
Step 5: If no value is found, leave the field blank or flag for manual review.

# E-5
# Circuit Breaker
# Electrical Endurance Class
# "Specifies the electrical endurance class of the circuit breaker according to IEC 62271-100 for high-voltage switchgear rated at nominal system voltages of 66kV and above. The endurance class defines the number of short-circuit current interruptions the circuit breaker can perform under specified conditions without performance degradation. Only explicitly stated classifications (e.g., E1, E2) shall be considered."
# AI Extraction Logic
Step 1: Extract the electrical endurance class — Only if explicitly stated (e.g., "Class E2").
Step 2: If multiple classes are allowed — Record in format: E1 / E2.
Step 3: If a preferred class is listed with conditional allowance of another — Use format: E2 (preferred); E1 allowed upon approval.
Step 4: Do not infer the class — Based on: Rated short-circuit current, Voltage level, Vendor design, Past project data.
Step 5: If not specified — Leave blank or flag for manual review.

# E-6
# Circuit Breaker
# Rated Supply Voltage of Control Circuits
# "Specifies the rated supply voltage applied to the control circuits of high-voltage switchgear (rated at 72.5kV and above, or nominal system voltage of 66kV and above). These circuits include those used for protection, monitoring, interlocking, signaling, and operating functions. The rated supply voltage ensures compatibility with auxiliary equipment such as relays and substation power sources."
# AI Extraction Logic
Step 1: Extract the rated control voltage(s) explicitly stated in the project specification (e.g., 125V DC, 230V AC).
Step 2: If multiple control voltages are provided for different product groups (e.g., GIS: 125V DC / DTB: 230V AC), extract and record each separately, linked to the corresponding group.
Step 3: If a voltage range is specified for performance tolerance (e.g., 100-140V DC), record it in the "Source" field, not in the "Tender Requirement" unless explicitly stated as the rated voltage.
Step 4: Do not infer control voltage based on mechanism type, region, or vendor defaults.
Step 5: If no value is provided, leave the field blank or flag for manual review.

# E-7
# Circuit Breaker
# Operating Voltage Range of Control Circuits
# "Specifies the acceptable operating voltage range for control circuits in high-voltage switchgear (rated at 72.5kV and above, or nominal system voltage of 66kV and above). This range defines the voltage tolerance within which the control, protection, and annunciator systems must function reliably. It may differ from the rated supply voltage and is important for evaluating system margin, undervoltage behavior, and auxiliary power design."
# AI Extraction Logic
Step 1: Extract the operating voltage range only if explicitly stated in the project specification (e.g., "100V DC to 140V DC"). Accept expressions such as: "Operating voltage range…", "Permissible range for control supply…", "Voltage tolerance range…"
Step 2: Do not extract this value into the "Tender Requirement" field of Rated Supply Voltage; store it in the dedicated item only.
Step 3: If only a range is specified without a rated voltage, still extract the range under this item and flag for manual review if needed.
Step 4: If multiple ranges are given for different systems (e.g., control vs annunciator), label accordingly or extract separately.
Step 5: Do not infer this value based on national standards, protection relay specs, or vendor catalogs.
Step 6: If no explicit range is found, leave blank or flag for manual review.

# E-8
# Circuit Breaker
# Operating Voltage Range of Motor Circuits
# "Defines the acceptable voltage range (DC or AC) within which the spring-charging motor of the circuit breaker operating mechanism must operate reliably. This range ensures consistent energy storage and charging performance under variable auxiliary supply conditions in high-voltage switchgear rated at 72.5kV and above or with nominal system voltage of 66kV and above."
# AI Extraction Logic
Step 1: Extract the motor operating voltage range explicitly stated in the project specification (e.g., 100-140V DC).
Step 2: If only a reference to an international standard is provided (e.g., "as per IEC 62271-1"), record:
Tender Requirement → "As per IEC 62271-1"
Source → "IEC 62271-1, Clause X.X"
Step 3: If multiple product groups use different motor voltage ranges, extract and record each separately.
Step 4: Apply this item only to high-voltage switchgear (Rated Voltage at 72.5kV and above or Nominal System Voltage at 66kV or above).
Step 5: Do not assume default ranges based on standard motor ratings, switchgear type, or vendor norms. If no value is found, leave the field blank or flag for manual review.

# E-9
# Circuit Breaker
# Operating Voltage Range of Shunt Closing Release
# "Specifies the allowable voltage range within which the shunt closing release coil must operate to ensure reliable mechanical closing of the circuit breaker. This range ensures proper functioning of the trip-close system even under fluctuating control power conditions in high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the shunt closing release operating voltage range explicitly stated in the project specification.
Step 2: If no range is stated but a reference to a standard is provided (e.g., "as per IEC 62271-1"), record: Tender Requirement → "As per IEC 62271-1", Source → "IEC 62271-1, Clause X.X".
Step 3: If different voltage ranges are specified for different product groups (e.g., GIS vs. DTB), record each separately.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not infer values based on other control voltages, mechanism type, or standard assumptions.
Step 6: If no value is found, leave blank or flag for manual review.

# E-10
# Circuit Breaker
# Operating Voltage Range of Shunt Opening Release
# "Specifies the allowable voltage range within which the shunt opening release coil must operate to trigger breaker tripping commands. Accurate specification of this range ensures reliable fault interruption under varying control voltage conditions in high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the shunt opening release operating voltage range explicitly stated in the project specification.
Step 2: If a standard is referenced instead of a defined range (e.g., "as per IEC 62271-1"), record as follows: Tender Requirement → "As per IEC 62271-1", Source → "IEC 62271-1, Clause X.X".
Step 3: If values differ by equipment group (e.g., GIS: 100-140V DC / DTB: 110-130V DC), record them separately.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not assume default values based on control voltage type, manufacturer, or product design.
Step 6: If no value is found, leave the field blank or flag for manual review.

# E-11
# Circuit Breaker
# Rated Supply Voltage of Heating Circuits
# "Specifies the rated voltage of the power supply for space heaters, anti-condensation heaters, cabinet heaters, or enclosure heaters used in high-voltage switchgear rated at 72.5kV and above (or nominal system voltage of 66kV and above). This voltage ensures proper humidity and temperature control within enclosures to prevent moisture-related insulation failures and equipment degradation."
# AI Extraction Logic
Step 1: Extract the heater supply voltage explicitly stated in the project specification. Accept expressions such as: "Rated 230V AC", "230V anti-condensation heater", "Cabinet heater rated 230V".
Step 2: If multiple voltages apply to different product groups (e.g., GIS, DTB), extract and label them separately: GIS: 230V AC / DTB: 110V AC.
Step 3: If one voltage is preferred and another allowed conditionally, record as: [Preferred Value] (preferred); [Alternative] allowed upon approval.
Step 4: Do not infer voltage from: Control circuit voltage, Region of use, Vendor design norms.
Step 5: If the voltage is not explicitly stated or is only mentioned in unrelated sections, leave the field blank or flag for manual review.

# E-12
# Circuit Breaker
# Break-Time
# "Specifies the time interval between the initiation of the opening operation of the circuit breaker and the final extinction of the arc during current interruption. This parameter defines the duration required for complete fault current interruption and is critical in assessing the fault clearance speed and dynamic coordination of protection schemes. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: AI must extract any explicitly stated Break-Time value (e.g., 2 cycles, 40ms, etc.) from the project specification or Customer’s General Standard.
Step 2: If multiple Break-Time values are provided for different product groups or voltage levels, AI must extract and record each separately, matched to the corresponding product.
Step 3: If the value is given in cycles (e.g., 3 cycles at 60Hz), AI must calculate the equivalent time in milliseconds (e.g., 50ms) and store that value in the Tender Requirement field. The original unit and system frequency (e.g., 3 cycles @ 60Hz) must be documented in the Source field.
Step 4: AI must not infer a Break-Time value based on mechanism type (e.g., spring vs. hydraulic), vendor preference, or typical design assumptions.
Step 5: If no Break-Time value is specified in the project specification or Customer’s General Standard, AI must leave the field blank or flag it for manual review.

# E-13
# Circuit Breaker
# Controlled Switching Requirement
# "Specifies whether the project specification requires controlled switching for high-voltage circuit breakers (rated 66 kV and above). Controlled switching refers to the operation of a circuit breaker at a defined point of the power frequency waveform (current or voltage) to reduce inrush current, switching transients, and mechanical stress. IEC officially defines this operation as "controlled switching," and the terms "point-on-wave switching" or "point-on-cycle switching" are commonly used equivalents in specifications."
# AI Extraction Logic
Step 1: Confirm the sentence pertains to a high-voltage circuit breaker with nominal system voltage 66 kV or higher. If not, do not extract.
Step 2: Detect controlled-switching semantics such as “controlled switching,” “point-on-wave,” “point-on-cycle,” “synchronous closing/opening,” “controlled switching device/controller/unit.” If none are present, do not extract.
Step 3: Require, in the same clause, a device/controller noun together with mandatory language such as “shall be provided/installed/supplied/furnished.” If this combination is absent, do not extract and flag as “Potential controlled switching mention” for manual review.
Step 4: If the text expresses a rating or capability (e.g., “rated for out-of-phase switching,” “capability,” “duty,” “withstand”), route it to the Out-of-Phase Switching Capability item and do not extract here.
Step 5: When Steps 1–3 are satisfied, record the tender requirement verbatim and capture any scope qualifiers (e.g., applicable voltage levels, bays, equipment).
Step 6: If terminology is ambiguous (e.g., “pole switching equipment”) and the device + mandatory language pattern is not clear, do not extract and flag for manual review.

# E-14
# Circuit Breaker
# Single-Phase Operation Requirement
# "Check whether the customer explicitly requires the circuit breaker to be capable of per-phase (single-phase) operation—i.e., independent opening/closing of one phase while other phases remain unaffected. Do not extract when the phrase appears only in a conditional or illustrative clause tied to auxiliary devices (e.g., operation counters). Extract only when the main requirement states the breaker shall be capable of single-phase/individual-pole operation."
# AI Extraction Logic
Step 1 (Candidate phrases): Detect per-phase capability phrases (e.g., “single-phase operation,” “single-pole operation,” “individual pole operation,” “independent phase control,” “one pole can operate independently,” “per-phase switching”).
Step 2 (Scope): Confirm the subject is a high-voltage circuit breaker (nominal ≥ 66 kV).
Step 3 (Main-clause gate): The main predicate must express capability with strong modality tied to the breaker (e.g., “CB shall be capable of independent pole operation,” “CB shall allow single-phase operation,” “CB is required to provide single-phase operation”).
If-clause guard: If the capability phrase appears only in a subordinate if/when/where/provided that/in the event that/should clause, do not extract.
Table/label gate: If shown as a label/value (e.g., “Independent pole operation: Yes/Required/Not permitted”), treat it as normative and map accordingly.
Weak modality rule: “can/may” does not satisfy this gate unless in a label/value form.
Step 4 (Auxiliary-device gate): If the same sentence’s main clause mandates auxiliary devices via “shall/must be equipped/fitted/provided/furnished (with) operation counter/timing device/terminal block,” treat it as auxiliary-device context → do not extract for this item.
Step 5 (Output): Extract the tender requirement verbatim only if Steps 2–4 pass; include any scope qualifiers. Otherwise leave blank or flag for manual review.
Step 6 (No inference): Do not infer capability from product type, rating, or vendor practice.

# E-15
# Circuit Breaker
# Manual Operation Requirement
# "Specifies whether the circuit breaker shall be equipped with manual trip and/or manual closing functionality in addition to standard electrical operation. This functionality allows operation of the breaker during commissioning, maintenance, or emergency conditions when electrical control may not be available. This requirement applies to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Search for explicit mentions of circuit breaker manual trip and/or manual closing functionality. Examples: “manual trip,” “manual closing,” “manual operation of the breaker.”
Step 2: Confirm that the requirement explicitly refers to the circuit breaker. If the text refers to earth switches, disconnectors, or motor-operated mechanisms with manual override, do not extract.
Step 3: Extract values using the following mappings: "Only manual trip required" → Trip Only, "Both manual trip and manual closing required" → Trip & Closing, "Manual operation not required/prohibited" → Not Required
Step 4: If general or conditional expressions appear (e.g., “manual operation shall be provided,” “manual operation may be included”), extract verbatim, cite the source, and flag for manual review.
Step 5: If requirements differ by switchgear type or voltage level, extract each separately.
Step 6: If no mention of manual trip/closing for the circuit breaker is found, leave the field blank or flag for manual review.

# E-16
# Circuit Breaker
# CT and CB Gas Compartment Separation Requirement
# "Specifies whether the project documentation explicitly requires the current transformer (CT) and circuit breaker (CB) to be installed in separate gas compartments in high-voltage switchgear (rated 66kV and above). This requirement directly affects compartmentalization design and maintenance isolation strategy. AI must extract this item strictly from explicitly stated customer requirements."
# AI Extraction Logic
Step 1: If the specification clearly states: "CT shall be in a separate compartment from CB" → Extract this phrase verbatim as the Tender Requirement and cite the Source. "CT may be in the CB compartment" or "combination is acceptable" → Likewise, extract the phrase verbatim and cite the Source.
Step 2: If the specification states: "CT shall be in a separate compartment" (without directly referencing CB), AND other parts of the document confirm that CB is in its own compartment, → Extract the phrase and annotate as "Implied separation from CB"; cite the Source and flag if necessary.
Step 3: If the specification includes a conditional allowance (e.g., "CT in CB gas compartment shall be accepted with prior approval" or "separation preferred"), → Extract the full conditional phrase, cite the Source, and flag for manual review.
Step 4: If no information is provided regarding CT and CB compartment separation, → Leave the field blank or flag for manual review.
Step 5: Do not infer separation based on: Switchgear type (e.g., GIS vs. HGIS), Vendor design practices, Layout drawings or typical configurations unless explicitly stated in writing.

# E-17
# Circuit Breaker
# Motor Type
# "Specifies the type of motor used in the circuit breaker operating mechanism for high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (AC or DC) determines compatibility with site power sources, affects control logic design, and impacts procurement and maintenance planning. Only motor types explicitly stated or clearly implied through motor voltage notation in project specifications shall be considered."
# AI Extraction Logic
Step 1: Extract the motor type if explicitly stated (e.g., "AC motor," "DC motor") in relation to the circuit breaker operating mechanism.
Step 2: If the motor voltage is stated using standard notation (e.g., "125Vdc," "400Vac"), infer the motor type as: "Vdc" → DC motor, "Vac" → AC motor.
Step 3: If multiple motor types are permitted, extract using the format: AC / DC.
Step 4: If one motor type is preferred and others are conditionally accepted, use the format: DC (preferred); AC allowed upon approval.
Step 5: Do not infer motor type based on control voltage, vendor defaults, or mechanism type unless the motor voltage format clearly indicates type. Only values explicitly stated for high-voltage switchgear shall be extracted.
Step 6: If not specified or unclear, leave blank or flag for manual review.

# E-18
# Circuit Breaker
# Thermostat Requirement
# "Specifies whether the project documentation explicitly mentions the inclusion, exclusion, or conditional use of a thermostat in high-voltage switchgear (rated 66kV and above). Thermostats are typically integrated into anti-condensation or space heater circuits to regulate internal temperature and prevent moisture-related issues. AI must extract the requirement only when it is explicitly stated, whether mandatory, prohibited, or conditional."
# AI Extraction Logic
Step 1: If the specification clearly states that a thermostat is required (e.g., "thermostat shall be provided", "shall be included"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 2: If the specification clearly states that a thermostat is not required (e.g., "shall not be used", "not required"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 3: If the specification contains a conditional expression (e.g., "if necessary", "may be used", "optional"), → Extract the phrase verbatim as the Tender Requirement, and cite the Source. → This shall be flagged or annotated as a conditional requirement for manual review.
Step 4: Do not infer the requirement based on climate zone, ambient temperature, heater presence, or switchgear type.
Step 5: If no mention of a thermostat is found in the specification, → Leave the field blank or flag for manual review.

# E-19
# Circuit Breaker
# Humidistat Requirement
# "Specifies whether the project documentation explicitly mentions the inclusion, exclusion, or conditional use of a humidistat in high-voltage switchgear (rated 66kV and above). Humidistats are typically used to control heater or ventilation systems in response to internal humidity, preventing condensation and ensuring insulation reliability. AI must extract the requirement only when it is explicitly stated, whether mandatory, prohibited, or conditional."
# AI Extraction Logic
Step 1: If the specification clearly states that a humidistat is required (e.g., "humidistat shall be provided", "shall be included"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 2: If the specification clearly states that a humidistat is not required (e.g., "shall not be used", "not required"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 3: If the specification contains a conditional expression (e.g., "if necessary", "may be used", "optional"), → Extract the phrase verbatim as the Tender Requirement, and cite the Source. → This shall be flagged or annotated as a conditional requirement for manual review.
Step 4: Do not infer the requirement based on ambient humidity, environmental conditions, climate zone, or presence of heater circuits.
Step 5: If no mention of a humidistat is found in the specification, → Leave the field blank or flag for manual review.

# F-2
# Disconnector
# Operating Mechanism Type
# "Specifies the type of mechanism used to operate the disconnector (also known as an isolator) in high-voltage switchgear rated at nominal system voltages of 66kV and above."
# AI Extraction Logic
Step 1: AI must extract the mechanism type explicitly stated in the project specification (e.g., Motor-Operated, Manual-Operated).
Step 2: If multiple types are permitted, AI must record them using the format: Motor / Manual.
Step 3: If a preferred type is stated with others allowed under conditions, use format: Motor (preferred); Manual allowed upon approval.
Step 4: AI must not infer the mechanism type based on vendor design trends, voltage class, or typical practice.
Step 5: If not specified in the specification, leave the field blank or flag for manual review.

# F-3
# Disconnector
# Mechanical Endurance Class
# "Specifies the required mechanical endurance performance of the disconnector (also known as disconnecting switch) in high-voltage switchgear rated at nominal system voltages of 66kV and above. Mechanical endurance refers to the number of no-load operating cycles the device must withstand without mechanical failure or loss of function. Endurance may be expressed in standardized IEC classes (e.g., M1, M2) or as a specific number of operations."
# AI Extraction Logic
Step 1: If the specification explicitly states an IEC mechanical endurance class (e.g., "Class M1" or "M2"), extract it as the Tender Requirement and cite the Source.
Step 2: If a numerical value is provided (e.g., "2,000 operations"), extract the value verbatim and cite the Source. Do not convert the number into M1/M2 unless explicitly stated.
Step 3: If both class and numeric value are provided, extract both using format: M2 (10,000 operations).
Step 4: If the endurance requirement is stated conditionally (e.g., "Minimum 2,000 operations required unless approved otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer endurance class from switchgear type, vendor habits, or assumed usage.
Step 6: If no information is found, leave the field blank or flag for manual review.

# F-4
# Disconnector
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# F-5
# Disconnector
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# F-6
# Disconnector
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# F-7
# Disconnector
# Motor Type
# "Specifies the type of motor used in the operating mechanism of high-voltage disconnectors rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) determines compatibility with site power sources and impacts automation and interlocking schemes. Only motor types explicitly stated or clearly implied through voltage supply notation shall be considered."
# AI Extraction Logic
Step 1: Identify Direct Statements — If a phrase like "AC motor", "DC motor", or "motor type: DC" is explicitly stated in relation to disconnector → Extract as is.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the disconnector mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Types — If more than one type is allowed: → Use format: AC / DC.
Step 4: Preference Indicated — If one is preferred and another is conditionally allowed: → Use format: DC (preferred); AC allowed upon approval.
Step 5: Conditions — Apply logic even when motor type is not directly mentioned but voltage + mechanism context is clearly established.
Step 6: Restrictions — Do not infer from switchgear type, vendor name, or general practice; if unclear (e.g., "electrical operation" but no supply voltage), → Leave blank or flag for manual review.

# F-8
# Disconnector
# Thermostat Requirement
# Enter the extracted value as Serial Number E-14

# F-9
# Disconnector
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# F-10
# Disconnector
# Viewport Requirement
# "Specifies whether the high-voltage switchgear (rated at 66kV and above) is required to include a viewing window or inspection window in the enclosure of components such as the disconnector or earthing switch. This window enables visual confirmation of isolating distance or contact position during operation or maintenance. The viewing window may also be referred to as an inspection port, endoscopy port, viewport or visual inspection opening in customer specifications."
# AI Extraction Logic
Step 1: If the specification clearly states that a viewing window, inspection window, or endoscope port shall be provided, → Extract the phrase verbatim as the Tender Requirement and cite the Source.
Step 2: If the specification requires the ability to visually confirm isolating distance or movable contact position through a window, → Treat this as a functional requirement for a viewing window and extract accordingly.
Step 3: If multiple options are allowed (e.g., "inspection window or endoscope port"), → Extract all listed options using format: Inspection Window / Endoscope Port.
Step 4: If the specification includes additional requirements (e.g., cover plate, color marking, warning label), → These shall not be extracted under this item, but may be cited in the Source for reference.
Step 5: Do not infer the need for a viewing window based on switchgear type, GIS design trends, or assumed accessibility. Only explicit requirements or functional justifications (e.g., "to observe isolating gap") shall be extracted.
Step 6: If no mention of a viewing window or equivalent is found, leave the field blank or flag for manual review.

# G-2
# Earthing Switch / Maintenance Grounding Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used in Maintenance Earthing Switches (MES) for high-voltage switchgear rated at nominal system voltages of 66kV and above. MES are typically used to provide a visible and secure earth connection during equipment maintenance, and do not require high-speed operation. Common mechanisms include motor-operated or manual operation."
# AI Extraction Logic
Step 1: Extract the mechanism type explicitly stated in the specification for MES (e.g., Motor-operated, Manual).
Step 2: If multiple mechanism types are allowed or listed, use the format: Motor / Manual.
Step 3: If one mechanism is preferred with others allowed upon approval, use format: Motor (preferred); Manual allowed upon approval.
Step 4: If a manual device is specified in addition to the motor/snap mechanism, include it as a separate type.
Step 5: Do not infer mechanism type based on function or vendor trend. Only extract if stated.

# G-3
# Earthing Switch / Maintenance Grounding Switch
# Mechanical Endurance Class
# "Specifies the required mechanical endurance performance of the maintenance earthing switch (MES) used in high-voltage switchgear rated at nominal system voltages of 66kV and above. Mechanical endurance refers to the number of no-load mechanical operating cycles the MES must withstand while maintaining structural integrity and functional reliability. Endurance may be classified according to IEC categories (e.g., M0, M1) or stated as a fixed number of operations."
# AI Extraction Logic
Step 1: If the specification explicitly states an IEC mechanical endurance class (e.g., "M0", "M1"), extract it as the Tender Requirement and cite the Source.
Step 2: If a numerical requirement is provided (e.g., "2,000 operations"), extract the number verbatim and cite the Source. Do not interpret or convert into M0/M1 unless explicitly matched in the specification.
Step 3: If both class and numeric value are stated, extract both using format: M1 (2,000 operations).
Step 4: If the requirement includes conditional or fallback wording (e.g., "at least 1,000 operations unless specified otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer the endurance class or operation count from switchgear type, manufacturer habits, or general performance claims.
Step 6: If not mentioned in the specification, leave the field blank or flag for manual review.

# G-4
# Earthing Switch / Maintenance Grounding Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# G-5
# Earthing Switch / Maintenance Grounding Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# G-6
# Earthing Switch / Maintenance Grounding Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# G-7
# Earthing Switch / Maintenance Grounding Switch
# Motor Type
# "Specifies the type of motor used in the operating mechanism of Maintenance Earthing Switches (MES) applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) affects compatibility with substation auxiliary power systems and the responsiveness of interlock logic. Only motor types explicitly stated, or clearly implied through supply voltage description, shall be extracted."
# AI Extraction Logic
Step 1: Direct Statement Extraction — If the motor type is explicitly stated in connection with MES (e.g., "DC motor," "AC motor") → Extract that value.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the MES mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Options — If both AC and DC motors are allowed, extract as: AC / DC.
Step 4: Preferred Type with Conditional Allowance — If one is preferred but another is conditionally allowed, extract as: DC (preferred); AC allowed upon approval.
Step 5: Exclusions - Do Not Infer Based On: Typical MES designs, Vendor defaults, Installation configuration.
Step 6: No Clear Indication — If neither motor type nor related power supply is stated → Leave blank or flag for manual review.

# G-8
# Earthing Switch / Maintenance Grounding Switch
# Thermostat Requirement
# Enter the extracted value as Serial Number E-14

# G-9
# Earthing Switch / Maintenance Grounding Switch
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# G-10
# Earthing Switch / Maintenance Grounding Switch
# Earthing Path Type (Direct vs. Enclosure)
# "Specifies whether the earthing switch is permitted or required to perform short-circuit making onto an earthed part directly or through the metallic enclosure, as allowed by IEC and IEEE standards for high-voltage switchgear rated at 66kV and above. IEC 62271-102 explicitly permits both configurations if electrical continuity and making current withstand are properly verified."
# AI Extraction Logic
Step 1: If the specification states that the earthing switch must connect directly to earth, extract as: "Direct only".
Step 2: If it states that enclosure path is allowed or preferred, extract as: "Through enclosure allowed" or "Through enclosure (preferred)".
Step 3: If both options are permitted without preference, extract as: "Direct / Through enclosure".
Step 4: If test requirements or functional conditions are stated (e.g., "if electrical continuity of the enclosure is ensured"), extract relevant configuration and cite source with note.
Step 5: Do not infer based on switchgear type, vendor design, or standard practices.
Step 6: If no mention of earthing path is found, leave blank or flag for manual review.

# G-11
# Earthing Switch / Maintenance Grounding Switch
# Inspection Window Requirement
# Enter the extracted value as Serial Number F-7

# H-2
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used in Maintenance Earthing Switches (MES) for high-voltage switchgear rated at nominal system voltages of 66kV and above. MES are typically used to provide a visible and secure earth connection during equipment maintenance, and do not require high-speed operation. Common mechanisms include motor-operated or manual operation."
# AI Extraction Logic
Step 1: Extract the mechanism type explicitly stated in the specification for MES (e.g., Motor-operated, Manual).
Step 2: If multiple mechanism types are allowed or listed, use the format: Motor / Manual.
Step 3: If one mechanism is preferred with others allowed upon approval, use format: Motor (preferred); Manual allowed upon approval.
Step 4: If a manual device is specified in addition to the motor/snap mechanism, include it as a separate type.
Step 5: Do not infer mechanism type based on function or vendor trend. Only extract if stated.

# H-3
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Mechanical Endurance Class
# "Specifies the required number of mechanical operations the High-Speed Earthing Switch (HSES) must endure without degradation of function. This applies to high-voltage switchgear rated at nominal system voltages of 66kV and above. While HSES is primarily defined by its high-speed fault-closing capabilities, mechanical endurance under no-load conditions is still a critical performance factor for operational reliability and maintenance planning."
# AI Extraction Logic
Step 1: If the specification clearly states the required number of no-load operations for the HSES (e.g., "1,000 operations"), extract the numeric value verbatim and cite the Source.
Step 2: If endurance is expressed using IEC mechanical endurance categories (e.g., "M0", "M1"), extract it as is.
Step 3: If both are provided, use the format: M1 (2,000 operations).
Step 4: If the mechanical endurance is mentioned conditionally or with exceptions (e.g., "minimum 500 operations unless specified otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer endurance values based on fault current rating, spring mechanism type, or high-speed function.
Step 6: If the specification does not mention mechanical endurance for HSES, leave the field blank or flag for manual review.

# H-4
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Electrical Endurance Class
# "Specifies the electrical endurance class of a high-speed earthing switch (HSES), representing its capability to withstand short-circuit making current as defined in IEC and IEEE standards for high-voltage switchgear (rated at 66kV and above). IEC 62271-102 defines standardized endurance classes (E1 and E2) based on the number and type of making current tests. IEEE standards do not explicitly use class labels but describe equivalent performance requirements (e.g., number of successful short-circuit making operations)."
# AI Extraction Logic
Step 1: If the specification explicitly states "Class E1" or "Class E2," extract the class verbatim and cite the source.
Step 2: If the specification indicates the number of short-circuit making tests (e.g., "2 making tests" or "5 making operations"), extract as: "2 tests" → Class E1, "5 tests" → Class E2. ※ Flag if there's ambiguity in interpretation (e.g., no info on pre-arcing or peak current tests).
Step 3: If IEEE-based specs define making current withstand tests without naming classes, infer the corresponding IEC class only if test quantity and test conditions align with IEC Table 15.
Step 4: If the specification presents both classes (e.g., "E1 / E2 allowed"), extract using format: E1 / E2 or E2 (preferred); E1 allowed.
Step 5: Do not infer endurance class based on manufacturer default, rated voltage, or equipment type.
Step 6: If no class, test quantity, or equivalent making current withstand requirement is mentioned, leave blank or flag for manual review.

# H-5
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# H-6
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# H-7
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# H-8
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Motor Type
# "Specifies the type of motor used in the operating mechanism of High-Speed Earthing Switches (HSES) applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) affects compatibility with substation auxiliary power systems and the responsiveness of interlock logic. Only motor types explicitly stated, or clearly implied through supply voltage description, shall be extracted."
# AI Extraction Logic
Step 1: Direct Statement Extraction — If the motor type is explicitly stated in connection with HSES (e.g., "DC motor," "AC motor") → Extract that value.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the HSES mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Options — If both AC and DC motors are allowed, extract as: AC / DC.
Step 4: Preferred Type with Conditional Allowance — If one is preferred but another is conditionally allowed, extract as: DC (preferred); AC allowed upon approval.
Step 5: Exclusions - Do Not Infer Based On: Typical HSES designs, Vendor defaults, Installation configuration.
Step 6: No Clear Indication — If neither motor type nor related power supply is stated → Leave blank or flag for manual review.

# H-9
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Thermostat Requirement
# Enter the extracted value as Serial Number  E-14

# H-10
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# H-11
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Earthing Path Type (Direct vs. Enclosure)
# Enter the extracted value as Serial Number G-10

# H-12
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Inspection Window Requirement
# Enter the extracted value as Serial Number F-7

''',
type="user",
author="탁근주",
version=250930)

description_prompt_A_H가 저장되었습니다.


In [10]:
# sys_prompt
manager.create_entry(subject="sys_prompt",text='''
You are an AI Virtual Machine Simulator reviewing specification documents in a high-voltage circuit breaker design department. Your task is to extract specific information from a provided PDF document based on predefined categories, specification items, descriptions, and extraction logic. Multiple PDF documents are provided. When printing the extraction results for each item, be sure to include the original text from which the content was extracted, along with the name of the PDF file to which the original text belongs.
''',
type="sys",
author="탁근주",
version=250724)

sys_prompt가 저장되었습니다.


In [10]:
# switchgeartype_prompt
manager.create_entry(subject="switchgeartype_prompt",text='''
## Variable Definition:
- SwitchgearType: The standard field name for the combination of Nominal System Voltage and Switchgear Type (e.g., "380 kV GIS").
- Always use "SwitchgearType" (case-sensitive) as the field name in all outputs and subsequent prompts.

## Purpose:
Extract and bind the High-Voltage Switchgear Type and Nominal System Voltage from project-binding documents.

## Extraction Logic:
Step 1: From project-binding documents (Project Spec/SOW/BOQ/project datasheets/drawings), extract all explicitly required switchgear types; capture the verbatim phrase as Source.  
Step 2: Normalize to these canonical labels:  
- Gas-Insulated Switchgear (GIS)  
- Hybrid Gas-Insulated Switchgear (HGIS / Hybrid GIS)  
- Dead Tank Breakers (DTB)  
- SF₆ Gas-Free Insulated Switchgear  
Exclude individual components such as disconnector switches(DS), earthing switches (ES), capacitive voltage transformer(CVT), Current Transformer(CT), Voltage Transformer(VT), and surge arresters(SA).  
Step 3: Collect the context ID (sentence / table row / scoped section) for each item.  
Step 4: Pair a type with the nominal voltage only if both occur within the same context ID.  
Step 5: Accept the type only if its paired nominal system voltage ≥ 66 kV.  
Step 6: Use the General Specification only when:  
(a) the project documents explicitly reference it as binding, or  
(b) the structure makes a single equipment family unambiguous.  
Step 7: Output the result as a string in the format:  
SwitchgearType: "{Nominal Voltage} {High-Voltage Switchgear Type}" (e.g., SwitchgearType: "380 kV GIS")

output format(list) : 
[Switchgear Type1, Switchgear Type2, Switchgear Type3...]
''',
type="user",
author="탁근주",
version=250930)

switchgeartype_prompt가 저장되었습니다.


In [ ]:
# description_prompt
manager.create_entry(subject="description_prompt",text='''
You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)

# A-1
# General Conditions
# Applicable International Standards
# "Specifies whether the high-voltage switchgear must comply with IEC or IEEE standards, which govern its design, testing, and performance requirements. This determines which standard group’s definitions, design principles, test protocols, and performance criteria must be followed. Only standards applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic:
Step 1: Extract only one primary applicable standard—either IEC Standards or IEEE Standards—applying to the high-voltage switchgear explicitly required in the project specification.
Step 2: If both standards are mentioned, AI must determine whether one governs the switchgear requirements (e.g., by direct reference, section title, or priority statement). If priority is unclear, but one is predominantly cited for switchgear-related clauses, treat it as primary.
Step 3: Do not list all referenced standards; identify only the controlling standard group for the high-voltage switchgear.

# C-2
# Electrical Ratings
# Rated Voltage (IEC)
# "Specifies the phase-to-phase RMS voltage corresponding to the maximum system voltage for which the high-voltage switchgear is designed. In IEC-based specifications, this is referred to as "Rated Voltage". In IEEE-based specifications, the equivalent term is "Rated Maximum Voltage." Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the Rated Voltage associated with each Nominal System Voltage and High-Voltage Switchgear Type explicitly stated in the project specification.
Step 2: If Rated Voltage is not explicitly provided, consult the Customer’s General Standard to extract the corresponding value based on the Nominal Voltage.
Step 3: If multiple Rated Voltage values exist for different product groups, extract and record each separately, ensuring clear alignment with the corresponding Nominal Voltage and Switchgear Type.


# C-3
# Electrical Ratings
# Rated Short-Duration Power-Frequency Withstand Voltage (IEC)
# "This item identifies the rated short-duration power-frequency withstand voltage (PFWV) required for high-voltage switchgear, typically expressed in kV (r.m.s.) for a 1-minute test. This value may be described in IEC or IEEE terms and is used to define the insulation strength of the equipment. Only the value explicitly stated in the customer specification should be extracted. No comparison with standard reference values shall be performed."
# AI Extraction Logic
Step 1: Search for keywords such as "Power-frequency withstand voltage", "1-minute withstand voltage", "AC withstand (r.m.s.)", or "Short-duration power-frequency test voltage", focusing on sections related to dielectric tests or insulation coordination.
Step 2: Extract the PFWV value in kV (r.m.s.), and if mentioned, also extract the test condition such as "phase-to-earth" or "across open switching device".
Step 3: Record the extracted value(s) exactly as written in the customer specification without interpretation or comparison against any standard reference values; if multiple values are given for different insulation paths, record each separately with its corresponding test condition.

# C-4
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage(Phase-to-Earth)
# "This item identifies the rated switching impulse withstand voltage (SIWV) between phase and earth, typically expressed in kV (peak). It is used to evaluate the capability of high-voltage switchgear to withstand slow-front transient overvoltages. Only the value explicitly stated in the customer specification should be extracted. No interpretation or comparison against standard reference values shall be performed."
# AI Extraction Logic
Step 1: Search for keywords such as "Switching impulse withstand voltage", "Switching impulse", "SIWV", "SIV", "Phase-to-earth", "Phase-to-ground", "to ground", "kV peak", "withstand voltage (peak)", focusing on sections related to insulation coordination or dielectric test levels.
Step 2: Extract the SIWV value in kV (peak); if stated, also extract the insulation path (e.g., "phase-to-ground").
Step 3: Record the extracted value(s) exactly as stated in the customer specification; do not compare against any IEC or IEEE standard values; if multiple values are listed (e.g., to ground vs. between phases), extract and label them separately.

# C-5
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage
# "This item identifies the rated lightning impulse withstand voltage (LIWV) required for high-voltage switchgear. It is typically expressed in kV (peak) and represents the equipment's ability to withstand fast-front transient overvoltages such as those caused by lightning. Only the value explicitly stated in the customer specification should be extracted. No interpretation or comparison against standard reference values shall be performed."
# AI Extraction Logic
Step 1: Search for keywords such as "Lightning impulse withstand voltage", "LIWV", "BIL", "Impulse level", "Impulse test", "kV peak", "withstand voltage (peak)", "Phase-to-earth", "across open contact", focusing on sections related to dielectric tests, impulse tests, or insulation coordination.
Step 2: Extract the LIWV value in kV (peak); if stated, extract the test condition (e.g., "to ground", "across open contact", "between phases").
Step 3: Record the extracted value(s) exactly as stated in the customer specification; do not apply any standard-based interpretation or comparison; if multiple LIWV values are listed, extract and label each based on its stated test condition.

# C-6
# Electrical Ratings
# Rated Frequency (IEC)
# "Specifies the nominal operating frequency of the high-voltage switchgear, which affects its electrical performance, insulation coordination, and compatibility with the power grid. The rated frequency is typically 50 Hz or 60 Hz, depending on the regional grid standard. Only frequencies applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the Rated Frequency explicitly stated in the project specification for high-voltage switchgear.
Step 2: If different rated frequency values are defined for multiple product groups, extract and record each separately per nominal voltage level and switchgear type.
Step 3: If no frequency is provided, do not infer a default value (e.g., 50/60 Hz). Leave the field blank or flag for manual review.

# C-7
# Electrical Ratings
# Rated Continuous Current
# "Specifies the RMS value of current that the high-voltage switchgear can carry continuously under its defined service conditions, without exceeding allowable temperature rise or mechanical limits of current-carrying components. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the Rated Continuous Current value explicitly stated in the project specification for each unique combination of Nominal System Voltage and High-Voltage Switchgear Type (e.g., GIS, HGIS, DTB).
Step 2: If multiple values are defined for different product groups or voltage levels, extract and record each value separately, maintaining linkage to its voltage level and switchgear type.
Step 3: Do not infer or estimate a value based on general standards or default assumptions; if no Rated Continuous Current is specified, leave the field blank or flag for manual review.

# C-8
# Electrical Ratings
# Rated Short-Time Withstand Current
# "Specifies the RMS value of the short-circuit current that the high-voltage switchgear can withstand in the closed position for a specified short duration (typically 1s or 3s), without mechanical deformation, internal damage, or insulation failure. This rating validates the switchgear’s thermal and electrodynamic withstand capability under fault conditions until the protection system isolates the fault. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the RMS value (e.g., 31.5kA, 40kA, etc.) of the Rated Short-Time Withstand Current explicitly specified for each product group.
Step 2: If multiple withstand current values are specified for different switchgear types or voltage levels, extract and record each value separately, linked to its product group.
Step 3: Do not infer values based on typical ratings or product type. If the value is not stated, leave the field blank or flag for manual review.

# C-9
# Electrical Ratings
# Rated Peak Withstand Current
# "Specifies the peak value of the first asymmetrical short-circuit current loop that the high-voltage switchgear must withstand without mechanical failure. This rating evaluates the equipment’s ability to endure electrodynamic forces caused by the highest momentary current. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic: 
Step 1: Extract the peak withstand current value (e.g., 100kA peak) if explicitly stated in the project specification or Customer’s General Standard.
Step 2: If the value is not directly provided but a Rated Short-Time Withstand Current (I<sub>th</sub>) and a Peak Factor (e.g., 2.5, 2.7) are both explicitly stated, AI must calculate the peak current using: Rated Peak Withstand Current = I<sub>th</sub> × Peak Factor
Step 3: Record both the calculated peak current in the Tender Requirement field and cite the Peak Factor source in the Source field.
Step 4: If no numerical value or valid Peak Factor is available, leave the field blank or flag for manual review.
Step 5: Do not assume or apply a default peak factor unless it is explicitly stated in the specification or General Standard.

# C-10
# Electrical Ratings
# Rated Duration of Short-Circuit
# "Specifies the time interval (typically 1s or 3s) during which the high-voltage switchgear must withstand the rated short-time current without mechanical or thermal failure while in the closed position. This parameter defines the thermal stress duration that the equipment can safely endure under fault conditions before current interruption. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic:
Step 1: Extract the duration value (e.g., 1s, 3s) explicitly stated in the project specification or Customer’s General Standard.
Step 2: If different durations are specified for different product groups (e.g., GIS vs. DTB), extract and record each value separately.
Step 3: Do not assume a default duration based on switchgear type or voltage level. If no value or reference is stated, leave the field blank or flag for manual review.

# B-1
# Environmental Conditions
# Installation Area
# "Specifies whether the high-voltage switchgear is designed for indoor, outdoor, or mixed installation, depending on the environmental and structural conditions defined in the project specification. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered. If different components (e.g., Gas-Insulated Bus, Hybrid Insulated Switchgear, Dead Tank Breakers) require different installation areas, the installation area must be recorded separately for each switchgear product group."
# AI Extraction Logic
Step 1: Extract whether the switchgear is designated for Indoor, Outdoor, or both, based on explicit statements in the project specification.
Step 2: If different installation areas are defined for different product groups (e.g., GIS: Indoor, GIB: Outdoor), extract and record each separately per group.
Step 3: Do not infer the installation area based on switchgear type or general design trends. If unclear, leave blank or flag for manual review.

# B-2
# Environmental Conditions
# Coastal Area Classification
# "Specifies whether the installation site is explicitly classified as a coastal area, where high humidity, salt, and airborne contaminants significantly increase the risk of corrosion, surface degradation, and long-term material wear. Only classifications applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered. This classification determines whether additional corrosion protection may be required (e.g., duplex coating, reinforced galvanization), but such protection measures shall be handled in separate specification items and shall not be used as indirect evidence."
# AI Extraction Logic
Step 1: Identify explicit statements — Extract only when the specification clearly states that the installation site is classified as a coastal area. Examples: "This site is classified as a coastal area.", "Installations are located in a coastal environment."
Step 2: Extract classification value — If coastal classification is clearly stated → Tender Requirement = Yes; If the site is explicitly stated as not a coastal area → Tender Requirement = No; If no classification is provided or the statement is ambiguous → Leave the field blank or flag for manual review.
Step 3: Prevent false positives (critical) — Do not extract "Yes" in the following cases: When only protective measures are mentioned: "If installed in coastal areas, apply duplex coating.", "Outdoor equipment shall be hot-dip galvanized.", "Coastal paint system shall be applied upon request."; When only geographic hints are given: "Project located near the Red Sea.", "Installed in humid conditions."

# B-3
# Environmental Conditions
# Ambient Air Temperature (Indoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated minimum indoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "indoor," AI must verify the surrounding context. It should be extracted only if clearly associated with indoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the indoor minimum temperature specific to high-voltage switchgear.
Step 4: Ignore general climate data, monthly/annual averages, or values unrelated to equipment installation.
Step 5: If no valid value is found, leave the field blank or flag for manual review.

# B-4
# Environmental Conditions
# Ambient Air Temperature (Indoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated maximum indoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "indoor," AI must verify the surrounding context. It may be extracted only if clearly associated with indoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the indoor maximum temperature applicable to high-voltage switchgear.
Step 4: AI must ignore general climate conditions, monthly/annual temperature data, or values not specifically tied to equipment installation.
Step 5: If no valid value is found, leave the field blank or flag for manual review.

# B-5
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated minimum outdoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "outdoor," AI must verify context and extract it only if clearly associated with outdoor high-voltage switchgear.
Step 3: If no such value is found in the project specification, AI should check the Customer’s General Standard for the applicable outdoor minimum temperature.
Step 4: AI must ignore general climatic or geographical information (e.g., regional climate data, annual weather summaries) unless directly tied to switchgear installation requirements.
Step 5: If no valid value is found, leave the field blank or flag for manual review.

# B-6
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated maximum outdoor ambient temperature for high-voltage switchgear installation from the project specification.
Step 2: If the value is not clearly labeled as "outdoor," AI must verify the surrounding context. Extract the value only if it is clearly associated with outdoor high-voltage switchgear.
Step 3: If the project specification does not provide a valid value, AI must refer to the Customer’s General Standard for an outdoor maximum temperature relevant to high-voltage switchgear.
Step 4: Ignore general climate data (e.g., hottest month average, yearly highs) unless explicitly tied to equipment installation.
Step 5: If no valid temperature is found, leave the field blank or flag for manual review.

# B-7
# Environmental Conditions
# Altitude
# "Specifies the installation altitude applicable to high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated installation altitude for high-voltage switchgear from the project specification.
Step 2: If a site-level altitude (e.g., for a substation or plant) is mentioned and no conflicting equipment-specific value is provided, AI should apply this site altitude to the switchgear.
Step 3: If multiple conflicting altitude values exist, AI must not select or infer—the field must be flagged for manual review.
Step 4: AI must not infer altitude from IEC or IEEE standard assumptions (e.g., "1,000 m default" in IEC 62271-1) unless such assumptions are explicitly referenced in the project specification.
Step 5: The extracted value must come from sections clearly related to equipment installation or site conditions for high-voltage switchgear.

# B-8
# Environmental Conditions
# Site Pollution Severity
# "Specifies the pollution severity level at the installation site, which affects the external insulation performance, creepage distance, and material durability of high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the pollution severity classification — (Light, Medium, Heavy, Very Heavy) explicitly stated in the project specification.
Step 2: If no severity level is specified — Extract contamination data such as Equivalent Salt Deposit Density (ESDD) and Non-Soluble Deposit Density (NSDD), if available.
Step 3: If ESDD/NSDD are provided — Classify severity based on IEC TS 60815-1 as follows: Light: ESDD ≤ 0.03 mg/cm², NSDD ≤ 0.4 mg/cm²; Medium: 0.03 < ESDD ≤ 0.10, 0.4 < NSDD ≤ 1.0; Heavy: 0.10 < ESDD ≤ 0.30, 1.0 < NSDD ≤ 3.0; Very Heavy: ESDD ≥ 0.30, NSDD ≥ 3.0.
Step 4: If the specification references IEEE classifications — (e.g., per IEEE 1313.2), extract the pollution level as defined without converting it to IEC format.
Step 5: Do not infer pollution level — If neither classification nor contamination data is provided; leave the field blank or flag for manual review.

# B-9
# Environmental Conditions
# Seismic Classification
# "Specifies the seismic performance requirements for the installation site of high-voltage switchgear, based on expected earthquake intensity, ground motion, or seismic zoning. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicit seismic classification (e.g., Low, Moderate, High, Zone IIB, IEEE 693 High Level) stated in the project specification.
Step 2: If no classification is provided, extract available seismic data such as Peak Ground Acceleration (PGA) in g, Seismic Response Spectrum, or Seismic Zone Identifier (e.g., Zone IIB, Zone 4).
Step 3: If both classification and raw data are available, extract both separately; do not convert PGA or response spectra into classification tiers unless the mapping is explicitly stated.
Step 4: If no seismic information is found, leave the field blank or flag for manual review.
Step 5: Do not infer or calculate classification from PGA or response data using IEC 62271-207 or IEEE 693 tables unless the document explicitly references such mapping.



# D-3
# Design Parameters
# Insulation Medium Requirement
# "Check whether the customer specifies a particular insulation medium to be used in the high-voltage switchgear. The requirement may include allowance or prohibition of SF₆ (sulfur hexafluoride), approval for SF₆-free alternatives (e.g., C4-FN, dry air, vacuum), or statements about environmental compliance. The insulation medium may apply to GIS, DTB, GIB, or other major gas-insulated components."
# AI Extraction Logic
Step 1: Identify references to insulation medium or insulating gas — Used in GIS, DTB, HGIS, GIB, or other HV switchgear types.
Step 2: Detect expressions — Such as: "SF6 shall be used as the insulation medium", "Only SF6-free solution shall be accepted", "Insulation gas must comply with F-gas regulation", "C4-FN or dry air shall be applied", "No fluorinated gas allowed".
Step 3: Extract the insulation medium type(s) mentioned — And note whether it is: Required (e.g., "SF6-free only"); Allowed (e.g., "SF6 permitted"); Prohibited (e.g., "SF6 not allowed"). If multiple options are listed (e.g., SF6 or dry air), extract all.

# D-4
# Design Parameters
# Enclosure Type
# "Specifies whether the high-voltage switchgear enclosure is designed as a single-phase or three-phase configuration. Only switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the enclosure type (Single-Phase or Three-Phase) explicitly stated in the project specification.
Step 2: If different enclosure types are defined for different voltage levels or product groups, extract them separately and link to the appropriate group.
Step 3: If both types are listed as acceptable (e.g., "either type is acceptable"), record them in the format: Single-Phase / Three-Phase.
Step 4: If one type is preferred and another conditionally allowed, record as: Single-Phase (preferred) or Single-Phase; Three-Phase allowed upon approval.
Step 5: Do not infer enclosure type from voltage level, product design, or general practice; only extract explicitly defined values.
Step 6: If enclosure type is not stated, leave the field blank or flag for manual review.

# D-5
# Design Parameters
# Enclosure Material
# "Specifies the primary metallic material used for the enclosure (housing) of high-voltage switchgear, which impacts corrosion resistance, mechanical strength, and environmental durability. Only enclosures for equipment rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated enclosure material from the project specification (e.g., Aluminum, Stainless Steel, Galvanized Steel, Al-Zn Coated Steel).
Step 2: If different materials are defined for separate voltage levels or product groups, extract and record them separately per group.
Step 3: If multiple materials are permitted (e.g., "Aluminum or Stainless Steel"), record them in the format: Aluminum / Stainless Steel.
Step 4: If one material is preferred and another is conditionally allowed, record as: Aluminum (preferred) or Aluminum; Stainless Steel allowed upon approval.
Step 5: Do not infer enclosure material from product family, insulation type, or coating reference unless explicitly stated.
Step 6: If no valid material is provided, leave the field blank or flag for manual review.

# D-6
# Design Parameters
# Conductor Material
# "Specifies the material used for the main current-carrying conductors (e.g., busbars, terminals) inside the high-voltage switchgear. The material affects electrical conductivity, current capacity, thermal performance, and mechanical stability. Only conductors used in switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the explicitly stated conductor material — (e.g., Copper, Aluminum) from the project specification.
Step 2: If both materials are allowed — Record in the format: Copper / Aluminum.
Step 3: If one is preferred and another allowed with conditions — Record as: Copper (preferred) or Copper; Aluminum allowed upon approval.
Step 4: Do not infer conductor material — Based on system voltage, current rating, or performance class.
Step 5: If not specified — Leave the field blank or flag for manual review.

D-7
Design Parameters
Partial Discharge Level
"Specifies the maximum permissible partial discharge (PD) intensity for internal components of high-voltage switchgear, including busbars, support insulators, bushings, and other insulation elements.
This requirement ensures compliance with insulation quality criteria under type test and routine test conditions.
PD limits are typically governed by international standards such as IEC 62271-203 or IEEE Std C37.122.
Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract any explicit numerical value for PD level (e.g., less than5pC) stated in the project specification or customer’s general standard.
Step 2: If only a reference to a standard is provided (e.g., "PD level as per IEC 62271-203" or "as per IEEE Std C37.122"), record the Tender Requirement as "As per [standard name]" and cite it in the Source field.
Step 3: If both a numerical value and a standard are given, record the numerical value as the Tender Requirement and the standard as the Source.
Step 4: AI must not infer or estimate PD levels based on equipment type or nominal voltage. If no clear information is available, the field should be left blank or flagged for review."

# D-8
# Design Parameters
# Maximum Relative Gas Leakage Rate
# "Specifies the maximum allowable annual gas leakage rate, expressed as a percentage of the total gas quantity within each gas-filled compartment of high-voltage switchgear. This requirement ensures long-term gas retention integrity, pressure stability, and compliance with environmental regulations for equipment using SF₆, SF₆ mixtures, or alternative insulating gases. Leakage limits are typically defined in international standards such as IEC 62271-203 or functionally equivalent clauses in IEEE guidelines, depending on the applicable standard. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract any explicit numerical value for the annual relative leakage rate per gas compartment (e.g., less than0.5% per year) from the project specification or customer’s general standard.
Step 2: If only a reference to a standard is provided (e.g., "as per IEC 62271-203" or "as per IEEE C37.x"), extract "As per [standard name]" as the Tender Requirement, and cite the reference in the Source field.
Step 3: If both a numerical value and a standard reference are provided, record the numerical value as the Tender Requirement and the referenced standard as the Source.
Step 4: AI must not infer default values based on voltage, product type, or insulation gas type. If no clear value or reference is provided, leave the field blank or flag it for manual review.

# D-9
# Design Parameters
# Internal Fault - Short-Circuit Current
# "Specifies the RMS value of short-circuit current the high-voltage switchgear must withstand during an internal arc fault. If the internal arc withstand current is not explicitly defined in the specification, it shall be assumed equal to the Rated Short-Time Withstand Current (as per IEC 62271-203 and industry practice)."
# AI Extraction Logic
Step 1: If the internal arc fault withstand current is explicitly stated, extract that value.
Step 2: If not stated, extract the Rated Short-Time Withstand Current and apply it as the internal fault current.
Step 3: Cite the source appropriately—either direct internal arc requirement or reference to Rated Short-Time Withstand Current.
Step 4: Do not infer any alternate value based on voltage or switchgear type.

# D-10
# Design Parameters
# Internal Fault - Duration of Current
# "Specifies the minimum time duration that the high-voltage switchgear enclosure must withstand the effects of an internal arc fault without loss of containment or unsafe failure, under the specified internal fault current level. This duration (commonly expressed as 300ms, 500ms, or 1s) represents the burn-through resistance of the enclosure before fragmentation or external effects occur. IEC 62271-203 and IEEE C37.122 provide standard classifications and test protocols for burn-through time. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the internal arc duration (e.g., 300ms, 0.5s, 1s) if explicitly stated in the project specification or customer’s general standard.
Step 2: Accept terminology such as "burn-through time," "internal arc duration," or "arc withstand time", if clearly linked to GIS, HGIS, or DTB arc fault containment.
Step 3: If different durations apply to different product groups or voltage levels, extract and record them separately.
Step 4: If no value is provided, leave the field blank or flag for manual review.
Step 5: If the internal arc duration is not explicitly defined but performance criteria (e.g., "must withstand arc without external effect for 0.3s") are provided, extract the time value only if unambiguous.
Note: Do not infer this value from general short-circuit duration unless the specification clearly states it applies to internal fault conditions.

# D-11
# Design Parameters
# Maximum Length of Gas Compartment
# "Specifies the maximum allowable physical length of an individual gas-tight compartment within high-voltage switchgear (e.g., GIS or HGIS), as constrained by insulation reliability, gas leakage risk, pressure stability, and maintenance considerations. In some customer standards, limits are defined either by maximum physical length (e.g., 18m to 22.5m) or by maximum total gas volume per compartment (e.g., 4m³ of SF₆). Extensions beyond this length may be permitted with prior customer approval, provided that the total gas volume in the compartment does not exceed 4m³. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: Extract the maximum allowable length value (e.g., 22.5m) explicitly stated in the project specification or customer’s general standard.
Step 2: If the specification includes a conditional exception (e.g., longer length allowed if gas volume is 4m³ or less), extract both the base value and the exception, and record the condition in the Source field.
Step 3: If both length and volume-based conditions are given, extract the primary length limit in the Tender Requirement field and reference the volume condition in the Source field.
Step 4: If different length limits apply to different product groups, extract and record each separately.
Step 5: If no value or conditional rule is stated, leave the field blank or flag for manual review.

# E-2
# Circuit Breaker
# Operating Mechanism Type
# "Specifies the type of mechanism used to operate the circuit breaker in high-voltage switchgear rated at nominal system voltages of 66kV and above."
# AI Extraction Logic
Step 1: AI must extract the mechanism type explicitly stated in the project specification (e.g., Spring-Operated, Hydraulic-Operated).
Step 2: If multiple types are allowed, AI must record them using the format: Spring / Hydraulic / Pneumatic.
Step 3: If a preferred type is mentioned and others are conditionally accepted, use format: Spring (preferred); Hydraulic allowed upon approval.
Step 4: AI must not infer mechanism type based on voltage class, vendor trend, or typical configurations.
Step 5: If not specified, leave the field blank or flag for manual review.

# E-3
# Circuit Breaker
# Rated First-Pole-to-Clear Factor
# "Specifies the factor used in insulation coordination and TRV assessment to account for the recovery voltage across the first pole to clear in a three-phase circuit breaker. It represents the maximum overvoltage stress that may occur on the first pole when the other two phases are still conducting. The factor is defined in IEC 62271-100 and IEEE C37.06 and is dependent on system grounding and voltage class."
# AI Extraction Logic
Step 1: Extract the numerical value of the Rated First-Pole-to-Clear Factor (e.g., 1.3, 1.5) explicitly stated in the project specification.
Step 2: If multiple values are described based on system grounding (e.g., grounded vs. ungrounded), extract both with clear conditional labeling.
Step 3: Do not record general references like "as per IEC 62271-100" unless accompanied by an actual numerical value.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not assume default values. If no value is found, leave the field blank or flag for manual review.

# E-4
# Circuit Breaker
# Rated Operating Sequence
# "Specifies the rated operating sequence of the circuit breaker, which defines the standard sequence of open and close operations the breaker is designed to perform under defined conditions. This value is used to evaluate the breaker's mechanical and thermal performance during normal and fault-handling scenarios. The requirement applies to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the rated operating sequence exactly as stated in the customer specification (e.g., "O - CO - CO").
Step 2: If multiple sequences are specified for different applications or breaker types, extract and record each separately.
Step 3: Do not infer or generate default sequences based on breaker type or system voltage.
Step 4: If the operating sequence is only referenced via a standard (e.g., "as per IEC 62271-100"), record: Tender Requirement → "As per IEC 62271-100", Source → Cite the standard and clause if given.
Step 5: If no value is found, leave the field blank or flag for manual review.

# E-5
# Circuit Breaker
# Electrical Endurance Class
# "Specifies the electrical endurance class of the circuit breaker according to IEC 62271-100 for high-voltage switchgear rated at nominal system voltages of 66kV and above. The endurance class defines the number of short-circuit current interruptions the circuit breaker can perform under specified conditions without performance degradation. Only explicitly stated classifications (e.g., E1, E2) shall be considered."
# AI Extraction Logic
Step 1: Extract the electrical endurance class — Only if explicitly stated (e.g., "Class E2").
Step 2: If multiple classes are allowed — Record in format: E1 / E2.
Step 3: If a preferred class is listed with conditional allowance of another — Use format: E2 (preferred); E1 allowed upon approval.
Step 4: Do not infer the class — Based on: Rated short-circuit current, Voltage level, Vendor design, Past project data.
Step 5: If not specified — Leave blank or flag for manual review.

# E-6
# Circuit Breaker
# Rated Supply Voltage of Control Circuits
# "Specifies the rated supply voltage applied to the control circuits of high-voltage switchgear (rated at 72.5kV and above, or nominal system voltage of 66kV and above). These circuits include those used for protection, monitoring, interlocking, signaling, and operating functions. The rated supply voltage ensures compatibility with auxiliary equipment such as relays and substation power sources."
# AI Extraction Logic
Step 1: Extract the rated control voltage(s) explicitly stated in the project specification (e.g., 125V DC, 230V AC).
Step 2: If multiple control voltages are provided for different product groups (e.g., GIS: 125V DC / DTB: 230V AC), extract and record each separately, linked to the corresponding group.
Step 3: If a voltage range is specified for performance tolerance (e.g., 100-140V DC), record it in the "Source" field, not in the "Tender Requirement" unless explicitly stated as the rated voltage.
Step 4: Do not infer control voltage based on mechanism type, region, or vendor defaults.
Step 5: If no value is provided, leave the field blank or flag for manual review.

# E-7
# Circuit Breaker
# Operating Voltage Range of Control Circuits
# "Specifies the acceptable operating voltage range for control circuits in high-voltage switchgear (rated at 72.5kV and above, or nominal system voltage of 66kV and above). This range defines the voltage tolerance within which the control, protection, and annunciator systems must function reliably. It may differ from the rated supply voltage and is important for evaluating system margin, undervoltage behavior, and auxiliary power design."
# AI Extraction Logic
Step 1: Extract the operating voltage range only if explicitly stated in the project specification (e.g., "100V DC to 140V DC"). Accept expressions such as: "Operating voltage range…", "Permissible range for control supply…", "Voltage tolerance range…"
Step 2: Do not extract this value into the "Tender Requirement" field of Rated Supply Voltage; store it in the dedicated item only.
Step 3: If only a range is specified without a rated voltage, still extract the range under this item and flag for manual review if needed.
Step 4: If multiple ranges are given for different systems (e.g., control vs annunciator), label accordingly or extract separately.
Step 5: Do not infer this value based on national standards, protection relay specs, or vendor catalogs.
Step 6: If no explicit range is found, leave blank or flag for manual review.

# E-8
# Circuit Breaker
# Operating Voltage Range of Motor Circuits
# "Defines the acceptable voltage range (DC or AC) within which the spring-charging motor of the circuit breaker operating mechanism must operate reliably. This range ensures consistent energy storage and charging performance under variable auxiliary supply conditions in high-voltage switchgear rated at 72.5kV and above or with nominal system voltage of 66kV and above."
# AI Extraction Logic
Step 1: Extract the motor operating voltage range explicitly stated in the project specification (e.g., 100-140V DC).
Step 2: If only a reference to an international standard is provided (e.g., "as per IEC 62271-1"), record:
Tender Requirement → "As per IEC 62271-1"
Source → "IEC 62271-1, Clause X.X"
Step 3: If multiple product groups use different motor voltage ranges, extract and record each separately.
Step 4: Apply this item only to high-voltage switchgear (Rated Voltage at 72.5kV and above or Nominal System Voltage at 66kV or above).
Step 5: Do not assume default ranges based on standard motor ratings, switchgear type, or vendor norms. If no value is found, leave the field blank or flag for manual review.

# E-9
# Circuit Breaker
# Operating Voltage Range of Shunt Closing Release
# "Specifies the allowable voltage range within which the shunt closing release coil must operate to ensure reliable mechanical closing of the circuit breaker. This range ensures proper functioning of the trip-close system even under fluctuating control power conditions in high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the shunt closing release operating voltage range explicitly stated in the project specification.
Step 2: If no range is stated but a reference to a standard is provided (e.g., "as per IEC 62271-1"), record: Tender Requirement → "As per IEC 62271-1", Source → "IEC 62271-1, Clause X.X".
Step 3: If different voltage ranges are specified for different product groups (e.g., GIS vs. DTB), record each separately.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not infer values based on other control voltages, mechanism type, or standard assumptions.
Step 6: If no value is found, leave blank or flag for manual review.

# E-10
# Circuit Breaker
# Operating Voltage Range of Shunt Opening Release
# "Specifies the allowable voltage range within which the shunt opening release coil must operate to trigger breaker tripping commands. Accurate specification of this range ensures reliable fault interruption under varying control voltage conditions in high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: Extract the shunt opening release operating voltage range explicitly stated in the project specification.
Step 2: If a standard is referenced instead of a defined range (e.g., "as per IEC 62271-1"), record as follows: Tender Requirement → "As per IEC 62271-1", Source → "IEC 62271-1, Clause X.X".
Step 3: If values differ by equipment group (e.g., GIS: 100-140V DC / DTB: 110-130V DC), record them separately.
Step 4: Apply this item only to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher.
Step 5: Do not assume default values based on control voltage type, manufacturer, or product design.
Step 6: If no value is found, leave the field blank or flag for manual review.

# E-11
# Circuit Breaker
# Rated Supply Voltage of Heating Circuits
# "Specifies the rated voltage of the power supply for space heaters, anti-condensation heaters, cabinet heaters, or enclosure heaters used in high-voltage switchgear rated at 72.5kV and above (or nominal system voltage of 66kV and above). This voltage ensures proper humidity and temperature control within enclosures to prevent moisture-related insulation failures and equipment degradation."
# AI Extraction Logic
Step 1: Extract the heater supply voltage explicitly stated in the project specification. Accept expressions such as: "Rated 230V AC", "230V anti-condensation heater", "Cabinet heater rated 230V".
Step 2: If multiple voltages apply to different product groups (e.g., GIS, DTB), extract and label them separately: GIS: 230V AC / DTB: 110V AC.
Step 3: If one voltage is preferred and another allowed conditionally, record as: [Preferred Value] (preferred); [Alternative] allowed upon approval.
Step 4: Do not infer voltage from: Control circuit voltage, Region of use, Vendor design norms.
Step 5: If the voltage is not explicitly stated or is only mentioned in unrelated sections, leave the field blank or flag for manual review.

# E-12
# Circuit Breaker
# Break-Time
# "Specifies the time interval between the initiation of the opening operation of the circuit breaker and the final extinction of the arc during current interruption. This parameter defines the duration required for complete fault current interruption and is critical in assessing the fault clearance speed and dynamic coordination of protection schemes. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
Step 1: AI must extract any explicitly stated Break-Time value (e.g., 2 cycles, 40ms, etc.) from the project specification or Customer’s General Standard.
Step 2: If multiple Break-Time values are provided for different product groups or voltage levels, AI must extract and record each separately, matched to the corresponding product.
Step 3: If the value is given in cycles (e.g., 3 cycles at 60Hz), AI must calculate the equivalent time in milliseconds (e.g., 50ms) and store that value in the Tender Requirement field. The original unit and system frequency (e.g., 3 cycles @ 60Hz) must be documented in the Source field.
Step 4: AI must not infer a Break-Time value based on mechanism type (e.g., spring vs. hydraulic), vendor preference, or typical design assumptions.
Step 5: If no Break-Time value is specified in the project specification or Customer’s General Standard, AI must leave the field blank or flag it for manual review.

# E-13
# Circuit Breaker
# Controlled Switching Requirement
# "Specifies whether the project specification requires controlled switching for high-voltage circuit breakers (rated 66 kV and above). Controlled switching refers to the operation of a circuit breaker at a defined point of the power frequency waveform (current or voltage) to reduce inrush current, switching transients, and mechanical stress. IEC officially defines this operation as \"controlled switching,\" and the terms \"point-on-wave switching\" or \"point-on-cycle switching\" are commonly used equivalents in specifications."
# AI Extraction Logic
Step 1: Extract the requirement if the specification refers to "controlled switching," "point-on-wave controller," or "point-on-cycle operation" as required, prohibited, or conditionally permitted. → Record the phrase verbatim as the Tender Requirement and cite the Source.
Step 2: If multiple terms (e.g., "Controlled Switching Device," "Pole Switching Equipment") are used ambiguously, flag for manual review with a note: "Potential controlled switching terminology used."
Step 3: Do not infer controlled switching requirements based on breaker type, rating, or expected application (e.g., shunt reactor).
Step 4: If no relevant term is found, leave the field blank or flag for review.

# E-14
# Circuit Breaker
# Single-Phase Operation Requirement
# "Check whether the customer explicitly requires the circuit breaker to be capable of per-phase (single-phase) operation—i.e., independent opening/closing of one phase while other phases remain unaffected. Do not extract when the phrase appears only in a conditional or illustrative clause tied to auxiliary devices (e.g., operation counters). Extract only when the main requirement states the breaker shall be capable of single-phase/individual-pole operation."
# AI Extraction Logic
Step 1: (Candidate phrases): Detect per-phase capability phrases (e.g., “single-phase operation,” “single-pole operation,” “individual pole operation,” “independent phase control,” “one pole can operate independently,” “per-phase switching”).
Step 2: (Scope): Confirm the subject is a high-voltage circuit breaker (nominal ≥ 66 kV).
Step 3: (Main-clause gate): The main predicate must express capability with strong modality tied to the breaker (e.g., “CB shall be capable of independent pole operation,” “CB shall allow single-phase operation,” “CB is required to provide single-phase operation”).
If-clause guard: If the capability phrase appears only in a subordinate if/when/where/provided that/in the event that/should clause, do not extract.
Table/label gate: If shown as a label/value (e.g., “Independent pole operation: Yes/Required/Not permitted”), treat it as normative and map accordingly.
Weak modality rule: “can/may” does not satisfy this gate unless in a label/value form.
Step 4: (Auxiliary-device gate): If the same sentence’s main clause mandates auxiliary devices via “shall/must be equipped/fitted/provided/furnished (with) operation counter/timing device/terminal block,” treat it as auxiliary-device context → do not extract for this item.
Step 5: (Output): Extract the tender requirement verbatim only if Steps 2–4 pass; include any scope qualifiers. Otherwise leave blank or flag for manual review.
Step 6: (No inference): Do not infer capability from product type, rating, or vendor practice.

# E-15
# Circuit Breaker
# Manual Operation Requirement
# "Specifies whether the circuit breaker shall be equipped with manual trip and/or manual closing functionality in addition to standard electrical operation. This functionality allows operation of the breaker during commissioning, maintenance, or emergency conditions when electrical control may not be available. This requirement applies to high-voltage switchgear rated at 72.5kV or higher, or with nominal system voltage at 66kV or higher."
# AI Extraction Logic
Step 1: If the specification explicitly states: Only manual trip is required → record: Trip Only, Both manual trip and manual closing are required → record: Trip & Closing, Manual operation is not required or prohibited → record: Not Required.
Step 2: If the specification contains general or conditional expressions such as: "manual operation shall be provided", "if necessary, manual operation may be included" → Extract the phrase verbatim as the Tender Requirement, cite the Source, and flag for manual review.
Step 3: If different requirements are specified for different switchgear types or voltage levels, extract and record each separately.
Step 4: Do not infer operation mode based on product type, manufacturer practice, or standard design assumptions.
Step 5: If no mention of manual operation is found in the specification, leave the field blank or flag for manual review.

# E-16
# Circuit Breaker
# CT and CB Gas Compartment Separation Requirement
# "Specifies whether the project documentation explicitly requires the current transformer (CT) and circuit breaker (CB) to be installed in separate gas compartments in high-voltage switchgear (rated 66kV and above). This requirement directly affects compartmentalization design and maintenance isolation strategy. AI must extract this item strictly from explicitly stated customer requirements."
# AI Extraction Logic
Step 1: If the specification clearly states: "CT shall be in a separate compartment from CB" → Extract this phrase verbatim as the Tender Requirement and cite the Source. "CT may be in the CB compartment" or "combination is acceptable" → Likewise, extract the phrase verbatim and cite the Source.
Step 2: If the specification states: "CT shall be in a separate compartment" (without directly referencing CB), AND other parts of the document confirm that CB is in its own compartment, → Extract the phrase and annotate as "Implied separation from CB"; cite the Source and flag if necessary.
Step 3: If the specification includes a conditional allowance (e.g., "CT in CB gas compartment shall be accepted with prior approval" or "separation preferred"), → Extract the full conditional phrase, cite the Source, and flag for manual review.
Step 4: If no information is provided regarding CT and CB compartment separation, → Leave the field blank or flag for manual review.
Step 5: Do not infer separation based on: Switchgear type (e.g., GIS vs. HGIS), Vendor design practices, Layout drawings or typical configurations unless explicitly stated in writing.

# E-17
# Circuit Breaker
# Motor Type
# "Specifies the type of motor used in the circuit breaker operating mechanism for high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (AC or DC) determines compatibility with site power sources, affects control logic design, and impacts procurement and maintenance planning. Only motor types explicitly stated or clearly implied through motor voltage notation in project specifications shall be considered."
# AI Extraction Logic
Step 1: Extract the motor type if explicitly stated (e.g., "AC motor," "DC motor") in relation to the circuit breaker operating mechanism.
Step 2: If the motor voltage is stated using standard notation (e.g., "125Vdc," "400Vac"), infer the motor type as: "Vdc" → DC motor, "Vac" → AC motor.
Step 3: If multiple motor types are permitted, extract using the format: AC / DC.
Step 4: If one motor type is preferred and others are conditionally accepted, use the format: DC (preferred); AC allowed upon approval.
Step 5: Do not infer motor type based on control voltage, vendor defaults, or mechanism type unless the motor voltage format clearly indicates type. Only values explicitly stated for high-voltage switchgear shall be extracted.
Step 6: If not specified or unclear, leave blank or flag for manual review.

# E-18
# Circuit Breaker
# Thermostat Requirement
# "Specifies whether the project documentation explicitly mentions the inclusion, exclusion, or conditional use of a thermostat in high-voltage switchgear (rated 66kV and above). Thermostats are typically integrated into anti-condensation or space heater circuits to regulate internal temperature and prevent moisture-related issues. AI must extract the requirement only when it is explicitly stated, whether mandatory, prohibited, or conditional."
# AI Extraction Logic
Step 1: If the specification clearly states that a thermostat is required (e.g., "thermostat shall be provided", "shall be included"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 2: If the specification clearly states that a thermostat is not required (e.g., "shall not be used", "not required"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 3: If the specification contains a conditional expression (e.g., "if necessary", "may be used", "optional"), → Extract the phrase verbatim as the Tender Requirement, and cite the Source. → This shall be flagged or annotated as a conditional requirement for manual review.
Step 4: Do not infer the requirement based on climate zone, ambient temperature, heater presence, or switchgear type.
Step 5: If no mention of a thermostat is found in the specification, → Leave the field blank or flag for manual review.

# E-19
# Circuit Breaker
# Humidistat Requirement
# "Specifies whether the project documentation explicitly mentions the inclusion, exclusion, or conditional use of a humidistat in high-voltage switchgear (rated 66kV and above). Humidistats are typically used to control heater or ventilation systems in response to internal humidity, preventing condensation and ensuring insulation reliability. AI must extract the requirement only when it is explicitly stated, whether mandatory, prohibited, or conditional."
# AI Extraction Logic
Step 1: If the specification clearly states that a humidistat is required (e.g., "humidistat shall be provided", "shall be included"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 2: If the specification clearly states that a humidistat is not required (e.g., "shall not be used", "not required"), → Extract the phrase as the Tender Requirement, and cite the Source.
Step 3: If the specification contains a conditional expression (e.g., "if necessary", "may be used", "optional"), → Extract the phrase verbatim as the Tender Requirement, and cite the Source. → This shall be flagged or annotated as a conditional requirement for manual review.
Step 4: Do not infer the requirement based on ambient humidity, environmental conditions, climate zone, or presence of heater circuits.
Step 5: If no mention of a humidistat is found in the specification, → Leave the field blank or flag for manual review.

# F-2
# Disconnector
# Operating Mechanism Type
# "Specifies the type of mechanism used to operate the disconnector (also known as an isolator) in high-voltage switchgear rated at nominal system voltages of 66kV and above."
# AI Extraction Logic
Step 1: AI must extract the mechanism type explicitly stated in the project specification (e.g., Motor-Operated, Manual-Operated).
Step 2: If multiple types are permitted, AI must record them using the format: Motor / Manual.
Step 3: If a preferred type is stated with others allowed under conditions, use format: Motor (preferred); Manual allowed upon approval.
Step 4: AI must not infer the mechanism type based on vendor design trends, voltage class, or typical practice.
Step 5: If not specified in the specification, leave the field blank or flag for manual review.

# F-3
# Disconnector
# Mechanical Endurance Class
# "Specifies the required mechanical endurance performance of the disconnector (also known as disconnecting switch) in high-voltage switchgear rated at nominal system voltages of 66kV and above. Mechanical endurance refers to the number of no-load operating cycles the device must withstand without mechanical failure or loss of function. Endurance may be expressed in standardized IEC classes (e.g., M1, M2) or as a specific number of operations."
# AI Extraction Logic
Step 1: If the specification explicitly states an IEC mechanical endurance class (e.g., "Class M1" or "M2"), extract it as the Tender Requirement and cite the Source.
Step 2: If a numerical value is provided (e.g., "2,000 operations"), extract the value verbatim and cite the Source. Do not convert the number into M1/M2 unless explicitly stated.
Step 3: If both class and numeric value are provided, extract both using format: M2 (10,000 operations).
Step 4: If the endurance requirement is stated conditionally (e.g., "Minimum 2,000 operations required unless approved otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer endurance class from switchgear type, vendor habits, or assumed usage.
Step 6: If no information is found, leave the field blank or flag for manual review.

# F-4
# Disconnector
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# F-5
# Disconnector
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# F-6
# Disconnector
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# F-7
# Disconnector
# Motor Type
# "Specifies the type of motor used in the operating mechanism of high-voltage disconnectors rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) determines compatibility with site power sources and impacts automation and interlocking schemes. Only motor types explicitly stated or clearly implied through voltage supply notation shall be considered."
# AI Extraction Logic
Step 1: Identify Direct Statements — If a phrase like "AC motor", "DC motor", or "motor type: DC" is explicitly stated in relation to disconnector → Extract as is.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the disconnector mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Types — If more than one type is allowed: → Use format: AC / DC.
Step 4: Preference Indicated — If one is preferred and another is conditionally allowed: → Use format: DC (preferred); AC allowed upon approval.
Step 5: Conditions — Apply logic even when motor type is not directly mentioned but voltage + mechanism context is clearly established.
Step 6: Restrictions — Do not infer from switchgear type, vendor name, or general practice; if unclear (e.g., "electrical operation" but no supply voltage), → Leave blank or flag for manual review.

# F-8
# Disconnector
# Thermostat Requirement
# Enter the extracted value as Serial Number E-14

# F-9
# Disconnector
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# F-10
# Disconnector
# Viewport Requirement
# "Specifies whether the high-voltage switchgear (rated at 66kV and above) is required to include a viewing window or inspection window in the enclosure of components such as the disconnector or earthing switch. This window enables visual confirmation of isolating distance or contact position during operation or maintenance. The viewing window may also be referred to as an inspection port, endoscopy port, viewport or visual inspection opening in customer specifications."
# AI Extraction Logic
Step 1: If the specification clearly states that a viewing window, inspection window, or endoscope port shall be provided, → Extract the phrase verbatim as the Tender Requirement and cite the Source.
Step 2: If the specification requires the ability to visually confirm isolating distance or movable contact position through a window, → Treat this as a functional requirement for a viewing window and extract accordingly.
Step 3: If multiple options are allowed (e.g., "inspection window or endoscope port"), → Extract all listed options using format: Inspection Window / Endoscope Port.
Step 4: If the specification includes additional requirements (e.g., cover plate, color marking, warning label), → These shall not be extracted under this item, but may be cited in the Source for reference.
Step 5: Do not infer the need for a viewing window based on switchgear type, GIS design trends, or assumed accessibility. Only explicit requirements or functional justifications (e.g., "to observe isolating gap") shall be extracted.
Step 6: If no mention of a viewing window or equivalent is found, leave the field blank or flag for manual review.

# G-2
# Maintenance Earthing Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used in Maintenance Earthing Switches (MES) for high-voltage switchgear rated at nominal system voltages of 66kV and above. MES are typically used to provide a visible and secure earth connection during equipment maintenance, and do not require high-speed operation. Common mechanisms include motor-operated or manual operation."
# AI Extraction Logic
Step 1: Extract the mechanism type explicitly stated in the specification for MES (e.g., Motor-operated, Manual).
Step 2: If multiple mechanism types are allowed or listed, use the format: Motor / Manual.
Step 3: If one mechanism is preferred with others allowed upon approval, use format: Motor (preferred); Manual allowed upon approval.
Step 4: If a manual device is specified in addition to the motor/snap mechanism, include it as a separate type.
Step 5: Do not infer mechanism type based on function or vendor trend. Only extract if stated.

# G-3
# Maintenance Earthing Switch
# Mechanical Endurance Class
# "Specifies the required mechanical endurance performance of the maintenance earthing switch (MES) used in high-voltage switchgear rated at nominal system voltages of 66kV and above. Mechanical endurance refers to the number of no-load mechanical operating cycles the MES must withstand while maintaining structural integrity and functional reliability. Endurance may be classified according to IEC categories (e.g., M0, M1) or stated as a fixed number of operations."
# AI Extraction Logic
Step 1: If the specification explicitly states an IEC mechanical endurance class (e.g., "M0", "M1"), extract it as the Tender Requirement and cite the Source.
Step 2: If a numerical requirement is provided (e.g., "2,000 operations"), extract the number verbatim and cite the Source. Do not interpret or convert into M0/M1 unless explicitly matched in the specification.
Step 3: If both class and numeric value are stated, extract both using format: M1 (2,000 operations).
Step 4: If the requirement includes conditional or fallback wording (e.g., "at least 1,000 operations unless specified otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer the endurance class or operation count from switchgear type, manufacturer habits, or general performance claims.
Step 6: If not mentioned in the specification, leave the field blank or flag for manual review.

# G-4
# Maintenance Earthing Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# G-5
# Maintenance Earthing Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# G-6
# Maintenance Earthing Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# G-7
# Maintenance Earthing Switch
# Motor Type
# "Specifies the type of motor used in the operating mechanism of Maintenance Earthing Switches (MES) applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) affects compatibility with substation auxiliary power systems and the responsiveness of interlock logic. Only motor types explicitly stated, or clearly implied through supply voltage description, shall be extracted."
# AI Extraction Logic
Step 1: Direct Statement Extraction — If the motor type is explicitly stated in connection with MES (e.g., "DC motor," "AC motor") → Extract that value.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the MES mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Options — If both AC and DC motors are allowed, extract as: AC / DC.
Step 4: Preferred Type with Conditional Allowance — If one is preferred but another is conditionally allowed, extract as: DC (preferred); AC allowed upon approval.
Step 5: Exclusions - Do Not Infer Based On: Typical MES designs, Vendor defaults, Installation configuration.
Step 6: No Clear Indication — If neither motor type nor related power supply is stated → Leave blank or flag for manual review.

# G-8
# Maintenance Earthing Switch
# Thermostat Requirement
# Enter the extracted value as Serial Number E-14

# G-9
# Maintenance Earthing Switch
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# G-10
# Maintenance Earthing Switch
# Earthing Path Type (Direct vs. Enclosure)
# "Specifies whether the earthing switch is permitted or required to perform short-circuit making onto an earthed part directly or through the metallic enclosure, as allowed by IEC and IEEE standards for high-voltage switchgear rated at 66kV and above. IEC 62271-102 explicitly permits both configurations if electrical continuity and making current withstand are properly verified."
# AI Extraction Logic
Step 1: If the specification states that the earthing switch must connect directly to earth, extract as: "Direct only".
Step 2: If it states that enclosure path is allowed or preferred, extract as: "Through enclosure allowed" or "Through enclosure (preferred)".
Step 3: If both options are permitted without preference, extract as: "Direct / Through enclosure".
Step 4: If test requirements or functional conditions are stated (e.g., "if electrical continuity of the enclosure is ensured"), extract relevant configuration and cite source with note.
Step 5: Do not infer based on switchgear type, vendor design, or standard practices.
Step 6: If no mention of earthing path is found, leave blank or flag for manual review.

# G-11
# Maintenance Earthing Switch
# Viewport Requirement
# Enter the extracted value as Serial Number F-7

# H-2
# High Speed Earthing Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used in Maintenance Earthing Switches (MES) for high-voltage switchgear rated at nominal system voltages of 66kV and above. MES are typically used to provide a visible and secure earth connection during equipment maintenance, and do not require high-speed operation. Common mechanisms include motor-operated or manual operation."
# AI Extraction Logic
Step 1: Extract the mechanism type explicitly stated in the specification for MES (e.g., Motor-operated, Manual).
Step 2: If multiple mechanism types are allowed or listed, use the format: Motor / Manual.
Step 3: If one mechanism is preferred with others allowed upon approval, use format: Motor (preferred); Manual allowed upon approval.
Step 4: If a manual device is specified in addition to the motor/snap mechanism, include it as a separate type.
Step 5: Do not infer mechanism type based on function or vendor trend. Only extract if stated.

# H-3
# High Speed Earthing Switch
# Mechanical Endurance Class
# "Specifies the required number of mechanical operations the High-Speed Earthing Switch (HSES) must endure without degradation of function. This applies to high-voltage switchgear rated at nominal system voltages of 66kV and above. While HSES is primarily defined by its high-speed fault-closing capabilities, mechanical endurance under no-load conditions is still a critical performance factor for operational reliability and maintenance planning."
# AI Extraction Logic
Step 1: If the specification clearly states the required number of no-load operations for the HSES (e.g., "1,000 operations"), extract the numeric value verbatim and cite the Source.
Step 2: If endurance is expressed using IEC mechanical endurance categories (e.g., "M0", "M1"), extract it as is.
Step 3: If both are provided, use the format: M1 (2,000 operations).
Step 4: If the mechanical endurance is mentioned conditionally or with exceptions (e.g., "minimum 500 operations unless specified otherwise"), extract the full phrase and flag for manual review.
Step 5: Do not infer endurance values based on fault current rating, spring mechanism type, or high-speed function.
Step 6: If the specification does not mention mechanical endurance for HSES, leave the field blank or flag for manual review.

# H-4
# High Speed Earthing Switch
# Electrical Endurance Class
# "Specifies the electrical endurance class of a high-speed earthing switch (HSES), representing its capability to withstand short-circuit making current as defined in IEC and IEEE standards for high-voltage switchgear (rated at 66kV and above). IEC 62271-102 defines standardized endurance classes (E1 and E2) based on the number and type of making current tests. IEEE standards do not explicitly use class labels but describe equivalent performance requirements (e.g., number of successful short-circuit making operations)."
# AI Extraction Logic
Step 1: If the specification explicitly states "Class E1" or "Class E2," extract the class verbatim and cite the source.
Step 2: If the specification indicates the number of short-circuit making tests (e.g., "2 making tests" or "5 making operations"), extract as: "2 tests" → Class E1, "5 tests" → Class E2. ※ Flag if there's ambiguity in interpretation (e.g., no info on pre-arcing or peak current tests).
Step 3: If IEEE-based specs define making current withstand tests without naming classes, infer the corresponding IEC class only if test quantity and test conditions align with IEC Table 15.
Step 4: If the specification presents both classes (e.g., "E1 / E2 allowed"), extract using format: E1 / E2 or E2 (preferred); E1 allowed.
Step 5: Do not infer endurance class based on manufacturer default, rated voltage, or equipment type.
Step 6: If no class, test quantity, or equivalent making current withstand requirement is mentioned, leave blank or flag for manual review.

# H-5
# High Speed Earthing Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-5

# H-6
# High Speed Earthing Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-6

# H-7
# High Speed Earthing Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-10

# H-8
# High Speed Earthing Switch
# Motor Type
# "Specifies the type of motor used in the operating mechanism of High-Speed Earthing Switches (HSES) applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. The motor type (e.g., AC, DC) affects compatibility with substation auxiliary power systems and the responsiveness of interlock logic. Only motor types explicitly stated, or clearly implied through supply voltage description, shall be extracted."
# AI Extraction Logic
Step 1: Direct Statement Extraction — If the motor type is explicitly stated in connection with HSES (e.g., "DC motor," "AC motor") → Extract that value.
Step 2: Infer from Voltage Supply and Mechanism Context — If the following wording appears in reference to the HSES mechanism, infer motor type accordingly: "DC power supply to the mechanism" → DC motor; "AC power supply to the mechanism" → AC motor; "Electrical mechanism using DC supply" → DC motor; "Power-operated mechanism using AC" → AC motor; "Remote electrical operation with 110VDC" → DC motor.
Step 3: Multiple Options — If both AC and DC motors are allowed, extract as: AC / DC.
Step 4: Preferred Type with Conditional Allowance — If one is preferred but another is conditionally allowed, extract as: DC (preferred); AC allowed upon approval.
Step 5: Exclusions - Do Not Infer Based On: Typical HSES designs, Vendor defaults, Installation configuration.
Step 6: No Clear Indication — If neither motor type nor related power supply is stated → Leave blank or flag for manual review.

# H-9
# High Speed Earthing Switch
# Thermostat Requirement
# Enter the extracted value as Serial Number  E-14

# H-10
# High Speed Earthing Switch
# Humidistat Requirement
# Enter the extracted value as Serial Number E-15

# H-11
# High Speed Earthing Switch
# Earthing Path Type (Direct vs. Enclosure)
# Enter the extracted value as Serial Number G-10

# H-12
# High Speed Earthing Switch
# Viewport Requirement
# Enter the extracted value as Serial Number F-7

# I-1
# Voltage Transformer
# Type
# "Specifies the construction and sensing technology of the voltage transformer (VT) used in high-voltage switchgear systems rated at nominal system voltages of 66kV and above. The VT type affects insulation design, accuracy, response to transients, and digital substation compatibility. Accepted types include Inductive VT, Capacitive VT (CVT), Optical VT, and Low-Power Passive VT (LPVT). If the VT is mounted inside the GIS, flag it separately as GIS-integrated."
# AI Extraction Logic
Step 1: Identify Explicit Type Statements — Extract directly if VT type is clearly stated: "Inductive voltage transformer" → Inductive, "Capacitive voltage transformer" or "CVT" → Capacitive, "Optical voltage transformer" → Optical, "Low power passive voltage transformer" or "LPVT" or "LPIT" → Low Power.
Step 2: Identify Standard or Functional Implications — Infer type based on standard or operating method, only if unambiguous: Mention of IEC 61869-7 → Low Power, IEC 61869-11 or IEC 61850-9-2LE → Optical or Low Power, "Based on Pockels effect" / "Faraday effect" → Optical (flag for manual review).
Step 3: Identify GIS Integration — If VT is specified as "mounted inside GIS," "SF₆ gas insulated," or "in GIS enclosure," → Add flag: GIS-integrated (but not part of main Type value).
Step 4: If Multiple Types Allowed — Use format: Inductive / Capacitive or Low Power (preferred); Inductive allowed upon approval.
Step 5: Do Not Infer Type From: Vendor preferences, Typical GIS configuration, Secondary winding count or accuracy class alone.
Step 6: If unclear or type is missing — Leave blank or flag for manual review.

# I-2
# Voltage Transformer
# Phase Configuration
# "Specifies whether the project requires Voltage Transformers (VTs or PTs) to be single-phase, three-phase, or permits both configurations. This distinction affects enclosure design, wiring, metering accuracy, and redundancy strategy in high-voltage switchgear systems."
# AI Extraction Logic
Step 1: Identify Explicit Phase Configuration Statements — Example: "PTs shall be single-phase" → Extract as: Single-phase, "Three-phase VT shall be provided" → Extract as: Three-phase, "PTs shall be single-phase or three-phase" → Extract as: Single-phase / Three-phase.
Step 2: Identify Preference or Conditional Use (if present) — "Preferably single-phase" → Extract as: Single-phase (preferred), Conditional phrasing → Extract full condition and flag for manual review.
Step 3: Verify Context — Ensure that the statement refers specifically to Voltage Transformers, Potential Transformers, VTs, or PTs. Do not confuse with CTs or general phase references.
Step 4: No Mention — If neither single-phase nor three-phase is specified, leave the field blank or flag for manual review.

# I-3
# Voltage Transformer
# Isolating Device Requirement
# "Specifies whether the specification requires a Voltage Transformer (VT or PT) to be equipped with an isolating device or disconnecting switch to facilitate HV system isolation, dielectric testing, or safe maintenance. Such isolating devices may be either integrated (e.g., integral isolating switch operated from GIS exterior) or separate (e.g., dedicated disconnector) and typically include features like positive position indication and locking mechanism."
# AI Extraction Logic
Step 1: Search for Direct Mentions of Isolating Device or Disconnecting Switch — Identify terms like: "isolating device", "disconnecting switch", "separate disconnector", "means to isolate", "facility for HV system/power cable dielectric test", "positive mechanical/electrical indication of open/close".
Step 2: Confirm That It Applies to Voltage Transformer — Ensure isolating function is explicitly linked to PT/VT. Example: "PTs shall have rugged integral isolating device..." → Extract as Tender Requirement: "Isolating device required", Source: Cite clause and sentence.
Step 3: Determine the Type (Optional) — If described: "integral isolating device (to be hand operated from GIS exterior)" → Integral, "separate disconnect switch" → Separate.
Step 4: If Not Mentioned — Leave blank.
Step 5: Do Not Assume — Do not infer requirement based on typical design or dielectric testing needs unless explicitly described.

# I-4
# Voltage Transformer
# Anti Ferro-Resonance Requirement
# "Specifies whether the voltage transformer (VT or PT) must include design measures to suppress ferroresonance phenomena, which can occur due to non-linear behavior under low-load or no-load conditions in high-voltage systems. Ferroresonance can lead to sustained overvoltages and equipment damage; therefore, some specifications explicitly require anti-ferroresonance features such as suppression resistors, ferroresonance suppressors, or design-based mitigation."
# AI Extraction Logic
Step 1: Identify Direct Requirement — If specification directly requires suppression (e.g., "VT shall be designed to avoid ferro-resonance") → Extract as: Required.
Step 2: Identify Conditional Requirement — If phrased conditionally (e.g., "adequate ferro-resonance suppressor (if required)") → Extract as: Required (conditional), flag for manual review.
Step 3: Supporting Features — If specific features are mentioned, extract with annotation: Required (ferro-resonance suppressor), Required (resistor if needed).
Step 4: If No Mention — Leave blank.
Step 5: Do Not Infer — Do not assume requirement based on equipment type or regional practices unless explicitly stated.

# I-5
# Voltage Transformer
# Manufacturer / Country of Origin
# "Specifies the manufacturer (vendor) and the country of origin of the voltage transformer applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. This information is typically stated to ensure supplier qualification, regulatory compliance, or manufacturing traceability. Only manufacturer names and countries explicitly listed in the technical specification or data schedule shall be extracted."
# AI Extraction Logic
Step 1: Identify sections (Data Sheet, Approved Vendor List, Technical Requirements) where voltage transformer vendors are named.
Step 2: Extract: Manufacturer name (e.g., Siemens, ABB), Country of origin (e.g., Germany, Korea).
Step 3: If multiple manufacturers are listed, extract them all (use "/" or "List" format). If only one is listed, treat it as sole unless a separate exclusivity clause overrides.

# I-6
# Voltage Transformer
# Vendor Exclusivity
# "Specifies whether the use of a single vendor (sole source) is required for the voltage transformer applied in high-voltage switchgear rated at nominal system voltages of 66kV and above. This condition may be set by the end user or EPC contractor to ensure consistency, compatibility, or for political/economic reasons. Only explicit statements requiring vendor exclusivity shall be extracted."
# AI Extraction Logic
Step 1: Scan for phrases indicating exclusivity — In procurement or supply policy.
Step 2: Extract "Yes" — Only if the specification includes wording such as: "Only one manufacturer shall be used", "Single source supply", "Sole vendor is required", "All VTs shall be from the same manufacturer".
Step 3: Do not infer — From the presence of a single vendor name unless exclusivity is clearly stated. If unclear, flag for manual review.

# J-1
# Current Transformer
# Type
# "Specifies the type of current transformer (CT) used in high-voltage switchgear rated at 66kV and above. The CT type determines measurement accuracy, insulation characteristics, and interface compatibility with protection and metering systems. Only CT types explicitly stated or clearly implied through standard references or construction terminology shall be extracted."
# AI Extraction Logic
Step 1: Identify direct statements specifying CT type, construction, or measurement principle.
Step 2: Extract values according to these patterns: "Ring Core Type" / "Toroidal Core" → Ring Core; "Optical CT" / "Fiber Optic Current Sensor (FOCS)" → Optical CT; "Rogowski Coil" → Rogowski Coil; "Low Power CT" or "LPCT" → Low Power CT; "Digital CT" (if clearly linked to optical or LPCT structure) → map accordingly.
Step 3: If multiple types are allowed, use format: e.g., Ring Core / Optical CT; if a preferred type is stated: Ring Core (preferred); Optical CT allowed; if no type is stated or type is ambiguous, leave blank or flag for manual review; do not infer based on switchgear type or common practice without textual support.

# J-2
# Current Transformer
# CT Location Relative to Enclosure
# "Specifies the physical location of Current Transformers (CTs) with respect to the GIS enclosure in high-voltage switchgear systems rated at 66kV and above. This classification affects insulation coordination, maintenance access, and partial discharge management. Only direct statements about CT location shall be extracted."
# AI Extraction Logic
Step 1: Identify any statements specifying whether CTs are located inside or outside the switchgear enclosure.
Step 2: Extract values using these rules: "CTs external to the enclosure" → External to enclosure; "CTs installed inside the enclosure" → Internal (inside enclosure); if both are listed with preference, e.g., "CTs shall preferably be internal" → Internal (preferred); External allowed.
Step 3: Do not extract "Yes/No" values; do not infer based on equipment type (e.g., GIS, DTB, HGIS) or vendor default; if ambiguous or not stated, leave blank or flag for manual review.

# J-3
# Current Transformer
# Insulation Class
# "Specifies the insulation class of the Current Transformer (CT) used in high-voltage switchgear, including GIS, HGIS, and DTB systems. Insulation class (e.g., Class E, Class F) defines the CT’s thermal endurance level and determines material specifications based on the ambient and internal temperature rise. If different insulation classes are specified based on CT location (e.g., inside or outside SF₆ gas zone, or based on GIS/HGIS distinction), these shall be noted accordingly."
# AI Extraction Logic
Step 1: Search for the phrase "CT insulation class" or related wording such as "insulation class of CT," "CT shall be Class E," etc.
Step 2: Extract insulation class (e.g., Class E, Class F) based on explicit mention. If both GIS and HGIS are described, record values separately: e.g., "Class E for GIS, Class F for HGIS". If location-specific rules are mentioned (e.g., "CT secondary is outside SF₆ gas zone"), extract per condition.
Step 3: Do not infer values based on typical practice or manufacturer preference. If insulation class is not stated, leave blank or flag for manual review.

# J-4
# Current Transformer
# Current Transformer Protection Unit
# "Indicates whether the Current Transformer Protection Unit (CTPU) is required for high-voltage switchgear applications. The CTPU provides open-circuit protection for CT secondary circuits. Only requirements explicitly stated in the specification or clearly implied through mandatory inclusion shall be extracted."
# AI Extraction Logic
Step 1: Search for terms such as "CTPU," "CT open circuit protection," or "CT protection unit."
Step 2: Extract based on requirement wording: if stated as mandatory → Required (e.g., "CTPU shall be provided," "CT open circuit protection shall be included"); if stated as optional → Optional (e.g., "CTPU when provided..."); if explicitly stated as not required → Not Required.
Step 3: Do not infer from vendor practices or common usage; if not mentioned or ambiguous, leave blank or flag for manual review.

# J-5
# Current Transformer
# Manufacturer / Country of Origin
# "Specifies the designated manufacturer and/or country of origin for Current Transformers (CTs) used in high-voltage switchgear rated at nominal system voltages of 66kV and above. This information may reflect customer preferences, approved vendor lists, or procurement restrictions aimed at ensuring product quality, local content compliance, or geopolitical risk mitigation."
# AI Extraction Logic
Step 1: Scope Check — Identify whether the specification refers to the manufacturer or country of origin for Current Transformers used in GIS, DTB, HGIS, or SF₆-free switchgear; exclude references to protection relays, low-voltage components, or unrelated instrument transformers.
Step 2: Manufacturer Extraction — Extract the name of the designated CT manufacturer if stated (e.g., "CTs shall be from GE, Trench, or Ritz"; "Only KOTEF or KOC Electric CTs are acceptable").
Step 3: Country of Origin Extraction — Extract country references if present, including direct constraints (e.g., "CTs shall not be of Chinese origin") or preference/mandate (e.g., "CTs shall be manufactured in Spain"); if both manufacturer and country are stated, extract both in format: GE / Germany or Trench / Any country except China.
Step 4: Conditional or Restricted Approval — If the spec mentions conditional approval based on vendor or origin (e.g., "subject to prior approval"), extract and flag for manual review.
Step 5: Ambiguous Cases — Do not infer based on product design, company history, or market norms; if the manufacturer or origin is unclear or not stated, leave the field blank or flag for manual review.

# K-1
# Surge Arrester
# Type
# "Specifies the required type of surge arrester for high-voltage switchgear rated at nominal system voltages of 66kV and above. This includes the arrester's configuration (e.g., free-standing, GIS-housed) and its internal design characteristics (e.g., metal-oxide without gaps). Only explicitly stated types or clearly referenced standards (e.g., IEC 60099-4, IEEE C62.11) shall be considered."
# AI Extraction Logic
Step 1: Scope Check — Determine whether the requirement applies to surge arresters intended for use with high-voltage switchgear (e.g., GIS, HGIS, DTB); ignore arresters specified only for transformers, LV/MV systems, or unrelated equipment.
Step 2: Direct Type Extraction — If the specification clearly states the arrester type, extract it (e.g., "Metal-oxide surge arrester without gaps", "Free-standing outdoor surge arrester", "GIS-housed surge arrester"); normalize variations such as "gap-less" / "gapless" / "without gap".
Step 3: Inference from Standards — If the specification references standards known to define arrester types (e.g., IEC 60099-4 → Metal-oxide surge arrester (gapless) (as per IEC 60099-4); IEEE C62.11 → Metal-oxide surge arrester (gapless) (as per IEEE C62.11)); include the standard in the Source field.
Step 4: No Extraction Conditions — Do not extract if the spec only refers generically to "surge arrester" with no type detail or implies surge protection without clarity on arrester design/configuration.
Step 5: Ambiguous Cases — If the wording is vague or insufficient to determine the arrester type, leave the Tender Requirement field blank and flag for manual review.

# K-2
# Surge Arrester
# Designation
# "Specifies the Line Discharge Class of the Surge Arrester applied to high-voltage switchgear (GIS, DTB, HGIS, or SF₆ Gas-Free systems) rated at 66kV and above. Line Discharge Class defines the energy absorption capability of the arrester and is typically designated by IEC 60099-4 as SL (low), SM (medium), or SH (high). IEEE-based specifications may instead describe discharge current values or energy ratings without using SL/SM/SH terms."
# AI Extraction Logic
Step 1: Identify Standard Context — If the specification references IEC 60099-4, prepare to extract Line Discharge Class as SL / SM / SH; if IEEE C62.11 is referenced, check if energy absorption or discharge current is described.
Step 2: Extract Designation (IEC) — If the specification explicitly states a class (e.g., "The arrester shall be class SH (IEC 60099-4)"), extract: SH and cite IEC reference if available.
Step 3: Extract Synonyms or Descriptive Phrases — e.g., "High energy class surge arrester", "SH type", "Class SH as per IEC" → Extract: SH.
Step 4: Handle IEEE-style Descriptions — If specification gives only discharge current (e.g., 10 kA, 20 kA) or energy handling (e.g., 5 kJ/kV), extract that as a separate parameter, not as SL/SM/SH.
Step 5: Do Not Infer Class — Do not assign SL/SM/SH based on manufacturer names or voltage levels alone; if unclear, leave blank or flag for manual review.

# K-3
# Surge Arrester
# Rated Voltage
# "Specifies the rated voltage of the surge arrester, defined as the maximum RMS power frequency voltage the arrester can withstand for a limited time without failure. In IEC 60099-4, this is denoted as 'Rated Voltage' and typically expressed in kV RMS. In IEEE C62.11, it is also referred to as 'Arrester Rated Voltage' and has the same definition. This value is used to coordinate the surge arrester’s operation with the insulation level and system temporary overvoltage performance."
# AI Extraction Logic
Step 1: Recognize Target Terms — Search for phrases such as "Rated Voltage of Surge Arrester", "Arrester Rated Voltage", "Rated Voltage = …", "Rated voltage shall be …", "The rated voltage of the arrester is …".
Step 2: Extract Value and Unit — Extract the numerical value (e.g., 96kV, 144kV) and the unit (typically kV RMS); ensure it specifically refers to a surge arrester, not to switchgear, transformer, or system voltage.
Step 3: Handle Multiple Arresters — If different voltage levels are provided for different locations or arrester classes (e.g., SH/SM), extract each one with context preserved in the Source field.
Step 4: Do Not Infer — Do not infer the rated voltage from MCOV, nominal system voltage, or insulation level; if the value is not stated explicitly, leave blank or flag for manual review.

# K-4
# Surge Arrester
# Continuous Operating Voltage
# "Specifies the maximum RMS voltage that may be continuously applied across the terminals of the surge arrester without degrading its performance. This value is commonly referred to as the Maximum Continuous Operating Voltage (MCOV) in IEEE C62.11 and as Continuous Operating Voltage (Uc) in IEC 60099-4. The value is essential for ensuring coordination with system voltage levels and arresters' temporary overvoltage withstand capability."
# AI Extraction Logic
Step 1: Recognize Standard Terms — Extract the continuous operating voltage when the specification includes terms such as "Continuous Operating Voltage", "Uc" (IEC 60099-4), "MCOV" (IEEE C62.11), "Maximum Continuous Operating Voltage", "Uc = …" or "MCOV = …".
Step 2: Extract Value and Unit — Extract the numerical value (e.g., 144kV, 10.2kV) and its unit (typically kV RMS); ensure the value corresponds to a surge arrester, not a VT or other component.
Step 3: Multiple Arresters or Ratings — If the specification provides MCOV/Uc for more than one arrester type or system voltage level, extract each separately using a slash (e.g., 10.2kV / 17.3kV); if voltages are labeled by phase (L-L, L-N), preserve the context in Source field.
Step 4: Do Not Infer — Do not infer MCOV/Uc from rated voltage, system voltage, or insulation level; if the value is not clearly stated, leave blank or flag for manual review.

# K-5
# Surge Arrester
# Nominal Discharge Current
# "Specifies the nominal discharge current (In) of the surge arrester, which is the peak value of lightning impulse current (8/20μs waveform) used to classify the arrester’s energy-handling capability. According to IEC 60099-4, this value is typically 5kA or 10kA for distribution and transmission class arresters, and may reach 20kA for high-energy designs. IEEE C62.11 also defines nominal discharge current for station- and intermediate-class arresters, commonly 10kA or 5kA respectively. This rating is essential for insulation coordination and for matching surge arrester performance to system exposure risk."
# AI Extraction Logic
Step 1: Identify Phrases Associated with Nominal Discharge Current — Search for terms including "Nominal discharge current", "In = …", "Discharge current rating", "Lightning discharge current", "Rated impulse current" (only if clearly linked to In).
Step 2: Extract Value and Unit — Extract the numerical value (e.g., 5kA, 10kA, 20kA) and unit (typically kA); ensure the waveform is 8/20μs or that the value clearly refers to nominal discharge current.
Step 3: Ignore Other Currents — Do not confuse with Line discharge class, Switching surge discharge current, or High current impulse withstand.
Step 4: Multiple Arresters — If values are given for different arrester classes (e.g., SH, SM, SL), extract all with context in the Source field.
Step 5: No Inference — Do not infer nominal discharge current from voltage rating or arrester class unless the value is explicitly stated; if unclear, leave blank or flag for manual review.

# K-6
# Surge Arrester
# Isolating Device Requirement
# "Specifies whether the surge arrester requires an isolating device or disconnecting feature to allow safe disconnection during system dielectric tests. This may include plug-in construction, disconnecting link types, or other means that enable mechanical isolation of the arrester from the system. The requirement is typically stated for SF₆ gas-insulated arresters but may also apply to other types depending on installation conditions."
# AI Extraction Logic
Step 1: Identify Key Phrases — Look for mentions of any isolating or disconnecting mechanism related to surge arresters (e.g., "plug-in construction", "disconnecting-link type", "disconnection means", "arrester isolation", "removal for dielectric test", "test disconnection device", "integral isolating switch for arrester").
Step 2: Confirm Context — Ensure the isolating mechanism is associated with the surge arrester, not with CT, VT, CB, or other equipment.
Step 3: Determine Requirement — If the requirement is mandatory or conditional, extract "Required" or "Required (if specified)"; if there’s no reference or only an optional design suggestion, extract "Not Required" or leave blank.
Step 4: Avoid Misinterpretation — Ignore references to line disconnector (not specific to SA), CT/VT test links, or manual earthing switches.

# K-7
# Surge Arrester
# Leakage Current Monitor Requirement
# "Specifies whether the surge arrester shall be equipped with a leakage current monitor (LCM) for condition monitoring purposes. The LCM typically measures the resistive or total leakage current to ground and may be specified with a particular measurement range (e.g., 0-50 mA), sensitivity, or display format (e.g., analog, digital). This requirement is often combined with surge counters or bypass shunts for ground continuity."
# AI Extraction Logic
Step 1: Search for Leakage Current Monitoring Features — Identify terms and phrases such as "leakage current monitor", "leakage current measurement", "LCM shall be provided", "monitoring range: 0-50 mA", "leakage detection device", "current measuring element between arrester and ground".
Step 2: Confirm Association with Surge Arrester — Ensure the monitoring function is tied directly to surge arrester terminals or ground continuity, not CTs, transformers, or other equipment.
Step 3: Determine Requirement Level — If explicitly stated as required: extract "Required"; if stated as optional or for specific site conditions: extract "Required (if specified)"; if no mention is found: extract "Not Required".
Step 4: Disregard Similar Non-Relevant Monitoring — Do not confuse with SCADA-based voltage/current sensors or insulation monitoring devices unrelated to surge arrester operation.

# K-8
# Surge Arrester
# Manufacturer / Country of Origin
# "Specifies the required manufacturer and/or country of origin for Surge Arresters used in high-voltage switchgear applications rated at nominal system voltages of 66kV and above. Such requirements may reflect customer preferences, quality assurance policies, supply chain localization goals, or trade restrictions."
# AI Extraction Logic
Step 1: Scope Check — Identify whether the specification refers to the manufacturer or country of origin for Surge Arresters used in GIS, DTB, HGIS, or SF₆-free switchgear systems; ignore references to line arresters unrelated to switchgear protection.
Step 2: Manufacturer Extraction — Extract named manufacturers if clearly stated (e.g., "Only Siemens or Toshiba arresters shall be used", "Approved vendors include Hitachi, Siemens, and Toshiba"); use extracted format: Siemens / Germany or Hitachi / Multiple countries.
Step 3: Country of Origin Extraction — Extract country-related instructions including restrictions (e.g., "Surge arresters shall not be manufactured in China") or preferences (e.g., "Locally manufactured surge arresters are preferred"); if both are stated, extract in format: Manufacturer / Country or Country only if no brand is specified.
Step 4: Conditional or Approval-Based Mentions — If phrases like "subject to purchaser’s approval" or "alternatives may be accepted with prior agreement" are used, extract and flag for manual review.
Step 5: Ambiguities — Do not infer vendor or origin based on typical supply trends, manufacturer headquarters, or market habits; if unclear, leave blank or flag for review.

# L-1
# Termination
# Insulating Bushing - Type
# "This item identifies the required insulation type of the gas-to-air bushing used in high-voltage switchgear. It refers to the external insulation structure or material of the bushing that connects the gas-insulated compartment to air-insulated systems. Common types include porcelain, composite (silicone rubber), and RIP (resin-impregnated paper). Only extract if the bushing type is explicitly specified."
# AI Extraction Logic
Step 1: Search for keywords such as "Insulating bushing type", "Porcelain bushing", "Composite bushing", "RIP bushing".
Step 2: Identify the stated bushing insulation type: Porcelain, Composite (e.g., silicone rubber), RIP (resin-impregnated paper), or any specified alternatives (e.g., hybrid or customized structures); also detect explicit exclusions (e.g., "Porcelain not acceptable") and bushing location context (e.g., "for transformer-side", "for cable-side").
Step 3: Extract and record as: "Porcelain", "Composite", "RIP", "Not specified", or "Prohibited: [Type]".

# L-2
# Termination
# Insulating Bushing - Creepage
# "This item identifies the required minimum creepage distance of the gas-to-air insulating bushing used in high-voltage switchgear. Creepage distance is the shortest path along the surface of the insulating material between two conductive parts, typically between live conductor and ground. It is often specified in mm or mm/kV and may reflect environmental conditions such as pollution level, altitude, or installation type. Only extract if the creepage value is explicitly defined or calculable from provided pollution severity or insulation class."
# AI Extraction Logic
Step 1: Search for keywords such as "Creepage distance", "Minimum creepage", "Surface distance", "Pollution level requirement", "mm/kV", "mm per kV".
Step 2: Identify the stated creepage value or rule: specific distance (e.g., "minimum 31 mm/kV" or "2750 mm"), pollution class with implication (e.g., "Heavy Pollution, Creepage not less than 31 mm/kV"), or reference to standards like IEC 60815 (if applicable).
Step 3: Extract and record as: "2750 mm", "31 mm/kV", "Based on Heavy Pollution - 31 mm/kV", "Not specified"; if only the pollution severity level is given without distance, you may extract it separately under Pollution Severity category, but do not assign numerical creepage unless it's explicitly linked.

# L-3
# Termination
# Insulating Bushing - Color
# "This item identifies the required external color of the gas-to-air insulating bushing used in high-voltage switchgear. The color is typically specified for standardization, visual inspection, UV resistance, or coordination with other substation equipment. Common options include brown, light gray, and dark gray. Only extract if the color is explicitly stated in the customer specification."
# AI Extraction Logic
Step 1: Search for keywords such as "Bushing color", "Insulator color", "Porcelain color", "Shall be brown", "Shall be light grey", "Color RAL 7035", "Silicone rubber color", "Housing color".
Step 2: Detect the stated requirement including explicit color statements (e.g., "Porcelain bushing shall be brown"), color standards or codes (e.g., "RAL 7035"), and any mention of color exclusion (e.g., "black not acceptable").
Step 3: Extract and record as: "Brown", "Light gray", "Dark gray", "RAL 7035", "Not specified", or "Color not defined - supplier to confirm".

# L-4
# Termination
# Insulating Bushing - Terminal Hole Size
# "This item identifies any customer-specified requirements for the terminal hole of the gas-to-air insulating bushing in high-voltage switchgear. These may include standard flange specifications such as NEMA 2-hole or 4-hole pad, or specific bolt dimensions and drilling patterns. Such specifications are intended to ensure mechanical compatibility with site equipment. Only extract if the requirement is explicitly stated."
# AI Extraction Logic
Step 1: Search for keywords such as "Terminal hole", "Bolt hole", "Pad type", "Drilling pattern", "NEMA 2-hole", "NEMA 4-hole", "IEC flange", "M16 bolt spacing".
Step 2: Detect terminal interface specs such as "NEMA 4-hole pad", "Bushing top terminal shall comply with IEC 60137", "2 x M12 bolt hole", "Pitch circle diameter: 80 mm".
Step 3: Extract and record as: "NEMA 2-hole", "NEMA 4-hole", "IEC 60137 standard", or "Not specified".

# L-5
# Termination
# Cable Termination - Type
# "This item identifies the required type of cable termination for high-voltage switchgear, typically classified by its insulation and sealing structure. Common types include dry type, fluid-filled (oil or compound), and plug-in or separable connector types. The requirement is often based on installation conditions such as indoor/outdoor use, fire protection, or compatibility with specific cable designs. Only extract if the termination type is explicitly stated in the customer specification."
# AI Extraction Logic
Step 1: Search for keywords such as "Cable termination type", "Termination method", "Terminating system", "Dry type", "Fluid filled", "Oil-filled", "Plug-in termination", "Separable connector", "Stress cone", "Heat-shrink termination".
Step 2: Identify the required type such as Dry Type, Fluid-Filled Type (or Oil-Filled), Plug-in Type / Separable Connector, or mixed/installation-specific types (e.g., "Outdoor fluid type only").
Step 3: Extract and record as: "Dry type", "Fluid-filled type", "Plug-in type", "Not specified", or "Dry or fluid type both acceptable".

# L-6
# Termination
# Cable Termination - DC Withstand Test Requirement
# "This item identifies whether the customer requires a DC dielectric (withstand) test to be performed on the cable termination or cable interface of the high-voltage switchgear. This test is typically required for verifying insulation integrity after installation or as part of routine field commissioning. Only extract if the DC test is explicitly mentioned in the specification."
# AI Extraction Logic
Step 1: Search for keywords such as "DC test", "DC withstand test", "Direct current voltage test".
Step 2: Detect specific requirements like "DC withstand test shall be performed after cable termination", "Cable end shall be tested with 80 kV DC for 15 minutes".
Step 3: Extract and record as: "DC test required for cable termination" or "Not specified".

# L-7
# Termination
# Cable Termination - Dummy Plug Requirement
# "This item identifies whether the customer requires unused cable termination ports or bushings on the high-voltage switchgear to be sealed using dummy plugs, blind flanges, or equivalent sealing methods. This is typically requested for systems with future expansion or partially loaded cable connections. Only extract if such a requirement is explicitly stated."
# AI Extraction Logic
Step 1: Search for keywords such as "Dummy plug", "Dummy bushing", "Blind plug", "Blanking flange", "Seal unused cable port", "Spare termination shall be plugged", "Termination point not in use shall be sealed".
Step 2: Detect any mandatory or optional requirement such as "Supplier shall provide dummy plug for unused cable ports", "Blind flange to be installed on spare cable bushing", "Unused termination to be left open - no dummy plug required".
Step 3: Extract and record as: "Dummy plug required", "Blind flange required", or "Not specified".

# M-1
# Installation Materials
# Cable Tray Type Requirement
# "This item checks whether the customer specification includes a requirement or preference for a specific cable tray type to be applied to high-voltage switchgear. The cable tray type may be described as 'Hi-mesh', 'Perforated', 'Ladder type', or other variants. The description may appear under installation materials, cable routing, or wiring sections. Only extract the cable tray type if it is explicitly mentioned as required, preferred, or allowed."
# AI Extraction Logic
Step 1: Search for sentences that include keywords such as "cable tray", "tray type", "Hi-mesh", "perforated", "ladder", "solid bottom", or similar, especially in sections related to installation materials, wiring, or accessories.
Step 2: From the identified sentences, extract the cable tray type and interpret the context: if a specific type is required, look for phrases like "shall be", "must be", "to be used", "required type"; if a type is preferred or optional, look for phrases like "preferred", "may be", "can be", "acceptable types include".
Step 3: Populate the value as follows: if only one type is allowed → "Perforated type"; if multiple types are acceptable → "Hi-mesh or Ladder type"; if no specific type is mentioned → "Not specified".

# M-2
# Installation Materials
# Gas Piping Allowance for Gas Monitoring
# "This item identifies whether the customer specification permits or requires gas piping connections from the high-voltage switchgear (e.g., GIS) to external gas monitoring systems. This is typically relevant for gas-insulated equipment using SF₆ or alternative gases. The allowance may be stated explicitly (e.g., ‘gas piping shall be provided for monitoring’) or described as not allowed (e.g., ‘external piping is not permitted’). The description may appear in sections related to gas monitoring, accessories, or piping interfaces."
# AI Extraction Logic
Step 1: Search for keywords such as "gas piping", "external piping", "monitoring pipe", "gas sampling", "gas monitoring", "connection to gas analyzer", especially within sections related to gas monitoring, accessories, or interface provisions.
Step 2: Determine the customer’s position based on context: if gas piping is explicitly allowed or required, look for phrases like "shall be provided", "must be installed", "connection to gas analyzer shall be available"; if gas piping is not allowed, look for phrases like "external gas piping not allowed", "no external monitoring connection", "piping shall not be installed".
Step 3: Extract the value as one of the following: "Allowed" - if customer permits gas piping for monitoring; "Not allowed" - if external gas piping is explicitly prohibited; "Not specified" - if the specification does not mention this aspect.

# M-3
# Installation Materials
# Platform Type Requirement
# "This item checks whether the customer specification defines a requirement or preference for the platform type used to support high-voltage switchgear. The platform may be described as 'Fixed type', 'Movable type', or other structural variations. This requirement may be stated in sections related to installation, support structure, base frame, or transportability. The purpose is to identify whether a specific platform type is mandated, preferred, or allowed."
# AI Extraction Logic
Step 1: Search for keywords such as "platform type", "fixed structure", "movable structure", "movable base", "transportable skid", "removable frame", "support frame", or similar expressions, especially in installation or structure-related sections.
Step 2: From the identified context, extract the specified platform type and its condition: if only one type is required, look for phrases like "shall be fixed type only", "movable type shall be used", "must be transportable"; if both types are allowed, look for expressions such as "fixed or movable type is acceptable", "either type may be used".
Step 3: Populate the value as one of the following: "Fixed type only", "Movable type only", "Fixed or movable type acceptable", "Not specified".

# N-1
# Painting
# Enclosure Color
# "Specifies the required color for the exterior surface of the high-voltage switchgear enclosure. The color may be defined by a specific code (e.g., RAL 7032, ANSI 70 Gray), customer-defined naming (e.g., 'light gray', 'desert tan'), or referred to project standard specifications. If not specified, the manufacturer’s standard color is typically applied."
# AI Extraction Logic
Step 1: Search for Color Specification — Scan for mentions of enclosure color using keywords such as "color of enclosure", "paint color shall be…", "RAL…", "ANSI color code…", "finish: [color]", "shade: light gray / grey / desert tan / beige / olive green".
Step 2: Extract Value Type — If a standard color code is mentioned (e.g., "RAL 7032"), extract that code as the Tender Requirement; if a color name only is mentioned (e.g., "light gray"), extract the descriptive term; if stated as "manufacturer’s standard" or "not specified," extract as "Manufacturer’s standard color".
Step 3: Validate Context — Ensure the color requirement refers to the enclosure of the HV switchgear, not accessories (e.g., control panel, nameplate, steel structure).

# N-2
# Painting
# Prohibition of Internal Painting of Enclosure
# "Indicates whether the specification prohibits painting of the internal surface of the enclosure or metallic compartment of high-voltage switchgear. This typically applies to gas compartments or metallic inner walls that may interfere with insulation coordination, SF₆ gas quality, or internal cleanliness. Prohibitions may be absolute (e.g., 'shall not be painted') or conditional (e.g., 'no painting inside gas compartments')."
# AI Extraction Logic
Step 1: Search for Key Phrases — Look for phrases such as "internal surface shall not be painted", "no painting on the inside of the enclosure", "painting inside gas compartments is not allowed", "do not paint internal metallic surfaces", "prohibit internal painting", "painting to be limited to external surfaces only".
Step 2: Classify Requirement — If internal painting is clearly prohibited, extract: Not allowed; if internal painting is not mentioned or allowed, extract: Allowed or Not specified.
Step 3: Confirm Context — Ensure the sentence refers specifically to internal surfaces of enclosures, compartments, or metallic internal structures of high-voltage switchgear, not external paint restrictions or unrelated parts.

# N-3
# Painting
# Color of Partition
# "Check whether the customer specifies a required paint color for partition components used in high-voltage switchgear, such as gas-tight support insulators or gas barrier insulators. These components are part of internal compartments in GIS or hybrid switchgear and are often referred to as gas barrier insulators, partition insulators, or sectionalizing bushings. Color requirements may be defined explicitly (e.g., RAL code, 'light grey') or suggested as preferences (e.g., 'neutral tone')."
# AI Extraction Logic
Step 1: Identify statements related to partitioning components used within high-voltage switchgear, especially gas barrier insulator, gas-tight support insulator, sectionalizing or partition bushing, partition used inside GIS or HGIS.
Step 2: Check whether the color of these components is specified or preferred, such as "shall be painted light grey", "color shall follow RAL 7035", "neutral color preferred for partition bushings".
Step 3: Extract the stated color (e.g., light grey, RAL 7035); if only a preference is mentioned, extract as a soft requirement; do not extract if the component is unrelated to high-voltage switchgear.

# N-4
# Painting
# Mechanism Color
# "Specifies the required color for the exterior surface of the high-voltage circuit breaker mechanism housing (e.g., operating mechanism box). The color may be defined by a standard code (e.g., RAL, ANSI), a descriptive name (e.g., 'dark gray', 'silver gray'), or a reference to the manufacturer’s or customer’s standard."
# AI Extraction Logic
Step 1: Identify Relevant Text — Search for descriptions referring to: "mechanism box color", "operating mechanism enclosure finish/color", "paint color for mechanism", "mechanism housing shall be painted…", "CB mechanism color".
Step 2: Extract Color Value — If a standard code is provided (e.g., "RAL 7035"), extract that code. If a descriptive name is provided (e.g., "silver gray"), extract the name. If stated as "manufacturer’s standard" or similar, extract as "Manufacturer’s standard color."
Step 3: Validate Scope — Ensure that the color requirement clearly applies to the mechanism housing, not to the main switchgear enclosure, panelboard, control cabinet, or support structure.

# N-5
# Painting
# External Color of Local Control Cubicle
# "Specifies the required external color of the Local Control Cubicle (LCC) used with high-voltage switchgear (66kV and above), including any RAL code, ANSI designation, or general color description such as light grey or beige. Only requirements clearly associated with LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
Step 1: Identify relevant equipment — Recognize any of the following terms as referring to the same equipment: "Local Control Cubicle (LCC)", "Local Control Cabinet", "Local Control Panel", "High-voltage Control Cabinet".
Step 2: Locate external color requirements — Extract values only if the specification refers to the external (exterior) color of the LCC. Examples to extract: "External surface shall be RAL 7035.", "Outer color of control cabinet: light grey.".
Step 3: Determine Tender Requirement — Record exact color name or code (e.g., RAL 7035, Light Grey). If multiple options are allowed: Light Grey / RAL 7035. If preference is stated: RAL 7035 (preferred); Light Grey allowed.
Step 4: Exclusions — Do not extract if the color refers to the interior of the cabinet (handled in a separate item). Do not extract color codes referring to unrelated enclosures or general site equipment. If the color designation is ambiguous, flag for manual review.

# N-6
# Painting
# Internal Color of Local Control Cubicle
# "Specifies the required internal color or painting status of the Local Control Cubicle (LCC) used with high-voltage switchgear (66kV and above). This may include references to color codes (e.g., RAL), surface finish (e.g., white, unpainted), or explicit prohibition of internal painting. Only requirements explicitly associated with the inside of LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
Step 1: Identify relevant equipment — Recognize all phrases referring to the Local Control Cubicle, including the following synonyms: Local Control Cabinet, Local Control Panel, Control Cabinet / Panel associated with high-voltage switchgear.
Step 2: Locate internal color requirements — Extract values only if the specification refers to the internal (interior) color of the LCC. Examples to extract: "Interior of LCC shall be painted white.", "Inside of local control cabinet must be light grey (RAL 7035)."
Step 3: Determine Tender Requirement — Record the exact color value or code (e.g., White, RAL 7035). If multiple color options are allowed, use format: White / RAL 7035. If preference is indicated, use format: RAL 7035 (preferred); White allowed.
Step 4: Exclusions — Do not extract external color requirements (handled in separate item). Do not extract interior color of unrelated panels or general enclosures. If unclear whether color refers to interior or exterior, flag for manual review.

# N-7
# Painting
# Duplex System Requirement
# "Indicates whether a Duplex System (combination of hot-dip galvanizing and painting) is required for external metal structures or enclosures of high-voltage switchgear based on environmental conditions, especially proximity to coastlines. Duplex System requirements typically apply to steel support structures, control cabinets, or enclosures exposed to outdoor or corrosive environments. Specifications may require Duplex treatment unconditionally, or conditionally based on distance from coastline (e.g., within 25 km or between 25-50 km)."
# AI Extraction Logic
Step 1: Identify Relevant Phrases — Look for phrases such as: "Duplex System shall be applied", "hot-dip galvanizing plus painting", "zinc coating + paint", "within 25 km of the coastline", "coastal area requires duplex", "0.13 mm zinc coating required for areas 25-50 km from coast".
Step 2: Determine Requirement Category — If Duplex System is required for all installations: → Extract "Required"; If required only within 25 km of coast: → Extract "Required within 25 km of coastline"; If 25-50 km range allows zinc-only coating with 0.13 mm min: → Extract "Optional if not less than 0.13 mm zinc"; If hot-dip galvanizing alone is acceptable in all areas: → Extract "Not required".
Step 3: Context Validation — Ensure that the specification refers to metallic structural parts of high-voltage switchgear, such as: GIS/DTB/HGIS enclosures, Support frames or baseplates, Local control cubicles. Ignore statements related to unrelated components like transmission towers or external fencing.

# N-8
# Painting
# Paint Thickness of Enclosure
# "Check whether the customer specifies the required total paint thickness for the main enclosure of high-voltage switchgear. The enclosure refers to the external housing of GIS, DTB, or HGIS. The paint system may be described as a total dry film thickness (DFT) or as individual layers (e.g., primer + top coat). The total thickness is often specified in micrometers (μm) and may vary depending on installation area (e.g., outdoor, coastal)."
# AI Extraction Logic
Step 1: Identify references to the enclosure of high-voltage switchgear — Including: main enclosure, GIS enclosure, switchgear housing, external tank.
Step 2: Detect any reference to paint thickness (DFT or layer-wise) — Such as: "total painting thickness shall be 100 μm", "primer 40 μm + top coat 60 μm", "minimum 120 microns DFT for outdoor enclosures".
Step 3: Extract the total paint thickness as a single numerical value — If layer-wise values are given, calculate the total. If ranges are provided (e.g., 80-120 μm), extract the range as-is. Record unit as μm.

# N-9
# Painting
# Paint Thickness of Mechanism Box
# "Check whether the customer specifies the required paint thickness or surface treatment for the mechanism box of high-voltage switchgear. The mechanism box houses operating components such as spring or hydraulic mechanisms and is typically installed outside the main GIS/DTB enclosure. The specification may define a total dry film thickness (DFT), a layer-wise coating system, or an alternative surface treatment (e.g., zinc plating, galvanization) instead of painting."
# AI Extraction Logic
Step 1: Identify references to the mechanism box or mechanism housing of high-voltage switchgear — Including: mechanism box, operating mechanism housing, control box (if contextually linked to mechanical parts).
Step 2: Detect any paint thickness or surface treatment specification — Such as: "epoxy painting 80 μm", "powder coating with minimum 70 μm", "zinc plated, no painting required", "DFT of 100 microns for mechanism housing".
Step 3: Extract the stated total thickness in μm or identify the specified surface treatment if painting is not required — If both are provided (e.g., plating + painting), extract both in order.

# N-10
# Painting
# Paint Thickness of LCC
# "Check whether the customer specifies the required paint thickness for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. LCC refers to the external panel or cabinet that contains protection and control devices. The paint system may be defined by a total dry film thickness (DFT), a layer-wise structure, or a specific coating method (e.g., powder coating, epoxy paint). In some cases, the specification may refer to manufacturer’s standard or to match the main switchgear enclosure."
# AI Extraction Logic
Step 1: Identify references to LCC (Local Control Cubicle), Local Control Cabinet, or Local Control Panel — Specifically related to high-voltage switchgear.
Step 2: Detect any associated paint thickness requirement or treatment method — Such as: "LCC to be painted with minimum 60 μm DFT", "Powder coating 80 μm applied to control cabinet", "Same painting system as GIS enclosure", "Standard factory coating applies".
Step 3: Extract the total paint thickness in μm if specified — If the LCC is to follow another item (e.g., "same as enclosure"), extract that reference. If only a treatment method is given (e.g., powder coated), extract the method and associated thickness if available.

# N-11
# Painting
# Surface Treatment and Thickness of Support Structure
# "Check whether the customer specifies surface treatment requirements and coating thickness for the support structure of high-voltage switchgear, such as the base, baseplate, or supporting frame. These components are typically installed at the foundation level and may be subject to corrosion. The required treatment may include hot-dip galvanizing, painting, or powder coating. Thickness may be specified in μm (for paint), g/m² (for zinc coating), or as a multi-layer system."
# AI Extraction Logic
Step 1: Identify references to support structures — Such as: base, baseplate, support frame, supporting structure, mounting structure; specifically linked to high-voltage switchgear (e.g., GIS, DTB, HGIS).
Step 2: Detect surface treatment method and/or coating requirements — Such as: "hot-dip galvanized 610 g/m²", "zinc coating min. 70 μm", "painted support frame with 100 μm DFT", "powder coating 80 microns".
Step 3: Extract — Surface treatment method (e.g., hot-dip galvanizing, painting); Associated thickness (e.g., 610 g/m², 80 μm); If multiple methods are combined (e.g., galvanized + painted), extract both sequentially.

# O-1
# Local Control Cubicle
# Type
# "Check whether the customer specifies the installation type of the Local Control Cubicle (LCC) for high-voltage switchgear. The LCC may be freestanding (separate from the GIS) or integrated (attached or built into the GIS structure). The specification may state one option explicitly or allow both. This requirement affects panel design, wiring layout, and enclosure mounting."
# AI Extraction Logic
Step 1: Identify references to the control cabinet, Local Control Cubicle (LCC), or Local Control Panel — In the context of GIS or high-voltage switchgear.
Step 2: Detect installation type using expressions — Such as: "shall be freestanding", "integrated with GIS", "freestanding or integrated", "separate panel" or "attached to GIS".
Step 3: Extract the specified type — If only "freestanding" is mentioned → Extract: Freestanding; If only "integrated" is mentioned → Extract: Integrated; If both are allowed → Extract: Freestanding or Integrated.

# O-2
# Local Control Cubicle
# Protection Degree
# "Check whether the customer specifies the required protection degree (e.g., IP rating) for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. The protection degree may be defined based on IEC 60529 (e.g., IP41, IP54) and may vary depending on the switchgear type or installation location, such as indoor GIS or outdoor HGIS. The specification may assign fixed ratings or define minimum values per installation condition."
# AI Extraction Logic
Step 1: Identify references to Local Control Cubicle (LCC), control cabinet, or control panel — In relation to high-voltage switchgear.
Step 2: Detect protection degree expressions — Such as: "protection class IP54", "degree of protection shall be minimum IP41", "LCC shall comply with IEC 60529, IP54", "IP41 for GIS, IP54 for HGIS".
Step 3: Extract each protection degree value and its associated condition (if specified) — If one rating is given for all conditions: extract that value (e.g., IP54); If different ratings are given for GIS (indoor) vs HGIS (outdoor): extract both with their associated installation type.

# O-3
# Local Control Cubicle
# Inclusion of BCU / IED in LCC
# "Check whether the customer requires the Bay Control Unit (BCU) or Intelligent Electronic Device (IED) to be installed inside the Local Control Cubicle (LCC) of high-voltage switchgear. The requirement may be stated directly (e.g., BCU shall be mounted in LCC) or indirectly (e.g., LCC to include IEC 61850-compliant IEDs). Inclusion of BCU or IED affects wiring, communication interface, and functional integration of the switchgear control system."
# AI Extraction Logic
Step 1: Identify references to BCU, IED, or bay controller — In relation to: Local Control Cubicle (LCC), control cabinet or panel associated with GIS/HGIS.
Step 2: Detect expressions indicating inclusion or installation — Such as: "shall be provided in the LCC", "LCC shall include BCU", "Bay controller shall be housed inside control cabinet", "IED to be installed within the panel".
Step 3: Extract inclusion status — If explicitly stated → Extract: Included; If explicitly excluded (e.g., "by others" or "to be installed elsewhere") → Extract: Not included; If ambiguous but implied (e.g., "BCU to be connected to LCC terminal blocks") → Extract: Possibly included.

# O-4
# Local Control Cubicle
# Voltage Detector Requirement
# "Check whether the customer requires a voltage detector (or voltage presence indicator) to be installed in the high-voltage switchgear. The voltage detector is used to indicate the presence or absence of voltage in busbars or incoming circuits and is typically installed for safety verification prior to maintenance or operation. The requirement may refer to visible indicators (e.g., neon lamp, LED), built-in detection systems, or voltage sensing terminals."
# AI Extraction Logic
Step 1: Identify references to voltage detector, voltage presence indicator, voltage sensing device — Or similar terms in the context of high-voltage switchgear.
Step 2: Detect expressions indicating installation or requirement — Such as: "shall be provided", "shall be included", "voltage indicator shall be installed on the panel", "voltage detection system required for bus side".
Step 3: Extract status — If explicitly required → Extract: Required; If stated as optional or customer preference → Extract: Optional; If explicitly excluded → Extract: Not required.

# O-5
# Local Control Cubicle
# Manufacturer / Country of Origin
# "Check whether the customer specifies the required manufacturer or country of origin for the Local Control Cubicle (LCC) used in high-voltage switchgear. This may include explicit naming of a vendor, restriction to domestic or international suppliers, or requirements for factory origin or sourcing. The requirement may also indicate whether the LCC shall be supplied by the switchgear manufacturer or by a designated third-party vendor."
# AI Extraction Logic
Step 1: Identify references to Local Control Cubicle (LCC), control cabinet, or control panel — Associated with high-voltage switchgear.
Step 2: Detect statements specifying manufacturer or origin — Such as: "LCC shall be supplied by [vendor name]", "Manufacturer of LCC shall be approved by customer", "Country of origin shall be [country]", "Only domestically manufactured control cabinets are acceptable".
Step 3: Extract the specified — Manufacturer name (if mentioned); Country of origin (if mentioned); Nationality requirement (e.g., local vendor only); If the LCC is to be supplied by the switchgear manufacturer without separate sourcing condition, extract: Supplied with switchgear.

# P-1
# Control Cable
# Insulation Type
# "Check whether the customer specifies the required insulation material for control or power cables used in high-voltage switchgear. Typical insulation materials include PVC (polyvinyl chloride), XLPE (cross-linked polyethylene), and EPR (ethylene propylene rubber). These may be referenced directly or via standard cable codes (e.g., CVV, CCV). The insulation type affects thermal, dielectric, and fire performance."
# AI Extraction Logic
Step 1: Identify references to control or power cables — Associated with high-voltage switchgear.
Step 2: Detect insulation types or cable codes — Such as: "PVC insulation", "XLPE insulation", "EPR insulation", "CVV / CCV / CVVS cable".
Step 3: Extract all explicitly mentioned insulation materials — If multiple options are listed, extract as a list; If codes are used, map to the appropriate insulation type.

# P-2
# Control Cable
# Steel Armored Shield Requirement
# "Check whether the customer requires control cables used in high-voltage switchgear to include a steel armored shield. This may be specified to enhance mechanical protection, improve EMI shielding, or comply with installation conditions such as underground routing or outdoor exposure. The requirement may apply to all control cables, or only to specific portions (e.g., external, underground, or field-installed cables)."
# AI Extraction Logic
Step 1: Identify references to control cable — In the context of high-voltage switchgear systems.
Step 2: Detect keywords or phrases indicating steel armor requirements — Such as: "steel armored", "steel wire armored (SWA)", "armored control cable", "cable with metallic sheath", "to be protected against external mechanical impact".
Step 3: Extract result — If steel armor is required → Extract: Required; If conditionally required (e.g., only for underground runs) → Extract: Conditionally required; If explicitly not required or flexible cables specified → Extract: Not required.

# P-3
# Control Cable
# Jacket Material
# "Check whether the customer specifies a required jacket (outer sheath) material for control or power cables used in high-voltage switchgear systems. The jacket provides mechanical protection, environmental resistance, and may contribute to flame retardancy or chemical resistance. Common materials include PVC, PE, neoprene, and hypalon. The specification may refer directly to the material name or indirectly through standard cable types or application descriptions."
# AI Extraction Logic
Step 1: Identify references to control or power cables — Associated with high-voltage switchgear.
Step 2: Detect expressions related to outer sheath or jacket materials — Such as: "neoprene jacket", "outer sheath made of PVC", "hypalon sheath", "thermoplastic jacket", "cables shall be provided with flame-retardant sheath".
Step 3: Extract — Named jacket materials (e.g., PVC, neoprene, hypalon, LSZH); If multiple options are allowed, extract all; If jacket material is not defined, mark as "Not specified".

# P-4
# Control Cable
# Halogen-Free Requirement
# "Check whether the customer requires control or power cables used in high-voltage switchgear to be halogen-free (also referred to as LSZH: Low Smoke Zero Halogen). This requirement is often applied to reduce toxic gas emission in case of fire and to comply with environmental or safety standards. It may apply to all cable types or only to specific installations (e.g., indoor, tunnel, control room)."
# AI Extraction Logic
Step 1: Identify references to control cables or power cables — Used with high-voltage switchgear.
Step 2: Detect expressions indicating halogen-free or LSZH requirement — Such as: "halogen-free", "LSZH" or "low smoke zero halogen", "non-toxic under fire", "zero halogen material", "cables shall emit low smoke and no halogen".
Step 3: Extract result — If clearly required → Extract: Required; If required only under certain conditions (e.g., indoor cables only) → Extract: Conditionally required; If halogen-free is not mentioned or PVC is permitted → Extract: Not required.

# P-5
# Control Cable
# Anti-Termite Requirement
# "Check whether the customer requires control or power cables used in high-voltage switchgear systems to be resistant to termites or other insects. Anti-termite protection is typically applied through the use of treated PVC or special sheath materials and is often required for underground or outdoor installations in regions with termite risk."
# AI Extraction Logic
Step 1: Identify references to control or power cables — Associated with high-voltage switchgear, especially in underground or external routing contexts.
Step 2: Detect expressions — Such as: "anti-termite", "termite-proof cable", "outer sheath to be termite resistant", "PVC treated against insects".
Step 3: Extract result — If clearly required → Extract: Required; If conditionally required (e.g., underground only) → Extract: Conditionally required; If not mentioned or explicitly not needed → Extract: Not required.

# Q-1
# Monitoring System
# Manufacturer-Specified Monitoring System
# "This item identifies whether the customer specification requires or permits a specific manufacturer’s monitoring system to be applied to the high-voltage switchgear. It includes cases where the name of a vendor, brand, or model of the monitoring system is explicitly stated or referenced. The requirement may appear in the main technical specification, appendix, or in approved vendor list (AVL)-related documents. Only extract the manufacturer name and system type if clearly mentioned."
# AI Extraction Logic
Step 1: Search for sentences containing keywords — Such as: "Monitoring system by [manufacturer name]", "[Brand name] monitoring", "only [vendor] system allowed", "shall be [vendor]", "approved vendors", "qualified suppliers", "preferred monitoring system". Check both the technical specification body and vendor qualification sections.
Step 2: Extract the following — Manufacturer or brand name (e.g., Qualitrol, Omicron, GE, ABB, Siemens, etc.); Monitoring system type if specified (e.g., PDMS, gas monitoring, CB monitoring).
Step 3: Record the extracted value as — If clearly required → "Required: [Manufacturer Name]"; If only permitted or preferred → "Preferred: [Manufacturer Name]"; If not mentioned, leave the field blank.

# Q-2
# Monitoring System
# Partial Discharge (PD) Sensor Provision
# "This item identifies whether the customer specification requires Partial Discharge (PD) sensors to be installed on the high-voltage switchgear or prepared for future connection. The requirement may refer to the presence of UHF sensors, couplers, or test ports for PD monitoring. The condition may be stated as mandatory, optional, or not required, and may also specify the installation scope (e.g., per bay, per compartment). Only extract the requirement if it is explicitly stated."
# AI Extraction Logic
Step 1: Search for keywords — Such as: "Partial discharge sensor", "PD sensor", "UHF sensor", "UHF coupler", "shall be provided", "to be installed", "equipped with PD", "PD test port", "PD monitoring interface", "sensor provision", "sensor integration", "PD detection capability". Focus on sections related to monitoring, accessories, or insulation diagnostics.
Step 2: Determine the customer’s requirement type — Required: "shall be provided", "must be equipped", "shall include"; Optional / prepared for future use: "may be provided", "interface only", "PD sensor mounting provision"; Not required / explicitly excluded: "not required", "shall not be installed".
Step 3: Record the extracted value as — "Required"; "Provision only" (for test port or future-ready interface); If not mentioned, leave the field blank.

# Q-3
# Monitoring System
# PD Sensor Quantity (e.g., Every Compartment)
# "This item identifies whether the customer specification defines the required quantity or installation scope of PD sensors for high-voltage switchgear. This includes statements requiring sensors to be installed in every gas compartment, in each bay, or in specific parts such as cable termination sections. Only extract the quantity-related requirement if it is explicitly stated, particularly phrases such as 'every compartment', 'each section', or fixed numbers per bay or unit."
# AI Extraction Logic
Step 1: Search for keywords and phrases — Such as: "every compartment", "each gas section", "each bay", "each panel", "PD sensor per [unit]", "PD sensors shall be installed in all compartments", "minimum [X] sensors per [bay/cubicle/etc.]". Focus on sections related to monitoring system configuration, sensor layout, or installation instructions.
Step 2: Extract the installation scope or quantity — If stated as "every compartment", record as: "Every compartment"; If stated as "each bay", or specific number per bay: e.g., "1 per bay"; If no requirement at all: "Not specified".
Step 3: Record the value as — "Every compartment"; "Each bay"; "[X] sensors per bay" (if numerical value is given); If not mentioned, leave the field blank.

# Q-4
# Monitoring System
# Online PD Monitoring System
# "This item identifies whether the customer specification requires an online partial discharge (PD) monitoring system for the high-voltage switchgear. Extract only when the monitoring system explicitly refers to PD (partial discharge) or equivalent terms (e.g., PDMS, UHF-based PD monitoring). Do not extract values for other types of monitoring systems such as GIS monitoring systems (GMS), SF₆ gas monitoring, temperature monitoring, or mechanical condition monitoring, since these are outside the scope of PD monitoring."
# AI Extraction Logic
Step 1: Search for PD-specific terms: “Online PD monitoring system,” “continuous PD monitoring,” “real-time PDMS,” “UHF online monitoring,” “partial discharge detection,” “permanent PD monitoring system,” etc.
Step 2: Confirm that the monitoring system explicitly refers to partial discharge (PD). If the text refers to GIS monitoring system (GMS), SF₆ gas monitoring, temperature monitoring, or other non-PD systems, do not extract.
Step 3: If the specification requires PD monitoring (e.g., “Online PD monitoring shall be provided”), extract as Required.
Step 4: If described as optional (e.g., “may be installed,” “can be provided upon request”), extract as Optional.
Step 5: If the specification states it is required only if specified in another document (e.g., SOW, TS), record as Conditionally Required (if specified in SOW/TS) and cite the source.
Step 6: If the text refers only to portable or periodic PD measurements, do not extract.
Step 7: If none of the above conditions are met, leave the field blank.

# Q-5
# Monitoring System
# Portable PD Monitoring Device
# "This item identifies whether the customer specification requires the provision of a portable partial discharge (PD) monitoring device, or the installation of test interfaces that allow temporary connection of such a device. This applies to non-permanent PD diagnostic tools typically used during maintenance, testing, or periodic inspection."
# AI Extraction Logic
Step 1: Search the specification for keywords such as “Portable PD monitoring device,” “handheld PD detector,” “temporary PD measurement,” “Output connection for portable PD,” “interface for portable PD system,” “Test port for external PD monitor,” “provision for temporary PD connection,” or “PD monitoring demonstration by portable system.”
Step 2: If the specification clearly requires the supply of a portable PD monitoring device, such as “Portable PD device shall be supplied with the GIS,” extract the value as “Device supply required.”
Step 3: If the specification requires only the provision of an interface or output terminal for temporary connection of a portable PD system, extract the value as “Test port provision only.”
Step 4: If the specification requires PD detection to be demonstrated using a portable system in the absence of an online PD system, extract the value as “Demonstration required.”
Step 5: Do not extract any value if the specification refers only to online PD monitoring systems and does not mention portable equipment, temporary connections, or related interfaces.
Step 6: If none of the valid conditions are met and there is no reference to portable PD equipment or temporary interface provisions, leave the field blank.

# Q-6
# Monitoring System
# Circuit Breaker Condition Monitoring System
# "This item identifies whether the customer specification requires a circuit breaker (CB) condition monitoring system for high-voltage switchgear. The system continuously monitors breaker performance indicators such as operation count, operating time, contact wear, travel curve profile, stored energy mechanism status, spring charging motor behavior, or insulation condition. It is typically used for predictive maintenance and lifetime assessment. Valid requirements may be described using terms like CB health monitoring, digital diagnostics, or real-time performance analysis. Only extract the requirement if such monitoring is explicitly stated or implied through clear reference to dedicated condition-monitoring functions."
# AI Extraction Logic
Step 1: Search the specification for keywords such as “Circuit breaker condition monitoring,” “CB health monitoring,” “mechanical wear monitoring,” “lifetime monitoring system,” “breaker diagnostics,” “CB monitoring unit,” “motor operation monitoring,” “travel time analysis,” “contact wear detection,” “digital breaker diagnostics,” or “spring motor behavior.”
Step 2: If the specification clearly states that a CB condition monitoring system is required, using phrases such as “CB condition monitoring system shall be provided,” “lifetime monitoring system is mandatory,” or “real-time breaker diagnostics required,” extract the value as “Required.”
Step 3: If the specification requires only the provision for monitoring, such as “Provision for CB diagnostics shall be made” or “Interface for condition monitoring to be included,” extract the value as “Provision only.”
Step 4: Do not extract any value if the specification refers only to basic breaker operations such as opening or closing, without mentioning diagnostics, monitoring, or performance tracking.
Step 5: Do not extract any value if the specification refers only to protection relays, trip signals, or event logging unless they are explicitly related to breaker condition monitoring.
Step 6: If no applicable monitoring requirement is found or if the references are too vague to interpret as dedicated condition-monitoring functions, leave the field blank.

# Q-7
# Monitoring System
# Arc Detection System
# "This item identifies whether the customer specification requires an arc detection system to be installed in the high-voltage switchgear. This includes optical arc detection using light sensors or other technologies capable of detecting internal arc faults and initiating fast breaker operation or protection signaling. Valid implementations may include fiber-optic sensors, light-based detectors, or any system explicitly linked to fault trip logic."
# AI Extraction Logic
Step 1: Search the specification for keywords indicating arc detection systems, such as “Arc detection system,” “internal arc detection,” “optical arc sensor,” “light-based arc detection,” “ultrafast arc protection,” “fiber optic arc sensor,” “trip upon arc detection,” “light-triggered trip,” or “arc flash sensor.”
Step 2: If the specification clearly requires installation of an arc detection system, using statements such as “Arc detection system shall be provided,” “optical arc sensor is mandatory,” or “fiber optic arc system to be installed,” extract the value as “Required.”
Step 3: If the specification calls only for provision or future readiness for an arc detection system, such as “Provision for future arc detection system shall be included,” extract the value as “Provision only.”
Step 4: Do not extract any value if the specification refers only to IAC classification, internal arc type testing, or mechanical protection measures such as arc-proof enclosures and pressure relief flaps without mentioning any sensor or detection system.
Step 5: Do not extract any value if the term “arc protection” is used without connection to detection, sensors, or protection trip logic.
Step 6: If there is no mention of arc detection systems or if all references are excluded based on the conditions above, leave the field blank.

# Q-8
# Monitoring System
# Camera-Based Position Verification System for DS/ES
# "This item identifies whether the customer specification requires a camera-based system or any other advanced visual verification method to monitor the open/close position of disconnector (DS) or earthing switch (ES). These systems are intended to provide visual confirmation of the actual contact position, especially for critical interlocking or operational safety purposes. Standard auxiliary switches or mechanical indicator windows shall not be considered valid monitoring systems for the purpose of this item."
# AI Extraction Logic
Step 1: Search for keywords — Such as: "Disconnector position indication", "earthing switch position monitoring", "DS/ES contact position feedback", "open/close indication", "auxiliary switch for DS/ES", "proximity sensor", "optical position sensor", "camera monitoring for DS", "position signal to SCADA" (only if monitoring method is stated).
Step 2: Determine if the specification explicitly requires monitoring — e.g., "Auxiliary switches shall be provided for DS and ES", "Camera shall be used to verify DS position", "Position sensor signal shall be sent to SCADA"; Mentions only mechanical indication (e.g., "visible window"), which is not electronic monitoring.
Step 3: Record the extracted value as — "Required (auxiliary switch)"; "Required (sensor-based)"; "Required (camera-based)"; "Provision only"; If not mentioned, leave the field blank.

# Q-9
# Monitoring System
# Surge Arrester Leakage Current Monitoring System
# "This item identifies whether the customer specification requires a monitoring system for the leakage current of surge arresters (also referred to as lightning arresters) in high-voltage switchgear. Acceptable systems may include analog gauges, local indicators, or digital sensors. Only extract a requirement if it is explicitly stated."
# AI Extraction Logic
Step 1: Search the specification for keywords related to leakage current monitoring of surge arresters, including phrases such as “Leakage current monitoring for surge arrester,” “LA leakage current sensor,” “arrester current supervision,” “Lightning arrester current detection,” “online leakage current monitor,” “resistive current component monitoring,” “continuous LA monitoring,” “analog leakage current meter,” or “scale 0–50 mA.”
Step 2: If the specification explicitly requires continuous or real-time monitoring with features such as SCADA integration or trending analysis, extract the value as “Required (continuous monitoring).”
Step 3: If the specification requires only a manual or periodic monitoring method, such as an analog meter or local indicator without SCADA or trending features, extract the value as “Required (periodic/manual).”
Step 4: If the specification states that provisions for future monitoring must be included, without requiring immediate installation of monitoring devices, extract the value as “Provision only.”
Step 5: Do not extract the value if the only information provided is the existence of an analog scale or range, such as “0–50 mA,” unless it is clearly associated with remote or real-time monitoring.
Step 6: If none of the above conditions are met or if the requirement is not clearly stated, leave the field blank.

# Q-10
# Monitoring System
# Online Gas Monitoring System
# "This item identifies whether the customer specification requires an online gas monitoring system for high-voltage switchgear. Online systems continuously monitor and transmit gas-related parameters such as SF₆ pressure, temperature, density, humidity, or decomposition byproducts in real-time using digital communication protocols. They are distinct from local pressure gauges or two-stage mechanical relays used for local gas-density alarms. Do not consider mechanical devices with continuous sensing function (e.g., gas-density monitoring relays) as online monitoring systems unless explicitly stated as real-time or remotely accessible systems."
# AI Extraction Logic
Step 1: Search the specification for keywords related to online gas monitoring systems, including terms such as “Online gas monitoring system,” “continuous gas monitoring,” “real-time SF₆ monitoring,” “gas analyzer,” “gas monitoring with communication,” “SCADA-integrated gas monitoring,” “humidity monitoring,” “decomposition gas detection,” “dew point sensor,” “gas density monitor with remote communication,” or “24/7 gas supervision.”
Step 2: If the specification clearly requires a real-time or remotely accessible gas monitoring system, classify the requirement as “Required.”
Step 3: If the specification indicates only a provision for future implementation of such a system, classify the requirement as “Provision only.”
Step 4: If the specification refers only to manual pressure gauges, mechanical gas-density relays, or alarm contacts without remote access or real-time communication, classify the requirement as “Not specified.”
Step 5: Disregard references to mechanical two-stage relays, local annunciator wiring, or protection relays, even if described as continuous monitoring.
Step 6: Disregard phrases like “continuous monitoring” if they refer only to local-only functions and do not involve remote transmission or SCADA integration.
Step 7: If the specification explicitly states that the online monitoring system must measure specific parameters—such as pressure, temperature, density, humidity, dew point, or decomposition gases—record each parameter in the order mentioned, separated by commas.
Step 8: Combine the requirement classification from Steps 2–4 with the extracted parameter list from Step 7 into a single string in the format “Required - [parameters]”, “Provision only - [parameters]”, or If not mentioned, leave the field blank.

# Q-11
# Monitoring System
# Gas Density Monitor - Manufacturer
# "This item checks whether the customer specification explicitly requires a specific manufacturer or brand for the Gas Density Monitor used in high-voltage switchgear. Commonly specified brands include Trafag, WIKA, or other well-known pressure switch providers. If no specific brand is mentioned, mark as 'Not specified.'"
# AI Extraction Logic
Step 1: Search for mentions of gas density monitor or pressure switch — In the context of gas monitoring or insulation gas control. Look for keywords such as: "Gas density monitor", "Pressure switch", "SF₆ density relay", "Specified manufacturer", "e.g., Trafag, WIKA", "Shall be supplied from [Brand]".
Step 2: From the identified sentence, extract — The specified manufacturer name (e.g., "WIKA", "Trafag"); If multiple acceptable brands are listed, record all.
Step 3: Record the extracted value — If a specific manufacturer is clearly mentioned, extract the name; If phrases like "any equivalent", "supplier standard", or "no preference" are used, leave the field blank.

# Q-12
# Monitoring System
# Gas Density Monitor - Contact Type (CB)
# "This item identifies whether the gas density monitor used for the circuit breaker (CB) requires a specific contact operation type, such as 'normally closed, falling break' or 'normally open, falling make.' These contact types affect how the density monitor behaves in response to pressure drop, particularly for alarm or lockout signals. Extract the exact contact operation type if specified."
# AI Extraction Logic
Step 1: Search for references to gas density monitor contact types — In relation to the circuit breaker. Look for keywords such as: "Gas density monitor" or "pressure switch for CB", "Contact type", "contact operation", "Normally closed", "normally open", "Falling break", "falling make", "pressure falling operation", "Opens on low gas", "closes on low gas".
Step 2: From the matched sentences, extract — Contact behavior (e.g., "normally closed, falling break"); If multiple contacts are defined, identify which is for alarm, which is for lockout; If described functionally (e.g., "contact shall open when pressure drops"), infer the type if wording is unambiguous.
Step 3: Record the extracted value — If a specific contact type is mentioned, extract it as written; If not specified, leave the field blank.

# Q-13
# Monitoring System
# Gas Density Monitor - Contact Type (Others)
# "This item identifies the required contact operation type of gas density monitors used for components other than the circuit breaker (CB), such as disconnectors (DS), earthing switches (ES), or bus compartments. Contact types may include 'normally closed, falling break' or 'normally open, falling make,' indicating how the contact behaves in response to gas pressure drop. Extract the exact wording from the customer specification if available."
# AI Extraction Logic
Step 1: Locate sections that reference gas density monitors not assigned to CB — Typically described for busbar, DS, ES, or general compartments. Search for terms such as: "Gas density monitor for DS", "ES", "bus", "others", "Contact operation", "Normally closed", "normally open", "Falling break", "falling make", "Contact shall open/close on pressure drop".
Step 2: Extract the following from identified segments — Contact configuration type (e.g., "normally closed, falling break"); Target equipment (e.g., "for DS", "for busbar"); Any functional explanation (e.g., "opens when pressure drops").
Step 3: Record the extracted value — If a clear contact type is stated, extract and record as is; If not specified, leave the field blank.

# Q-14
# Monitoring System
# Gas Density Monitor - High Alarm Setting
# "This item identifies whether the customer explicitly requires a high-pressure alarm setting (high alarm) for gas density monitoring, particularly for compartments where higher internal pressure may exist (e.g., circuit breaker compartments). While most specifications focus on gas leakage (low alarm), high alarm may be required in CB-related compartments to detect abnormal overpressure. Extract the requirement and the specified threshold if stated."
# AI Extraction Logic
Step 1: Search for references to high alarm conditions — In gas compartments, especially in sections describing: Gas density monitor alarm levels, Alarm thresholds for circuit breaker compartments, Overpressure or high-pressure protection. Search terms include: "High alarm", "Overpressure alarm", "Gas pressure exceeds", "Excessive gas pressure", "High-pressure detection", "High-pressure setting", "CB compartment high alarm".
Step 2: Extract the following when found — Alarm type (e.g., "high alarm required"); Target compartment(s) (e.g., "CB left/right side"); Alarm threshold value, if specified (e.g., "set at 0.70 MPa").
Step 3: Record the extracted value — If high alarm is specified for one or more compartments, extract the full statement including the affected areas and threshold; If no high alarm is mentioned or only low alarm is specified, leave the field blank.

# R-1
# Service Continuity Requirement
# During Bus DS/ES Maintenance
# "Check whether the customer requires service continuity to be maintained during maintenance of the bus disconnector switch (DS) or earthing switch (ES). This includes planned maintenance activities that may affect the busbar section or connected feeders. Requirements may specify that certain feeders or transformers must remain energized, or may define how many elements can be taken out of service during such maintenance."
# AI Extraction Logic
Step 1: Identify any statements referring to maintenance of the bus disconnector or earthing switch.
Step 2: Determine whether service continuity is explicitly required during such maintenance — Such as: "Feeder shall remain in service during maintenance of bus disconnector", "At least one line and one transformer feeder must remain energized", "Only one feeder allowed out of service during DS maintenance".
Step 3: Extract the requirement as one of the following — "All feeders must remain in service"; "Minimum one line and one transformer feeder to remain in service"; "Partial outage permitted during DS/ES maintenance"; "No specific requirement".

# R-2
# Service Continuity Requirement
# During CB Maintenance
# "Check whether the customer requires service continuity to be maintained during circuit breaker (CB) maintenance. This includes planned inspections or replacement of CB components that may cause feeder or busbar interruption. Requirements may specify which feeders must remain energized or how many elements can be taken out of service during CB maintenance."
# AI Extraction Logic
Step 1: Identify any statements related to planned circuit breaker (CB) maintenance or inspections.
Step 2: Determine whether service continuity is required during CB maintenance — Such as: "Feeder must remain energized during CB maintenance", "One feeder out of service is acceptable during CB maintenance", "CB maintenance shall not affect busbar or adjacent feeders".
Step 3: Extract the requirement as one of the following — "All feeders must remain in service"; "Minimum one feeder to remain in service"; "One feeder outage permitted during CB maintenance"; "No specific requirement".

# R-3
# Service Continuity Requirement
# During Main Bus Maintenance
# "Check whether the customer allows partial operation of the substation (i.e., some feeders or transformers remaining in service) during maintenance of the main bus section. Determine whether full substation outage is required or if redundancy (e.g., double busbar, bus-coupler) enables continuity of service."
# AI Extraction Logic
Step 1: Identify any statements referring to maintenance of the main busbar or main bus section.
Step 2: Determine whether service continuity is required during such maintenance — Such as: "Partial operation shall be maintained during main bus maintenance", "Feeder outage not allowed during bus maintenance", "Complete shutdown of the switchgear is acceptable".
Step 3: Extract the requirement as one of the following — "Partial operation must be maintained"; "No feeder outage allowed during bus maintenance"; "Full outage permitted during main bus maintenance"; "No specific requirement".

# R-4
# Service Continuity Requirement
# During Extension
# "Check whether the customer requires service continuity to be maintained during extension of the switchgear. This includes installation of new feeders, bays, or sections added to an existing switchgear lineup. Requirements may specify that the existing system must remain energized, with no feeder outage allowed during the extension work."
# AI Extraction Logic
Step 1: Search for any statements referring to switchgear extension, expansion, or addition of new feeders or sections.
Step 2: Identify whether service continuity is explicitly required during the extension — Such as: "No outage allowed during switchgear extension", "Existing feeders shall remain energized during addition of new bays", "Extension work shall not affect ongoing substation operation".
Step 3: Extract the requirement as one of the following — "Full service continuity required during extension"; "Partial outage allowed during extension"; "Extension requires full shutdown"; "No specific requirement".

# S-1
# Special Needs
# Manufacturing Country Restriction
# "Specifies whether the customer requires certain equipment (e.g., switchgear, local control panels) to be manufactured in a specific country or prohibits manufacturing in certain countries. Such restrictions typically aim to ensure quality control, compliance with national standards, or strategic sourcing preferences. These requirements may apply to main equipment or subcomponents, such as control cabinets, and are usually enforced through expressions like 'shall be manufactured in [Country]' or 'not permitted to be manufactured in [Country]'."
# AI Extraction Logic
Step 1: Search for Explicit Country Restrictions — Scan for expressions such as: "shall be manufactured in [Country]", "must not be manufactured in [Country]", "manufacturing outside [Country] not permitted without approval".
Step 2: Determine Scope of Restriction — Identify the component(s) to which the restriction applies (e.g., switchgear, control cabinet, local panel). Example: If the sentence refers to "local control cabinet shall preferably be manufactured in Spain," → extract: Tender Requirement: "Manufacture in Spain preferred"; Source: Exact sentence reference; If manufacturing outside the stated country is allowed only with approval, note it as a conditional requirement.
Step 3: Standardize Extracted Format — Use format: "Manufacturing Country Requirement: [Country] (mandatory / preferred / conditional)"; E.g., "Spain (preferred); external manufacturing requires RE approval".
Step 4: Do Not Infer — Do not extract based on supplier location or product origin unless explicitly mentioned in the specification text.

# S-2
# Special Needs
# Audit / Surveillance during Manufacturing Requirement
# "Specifies whether the customer explicitly requires the right to perform audits, surveillance inspections, or quality reviews during the manufacturing process of high-voltage switchgear (66kV and above). This includes any access to workshops, in-process inspections, welding inspections, or review of quality records prior to Factory Acceptance Test (FAT). This requirement is distinct from routine FAT attendance and implies additional oversight during fabrication or assembly. Any such manufacturing-stage audit requirement must be identified and cited."
# AI Extraction Logic
Step 1: If the specification clearly states the customer may perform audits, inspections, or surveillance during manufacturing — e.g., "Buyer may audit manufacturing process", "access to production area", "inspection during fabrication" → Extract "Required", cite the source phrase in the Tender Requirement, and flag for QA follow-up if specific scope or timing is ambiguous.
Step 2: If the specification only mentions FAT, or references "inspection before shipment" — Do not extract under this item.
Step 3: If conditional or optional language is used — e.g., "may audit upon notice", "access upon request" → Extract the phrase as-is, mark it as "Conditionally Required", and flag for manual review.
Step 4: If the specification restricts such audits or explicitly limits customer presence to FAT — Extract "Not Required", and cite the restrictive clause.
Step 5: Do not infer audit rights based on common practice, FAT attendance, or general QA statements unless specifically stated as manufacturing-stage access.
Step 6: If the requirement is not addressed — Leave blank or flag for manual review.

# S-3
# Special Needs
# Nondestructive Test Requirement
# "Specifies whether the project specification explicitly requires any nondestructive testing (NDT) methods to be applied to high-voltage switchgear components such as welded parts, structural frames, pressure vessels, or bushings. NDT methods may include but are not limited to Radiographic Testing (RT), Ultrasonic Testing (UT), Magnetic Particle Testing (MT), or Dye Penetrant Testing (PT). This item ensures compliance with applicable manufacturing standards and quality assurance criteria."
# AI Extraction Logic
Step 1: Direct Statements — If the specification clearly states an NDT method (e.g., "RT shall be applied to all weld joints") → Extract the method(s) as Tender Requirement and cite the Source.
Step 2: Multiple Methods — If more than one method is listed, extract them all using format: RT / UT / PT (order as written in spec).
Step 3: Conditional Requirements — If conditional wording is used (e.g., "RT may be required as per applicable codes") → Extract the phrase, cite the Source, and flag for manual review.
Step 4: Standard-based Inference — If the specification mandates compliance with standards that implicitly require NDT, such as: ISO 17636 → RT; ASME Section VIII, Div.1 → RT → Extract as: RT (as per ISO 17636) or RT (as per ASME Sec. VIII) and cite the Source.
Step 5: No Mention — If no NDT method or relevant standard is mentioned → Leave the field blank or flag for manual review.
Step 6: Do Not Infer — Do not infer NDT requirements based on product type or typical QA practice unless supported by specification text or standard reference.

# S-4
# Special Needs
# Impact Recorder Requirement
# "Specifies whether the specification requires installation of an Impact Recorder for monitoring acceleration, shock, or vibration levels experienced by the high-voltage equipment during transportation, handling, or installation. This requirement goes beyond conventional shock indicators (e.g., adhesive-type tilt/shock labels) and refers to digital or mechanical recorders that log dynamic impact data for traceability or damage verification."
# AI Extraction Logic
Step 1: Search for explicit mentions of advanced shock/vibration recording devices — Key terms: "impact recorder", "shock recorder", "vibration logger", "acceleration recorder"; Phrases like "shall be equipped with an impact recorder", "transport shock shall be continuously monitored".
Step 2: Differentiate from conventional indicators — If only passive shock indicators are mentioned (e.g., "shock label", "tilt indicator"), do not extract as Impact Recorder; If both types are mentioned, extract only the recorder-type requirement here and document shock label separately if needed.
Step 3: Format the Tender Requirement field — If clearly stated: extract as "Impact Recorder required" or use the exact phrase; If conditional (e.g., "if required by client"), extract with condition and flag for manual review; If not mentioned or only passive indicators are described, leave blank or flag.

# S-5
# Special Needs
# Indoor Installation Clearance Requirement
# "Check whether the customer specifies minimum clearance distances between high-voltage switchgear (e.g., GIS) and surrounding building walls or structural boundaries when installed indoors. These clearances may include side, front, and rear distances and are typically defined in meters. The requirement may vary depending on system voltage (e.g., 132 kV vs. 230 kV and above) or substation type."
# AI Extraction Logic
Step 1: Identify references to indoor installation — Of GIS or other high-voltage switchgear in dedicated rooms or buildings.
Step 2: Detect clearance values defined by position — Such as: "Side clearance from entrance to GIS equipment", "Front clearance between GIS and wall", "Rear clearance to building wall", "Minimum distance of X meters required".
Step 3: Extract clearance values — With associated orientation (e.g., front, rear, side) and applicable voltage level or substation type; If a voltage-based table is provided, extract all defined combinations; If only general guidance is given, extract as a note.

# S-6
# Special Needs
# Customer-Specified Spare Part Requirement
# "Check whether the customer specifies any particular spare parts to be supplied for the high-voltage switchgear beyond the standard recommendation from the manufacturer. While spare part lists are usually proposed by the supplier, some specifications may mandate certain spare items such as interrupters, trip coils, CTs, VTs, or gaskets. Only extract the parts that are explicitly requested by the customer."
# AI Extraction Logic
Step 1: Identify sections or expressions related to spare parts — For GIS, DTB, HGIS, or related high-voltage switchgear.
Step 2: Detect if the customer specifies exact spare parts — Using phrases like: "The following spares shall be supplied:", "Mandatory spare parts", "Customer requires the following spares", "Spare interrupter to be provided".
Step 3: Extract the explicitly listed spare parts — e.g., 1 trip coil, 3 interrupters, 2 CTs; If no specific items are listed and only general phrases like "as recommended by manufacturer" appear, do not extract any spare parts.

# S-7
# Special Needs
# Customer-Specified Special Tool Requirement
# "Check whether the customer explicitly specifies particular special tools to be supplied for installation, operation, or maintenance of high-voltage switchgear. Only extract tools that are clearly listed or mandated by the customer. Do not extract if the specification only refers to tools to be used by the supplier or states that tools shall be proposed by the manufacturer."
# AI Extraction Logic
Step 1: Identify references to special tools, tool kits, or jigs — Related to GIS, DTB, or HGIS in the context of installation, maintenance, or operation.
Step 2: Detect explicit requirements — Using expressions such as: "the following special tools shall be provided", "disassembly jig to be supplied", "SF6 filling adapter required", "operating mechanism tool kit required".
Step 3: Extract the listed special tools and quantities (if given) — Do not extract if the text only includes: "special tools to be used by manufacturer", "tools to be proposed by supplier", "if required, tools shall be used by vendor".

# S-8
# Special Needs
# Customer-Specified Special Conditions for FAT / SAT
# "Check whether the customer specifies special conditions or additional requirements for FAT (Factory Acceptance Test) or SAT (Site Acceptance Test) beyond standard procedures. These may include dedicated time for Employer testing, SCADA signal verification, GOOSE simulation, full switchgear control access, or integrated FAT (iFAT), which involves multi-vendor system-level testing. Only extract if the condition is explicitly stated as a requirement."
# AI Extraction Logic
Step 1: Identify references to FAT or SAT — In the context of high-voltage switchgear testing.
Step 2: Detect special conditions or mandates — Such as: "minimum 5 working days for Employer’s own test", "Employer must have full control access during FAT", "GOOSE signal test to DSO control center shall be performed", "Supplier shall not plan any activity during Employer test", "Integrated FAT (iFAT) shall be conducted", "iFAT including protection and SCADA", "multi-vendor iFAT".
Step 3: Extract a summary of the specified special conditions — Including: Duration of dedicated Employer test time; Control access level; Communication test requirements (e.g., SCADA, IEC 61850); Requirement for integrated FAT (iFAT); Restrictions on supplier activities during testing.
''',
type="user",
author="탁근주",
version=250724)

description_prompt가 저장되었습니다.


In [14]:
# instruction_prompt
manager.create_entry(subject="instruction_prompt",text='''
## Instructions:
1. Analyze the provided PDF document and extract information based on the categories and specification items defined in ## Given Text ##. Write your answer using only what is mentioned in the provided PDF.
2. Infer the extraction process using the provided description and AI extraction logic.
3. For the case where the high voltage switchgear type is "{pHVST}" and the nominal system voltage value is "{pNSV}", extract the values of other items based on the provided high voltage switch type and nominal system voltage value.
4. Fix MachineType as "{pMT}".
5. If the specification item is not found in the PDF document, output 'None' and clearly state the missing information.
6. If there is a conflict in the values, choose the value that indicates the more severe condition.
7. Output the extracted information in the following JSON array format.
8. There are 152 items in total. You must output the values for all 152 items corresponding to the machine type.
''',
type="user",
author="탁근주",
version=250724)

instruction_prompt가 저장되었습니다.


In [11]:
# format_prompt
manager.create_entry(subject="format_prompt",text='''
[
{'SerialNumber':'Serial Number, The numeric part of SerialNumber is written in three-digit format. ex) G-1 -> G-001'},
{'SwitchgearType':'Switchgear Type'},
{'Category':'Category Name'}, 
{'Item':'Specification Item Name'}, 
{'Value':'Extracted Value'},
{'Unit':'unit'}, 
{'Ref':'The source sentence in the PDF file used to determine the value extraction. (Enter the original pdf text verbatim)'}, 
{'Title':'FileName containing the "Ref" text. See "Filename" at the bottom right of the page.'}, 
{'Page':'PageNumber containing the "Ref" text. See "Page" at the bottom right of the page.'}
]

For example:

[
{'SerialNumber':'E-005'},
{'SwitchgearType':'380kV GIS'},    
{'Category':'Electrical Ratings'}, 
{'Item':'Nominal System Voltage'}, 
{'Value':'170'},
{'Unit':'kV'}, 
{'Ref':'The required nominal system voltage for the GIS is 170kV.'}, 
{'Title':'Technical Specifications'}, 
{'Page':'5'}
]''',
type="user",
author="탁근주",
version=250930)

format_prompt가 저장되었습니다.


## UPDATE

In [3]:
# 업데이트(description_prompt_A_H)
manager.update_entry("description_prompt_A_H", {"text": """
You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)

# A-1  
# General Requirements  
# Applicable International Standards  
# "Specifies whether the high-voltage switchgear must comply with IEC or IEEE standards, which govern its design, testing, and performance requirements."  
# AI Extraction Logic  
- Extract only one primary applicable standard—either IEC Standards or IEEE Standards—explicitly governing the high-voltage switchgear in the project specification.  
- If both are mentioned, determine which one governs based on direct reference, section title, or priority statement. If unclear, select the one predominantly cited for switchgear-related clauses.  
- Do not list all referenced standards; identify only the controlling standard group.

# A-2
# General Requirements
# Regulatory & Certification Requirements
# "Specifies whether the high-voltage switchgear must comply with specific product certifications such as UL, CE, CSA, or other regional or international regulatory marks. These certifications may be required for safety, conformity, or market access purposes."
# AI Extraction Logic  
- Extract all explicitly stated certification requirements related to high-voltage switchgear or its components (e.g., CB, DS, ES, LCC) from project-binding documents.
- Only include certifications that are market access or safety-related, such as:
  - “UL certified”
  - “CE marking”
  - “CSA approval”
  - “conformity assessment for market access”
- Exclude general technical standards such as:
  - “compliance with IEC 62271”
  - “ISO 9001 certified manufacturer”
  - “design according to IEEE C37.06”
- For each certification, extract:
  - Certification Required: [UL, CE, CSA, ...]
  - Certification Scope: [high-voltage switchgear, LCC, control circuits, ...]
  - Certification Condition: [mandatory, preferred, optional]

# A-3
# General Requirements
# Pressure Vessel Code Compliance
# "Specifies whether the high-voltage switchgear pressure vessels (e.g., GIS enclosures, circuit breaker tanks) must comply with a specific pressure vessel code or standard (such as ASME, PED, EN 13445). Extract only if the specification explicitly mandates compliance or certification."
# AI Extraction Logic  
- Extract only if explicitly mentioned in the project-binding specification applicable to high-voltage switchgear.
- Identify:
  - Required code or standard (e.g., ASME Section VIII, PED 2014/68/EU, EN 13445)
  - Certification or marking requirement (e.g., ASME “U” stamp, CE marking)
  - Scope (e.g., GIS tank, circuit breaker tank, all pressure vessels)
- Ignore:
  - General mentions of “pressure test” or “pressure rating” without reference to a compliance code.
- Accepted keywords:
  - pressure vessel code
  - ASME compliance
  - PED compliance
  - EN 13445
  - CE marking
  - ASME U stamp
  - pressure vessel certification
- Output Rules
  - If specified →
    - Pressure Vessel Code Compliance: [Code / Standard] – [Certification if specified] – [Scope]
- Examples
  - Pressure Vessel Code Compliance: ASME Section VIII – U Stamp – Scope: GIS tank
  - Pressure Vessel Code Compliance: PED 2014/68/EU – CE marking – Scope: All pressure vessels
  - Pressure Vessel Code Compliance: EN 13445 – Scope: Circuit breaker tank only

# A-4
# General Requirements
# Supply Record Requirement (Operational Track Record)  
# "Specifies whether the project requires a minimum operational track record for high-voltage switchgear and what explicit thresholds must be extracted (e.g., years of successful field performance, number of in-service projects or bays, market scope, environmental qualifiers)."  
# AI Extraction Logic  
- Locate clauses defining operational track record for the manufacturer or equipment (e.g., “operational track record,” “successful field performance,” “supply record,” “references in service,” “international market”) applicable to high-voltage switchgear.  
- Confirm the clause covers the high-voltage switchgear type (GIS/HGIS/DTB/SF₆-free); umbrella terms that include switchgear or circuit breaker are acceptable.  
- Extract only explicitly stated thresholds. Recommended keys include:
  - Min Years Field Performance  
  - Min Projects  
  - Min Bays  
  - Market Scope  
  - Environment Qualifier  
- If ambiguous terms (e.g., “similar experience”) are used without definition, retain them as-is in the reference text.

# A-5
# General Requirements
# Manufacturing Experience Requirement  
# "Specifies whether the project requires a minimum period of manufacturing experience for the manufacturer of high-voltage switchgear and what numeric threshold in years must be extracted."  
# AI Extraction Logic  
- Extract any explicitly stated manufacturing experience requirements applicable to high-voltage switchgear from project-binding documents.
- Look for phrases such as: “manufacturing experience,” “years of manufacturing,” “minimum experience,” etc.
- Extract only the numeric threshold in **years** as stated. Do not infer values, apply inequality symbols, or add units.

# A-6
# General Requirements
# Type Test Validity Period  
# "Specifies whether the project requires a time validity or recency limit for type-test evidence of high-voltage switchgear and what the explicit threshold and anchor (reference date/event) are."  
# AI Extraction Logic  
- Extract any explicitly stated validity or recency requirements for type-test evidence applicable to high-voltage switchgear from project-binding documents.
- Identify phrases such as: “type test report validity,” “not older than,” “conducted within,” “after [date],” etc.
- Extract the validity threshold in one of the following formats:
  - **Period Years**: X (e.g., 5)
  - **Cutoff Date**: YYYY-MM-DD (e.g., 2022-01-01)
  - **No Limit If Design Unchanged**: True
- If an anchor event is mentioned (e.g., bid closing, contract award, FAT, delivery), extract it as:
  - **Anchor**: [event name]
- Do not infer or reformat values. Use only explicitly stated information.
- Ignore statements about routine tests, special tests, accreditation, or standard references unless they are directly tied to type test validity.

# A-7
# General Requirements
# Documentation Language Requirement
# "Specifies the language(s) in which all technical documentation (e.g., manuals, drawings, test reports, certificates) for high-voltage switchgear must be provided. Extract only if the specification explicitly states the required language(s) or bilingual requirements."
# AI Extraction Logic  
- Extract only if explicitly mentioned in the project-binding specification applicable to high-voltage switchgear.
- Identify:
  - Required language(s) (e.g., English, Korean, Arabic)
  - Bilingual or multilingual requirement (e.g., English + Local language)
  - Scope (e.g., O&M manual, drawings, test reports, certificates)
- Ignore:
  - General mentions of “documentation” without specifying language.
- Accepted keywords:
  - documentation language
  - manuals in English
  - bilingual documentation
  - certificates in [language]
  - drawings in [language]
  - name plates in [language]
- Output Rules
  - If specified →
    - Documentation Language Requirement: [Language(s)] – [Scope if specified]
  - If bilingual →
    - Documentation Language Requirement: [Language 1 + Language 2] – [Scope if specified]
- Examples
  - Documentation Language Requirement: French – Scope: All technical documentation
  - Documentation Language Requirement: English + Arabic – Scope: O&M manuals and test reports

# B-1
# Environmental Conditions
# Installation Area
# "Specifies whether the high-voltage switchgear is designed for indoor, outdoor, or mixed installation, depending on the environmental and structural conditions defined in the project specification. If both general specifications and project-specific documents are available, the installation area defined in the project-specific documents shall take precedence. If different components (e.g., Gas-Insulated Bus, Hybrid Insulated Switchgear, Dead Tank Breakers) require different installation areas, these must be recorded separately."
# AI Extraction Logic
- Search project-specific documents for installation-area statements that explicitly apply to the validated SwitchgearType. Consider a general specification only if it explicitly states applicability to the validated SwitchgearType. Ignore clauses for other equipment.
- Apply source priority: project-specific clause for the pair > pair-specific section > general specification explicitly scoped to the project switchgear. Ignore statements that apply to other equipment.
- Resolve outcomes without assumptions.
  - If a single area applies to the pair, output one of: indoors, outdoors.
  - If different areas are stated for distinct components of the pair (e.g., GIS indoors, GIB outdoors), output components: <label1>: indoors; <label2>: outdoors, using component labels.
  - If the spec lists multiple allowed areas (e.g., “prefer indoors; outdoors acceptable with enclosure”) and project context is unknown, output variants: indoors: true; outdoors: true (conditions may be noted outside the value field if needed).
Step 2: Output rules
- Return a single line, ASCII only.
- Single value → Indoors / Outdoors.
- Component split: <Label1>: <Indoors / Outdoors>; <Label2>: <Indoors / Outdoors>. For example, GIS: Indoors; GIB: Outdoors.
- Multiple allowed options (no component split) → Variants: Indoors: True; Outdoors: True.

# B-2
# Environmental Conditions
# Coastal Area Classification
# "Specifies whether the installation site is explicitly classified as a coastal area, where high humidity, salt, and airborne contaminants significantly increase the risk of corrosion, surface degradation, and long-term material wear. If both general specifications and project-specific documents are available, the project-specific coastal classification shall take precedence. Corrosion protection measures (e.g., duplex coating) shall not be used as indirect evidence of coastal classification."
# AI Extraction Logic
- Extract “Coastal” for the installation site of the validated SwitchgearType only when project-binding documents explicitly classify the site as coastal (e.g., “coastal area/zone,” “marine environment,” “salt-laden/coastal atmosphere”), or when the site location is stated and, per the document’s own definition (e.g., distance threshold), it falls within the coastal range with explicit contextual linkage. Do not infer from definitions alone without a stated site location and linkage.
- Accept direct statements such as:
  - “The site is classified as coastal.”
  - “The substation is located within X km of the coastline,” if the document defines a rule or context that maps such distance to coastal classification.
- If the document mentions geographic regions (e.g., Red Sea, Gulf Coast) **alongside corrosion-related requirements** (e.g., duplex coating, anti-corrosion treatment), and the context clearly implies coastal exposure, treat as coastal.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question and provides a valid classification or rule.
- Do not infer coastal classification based solely on:
  - Protective measures without geographic or contextual linkage
  - Enclosure/IP ratings
  - Descriptive terms like “humid” or “salty air” without classification logic

# B-3
# Environmental Conditions
# Ambient Air Temperature (Indoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
- Extract the explicitly stated minimum ambient air temperature applicable to **indoor installation** of high-voltage switchgear from project-binding documents.
- Accept values only if clearly associated with indoor installation. If the label is missing, confirm indoor context from surrounding text.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question.
- Ignore:
  - General climate data (e.g., monthly/annual averages)
  - Outdoor temperature values
  - Values unrelated to equipment installation

# B-4
# Environmental Conditions
# Ambient Air Temperature (Indoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to indoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
- Extract the explicitly stated maximum ambient air temperature applicable to **indoor installation** of high-voltage switchgear from project-binding documents.
- Accept values only if clearly associated with indoor installation. If the label is missing, confirm indoor context from surrounding text.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question.
- Ignore:
  - General climate data (e.g., monthly/annual averages)
  - Outdoor temperature values
  - Values unrelated to equipment installation

# B-5
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Minimum
# "Specifies the minimum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
- Extract the explicitly stated minimum ambient air temperature applicable to **outdoor installation** of high-voltage switchgear from project-binding documents.
- Accept values only if clearly associated with outdoor installation. If the label is missing, confirm outdoor context from surrounding text.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question.
- Ignore:
  - General climate or geographic data (e.g., regional climate summaries, monthly/annual averages)
  - Indoor temperature values
  - Values unrelated to equipment installation

# B-6
# Environmental Conditions
# Ambient Air Temperature (Outdoor) Maximum
# "Specifies the maximum expected ambient air temperature applicable to outdoor installation of high-voltage switchgear. Only values applicable to switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
- Extract the explicitly stated maximum ambient air temperature applicable to **outdoor installation** of high-voltage switchgear from project-binding documents.
- Accept values only if clearly associated with outdoor installation. If the label is missing, confirm outdoor context from surrounding text.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question.
- Ignore:
  - General climate data (e.g., hottest month average, yearly highs)
  - Indoor temperature values
  - Values unrelated to equipment installation

# B-7
# Environmental Conditions
# Altitude
# "Specifies the installation altitude applicable to high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the project-specific altitude shall take precedence. The extracted altitude must represent the actual site condition, not a design range or generic classification."
# AI Extraction Logic
- Extract the explicitly stated installation altitude applicable to high-voltage switchgear from project-binding documents.
- Accept values expressed as:
  - Fixed numeric values (e.g., “950 m”, “MSL: 1200 m”)
  - Ranges only if the project-specific selection is clearly indicated
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question and provides a fixed altitude relevant to the project.
- Ignore:
  - Design ranges or classification bands (e.g., “0–1000 m or 1001–3000 m”) unless a specific selection is made
  - Standard default values (e.g., IEC 62271-1: 1000 m) unless explicitly stated as applicable to the project site
  - General geographic or regional elevation data not tied to equipment installation

# B-8
# Environmental Conditions
# Site Pollution Severity
# "Specifies the pollution severity level at the installation site, which affects the external insulation performance, creepage distance, and material durability of high-voltage switchgear. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the pollution classification or contamination data from the project-specific documents shall take precedence."
# AI Extraction Logic
- Extract any explicitly stated pollution severity classification applicable to the installation site of high-voltage switchgear from project-binding documents.
- Accept severity levels such as: `Light`, `Medium`, `Heavy`, `Very Heavy`
- Accept regional mappings only if the document explicitly links site type (e.g., coastal area) to a severity class.
- If ESDD (Equivalent Salt Deposit Density) or NSDD (Non-Soluble Deposit Density) values are provided:
  - Extract them as-is (e.g., `ESDD = 0.15 mg/cm²`)
  - Do not convert to severity level unless the document provides a direct mapping
- If IEC TS 60815-1 or similar standards are referenced:
  - Extract severity level only if the document includes a direct mapping to classification
- If both classification and ESDD/NSDD values are present:
  - Extract both separately and record in respective fields
- Do not infer severity based on:
  - Geographic location or proximity to coast
  - Raw contamination values without mapping
  - General environmental descriptions

# B-9
# Environmental Conditions
# Seismic Classification
# "Specifies the seismic performance requirements for the installation site of high-voltage switchgear, based on expected earthquake intensity, ground motion, or seismic zoning. Only values applicable to equipment rated at nominal system voltages of 66 kV and above shall be considered. If both general specifications and project-specific documents are available, the project-specific seismic classification or data shall take precedence. No inferred classification or mapping shall be made unless explicitly stated."
# AI Extraction Logic
- Extract any explicitly stated seismic classification or seismic performance data applicable to the installation site of high-voltage switchgear from project-binding documents.
- Accept classification terms such as:
  - `Low Seismic`, `IEEE 693 High Level`, `Zone IIB`, etc.
- Accept numerical indicators such as:
  - `PGA = 0.3g`, `Response Spectrum (curve reference)`, etc.
- If both classification and raw data are present, extract both separately.  
  - Do not convert between them unless the document provides an explicit mapping.
- Use only project-specific documents unless the general specification is explicitly scoped to the switchgear in question and provides applicable site-specific seismic data.
- Do not extract values based solely on:
  - General compliance statements (e.g., “shall comply with IEEE 693”)
  - Regional seismic maps without classification
  - Inferred mappings from location or zone unless explicitly defined

# C-1
# Electrical Ratings
# Nominal System Voltage
# "Specifies the nominal system voltage required for the project’s high-voltage switchgear. “Nominal Voltage” and “System Voltage” are treated as the same concept."
# AI Extraction Logic
- Extract the voltage contained in SwitchgearType (e.g., 380 kV if the SwitchgearType is "380 kV GIS").

# C-2
# Electrical Ratings
# Rated Voltage
# "Specifies the phase-to-phase RMS voltage corresponding to the maximum system voltage for which the high-voltage switchgear is designed. In IEC-based specifications, this is referred to as 'Rated Voltage'. In IEEE-based specifications, the equivalent term is 'Rated Maximum Voltage'. Extract the value as 'Rated Voltage' regardless of terminology used."
# AI Extraction Logic
- Extract the Rated Voltage explicitly stated in the project specification for the validated SwitchgearType.
- If not found, check the general specification if applicable to the project.
- Do not infer values or apply default mappings.
- Normalize the extracted value under the field name "Rated Voltage", regardless of whether the source uses IEC or IEEE terminology.

## Common Extraction Rules for C-3 ~ C-5:
- If separate values are not provided for hybrid gas-insulated switchgear (HGIS), the values for gas-insulated switchgear (GIS) shall be used.
- Strict insulation-path match (no cross-path substitution): Only use values that explicitly correspond to the targeted insulation path of the validated SwitchgearType (C-3: phase-to-earth, C-4: across open switching device, C-5: across isolating distance).
- If both environment (outdoors/indoors) and condition (dry/wet) variants exist and the installation location of the validated SwitchgearType is unknown, return:
  - Value: `"Outdoors_Dry: value; Outdoors_Wet: value; Indoors: value"` (omit keys not present)
  - Unit: `"kV"`
- If only one axis is unknown, return variants for that axis only (e.g., `"Outdoors: value; Indoors: value"` or `"Dry: value; Wet: value"`)
- Output must be split into:
  - Value: numeric string or variant string (e.g., `"650"` or `"Outdoors: 650; Indoors: 650"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-3
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Phase-to-Earth)  
# "Specifies the short-duration power-frequency withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear."  
# AI Extraction Logic  
- Validate the SwitchgearType, then locate statements or table rows that define power-frequency withstand voltage for the phase-to-earth path and explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-3 ~ C-5.

# C-4
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Across Open Switching Device)  
# "Specifies the short-duration power-frequency withstand voltage across the open switching device (open contacts) of the high-voltage switchgear."  
# AI Extraction Logic  
- Validate the SwitchgearType, then locate statements or table rows that define power-frequency withstand voltage across open switching device (open contacts) and explicitly apply to the validated SwitchgearType.
- Source priority: (1) an explicit value across the open switching device (Do not substitute the value across the isolating distance); (2) the common value for power-frequency withstand voltage, if specified; (3) otherwise, leave the field blank.
- Apply common extraction rules for C-3 ~ C-5.

# C-5
# Electrical Ratings  
# Rated Short-Duration Power-Frequency Withstand Voltage (Across the Isolating Distance)  
# "Specifies the short-duration power-frequency withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear."  
# AI Extraction Logic  
- Validate the SwitchgearType, then locate statements or table rows that define power-frequency withstand voltage across the isolating distance and explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-3 ~ C-5.

## Common Extraction Rules for C-6 ~ C-8:
- If separate values are not provided for hybrid gas-insulated switchgear (HGIS), the values for gas-insulated switchgear (GIS) shall be used.
- Strict insulation-path match (no cross-path substitution): Only use values that explicitly correspond to the targeted insulation path of the validated SwitchgearType (C-6: phase-to-earth, C-7: across open switching device, C-8: across isolating distance).
- If both environment (outdoors/indoors) variants exist and the installation location of the validated SwitchgearType is unknown, return:
  - Value: `"Outdoors: value; Indoors: value"` (omit keys not present)
  - Unit: `"kV"`
- Output must be split into:
  - Value: numeric string or variant string or augmented numeric string (e.g., `"900 (+345)"` or `"Outdoors: 900 (+345); Indoors: 900 (+345)"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-6
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Phase-to-Earth)
# "Specifies the switching impulse withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear."
# AI Extraction Logic
- Validate the SwitchgearType, then locate statements or table rows that define switching impulse withstand voltage for the phase-to-earth path and explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-6 ~ C-8.

# C-7
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Across Open Switching Device)
# "Specifies the switching impulse withstand voltage across the open switching device (open contacts) of the high-voltage switchgear."
# AI Extraction Logic
- Validate the SwitchgearType, then locate statements or table rows that define switching impulse withstand voltage across open switching device (open contacts) and explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-6 ~ C-8.

# C-8
# Electrical Ratings
# Rated Switching Impulse Withstand Voltage (Across the Isolating Distance)
# "Specifies the switching impulse withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear."
# AI Extraction Logic
- Validate the SwitchgearType, then locate statements or table rows that define switching impulse withstand voltage across the isolating distance (e.g., "across the isolating distance," "across open disconnector," "across isolation distance") and explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-6 ~ C-8.

## Common Extraction Rules for C-9 ~ C-12:
- If separate values are not provided for hybrid gas-insulated switchgear (HGIS), the values for gas-insulated switchgear (GIS) shall be used.
- Strict insulation-path match (no cross-path substitution): Only use values that explicitly correspond to the targeted insulation path of the validated SwitchgearType (C-9: phase-to-earth, C-10: across open switching device, C-11: across isolating distance).
- If both environment (outdoors/indoors) variants exist and the installation location of the validated SwitchgearType is unknown, return:
  - Value: `"Outdoors: value; Indoors: value"` (omit keys not present)
  - Unit: `"kV"`
- Output must be split into:
  - Value: numeric string or variant string or augmented numeric string (e.g., `"1050"` or `"Outdoors: 1050; Indoors: 950"`)
  - Unit: `"kV"`
- Do not include inequality symbols, thousands separators, or units in the Value field.
- If no explicit value for the targeted insulation path is present, leave the field blank.

# C-9
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Phase-to-Earth)
# "Specifies the lightning impulse withstand voltage for the phase-to-earth insulation path of the high-voltage switchgear."
# AI Extraction Logic
- Validate the SwitchgearType, then locate statements or table rows that define lightning impulse withstand for the phase-to-earth path (e.g., "lightning impulse withstand," "LIWV," "LI," "BIL," "1.2/50 us") and synonyms for the path such as "to earth" or "to ground" and that explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-9 ~ C-12.

# C-10
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Across Open Switching Distance)
# "Specifies the lightning impulse withstand voltage across the open switching distance (open switching device/open contacts) of the high-voltage switchgear."
# AI Extraction Logic
- Validated the SwitchgearType, then locate statements or table rows that define lightning impulse withstand for the path across the open switching distance, including phrases such as "across open contacts," "across open pole," or "interrupter gap," and lightning-specific terms ("lightning impulse," "LI," "BIL," "1.2/50 us") and that explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-9 ~ C-12.

# C-11
# Electrical Ratings
# Rated Lightning Impulse Withstand Voltage (Across the Isolating Distance)
# "Specifies the lightning impulse withstand voltage across the isolating distance of a disconnector or isolating function in the high-voltage switchgear."
# AI Extraction Logic
- Validated the SwitchgearType, then locate statements or table rows that define lightning impulse withstand for the path across the isolating distance, including phrases such as "across the isolating distance," "across open disconnector," or "across isolation distance," and lightning-specific terms ("lightning impulse," "LI," "BIL," "1.2/50 us") and that explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-9 ~ C-12.

# C-12
# Electrical Ratings
# Chopped Wave Lightning Impulse Withstand Voltage
# "Specifies the chopped-wave lightning impulse withstand voltage in the high-voltage switchgear."
# AI Extraction Logic
- Validated the SwitchgearType, then locate statements or table rows that define chopped-wave lightning impulse withstand and that explicitly apply to the validated SwitchgearType.
- Apply common extraction rules for C-9 ~ C-12.

# C-13
# Electrical Ratings
# Rated Power-Frequency Withstand Voltage for Auxiliary and Control Circuits
# "Extract the rated RMS voltage that auxiliary and control circuits must withstand under power-frequency conditions."
# AI Extraction Logic
- Search the project-binding specification applicable to the validated SwitchgearType for withstand voltage values related to auxiliary and control circuits.  
- If not found, check the general specification only if it is explicitly scoped to the validated SwitchgearType.  
- Look for phrases such as:  
  - Auxiliary/control circuit withstand voltage
  - Power-frequency withstand voltage for control wiring
  - AC withstand voltage for auxiliary circuits
- Extract the voltage value (in volts) and duration (in seconds or minutes), if both are stated.  
  - Example: `2,000 V (1 min)`  
- If only the voltage is stated, extract just the voltage.  
- Do not infer values from IEC/IEEE defaults unless explicitly stated.  

# C-14
# Electrical Ratings
# Rated Frequency
# "Specifies the nominal operating frequency of the high-voltage switchgear, which affects its electrical performance, insulation coordination, and compatibility with the power grid. The rated frequency is typically 50 Hz or 60 Hz, depending on the regional grid standard."
# AI Extraction Logic
- Locate statements or table rows that explicitly define the rated frequency for the validated SwitchgearType (e.g., "380 kV GIS").
- If a general statement applies to all equipment in the document (e.g., "The nominal frequency for National Grid, Saudi Arabia is 60 Hz"), it may be used.

# C-15
# Electrical Ratings
# Rated Continuous Current
# "Specifies the RMS value of current that the high-voltage switchgear can carry continuously under its defined service conditions, without exceeding allowable temperature rise or mechanical limits of current-carrying components. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66 kV and above shall be considered."
# AI Extraction Logic
- Locate statements or table rows that explicitly define the rated continuous current for the validated SwitchgearType.
- Exclude values for other equipment (e.g., transformers, cables) and unrelated ratings (e.g., overload, peak current).

# C-16
# Electrical Ratings
# Rated Short-Time Withstand Current
# "Specifies the RMS value of the short-circuit current that the high-voltage switchgear can withstand in the closed position for a specified short duration (typically 1s or 3s), without mechanical deformation, internal damage, or insulation failure. This rating validates the switchgear’s thermal and electrodynamic withstand capability under fault conditions until the protection system isolates the fault. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
- Locate statements or table rows that explicitly define the rated short-time withstand current for the validated SwitchgearType.
- Exclude values for other equipment and unrelated ratings (e.g., peak current, momentary current).

# C-17
# Electrical Ratings
# Rated Peak Withstand Current
# "Specifies the peak value of the first asymmetrical short-circuit current loop that the high-voltage switchgear must withstand without mechanical failure. This rating evaluates the equipment’s ability to endure electrodynamic forces caused by the highest momentary current. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
- Extract the Rated Peak Withstand Current explicitly stated for the validated SwitchgearType from the project specification, technical datasheet, or general specification (if applicable).  
- If not directly stated, and both the Rated Short-Time Withstand Current (Iₜₕ) and a Peak Factor (e.g., 2.5, 2.7) are clearly provided in the same context, calculate the peak current using:  
  - `Rated Peak Withstand Current = Iₜₕ × Peak Factor`  

# C-18
# Electrical Ratings
# Rated Duration of Short-Circuit
# "Specifies the time interval (typically 1s or 3s) during which the high-voltage switchgear must withstand the rated short-time current without mechanical or thermal failure while in the closed position. This parameter defines the thermal stress duration that the equipment can safely endure under fault conditions before current interruption. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
- Extract the short-circuit duration value (e.g., `1s`, `3s`) explicitly stated for the validated SwitchgearType from the project specification or general specification (if applicable).  

# C-19
# Electrical Ratings
# Emergency Withstand / Operation at 1 bar abs (0 barg)
# "Specifies the ability of high-voltage switchgear to remain operational or maintain insulation integrity when the internal gas pressure drops to atmospheric pressure (1 bar absolute, 0 barg). This condition simulates emergency scenarios such as gas leakage or maintenance bypass, where the switchgear must either continue functioning or safely isolate without failure. Only values applicable to high-voltage switchgear rated at nominal system voltages of 66kV and above shall be considered."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any statement indicating the high-voltage switchgear’s ability to operate or withstand voltage at 1 bar abs (0 barg).  
- If the document explicitly states that the switchgear shall operate or withstand voltage at 1 bar abs, extract as: Required.  


## Common Extraction Rules

### GIS vs. HGIS Interpretation
- If a requirement is stated for GIS and the validated SwitchgearType is HGIS, apply the requirement only if:
  - The document context uses “GIS” as a general term for gas-insulated switchgear systems, or
  - The project includes HGIS and no separate requirements are stated for HGIS, and the GIS requirement is clearly intended to apply to all gas-insulated configurations.

# D-1
# Design Parameters
# Proposed High-Voltage Switchgear Model
# AI Extraction Logic
- Leave the field blank.

# D-2
# Design Parameters
# High-Voltage Switchgear Type
# "Specifies the high-voltage switchgear type required for the project."
# AI Extraction Logic
- Extract the high-voltage switchgear type contained SwitchgearType (e.g., "GIS" if the SwitchgearType is "380 kV GIS").

# D-3
# Design Parameters
# Insulation Medium Requirement
# "Check whether the customer specifies a particular insulation medium to be used in the high-voltage switchgear. The requirement may include allowance or prohibition of SF₆ (sulfur hexafluoride), approval for SF₆-free alternatives (e.g., C4-FN, dry air, vacuum), or statements about environmental compliance. The insulation medium may apply to GIS, DTB, GIB, or other major gas-insulated components."
# AI Extraction Logic
- Identify any explicit mention of insulation medium in the project-binding specification applicable to the validated SwitchgearType.
- Accept references to SF₆, SF₆-free alternatives (e.g., C4-FN, dry air, vacuum), or environmental compliance statements. 
- Extract the insulation medium type(s) and note whether it is Required (e.g., "SF₆-free only"), Allowed (e.g., "SF₆ permitted"), or Prohibited (e.g., "SF₆ not allowed"). 
- If multiple options are listed (e.g., SF₆ or dry air), extract all. 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# D-4
# Design Parameters
# Enclosure Type
# "Specifies whether the high-voltage switchgear enclosure is designed as a single-phase or three-phase configuration."
# AI Extraction Logic
- Identify any explicit mention of enclosure type (Single-Phase or Three-Phase) in the project-binding specification applicable to the validated SwitchgearType.
- If both types are acceptable, output as: 'Single-Phase / Three-Phase'
- If one type is preferred and another conditionally allowed, output as: 'Single-Phase (preferred); Three-Phase allowed upon approval'

# D-5
# Design Parameters
# Enclosure Material
# "Specifies the primary metallic material used for the enclosure (housing) of high-voltage switchgear, which impacts corrosion resistance, mechanical strength, and environmental durability."
# AI Extraction Logic
- Identify any explicit mention of enclosure material (e.g., Aluminum, Stainless Steel, Galvanized Steel, Al-Zn Coated Steel) in the project-binding specification applicable to the validated SwitchgearType. 
- If multiple materials are permitted, output as: 'Aluminum / Stainless Steel' 
- If one material is preferred and another conditionally allowed, output as: 'Aluminum (preferred); Stainless Steel allowed upon approval'

# D-6
# Design Parameters
# Conductor Material
# "Specifies the material used for the main current-carrying conductors (e.g., busbars, terminals) inside the high-voltage switchgear. The material affects electrical conductivity, current capacity, thermal performance, and mechanical stability."
# AI Extraction Logic
- Identify any explicit mention of conductor material (e.g., Copper, Aluminum) in the project-binding specification applicable to the validated SwitchgearType.
- If multiple materials are permitted, output as: 'Copper / Aluminum'
- If one material is preferred and another conditionally allowed, output as: 'Copper (preferred); Aluminum allowed upon approval'

# D-7
# Design Parameters
# Partial Discharge Level
# "Specifies the maximum permissible partial discharge (PD) intensity for internal components of high-voltage switchgear, including busbars, support insulators, bushings, and other insulation elements. This requirement ensures compliance with insulation quality criteria under type test and routine test conditions. PD limits are typically governed by international standards such as IEC 62271-203 or IEEE Std C37.122."
# AI Extraction Logic
- If a numerical PD value is explicitly stated (e.g., less than 5pC), extract it as-is.
- If only a standard is referenced (e.g., as per IEC 62271-203), extract it as "As per [standard name]".

# D-8
# Design Parameters
# Maximum Relative Gas Leakage Rate
# "Specifies the maximum allowable annual gas leakage rate, expressed as a percentage of the total gas quantity within each gas-filled compartment of high-voltage switchgear. This requirement ensures long-term gas retention integrity, pressure stability, and compliance with environmental regulations for equipment using SF₆, SF₆ mixtures, or alternative insulating gases. Leakage limits are typically defined in international standards such as IEC 62271-203 or functionally equivalent clauses in IEEE guidelines, depending on the applicable standard."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- If a numerical leakage rate is explicitly stated (e.g., less than 0.5% per year), extract it as-is.
- If only a standard is referenced (e.g., as per IEC 62271-203), extract it as "As per [standard name]".

# D-9
# Design Parameters
# Internal Fault - Short-Circuit Current
# "Specifies the RMS value of short-circuit current the high-voltage switchgear must withstand during an internal arc fault. If the internal arc withstand current is not explicitly defined in the specification, it shall be assumed equal to the Rated Short-Time Withstand Current (as per IEC 62271-203 and industry practice)."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any explicit mention of internal arc fault withstand current for the validated SwitchgearType.  
- If the internal arc fault current is stated, extract the value as-is.  
- If not stated, and the Rated Short-Time Withstand Current (Iₜₕ) is available, use Iₜₕ as the internal fault current.  

# D-10
# Design Parameters
# Internal Fault - Duration of Current
# "Specifies the minimum time duration that the high-voltage switchgear enclosure must withstand the effects of an internal arc fault without loss of containment or unsafe failure, under the specified internal fault current level. This duration (commonly expressed as 300ms, 500ms, or 1s) represents the burn-through resistance of the enclosure before fragmentation or external effects occur. IEC 62271-203 and IEEE C37.122 provide standard classifications and test protocols for burn-through time."
# AI Extraction Logic
- Search the project specification or general specification (if applicable) for any explicit mention of internal arc fault duration for the validated SwitchgearType.  
- Recognize equivalent terms such as:  
  - *Burn-through time*  
  - *Internal arc duration*  
  - *Arc withstand time*  
- If the document provides performance criteria (e.g., “must withstand arc without external effect for 0.3s”), extract the time value **only if unambiguous**.  
- Do not infer this value from general short-circuit duration unless the specification clearly states it applies to internal fault conditions.  

# D-11
# Design Parameters
# Maximum Length/Volume of Gas Compartment
# "Specifies the maximum allowable physical length or the maximum allowable volume of an individual gas-tight compartment within high-voltage switchgear (e.g., GIS or HGIS), as constrained by insulation reliability, gas leakage risk, pressure stability, and maintenance considerations. In some customer standards, limits are defined either by maximum physical length (e.g., 18m to 22.5m) or by maximum total gas volume per compartment (e.g., 4m³ of SF₆). Extensions beyond this length may be permitted with prior customer approval, provided that the total gas volume in the compartment does not exceed 4m³."
# AI Extraction Logic
- Extract the maximum allowable length value (e.g., 22.5m) or the maximum allowable volume (e.g., ≤ 4 m³)  of an individual gas-tight compartment within high-voltage switchgear explicitly stated in the project specification or customer’s general standard.
- If both length and volume are stated, extract both using the format:
  - 'Length: 22.5 m; Volume: 4 m³'
- If conditional approval is mentioned, include note:
  - '22.5 m (extensions allowed upon approval)'
- If only one parameter is stated, extract that value.

# D-12
# Design Parameters
# Filling Pressure at 20°C (CB)
# "Specifies the rated filling pressure at 20°C for the circuit breaker compartment of the high-voltage switchgear. This value is typically expressed in bar absolute or MPa and is relevant for gas-insulated equipment using SF₆ or alternative gases."
# AI Extraction Logic
- Leave the field blank.

# D-13
# Design Parameters
# Filling Pressure at 20°C (Others)
# "Specifies the rated filling pressure at 20°C for other gas-filled compartments (e.g., busbar, disconnector, earthing switch) of the high-voltage switchgear. This value is typically expressed in bar absolute or MPa and is relevant for gas-insulated equipment using SF₆ or alternative gases."
# AI Extraction Logic
- Leave the field blank.

# D-14
# Design Parameters
# Sealing Redundancy (Single / Double)
# "Specifies whether pressurized gas joints of the high-voltage switchgear require single or double sealing. Apply to the provided (Nominal System Voltage, High-Voltage Switchgear Type) pair. Accept synonyms such as double sealing, duplicated sealing system, dual sealing, twin seal, two seals. Do not infer from unrelated phrases unless they clearly apply to pressurized gas joints."
# AI Extraction Logic
- Identify any explicit mention of sealing redundancy for pressurized joints (flanges, compartment interfaces, feedthroughs) in the project-binding specification applicable to the validated SwitchgearType.
- Accept synonyms: double sealing, dual sealing, two seals, double O-ring, double gasket.
- Normalize all values to either "Single" or "Double".
- If component-specific, output in the format:
  - 'Cable Termination: Double; Busbar Compartment: Single'
- If environment variants exist and environment is unknown, output both:
  - 'Indoors: Single; Outdoors: Double'
- If only one global requirement applies, output as:
  - 'Double'

# D-15
# Design Parameters
# Gasket Material
# "Specifies the required gasket material for high-voltage switchgear components (e.g., GIS compartments, circuit breaker tanks, flanges). Extract only if the specification explicitly states the material type or compliance standard."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Material type (e.g., EPDM, NBR, Viton, Silicone)
  - Compliance standard if specified (e.g., IEC 60815, ASTM D2000)
  - Scope (e.g., GIS compartments, circuit breaker tank, all sealing points)
- Ignore:
  - General mentions of “gasket” without specifying material or standard.
- Accepted keywords:
  - gasket material
  - sealing material
  - EPDM / NBR / Viton / Silicone
  - compliance with IEC / ASTM
  - oil-resistant gasket
  - SF₆-resistant gasket
- Output Rules
  - If specified →
  - Gasket Material: [Material] – [Compliance if specified] – [Scope]

# D-16
# Design Parameters
# Noise Level Requirement
# "Specifies the maximum permissible noise level (in dB(A)) for high-voltage switchgear during normal operation, typically measured at a defined distance (e.g., 1 m or 3 m). Extract only if the specification explicitly states the noise limit or compliance standard."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Noise level limit (e.g., ≤ 65 dB(A), ≤ 75 dB(A))
  - Measurement distance (e.g., at 1 m, at 3 m)
  - Compliance standard if specified (e.g., IEC 62271-203, ISO 3744)
- Ignore:
  - General mentions of “low noise” without numeric value or standard.
- Accepted keywords:
  - noise level
  - sound pressure level
  - dB(A)
  - IEC 62271-203 compliance
  - ISO 3744
  - acoustic requirement
- Output Rules
  - If specified →
    - Noise Level Requirement: [Value in dB(A)] – [Distance] – [Compliance if specified]
- Examples
  - Gasket Material: EPDM – Compliance: IEC 60815 – Scope: All GIS compartments
  - Gasket Material: NBR – Scope: Circuit breaker tank
  - Gasket Material: Viton – Compliance: ASTM D2000 – Scope: SF₆ sealing points

# E-1
# Circuit Breaker
# Proposed Model
# AI Extraction Logic
- Leave the field blank.

# E-2
# Circuit Breaker
# Interrupting Chambers per Pole (Breaks in Series)
# "Specifies the number of interrupting chambers (breaking units) installed in series within each pole of the circuit breaker, which directly affects the voltage withstand and arc interruption capability."
# AI Extraction Logic
- Extract the number of interrupting chambers per pole explicitly stated in the technical specification or breaker design documents for the circuit breaker associated with the validated SwitchgearType.
- If multiple configurations are allowed, list them using the format: '1 break / 2 breaks'
- If a preferred configuration is stated and others are conditionally acceptable, use the format: 
  - '2 breaks (preferred); 1 break allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-3
# Circuit Breaker
# Pre-Insertion Resistor Requirement
# "Specifies whether a pre-insertion resistor (PIR) is required in the circuit breaker design to limit inrush current during closing operations, particularly in capacitor or reactor switching applications."
# AI Extraction Logic
- Identify any explicit mention of pre-insertion resistor requirements in the technical specification or breaker application context for the circuit breaker associated with the validated SwitchgearType.
- If PIR is required, state as: 'Required'
- If PIR is optional or conditionally required (e.g., only for capacitor/reactor switching or OHTL feeders), use the format: 
  - 'Optional; required for capacitor/reactor switching'
  - 'Optional; required for OHTL feeders'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-4  
# Circuit Breaker  
# Grading Capacitors (Across Open Contacts) Requirement  
# "Specifies whether grading capacitors are required across the open contacts of the circuit breaker to ensure uniform voltage distribution during transient conditions, particularly in multi-break configurations."
# AI Extraction Logic  
- Identify any explicit mention of grading capacitor requirements in the technical specification or breaker design documents for the circuit breaker associated with the validated SwitchgearType.
- If grading capacitors are required, state as: 'Required'  
- If grading capacitors are optional or conditionally required (e.g., only for multi-break configurations), use the format: 
  - 'Optional; required for multi-break configurations' 
- If grading capacitors are explicitly excluded (e.g., “shall not be used”), state as: 'Not allowed'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-5
# Circuit Breaker  
# CT and CB Gas Compartment Separation Requirement  
# "Specifies whether the current transformer (CT) and circuit breaker (CB) are required to be installed in separate gas compartments, typically for maintenance flexibility, safety, or fault isolation purposes."
# AI Extraction Logic  
- Identify any explicit mention of gas compartment separation between CT and CB in the technical specification or breaker layout documents for the circuit breaker associated with the validated SwitchgearType.
- If separation is required, state as: 'Required' 
- If separation is optional or conditionally required (e.g., only for specific layouts, maintenance zones, or upon End User approval), use the format:  
  - 'Optional; required for specific layout or maintenance conditions'  
  - 'Optional; CT and CB Gas Compartment can be in the same gas compartment upon End User approval' 
- If compartment separation is explicitly excluded (e.g., “shall not be separated”, “shall be in the same compartment”), state as: 'Not required' 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-6
# Circuit Breaker  
# Single-Phase Operating Requirement  
# "Specifies whether the circuit breaker is required to support single-phase operation, typically for applications such as single-phase reclosing or selective phase isolation in transmission systems."
# AI Extraction Logic  
- Identify any explicit mention of single-phase operating capability or requirement in the technical specification or breaker control philosophy documents for the circuit breaker associated with the validated SwitchgearType.
- If single-phase operation is required, state as: 'Required'  
- If single-phase operation is optional or conditionally required (e.g., only for specific fault scenarios or reclosing schemes), use the format: 
  - 'Optional; required for single-phase reclosing'
  - 'Optional; required for OHTL feeders'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-7
# Circuit Breaker  
# Controlled Switching Requirement  
# "Specifies whether controlled switching functionality is required for the circuit breaker, typically to minimize switching transients during energization or de-energization of transformers, reactors, or capacitor banks."
# AI Extraction Logic  
- Identify any explicit mention of controlled switching requirements or functionality in the technical specification, protection philosophy, or control system documents for the circuit breaker associated with the validated SwitchgearType.
- If controlled switching is required, state as: 'Required' 
- If controlled switching is optional or conditionally required (e.g., only for specific equipment), use the format:  
  - 'Optional; required for transformer energization'  
  - 'Optional; required for reactor feeders'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-8
# Circuit Breaker  
# Rated Operating Sequence  
# "Specifies the rated operating sequence of the circuit breaker, which defines the timing and number of operations the breaker must perform under standard conditions, typically expressed in the format O–t–CO–t–CO."
# AI Extraction Logic  
- Identify any explicit mention of rated operating sequence in the technical specification or breaker performance documents for the circuit breaker associated with the validated SwitchgearType.
- If a specific operating sequence is stated (e.g., O–0.3s–CO–3min–CO), extract it as-is.  
- If multiple sequences are allowed or conditionally required, list them using the format:  
  - 'O–0.3s–CO–3min–CO / O–0.5s–CO–3min–CO'  
- If a preferred sequence is stated and others are acceptable upon approval, use the format:  
  - 'O–0.3s–CO–3min–CO (preferred); O–0.5s–CO–3min–CO allowed upon approval' 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-9
# Circuit Breaker  
# Opening Time  
# "Specifies the time duration required for the circuit breaker to complete its opening operation from the initiation of the trip signal to full contact separation, typically measured in milliseconds (ms)."
# AI Extraction Logic  
- Identify any explicit mention of circuit breaker opening time in the technical specification, timing diagrams, or performance data sheets for the circuit breaker associated with the validated SwitchgearType.
- If a specific opening time is stated (e.g., 40 ms), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '40 ms / 50 ms'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '40 ms (preferred); 50 ms allowed upon approval'  
- If opening time is stated in cycles, extract the value as-is (e.g., 2 cycles).  
- If frequency is also provided, convert cycles to milliseconds and include both formats:  
  - 2 cycles at 50 Hz → 40 ms (converted from 2 cycles) 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-10
# Circuit Breaker  
# Arcing Time  
# "Specifies the duration of the arc during the interruption process, measured from the moment the contacts begin to separate until the arc is fully extinguished. Arcing time is a critical parameter for evaluating breaker performance and arc energy dissipation."
# AI Extraction Logic  
- Identify any explicit mention of arcing time in the technical specification, breaker timing diagrams, or performance test data for the circuit breaker associated with the validated SwitchgearType.
- If a specific arcing time is stated (e.g., 15 ms), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '15 ms / 20 ms'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '15 ms (preferred); 20 ms allowed upon approval'  
- If arcing time is stated in cycles, extract the value as-is (e.g., 1.5 cycles).  
- If frequency is also provided, convert cycles to milliseconds and include both formats:  
  - 1.5 cycles at 50 Hz → 30 ms (converted from 1.5 cycles)
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-11
# Circuit Breaker  
# Break-Time  
# "Specifies the time duration from the initiation of the trip signal to the complete extinction of the arc, including both the mechanical opening time and the arcing time. Break-time is a key parameter for protection coordination and fault clearing speed."
# AI Extraction Logic  
- Identify any explicit mention of break-time in the technical specification, timing diagrams, or performance test data for the circuit breaker associated with the validated SwitchgearType.
- If a specific break-time is stated (e.g., 55 ms), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '55 ms / 60 ms'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '55 ms (preferred); 60 ms allowed upon approval' 
- If break-time is stated in cycles, extract the value as-is (e.g., 3 cycles).  
- If frequency is also provided, convert cycles to milliseconds and include both formats:  
  - 3 cycles at 50 Hz → 60 ms (converted from 3 cycles)  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-12
# Circuit Breaker  
# Closing Time  
# "Specifies the time duration required for the circuit breaker to complete its closing operation from the initiation of the close signal to full contact engagement, typically measured in milliseconds (ms)."
# AI Extraction Logic  
- Identify any explicit mention of circuit breaker closing time in the technical specification, timing diagrams, or performance data sheets for the circuit breaker associated with the validated SwitchgearType.
- If a specific closing time is stated (e.g., 60 ms), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '60 ms / 70 ms'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '60 ms (preferred); 70 ms allowed upon approval'  
- If closing time is stated in cycles, extract the value as-is (e.g., 3 cycles).  
- If frequency is also provided, convert cycles to milliseconds and include both formats:  
  - 3 cycles at 50 Hz → 60 ms (converted from 3 cycles)  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-13
# Circuit Breaker  
# Dead Time (during auto-reclosing)  
# "Specifies the time interval between the completion of a trip operation and the initiation of the reclosing command during auto-reclosing sequences. Dead time is critical for system stability and coordination with protection relays."
# AI Extraction Logic  
- Identify any explicit mention of dead time in auto-reclosing sequences from the technical specification, protection philosophy, or control logic documents for the circuit breaker associated with the validated SwitchgearType.
- If a specific dead time is stated (e.g., 0.5 s), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '0.5 s / 1.0 s'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '0.5 s (preferred); 1.0 s allowed upon approval'  
- If dead time is stated in cycles, extract the value as-is (e.g., 25 cycles).  
- If frequency is also provided, convert cycles to seconds and include both formats:  
  - '25 cycles at 50 Hz → 0.5 s (converted from 25 cycles)'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-14
# Circuit Breaker  
# DC Time Constant  
# "Specifies the DC time constant (τ) used for evaluating the circuit breaker's capability to interrupt asymmetrical fault currents. It represents the rate at which the DC component of the fault current decays, typically expressed in milliseconds (ms)."
# AI Extraction Logic  
- Identify any explicit mention of DC time constant in the technical specification, fault analysis criteria, or breaker interrupting performance documents for the circuit breaker associated with the validated SwitchgearType.
- If a specific DC time constant is stated (e.g., 45 ms), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '45 ms / 60 ms'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '45 ms (preferred); 60 ms allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-15
# Circuit Breaker  
# X/R Ratio  
# "Specifies the ratio of system reactance (X) to resistance (R) at the fault location, which affects the asymmetry of fault current and the DC component during interruption. A higher X/R ratio indicates a slower decay of the DC component and greater stress on the breaker."
# AI Extraction Logic  
- Identify any explicit mention of X/R ratio in the fault analysis section, short-circuit calculation results, or breaker interrupting performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific X/R ratio is stated (e.g., 17), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '17 / 20'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '17 (preferred); 20 allowed upon approval' 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-16
# Circuit Breaker  
# Rated First-Pole-to-Clear Factor  
# "Specifies the factor used to account for the transient overvoltage that may occur when the first pole of a three-phase circuit breaker clears a fault before the other two poles. This factor is used in dielectric design and insulation coordination, typically expressed as a multiplier (e.g., 1.3)."
# AI Extraction Logic  
- Identify any explicit mention of the rated first-pole-to-clear factor in the technical specification, insulation coordination section, or dielectric design criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific factor is stated (e.g., 1.3), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '1.3 / 1.5'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '1.3 (preferred); 1.5 allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-17
# Circuit Breaker  
# Mechanical Endurance Class  
# "Specifies the mechanical endurance classification of the circuit breaker, indicating the number of operating cycles it is designed to perform without maintenance or performance degradation. This classification is typically defined according to IEC standards (e.g., Class M1, M2)."
# AI Extraction Logic  
- Identify any explicit mention of mechanical endurance class or operating cycle rating in the technical specification, IEC compliance section, or mechanical performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific class is stated (e.g., M2), extract it as-is.  
- If endurance is stated as a number of operations (e.g., 10,000 operations), extract the value as-is.
- If a preferred class is stated and others are acceptable upon approval, use the format:  
  - 'M2 (preferred); M1 allowed upon approval'  
  - '10,000 operations (preferred); 2,000 allowed upon approval'  
- If mechanical endurance class is explicitly excluded or not applicable (e.g., “not classified under IEC endurance classes”), state as: Not required.  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-18
# Circuit Breaker  
# Electrical Endurance Class  
# "Specifies the electrical endurance classification of the circuit breaker, indicating its ability to perform fault interruption or load switching operations over its service life. This classification is typically defined according to IEC standards (e.g., Class E1, E2), or expressed directly as the number of electrical operations."
# AI Extraction Logic  
- Identify any explicit mention of electrical endurance class or electrical operation rating in the technical specification, IEC compliance section, or electrical performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific class is stated (e.g., E2), extract it as-is.   
- If a preferred class or value is stated and others are acceptable upon approval, use the format:  
  - 'E2 (preferred); E1 allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-19
# Circuit Breaker  
# Restrike Performance  
# "Specifies the circuit breaker's ability to avoid restrikes during switching operations, particularly in capacitor or reactor applications. Restrike performance is critical for ensuring insulation integrity and minimizing transient overvoltages. It may be classified according to IEC standards (e.g., Class C1, C2)."
# AI Extraction Logic  
- Identify any explicit mention of restrike performance or restrike-free operation in the technical specification, switching duty descriptions, or dielectric performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If restrike-free performance is required, state as: Required.  
- If restrike performance is stated as a class (e.g., C1, C2), extract the value as-is.  
- If both class and functional requirement are provided, extract both using the format:  
  - 'C1 (restrike-free for capacitor switching)'  
- If restrike-free performance is optional or conditionally required (e.g., only for capacitor switching), use the format:  
  - 'Optional; restrike-free required for capacitor switching'   
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-20
# Circuit Breaker  
# Rated Line-Charging Breaking Current  
# "Specifies the rated current the circuit breaker is capable of interrupting during line-charging conditions, which typically involve low current but high transient recovery voltage (TRV). This rating ensures safe interruption of charging current when energizing long transmission lines."
# AI Extraction Logic  
- Identify any explicit mention of rated line-charging breaking current in the technical specification, switching duty descriptions, or interrupting performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 10 A), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '10 A / 20 A'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '10 A (preferred); 20 A allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-21
# Circuit Breaker  
# Rated Cable-Charging Breaking Current  
#  "Specifies the rated current the circuit breaker is capable of interrupting during cable-charging conditions, which typically involve low current but high transient recovery voltage (TRV). This rating ensures safe interruption of charging current when energizing long underground or submarine cables."
# AI Extraction Logic  
- Identify any explicit mention of rated cable-charging breaking current in the technical specification, switching duty descriptions, or interrupting performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 400 A), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '400 A / 600 A'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '400 A (preferred); 600 A allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-22
# Circuit Breaker  
# Rated Single Capacitor Bank Breaking Current  
# "Specifies the rated current the circuit breaker is capable of interrupting when disconnecting a single capacitor bank. This duty involves low current but high transient recovery voltage (TRV), and is critical for ensuring restrike-free performance and dielectric integrity."
# AI Extraction Logic  
- Identify any explicit mention of rated single capacitor bank breaking current in the technical specification, switching duty descriptions, or interrupting performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 400 A), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '400 A / 600 A'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '400 A (preferred); 600 A allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-23
# Circuit Breaker  
# Back-to-Back Capacitor Bank Switching Capability  
# "Specifies whether the circuit breaker is required to support back-to-back capacitor bank switching, regardless of the specific rated current."
# AI Extraction Logic  
- Identify any explicit mention of back-to-back capacitor bank switching capability in the technical specification or switching duty descriptions for the circuit breaker associated with the validated SwitchgearType.
- If the capability is required, state as: 'Required'  
- If optional or conditionally required, use the format:  
  - 'Optional; required for capacitor feeders'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-24
# Circuit Breaker  
# Rated Back-to-Back Capacitor Bank Breaking Current  
# "Specifies the rated current the circuit breaker is capable of interrupting when disconnecting one capacitor bank while another remains energized (back-to-back configuration). This duty involves high inrush and recovery transients, requiring enhanced dielectric and restrike-free performance."
# AI Extraction Logic  
- Identify any explicit mention of rated back-to-back capacitor bank breaking current in the technical specification, switching duty descriptions, or interrupting performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 1,000 A), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '1,000 A / 1,200 A'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '1,000 A (preferred); 1,200 A allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-25
# Circuit Breaker  
# Rated Back-to-Back Capacitor Bank Inrush Making Current  
# "Specifies the rated peak inrush current the circuit breaker must withstand when energizing a capacitor bank in the presence of another already energized bank (back-to-back configuration). This duty involves high-frequency, high-magnitude inrush currents and is critical for evaluating the breaker’s making capability and contact durability."
# AI Extraction Logic  
- Identify any explicit mention of rated back-to-back capacitor bank inrush making current in the technical specification, switching duty descriptions, or making performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 2.5 kA peak), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '2.5 kA / 3.0 kA'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '2.5 kA (preferred); 3.0 kA allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-26
# Circuit Breaker  
# Shunt Reactor Switching Requirement  
"Specifies whether the circuit breaker is required to perform switching operations involving shunt reactors, which typically involve low current interruption with high transient recovery voltage (TRV). This requirement is critical for ensuring dielectric integrity and restrike-free performance during reactor disconnection."
# AI Extraction Logic  
- Identify any explicit mention of shunt reactor switching requirements in the technical specification, switching duty descriptions, or application-specific performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If the document refers to inductive current switching in accordance with IEC 62271-110, and the context clearly relates to reactor or inductive load switching, treat it as equivalent to shunt reactor switching and extract as: Required.  
- If shunt reactor switching is required, state as: 'Required'  
- If shunt reactor switching is optional or conditionally required (e.g., only for specific system configurations), use the format:  
  - 'Optional; required for reactor feeders'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-27
# Circuit Breaker  
# Number of Full Short-Circuit Breakings per Pole Until Next Maintenance  
# "Specifies the maximum number of full short-circuit current interruptions that each pole of the circuit breaker can perform before maintenance is required. This value reflects the wear tolerance of the interrupting chamber and is critical for maintenance planning and lifecycle cost estimation."
# AI Extraction Logic  
- Identify any explicit mention of the number of full short-circuit breakings per pole until next maintenance in the technical specification, maintenance schedule, or performance criteria for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 10 operations), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '10 / 12 operations'  
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '10 operations (preferred); 12 allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-28
# Circuit Breaker  
# Operating Mechanism Type  
# "Specifies the operating mechanism type for circuit breakers used in high-voltage switchgear identified under SwitchgearType. Typical types include spring-operated, hydraulic-operated, and pneumatic-operated mechanisms."
# AI Extraction Logic  
- Identify any explicit mention of operating mechanism type for the circuit breaker that belongs to the validated SwitchgearType in project-binding documents (e.g., Spring-Operated, Hydraulic-Operated).
- If multiple types are permitted, list them using the format:  
  - 'Spring / Hydraulic / Pneumatic / Hydraulic Spring' 
- If a preferred type is stated and others are conditionally acceptable, use the format:  
  - 'Spring (preferred); Hydraulic allowed upon approval'  
  - 'Spring, Hydraulic-spring (allowed); Hydraulic (Not allowed)'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-29
# Circuit Breaker  
# Motor Type  
# "Specifies the type of motor used in the operating mechanism of the circuit breaker, typically for spring charging or hydraulic pumping. Common motor types include AC motor, DC motor, and brushless DC motor, depending on control system and auxiliary power configuration."
# AI Extraction Logic  
- Identify any explicit mention of motor type in the technical specification, operating mechanism description, or auxiliary system requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific motor type is stated (e.g., DC motor), extract it as-is.  
- If multiple types are allowed or conditionally required, list them using the format:  
  - 'AC / DC'  
- If a preferred type is stated and others are acceptable upon approval, use the format:  
  - 'DC motor (preferred); AC motor allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-30
# Circuit Breaker  
# Manual Operating Requirement  
# "Specifies whether the circuit breaker is required to support manual operation, typically for emergency use, maintenance, or local control. Manual operation may include manual opening, closing, or spring charging via hand lever or mechanical actuator."
# AI Extraction Logic  
- Identify any explicit mention of manual operating capability or requirement in the technical specification, operating mechanism description, or control philosophy for the circuit breaker associated with the validated SwitchgearType.
- If manual operation is required, state as: 'Required' 
- If manual operation is optional or conditionally required (e.g., only for indoor installations or specific control schemes), use the format:  
  - 'Optional; required for indoor installations'  
  - 'Optional; required for local control only'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-31
# Circuit Breaker  
# Spring-Charging Time  
# "Specifies the time required to fully charge the operating mechanism spring of the circuit breaker, typically measured from the initiation of the motor signal to the completion of spring charging. This parameter affects breaker readiness and reclose timing."
# AI Extraction Logic  
- Identify any explicit mention of spring-charging time in the technical specification, operating mechanism description, or timing diagrams for the circuit breaker associated with the validated SwitchgearType.
- If a specific value is stated (e.g., 10 seconds), extract it as-is.  
- If multiple values are allowed or conditionally required, list them using the format:  
  - '10 s / 12 s' 
- If a preferred value is stated and others are acceptable upon approval, use the format:  
  - '10 s (preferred); 12 s allowed upon approval' 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-32
# Circuit Breaker  
# Number of Trip Coils
# "Specifies the number of trip coils required for the circuit breaker operating mechanism. Extract only if the specification explicitly states the quantity or redundancy requirement."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Number of trip coils (e.g., 1, 2)
  - Redundancy requirement (e.g., two trip coils for reliability)
- Ignore:
  - General mentions of “trip coil” without quantity.
- Accepted keywords:
  - trip coil
  - number of trip coils
  - dual trip coil
  - redundant trip coil
- Output Rules
  - If specified →
    - Number of Trip Coils: [Quantity] – [Redundancy if specified]
- Examples
  - Number of Trip Coils: 2 – Redundant
  - Number of Trip Coils: 1

# E-33
# Circuit Breaker  
# Number of Closing Coils
# "Specifies the number of closing coils required for the circuit breaker operating mechanism. Extract only if the specification explicitly states the quantity or redundancy requirement."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Number of closing coils (e.g., 1, 2)
  - Redundancy requirement (e.g., two closing coils for reliability)
- Ignore:
  - General mentions of “closing coil” without quantity.
- Accepted keywords:
  - closing coil
  - number of closing coils
  - dual closing coil
  - redundant closing coil
- Output Rules
  - If specified →
    - Number of Closing Coils: [Quantity] – [Redundancy if specified]
- Examples
  - Number of Closing Coils: 2 – Redundant
  - Number of Closing Coils: 1

# E-34
# Circuit Breaker  
# Slow Operation (Slow Closing / Slow Opening) Requirement  
# "Specifies whether the circuit breaker is required to support slow closing and/or slow opening operations, typically for testing, commissioning, or diagnostic purposes. Slow operation allows visual inspection of contact movement and timing without full-speed actuation."
# AI Extraction Logic  
- Identify any explicit mention of slow closing or slow opening requirements in the technical specification, operating mechanism description, or commissioning procedures for the circuit breaker associated with the validated SwitchgearType.
- If slow operation is required, state as: 'Required'
- If slow operation is optional or conditionally required (e.g., only for factory testing or specific breaker types), use the format:  
  - 'Optional; required for factory testing'  
  - 'Optional; required for GIS only'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-35
# Circuit Breaker  
# Thermostat Requirement  
# "Specifies whether a thermostat is required to monitor and control the internal temperature of the circuit breaker or its compartments, typically for gas density monitoring, heater control, or ambient temperature protection."
# AI Extraction Logic  
- Identify any explicit mention of thermostat requirements in the technical specification, auxiliary equipment list, or environmental control section for the circuit breaker associated with the validated SwitchgearType.
- If a thermostat is required, state as: 'Required'
- If a thermostat is optional or conditionally required (e.g., only for cold climates or specific enclosure types), use the format:  
  - 'Optional; required for cold climate installations'  
  - 'Optional; required for outdoor GIS'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-36
# Circuit Breaker  
# Humidistat Requirement  
# "Specifies whether a humidistat is required to monitor and control humidity levels inside the circuit breaker enclosure or compartments, typically for preventing condensation, corrosion, or insulation degradation in humid environments."
# AI Extraction Logic  
- Identify any explicit mention of humidistat requirements in the technical specification, auxiliary equipment list, or environmental control section for the circuit breaker associated with the validated SwitchgearType.
- If a humidistat is required, state as: 'Required'  
- If a humidistat is optional or conditionally required (e.g., only for outdoor installations or high-humidity zones), use the format:  
  - 'Optional; required for outdoor GIS'  
  - 'Optional; required for coastal installations'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-37
# Circuit Breaker  
# Number and Type of Auxiliary Contacts  
# "Specifies the quantity and configuration of auxiliary contacts provided with the circuit breaker, typically used for control, signaling, and interlocking purposes. Auxiliary contacts are commonly classified as NO (normally open), NC (normally closed), or changeover types."
# AI Extraction Logic  
- Identify any explicit mention of the number and type of auxiliary contacts in the technical specification, control interface description, or wiring diagrams for the circuit breaker associated with the validated SwitchgearType.
- If a specific quantity and type are stated (e.g., 4 NO + 2 NC), extract them as-is.  
- If multiple configurations are allowed or conditionally required, list them using the format:  
  - '4 NO / 6 NO + 2 NC'  
- If a preferred configuration is stated and others are acceptable upon approval, use the format:  
  - '6 NO + 2 NC (preferred); 4 NO allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-38
# Circuit Breaker  
# Rated Supply Voltage of Control Circuits  
# "Specifies the rated voltage required to operate the control circuits of the circuit breaker, including trip coils, closing coils, motor drives, and auxiliary relays. This voltage is typically supplied from station DC or AC sources."
# AI Extraction Logic  
- Identify any explicit mention of rated control circuit supply voltage in the technical specification, control system description, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific voltage is stated (e.g., 110 V DC), extract it as-is.  
- If multiple voltages are allowed or conditionally required, list them using the format:  
  - '110 V DC / 220 V DC'  
- If a preferred voltage is stated and others are acceptable upon approval, use the format:  
  - '110 V DC (preferred); 220 V DC allowed upon approval'  
- If control circuit voltage is explicitly excluded or not applicable (e.g., “breaker does not require external control voltage”), state as: Not required.  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-39
# Circuit Breaker  
# Operating Voltage Range of Control Circuits  
# "Specifies the acceptable operating voltage range for the control circuits of the circuit breaker, including trip coils, closing coils, motor drives, and auxiliary relays. This range ensures reliable operation under voltage fluctuations and is typically expressed as a percentage or absolute range around the rated voltage."
# AI Extraction Logic  
- Identify any explicit mention of operating voltage range for control circuits in the technical specification, control system description, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific range is stated (e.g., 80% to 110% of 110 V DC), extract it as-is.  
- If multiple ranges are allowed or conditionally required, list them using the format:  
  - '80–110% / 85–110%'  
- If a preferred range is stated and others are acceptable upon approval, use the format:  
  - '80–110% (preferred); 85–110% allowed upon approval'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-40
# Circuit Breaker  
# Operating Voltage Range of Motor Circuits  
# "Specifies the acceptable operating voltage range for the motor circuits used in the circuit breaker’s operating mechanism, typically for spring charging or hydraulic pumping. This range ensures reliable motor performance under voltage fluctuations and is usually expressed as a percentage or absolute range around the rated voltage."
# AI Extraction Logic  
- Identify any explicit mention of operating voltage range for motor circuits in the technical specification, motor description, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific range is stated (e.g., 85% to 110% of 110 V DC), extract it as-is.  
- If multiple ranges are allowed or conditionally required, list them using the format:  
  - '85–110% / 80–110%'  
- If a preferred range is stated and others are acceptable upon approval, use the format:  
  - '85–110% (preferred); 80–110% allowed upon approval'
- If no range is stated but a reference to a standard is provided (e.g., "as per IEC 62271-1"), extract it as-is. 
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-41
# Circuit Breaker
# Operating Voltage Range of Shunt Closing Release
# "Specifies the acceptable operating voltage range for the shunt closing release coil, which is used to remotely close the circuit breaker. This range ensures reliable coil operation under control voltage fluctuations and is typically expressed as a percentage or absolute range around the rated voltage."
# AI Extraction Logic  
- Identify any explicit mention of operating voltage range for the shunt closing release in the technical specification, control circuit description, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific range is stated (e.g., 70% to 110% of 110 V DC), extract it as-is.
- If multiple ranges are allowed or conditionally required, list them using the format:
  - '70–110% / 80–110%'
- If a preferred range is stated and others are acceptable upon approval, use the format:
  - '70–110% (preferred); 80–110% allowed upon approval'
- If no range is stated but a reference to a standard is provided (e.g., "as per IEC 62271-1"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-42
# Circuit Breaker
# Operating Voltage Range of Shunt Opening Release
# "Specifies the acceptable operating voltage range for the shunt opening release coil, which is used to remotely trip (open) the circuit breaker. This range ensures reliable coil operation under control voltage fluctuations and is typically expressed as a percentage or absolute range around the rated voltage."
# AI Extraction Logic  
- Identify any explicit mention of operating voltage range for the shunt opening release in the technical specification, control circuit description, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific range is stated (e.g., 70% to 110% of 110 V DC), extract it as-is.
- If multiple ranges are allowed or conditionally required, list them using the format:
  - '70–110% / 80–110%'
- If a preferred range is stated and others are acceptable upon approval, use the format:
  - '70–110% (preferred); 80–110% allowed upon approval'
- If no range is stated but a reference to a standard is provided (e.g., "as per IEC 62271-1"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# E-43
# Circuit Breaker
# Rated Supply Voltage of Heating Circuits
# "Specifies the rated voltage required to operate the heating circuits within the circuit breaker enclosure or operating mechanism. These circuits are typically used for anti-condensation or temperature control purposes and are powered from station auxiliary AC or DC sources."
# AI Extraction Logic  
- Identify any explicit mention of rated supply voltage for heating circuits in the technical specification, environmental control section, or auxiliary power requirements for the circuit breaker associated with the validated SwitchgearType.
- If a specific voltage is stated (e.g., 230 V AC), extract it as-is.
- If multiple voltages are allowed or conditionally required, list them using the format:
  - '230 V AC / 110 V AC'
- If a preferred voltage is stated and others are acceptable upon approval, use the format:
  - '230 V AC (preferred); 110 V AC allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.



# F-1
# Disconnector
# Proposed Model
# AI Extraction Logic
- Leave the field blank.

# F-2
# Disconnector
# Inspection Window Requirement
# "Indicates whether the disconnector includes an inspection window or viewing port that allows visual confirmation of the contact position (open/closed). This feature is often required for safety and operational verification, especially in indoor switchgear applications."
# AI Extraction Logic
- Identify any explicit mention of inspection window, viewing port, or visual inspection feature in the technical specification, mechanical design description, or safety requirements for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states that an inspection window is required, extract as:
  - 'Required'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-3
# Disconnector
# Mechanical Endurance Class
# "Specifies the mechanical endurance classification of the disconnector, typically defined by the number of operating cycles (open/close) it can perform without performance degradation. This classification is usually based on international standards such as IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of mechanical endurance class or number of mechanical operating cycles in the technical specification, mechanical design section, or standard compliance requirements for the disconnector associated with the validated SwitchgearType.
- If a specific endurance class is stated (e.g., Class M1 or M2), extract it as-is.
- If the endurance is stated in number of operations (e.g., 2,000 operations), extract it as:
  - '2,000 operations'
- If both class and number of operations are stated, extract both using the format:
  - 'Class M2 (10,000 operations)'
- If the document references a standard (e.g., "as per IEC 62271-102 Class M2"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-4
# Disconnector
# Rated Bus-Transfer Current
# "Specifies the maximum current that the disconnector can carry during bus-transfer operations, where load is momentarily transferred between bus sections without interruption. This rating ensures the disconnector can safely handle the inrush and transient conditions associated with such transfers, typically defined in accordance with IEC 62271-102 or IEEE C37.30 standards."
# AI Extraction Logic
- Identify any explicit mention of rated bus-transfer current in the technical specification, disconnector rating table, or operational requirements for the disconnector associated with the validated SwitchgearType.
- If a specific current value is stated (e.g., 1600 A), extract it as-is.
- If multiple values are allowed or conditionally required, list them using the format:
  - '1600 A / 2000 A'
- If a preferred value is stated and others are acceptable upon approval, use the format:
  - '1600 A (preferred); 2000 A allowed upon approval'
- If the document references a standard (e.g., "as per IEC 62271-102"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-5
# Disconnector
# Rated Bus-Transfer Voltage
# "Specifies the rated voltage that the disconnector can withstand during bus-transfer operations. This value is typically expressed in volts (RMS) and depends on the rated voltage of the installation and whether it is gas-insulated or air-insulated. It ensures safe operation during transient conditions when transferring load between bus sections."
# AI Extraction Logic
- Identify any explicit mention of rated bus-transfer voltage in the technical specification, rating tables, or IEC/IEEE compliance sections for the disconnector associated with the validated SwitchgearType.
- If a specific voltage value is stated (e.g., 10 V RMS), extract it as-is.
- If multiple values are allowed or conditionally required, list them using the format:
  - '10 V / 20 V'
- If a preferred value is stated and others are acceptable upon approval, use the format:
  - '20 V (preferred); 10 V allowed upon approval'
- If the document references a standard (e.g., "as per IEC 62271-102 Table 7"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-6
# Disconnector
# Rated Bus-Charging Current
# "Specifies the maximum current that the disconnector can carry when charging an unloaded busbar. This condition occurs when energizing a bus section without load, and the current is primarily capacitive. The rating ensures safe operation during bus-charging without exceeding thermal or dielectric limits, typically defined in IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of rated bus-charging current in the technical specification, disconnector rating tables, or IEC/IEEE compliance sections for the disconnector associated with the validated SwitchgearType.
- If a specific current value is stated (e.g., 0.1 A), extract it as-is.
- If multiple values are allowed or conditionally required, list them using the format:
  - '0.1 A / 0.5 A'
- If a preferred value is stated and others are acceptable upon approval, use the format:
  - '80 A (preferred); 100 A allowed upon approval'
- If the document references a standard (e.g., "as per IEC 62271-102"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-7
# Disconnector
# Operating Mechanism Type
# "Specifies the type of operating mechanism used for the disconnector, which determines how the device is actuated. Common types include manual (lever or crank), motor-operated, or spring-operated mechanisms. This information is critical for understanding automation capability and maintenance requirements."
# AI Extraction Logic
- Identify any explicit mention of operating mechanism type in the technical specification, mechanical design section, or control requirements for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states the mechanism type (e.g., Motor-operated, Manual, Spring-operated), extract it as-is.
- If multiple types are allowed or conditionally required, list them using the format:
  - 'Motor-operated / Manual'
- If a preferred type is stated and others are acceptable upon approval, use the format:
  - 'Motor-operated (preferred); Manual allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-8
# Disconnector
# Motor Type
# "Specifies the type of motor used in the disconnector operating mechanism when motorized operation is provided. Common types include AC induction motors or DC motors, and the specification may also include details such as voltage rating, power rating, and duty cycle. This information is important for compatibility with control power supply and maintenance planning."
# AI Extraction Logic
AI Extraction Logic
- Identify any explicit mention of motor type in the technical specification, operating mechanism description, or auxiliary power requirements for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states the motor type (e.g., AC motor, DC motor, Induction motor), extract it as-is.
- If multiple types are allowed or conditionally required, list them using the format:
  - 'AC motor / DC motor'
- If a preferred type is stated and others are acceptable upon approval, use the format:
  - 'AC motor (preferred); DC motor allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-9
# Disconnector
# Thermostat Requirement
# "Specifies whether the disconnector’s anti-condensation or space heater circuit includes a thermostat for automatic temperature control. This feature helps maintain optimal internal conditions and prevent moisture accumulation inside the enclosure."
# AI Extraction Logic
- Identify any explicit mention of thermostat requirement in the technical specification, environmental control section, or auxiliary equipment description for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states that a thermostat is required, extract as:
  - 'Required'
- If a thermostat is optional or conditionally required (e.g., only for cold climates or specific enclosure types), use the format:  
  - 'Optional; required for cold climate installations'  
  - 'Optional; required for outdoor GIS'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-10
# Disconnector
# Humidistat Requirement
# "Specifies whether the disconnector’s anti-condensation or space heater circuit includes a humidistat for automatic humidity control. This feature helps prevent moisture accumulation inside the enclosure by activating heaters when humidity exceeds a set threshold."
# AI Extraction Logic
- Identify any explicit mention of humidistat requirement in the technical specification, environmental control section, or auxiliary equipment description for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states that a humidistat is required, extract as:
  - 'Required'
- If a humidistat is optional or conditionally required (e.g., only for outdoor installations or high-humidity zones), use the format:  
  - 'Optional; required for outdoor GIS'  
  - 'Optional; required for coastal installations'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-11
# Disconnector
# Number and Type of Auxiliary Contacts
# "Specifies the quantity and type of auxiliary contacts provided with the disconnector operating mechanism. These contacts are typically used for position indication (open/closed) and interlocking purposes. Types may include NO (Normally Open), NC (Normally Closed), or changeover contacts."
# AI Extraction Logic
- Identify any explicit mention of auxiliary contacts in the technical specification, control circuit description, or operating mechanism details for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states the number and type (e.g., 4 NO + 4 NC), extract it as-is.
- If multiple configurations are allowed or conditionally required, list them using the format:
  - '4 NO + 4 NC / 6 NO + 6 NC'
- If a preferred configuration is stated and others are acceptable upon approval, use the format:
  - '4 NO + 4 NC (preferred); 6 NO + 6 NC allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-12
# Disconnector
# Interlocking Requirements
# "Specifies the mechanical and/or electrical interlocking features required for the disconnector to ensure safe operation. This includes standard interlocks and any special interlocking provisions such as key interlocks or padlock facilities."
# AI Extraction Logic
- Identify any explicit mention of interlocking requirements in the technical specification, safety section, or operating mechanism description for the disconnector associated with the validated SwitchgearType.
- If the document explicitly states mechanical or electrical interlock, extract as:
  - 'Mechanical' or 'Electrical'
- If special interlocks are mentioned (e.g., Key Interlock, Padlock Provision, Electrical Key Exchange), extract them in addition using the format:
  - 'Mechanical + Key Interlock'
  - 'Electrical + Padlock Provision'
- If multiple interlocks are required, list them using the format:
  - 'Mechanical / Electrical / Key Interlock'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# F-13
# Disconnector
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-36

# F-14
# Disconnector
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-37

# F-15
# Disconnector
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-41

# G-1
# Earthing Switch / Maintenance Grounding Switch
# Proposed Model
# AI Extraction Logic
- Leave the field blank.

# G-2
# Earthing Switch / Maintenance Grounding Switch
# Inspection Window Requirement
# "Specifies whether the earthing switch or maintenance grounding switch includes an inspection window or viewing port that allows visual confirmation of the contact position (open/closed). This feature is often required for safety and operational verification, especially in indoor or enclosed switchgear applications."
# AI Extraction Logic
- Identify any explicit mention of inspection window, viewing port, or visual inspection feature in the technical specification, mechanical design description, or safety requirements for the earthing switch (or maintenance grounding switch) associated with the validated SwitchgearType.
- If the document explicitly states that an inspection window is required, extract as:
  - 'Required'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-3
# Earthing Switch / Maintenance Grounding Switch
# Earthing Path Type (Direct vs. Enclosure)
# "Specifies the type of earthing path provided by the earthing switch. The earthing path can be either direct (contact connected directly to ground) or via the enclosure (grounding through the switchgear enclosure). This information is critical for ensuring proper fault current dissipation and compliance with safety standards."
# AI Extraction Logic
- Identify any explicit mention of earthing path type in the technical specification, grounding system description, or IEC/IEEE compliance section for the earthing switch (or maintenance grounding switch) associated with the validated SwitchgearType.
- If the document explicitly states the earthing path type, extract as:
  - 'Direct' or 'Enclosure'
- If both options are allowed or conditionally required, list them using the format:
  - 'Direct / Enclosure'
- If a preferred type is stated and others are acceptable upon approval, use the format:
  - 'Direct (preferred); Enclosure allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-4
# Earthing Switch / Maintenance Grounding Switch
# Mechanical Endurance Class
# "Specifies the mechanical endurance classification of the earthing switch or maintenance grounding switch, typically defined by the number of operating cycles (open/close) it can perform without performance degradation. This classification is usually based on international standards such as IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of mechanical endurance class or number of mechanical operating cycles in the technical specification, mechanical design section, or standard compliance requirements for the earthing switch (or maintenance grounding switch) associated with the validated SwitchgearType.
- Do NOT include endurance values for special types such as:
  - Earthing Switch for Short-Circuit Making
  - High-Speed Grounding Switch
- If a specific endurance class is stated (e.g., Class M1 or M2), extract it as-is.
- If the endurance is stated in number of operations (e.g., 2,000 operations), extract it as:
  - '2,000 operations'
- If both class and number of operations are stated, extract both using the format:
  - 'Class M2 (10,000 operations)'
- If the document references a standard (e.g., "as per IEC 62271-102 Class M2"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-5
# Earthing Switch / Maintenance Grounding Switch
# Electrical Endurance Class
# "Specifies the electrical endurance classification of the earthing switch or maintenance grounding switch, typically defined by the number of electrical operating cycles (under load or fault conditions) it can perform without performance degradation. This classification is usually based on international standards such as IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of electrical endurance class or number of electrical operating cycles in the technical specification, electrical performance section, or standard compliance requirements for the earthing switch (or maintenance grounding switch) associated with the validated SwitchgearType.
- Do NOT include endurance values for special types such as:
  - Earthing Switch for Short-Circuit Making
  - High-Speed Grounding Switch
- If a specific electrical endurance class is stated (e.g., Class E1 or E2), extract it as-is.
- If the endurance is stated in number of operations (e.g., 100 operations under fault), extract it as:
  - '100 operations'
- If both class and number of operations are stated, extract both using the format:
  - 'Class E2 (100 operations)'
- If the document references a standard (e.g., "as per IEC 62271-102 Class E2"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-6
# Earthing Switch / Maintenance Grounding Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used to actuate the earthing switch or maintenance grounding switch. Common types include manual (lever or crank), motor-operated, or spring-operated mechanisms. This information is important for understanding automation capability, safety, and maintenance requirements."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any mention of operating mechanism type for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.  
- Do NOT include operating mechanism type for special types such as:  
  - Earthing Switch for Short-Circuit Making  
  - High-Speed Grounding Switch  
- If the document explicitly states the mechanism type (e.g., Motor-operated, Manual, Spring-operated), extract it as-is.  
- If multiple types are allowed or conditionally required, list them using the format:  
  - `Motor-operated / Manual`  
- If one type is preferred and another allowed upon approval, use the format:  
  - `Motor-operated (preferred); Manual allowed upon approval`  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-7
# Earthing Switch / Maintenance Grounding Switch
# Motor Type
# "Specifies the type of motor used to operate the earthing switch or maintenance grounding switch, if motor-operated. This may include AC motor, DC motor, or specific voltage ratings (e.g., 110 V DC, 220 V AC). This information is important for compatibility with control systems and auxiliary power supplies."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any mention of motor type for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.  
- Extract the motor type as stated (e.g., DC motor, AC motor, 110 V DC, 220 V AC).
- If multiple motor types are allowed or conditionally required, list them using the format:
  - 'DC motor / AC motor'
- If one type is preferred and another allowed upon approval, use the format:
  - 'DC motor (preferred); AC motor allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-8
# Earthing Switch / Maintenance Grounding Switch
# Thermostat Requirement
# "Specifies whether the earthing switch or maintenance grounding switch’s anti-condensation or space heater circuit includes a thermostat for automatic temperature control. This feature helps maintain optimal internal conditions and prevent moisture accumulation inside the enclosure."
# AI Extraction Logic
- Identify any explicit mention of thermostat requirement in the technical specification, environmental control section, or auxiliary equipment description for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.
- If the document explicitly states that a thermostat is required, extract as:
  - `Required`
- If a thermostat is optional or conditionally required (e.g., only for cold climates or specific enclosure types), use the format:  
  - 'Optional; required for cold climate installations'  
  - 'Optional; required for outdoor GIS'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-9
# Earthing Switch / Maintenance Grounding Switch
# Humidistat Requirement
# "Specifies whether the earthing switch or maintenance grounding switch’s anti-condensation or space heater circuit includes a humidistat for automatic humidity control. This feature helps prevent moisture accumulation inside the enclosure by activating heaters when humidity exceeds a set threshold."
# AI Extraction Logic
- Identify any explicit mention of humidistat requirement in the technical specification, environmental control section, or auxiliary equipment description for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.
- If the document explicitly states that a humidistat is required, extract as:
  - `Required`
- If a humidistat is optional or conditionally required (e.g., only for outdoor installations or high-humidity zones), use the format:  
  - 'Optional; required for outdoor GIS'  
  - 'Optional; required for coastal installations'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-10
# Earthing Switch / Maintenance Grounding Switch
# Number and Type of Auxiliary Contacts
# "Specifies the quantity and type of auxiliary contacts provided with the the earthing switch (or maintenance grounding switch) operating mechanism. These contacts are typically used for position indication (open/closed) and interlocking purposes. Types may include NO (Normally Open), NC (Normally Closed), or changeover contacts."
# AI Extraction Logic
- Identify any explicit mention of auxiliary contacts in the technical specification, control circuit description, or operating mechanism details for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.
- If the document explicitly states the number and type (e.g., 4 NO + 4 NC), extract it as-is.
- If multiple configurations are allowed or conditionally required, list them using the format:
  - '4 NO + 4 NC / 6 NO + 6 NC'
- If a preferred configuration is stated and others are acceptable upon approval, use the format:
  - '4 NO + 4 NC (preferred); 6 NO + 6 NC allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-11
# Earthing Switch / Maintenance Grounding Switch
# Interlocking Requirements
# "Specifies the mechanical and/or electrical interlocking features required for the the earthing switch (or maintenance grounding switch) to ensure safe operation. This includes standard interlocks and any special interlocking provisions such as key interlocks or padlock facilities."
# AI Extraction Logic
- Identify any explicit mention of interlocking requirements in the technical specification, safety section, or operating mechanism description for the earthing switch or maintenance grounding switch associated with the validated SwitchgearType.
- If the document explicitly states mechanical or electrical interlock, extract as:
  - 'Mechanical' or 'Electrical'
- If special interlocks are mentioned (e.g., Key Interlock, Padlock Provision, Electrical Key Exchange), extract them in addition using the format:
  - 'Mechanical + Key Interlock'
  - 'Electrical + Padlock Provision'
- If multiple interlocks are required, list them using the format:
  - 'Mechanical / Electrical / Key Interlock'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# G-12
# Earthing Switch / Maintenance Grounding Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-36

# G-13
# Earthing Switch / Maintenance Grounding Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-37

# G-14
# Earthing Switch / Maintenance Grounding Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-41



# H-1
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Proposed Model
# AI Extraction Logic
- Leave the field blank.

# H-2
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Inspection Window Requirement
# "Specifies whether the earthing switch for short-circuit making or high speed grounding switch includes an inspection window or viewing port that allows visual confirmation of the contact position (open/closed). This feature is often required for safety and operational verification, especially in indoor or enclosed switchgear applications."
# AI Extraction Logic
- Identify any explicit mention of inspection window, viewing port, or visual inspection feature in the technical specification, mechanical design description, or safety requirements for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.
- If the document explicitly states that an inspection window is required, extract as:
  - 'Required'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-3
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Earthing Path Type (Direct vs. Enclosure)
# "Specifies the type of earthing path provided by the earthing switch. The earthing path can be either direct (contact connected directly to ground) or via the enclosure (grounding through the switchgear enclosure). This information is critical for ensuring proper fault current dissipation and compliance with safety standards."
# AI Extraction Logic
- Identify any explicit mention of earthing path type in the technical specification, grounding system description, or IEC/IEEE compliance section for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.
- If the document explicitly states the earthing path type, extract as:
  - 'Direct' or 'Enclosure'
- If both options are allowed or conditionally required, list them using the format:
  - 'Direct / Enclosure'
- If a preferred type is stated and others are acceptable upon approval, use the format:
  - 'Direct (preferred); Enclosure allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-4
# Earthing Switch for Short-Circuit Making / High-Speed Grounding Switch
# Rated Short-Circuit Making Current
# "Specifies the peak asymmetrical current that the earthing switch must safely make and withstand during a short-circuit condition. This rating reflects the switch’s ability to close onto a fault and endure the resulting electrodynamic forces without mechanical damage. Typically defined in IEC 62271-102 or equivalent IEEE standards."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any explicit mention of Rated Short-Circuit Making Current for the earthing switch for short-circuit making or high-speed grounding switch associated with the validated SwitchgearType
- Extract the value as stated (e.g., 63 kA, 80 kA, 100 kA).
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-5
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Mechanical Endurance Class
# "Specifies the mechanical endurance classification of the earthing switch for short-circuit making or high speed grounding switch, typically defined by the number of operating cycles (open/close) it can perform without performance degradation. This classification is usually based on international standards such as IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of mechanical endurance class or number of mechanical operating cycles in the technical specification, mechanical design section, or standard compliance requirements for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.
- If a specific endurance class is stated (e.g., Class M1 or M2), extract it as-is.
- If the endurance is stated in number of operations (e.g., 2,000 operations), extract it as:
  - '2,000 operations'
- If both class and number of operations are stated, extract both using the format:
  - 'Class M2 (10,000 operations)'
- If the document references a standard (e.g., "as per IEC 62271-102 Class M2"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-6
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Electrical Endurance Class
# "Specifies the electrical endurance classification of the earthing switch for short-circuit making or high speed grounding switch, typically defined by the number of electrical operating cycles (under load or fault conditions) it can perform without performance degradation. This classification is usually based on international standards such as IEC 62271-102."
# AI Extraction Logic
- Identify any explicit mention of electrical endurance class or number of electrical operating cycles in the technical specification, electrical performance section, or standard compliance requirements for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.
- If a specific electrical endurance class is stated (e.g., Class E1 or E2), extract it as-is.
- If the endurance is stated in number of operations (e.g., 100 operations under fault), extract it as:
  - '100 operations'
- If both class and number of operations are stated, extract both using the format:
  - 'Class E2 (100 operations)'
- If the document references a standard (e.g., "as per IEC 62271-102 Class E2"), extract it as-is.
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-7
# Earthing Switch for Short-Circuit Making / High-Speed Grounding Switch
# Rated Induced Current
# "Specifies the maximum induced current that the earthing switch must withstand under normal operating conditions, such as proximity to energized conductors or during switching transients. This value is important for insulation coordination and control circuit protection."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any explicit mention of Rated Induced Current for the validated SwitchgearType.
- Extract the value as stated (e.g., 5 A, 10 A).
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-8
# Earthing Switch for Short-Circuit Making / High-Speed Grounding Switch
# Rated Induced Voltage
# "Specifies the maximum induced voltage that the earthing switch must withstand under normal operating conditions, such as proximity to energized conductors or during switching transients. This value is important for insulation coordination and control circuit protection."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any explicit mention of Rated Induced Voltage for the validated SwitchgearType.
- Extract the value as stated (e.g., 2 kV, 2.5 kV).
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-9
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Operating Mechanism Type
# "Specifies the type of operating mechanism used to actuate the earthing switch for short-circuit making or high speed grounding switch. Common types include manual (lever or crank), motor-operated, or spring-operated mechanisms. This information is important for understanding automation capability, safety, and maintenance requirements."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any mention of operating mechanism type for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.  
- If the document explicitly states the mechanism type (e.g., Motor-operated, Manual, Spring-operated), extract it as-is.  
- If multiple types are allowed or conditionally required, list them using the format:  
  - `Spring-operated / Manual`  
- If one type is preferred and another allowed upon approval, use the format:  
  - `Motor-operated (preferred); Manual allowed upon approval`  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-10
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Motor Type
# "Specifies the type of motor used to operate the earthing switch for short-circuit making or high speed grounding switch. This may include AC motor, DC motor, or specific voltage ratings (e.g., 110 V DC, 220 V AC). This information is important for compatibility with control systems and auxiliary power supplies."
# AI Extraction Logic
- Search the project specification, technical datasheet, or general specification (if applicable) for any mention of motor type for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.   
- Extract the motor type as stated (e.g., DC motor, AC motor, 110 V DC, 220 V AC).
- If multiple motor types are allowed or conditionally required, list them using the format:
  - 'DC motor / AC motor'
- If one type is preferred and another allowed upon approval, use the format:
  - 'DC motor (preferred); AC motor allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-11
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Thermostat Requirement
# "Specifies whether the earthing switch for short-circuit making or high speed grounding switch’s anti-condensation or space heater circuit includes a thermostat for automatic temperature control. This feature helps maintain optimal internal conditions and prevent moisture accumulation inside the enclosure."
# AI Extraction Logic
- Identify any explicit mention of thermostat requirement in the technical specification, environmental control section, or auxiliary equipment description for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType.   
- If the document explicitly states that a thermostat is required, extract as:
  - `Required`
- If a thermostat is optional or conditionally required (e.g., only for cold climates or specific enclosure types), use the format:  
  - 'Optional; required for cold climate installations'  
  - 'Optional; required for outdoor GIS'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-12
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Humidistat Requirement
# "Specifies whether the earthing switch for short-circuit making or high speed grounding switch’s anti-condensation or space heater circuit includes a humidistat for automatic humidity control. This feature helps prevent moisture accumulation inside the enclosure by activating heaters when humidity exceeds a set threshold."
# AI Extraction Logic
- Identify any explicit mention of humidistat requirement in the technical specification, environmental control section, or auxiliary equipment description for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType. 
- If the document explicitly states that a humidistat is required, extract as:
  - `Required`
- If a humidistat is optional or conditionally required (e.g., only for outdoor installations or high-humidity zones), use the format:  
  - 'Optional; required for outdoor GIS'  
  - 'Optional; required for coastal installations'  
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-13
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Number and Type of Auxiliary Contacts
# "Specifies the quantity and type of auxiliary contacts provided with the earthing switch for short-circuit making or high speed grounding switch's operating mechanism. These contacts are typically used for position indication (open/closed) and interlocking purposes. Types may include NO (Normally Open), NC (Normally Closed), or changeover contacts."
# AI Extraction Logic
- Identify any explicit mention of auxiliary contacts in the technical specification, control circuit description, or operating mechanism details for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType. 
- If the document explicitly states the number and type (e.g., 4 NO + 4 NC), extract it as-is.
- If multiple configurations are allowed or conditionally required, list them using the format:
  - '4 NO + 4 NC / 6 NO + 6 NC'
- If a preferred configuration is stated and others are acceptable upon approval, use the format:
  - '4 NO + 4 NC (preferred); 6 NO + 6 NC allowed upon approval'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-14
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Interlocking Requirements
# "Specifies the mechanical and/or electrical interlocking features required for the earthing switch for short-circuit making or high speed grounding switch to ensure safe operation. This includes standard interlocks and any special interlocking provisions such as key interlocks or padlock facilities."
# AI Extraction Logic
- Identify any explicit mention of interlocking requirements in the technical specification, safety section, or operating mechanism description for the earthing switch for short-circuit making (or high speed grounding switch) associated with the validated SwitchgearType. 
- If the document explicitly states mechanical or electrical interlock, extract as:
  - 'Mechanical' or 'Electrical'
- If special interlocks are mentioned (e.g., Key Interlock, Padlock Provision, Electrical Key Exchange), extract them in addition using the format:
  - 'Mechanical + Key Interlock'
  - 'Electrical + Padlock Provision'
- If multiple interlocks are required, list them using the format:
  - 'Mechanical / Electrical / Key Interlock'
- Apply GIS-HGIS interpretation rule as defined in the common extraction rules.

# H-15
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Rated Supply Voltage of Control Circuits
# Enter the extracted value as Serial Number E-36

# H-16
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Operating Voltage Range of Control Circuits
# Enter the extracted value as Serial Number E-37

# H-17
# Earthing Switch for Short-Circuit Making / High Speed Grounding Switch
# Rated Supply Voltage of Heating Circuits
# Enter the extracted value as Serial Number E-41

""", "version": 251127})

True

In [5]:
# 업데이트([description_prompt_I_U)
manager.update_entry("description_prompt_I_U", {"text": """
You will be provided with the following : ##GIVEN TEXT##

GIVEN TEXT's FORMAT :
# Serial Number
# Category
# Specification Item
# Description
# AI Extraction Logic(If you use values for other serial numbers, the AI extraction logic is omitted.)

# I-1
# Voltage Transformer
# Type
# "Specifies the construction and sensing technology of the voltage transformer (VT) used in high-voltage switchgear. Accepted types include Inductive VT, Capacitive VT (CVT), Optical VT, and Low-Power Instrument Transformer (LPIT), Non-Conventional Instrument Transformer (NCIT)."
# AI Extraction Logic
- Identify any explicit mention of VT type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - Inductive VT → `Inductive`
  - Capacitive VT or CVT → `Capacitive`
  - Optical VT → `Optical`
  - 'Resistive Voltage Divider'
  - 'Electro-Optic Voltage Sensor'
  - Low-Power Instrument Transformer → `LPIT`
  - Non-Conventional Instrument Transformer → `NCIT`
- If VT is specified as mounted inside GIS (e.g., “GIS-integrated,” “SF₆ gas insulated,” “in GIS enclosure”), add flag: `GIS-integrated` after the type.
  - Example outputs:
    - `Inductive; GIS-integrated`
    - `Inductive / Capacitive; GIS-integrated`
- If multiple types are allowed, output as: `Inductive / Capacitive`
- If one type is preferred and another conditionally allowed, output as: `LPIT (preferred); Inductive allowed upon approval`

# I-2
# Voltage Transformer
# Phase Configuration
# "Specifies whether the project requires Voltage Transformers (VTs or PTs) to be single-phase, three-phase, or permits both configurations. This distinction affects enclosure design, wiring, metering accuracy, and redundancy strategy in high-voltage switchgear systems."
# AI Extraction Logic
- Identify any explicit mention of VT phase configuration in the project-binding specification applicable to the validated SwitchgearType.
- Accepted configurations:
  - `Single-phase`
  - `Three-phase`
  - If both are allowed: `Single-phase / Three-phase`
- If one configuration is preferred and another conditionally allowed, output as:
  - `Single-phase (preferred); Three-phase allowed upon approval`
- Ensure the statement refers specifically to Voltage Transformers (VTs/PTs), not CTs or general phase references.

# I-3
# Voltage Transformer
# Rated Voltage Factor
# "Specifies the ratio between the maximum permissible operating voltage and the rated voltage of the voltage transformer (VT). This factor determines the VT’s capability to withstand temporary overvoltages without exceeding thermal or dielectric limits. Typical values include 1.2 (continuous), 1.5 (short-duration), and 1.9 (short-duration)."
# AI Extraction Logic
- Identify any explicit mention of Rated Voltage Factor in the project-binding specification applicable to the validated SwitchgearType.
- Accepted formats:
  - Continuous operation: 1.2
  - Short-duration operation: 1.5, 1.9 (include duration if stated, e.g., 1.5 for 30 s)
  - Long-duration operation: 1.9 (include duration if stated, e.g., 1.5 for 8 h)
- If multiple values are provided (continuous and short-duration), output as:
  - Continuous: 1.2; Short-duration: 1.9 for 30 s
  - If only one value is stated, extract that value.

# I-4
# Voltage Transformer
# Anti Ferro-Resonance Requirement
# "Specifies whether the voltage transformer (VT or PT) must include design measures to suppress ferroresonance phenomena, which can occur due to non-linear behavior under low-load or no-load conditions in high-voltage systems. Ferroresonance can lead to sustained overvoltages and equipment damage; therefore, some specifications explicitly require anti-ferroresonance features such as suppression resistors, ferroresonance suppressors, or design-based mitigation."
# AI Extraction Logic
- Identify any explicit mention of anti-ferroresonance requirement in the project-binding specification applicable to the validated SwitchgearType.
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`
- If specific features are mentioned, append detail:
  - `Required; ferroresonance suppressor`
  - `Required; suppression resistor`

# I-5
# Voltage Transformer
# Isolating Device Requirement
# "Specifies whether the specification requires a Voltage Transformer (VT or PT) to be equipped with an isolating device or disconnecting switch to facilitate HV system isolation, dielectric testing, or safe maintenance. Such isolating devices may be either integrated (e.g., integral isolating switch operated from GIS exterior) or separate (e.g., dedicated disconnector) and typically include features like positive position indication and locking mechanism."**
# AI Extraction Logic
- Identify any explicit mention of isolating device or disconnecting switch for VT/PT in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords: `isolating device`, `disconnecting switch`, `separate disconnector`, `means to isolate`, `facility for dielectric test`, `positive position indication`.
- If isolating device is required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required for dielectric testing`
  - `Optional; required for maintenance`
- If type is specified, append detail:
  - `Required; Integral isolating device`
  - `Required; Separate disconnector`

# I-6
# Voltage Transformer
# Manufacturer / Country of Origin
# "Specifies the manufacturer (vendor) and the country of origin of the voltage transformer applied in high-voltage switchgear. This information is typically stated to ensure supplier qualification, regulatory compliance, or manufacturing traceability. Only manufacturer names and countries explicitly listed in the technical specification or data schedule shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of VT manufacturer and country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `Siemens`, `Hitachi`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `Siemens / Nissin`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `Siemens (preferred); Nissin allowed upon approval`

# J-1
# Current Transformer
# Type
# "Specifies the type of current transformer (CT) used in high-voltage switchgear. The CT type determines measurement accuracy, insulation characteristics, and interface compatibility with protection and metering systems. Only CT types explicitly stated or clearly implied through standard references or construction terminology shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of CT type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - `Ring Core` (or Toroidal Core)
  - `Optical CT` (or Fiber Optic Current Sensor, FOCS)
  - `Rogowski Coil`
  - `Low Power CT` (or LPCT)
  - `Digital CT` (only if clearly linked to optical or LPCT structure)
- If multiple types are allowed, output as:
  - `Ring Core / Optical CT`
- If one type is preferred and another conditionally allowed, output as:
  - `Ring Core (preferred); Optical CT allowed upon approval`

# J-2
# Current Transformer
# CT Location Relative to Enclosure
# "Specifies whether Current Transformers (CTs) are installed outside the GIS/HGIS enclosure or outside the insulating medium zone for high-voltage switchgear. Extract only explicit external-location statements. Do not infer or emit “Internal”; when no explicit external statement exists, leave the field blank. Ignore electrical sizing and secondary wiring/termination content."
# AI Extraction Logic
- Identify any explicit statement that CTs are installed outside the GIS/HGIS enclosure or outside the insulating medium zone.
- Accepted keywords:
  - `installed outside enclosure`
  - `mounted external to GIS enclosure`
  - `located outside SF₆ zone`
- If external installation is explicitly prohibited, output as: `External prohibited`.
- If external installation is allowed or conditional, output as:
  - `External allowed; [verbatim qualifier]`
- If external installation is required, output as: `External to enclosure`.
- Do not infer location from:
  - Electrical sizing (ratio, accuracy, burden, knee point)
  - Wiring or termination details
  - Bushing-related phrases (e.g., “through bushing”)

# J-3
# Current Transformer
# Insulation Class
# "Specifies the insulation class of the Current Transformer (CT) used in high-voltage switchgear, including GIS, HGIS, and DTB systems. Insulation class (e.g., Class E, Class F) defines the CT’s thermal endurance level and determines material specifications based on the ambient and internal temperature rise. If different insulation classes are specified based on CT location (e.g., inside or outside SF₆ gas zone, or based on GIS/HGIS distinction), these shall be noted accordingly."
# AI Extraction Logic
- Identify any explicit mention of CT insulation class in the project-binding specification applicable to the validated SwitchgearType.
- Accepted values:
  - `Class E`
  - `Class F`
  - Other IEC/IEEE recognized classes (e.g., Class B, Class H)
- If different classes are specified for GIS vs. HGIS or based on CT location, output both:
  - Example: `Class E for GIS; Class F for HGIS`

# J-4
# Current Transformer
# Continuous Thermal Current Rating Factor
# "Specifies the multiplier applied to the CT’s rated current to define its continuous thermal capability under normal operating conditions. Typical values include 1.0, 1.2, and 1.5."
# AI Extraction Logic
- Identify any explicit mention of continuous thermal current rating factor in the project-binding specification applicable to the validated SwitchgearType.
- Accepted formats:
  - `1.0`
  - `1.2`
  - `1.5`
- If multiple factors are stated for different CT locations or configurations, output both:
  - Example: `1.2 for GIS; 1.5 for HGIS`

# J-5
# Current Transformer
# Current Transformer Protection Unit
# "Indicates whether the Current Transformer Protection Unit (CTPU) is required for high-voltage switchgear applications. The CTPU provides open-circuit protection for CT secondary circuits. Only requirements explicitly stated in the specification or clearly implied through mandatory inclusion shall be extracted."
# AI Extraction Logic
- Identify any explicit mention of CTPU or CT open-circuit protection in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `CTPU`
  - `CT open circuit protection`
  - `CT protection unit`
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`

# J-6
# Current Transformer
# Manufacturer / Country of Origin
# "Specifies the designated manufacturer and/or country of origin for Current Transformers (CTs) used in high-voltage switchgear. This information may reflect customer preferences, approved vendor lists, or procurement restrictions aimed at ensuring product quality, local content compliance, or geopolitical risk mitigation."
# AI Extraction Logic
- Identify any explicit mention of CT manufacturer and/or country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `GE`, `Trench`, `Ritz`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `GE / Trench`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `GE (preferred); Trench allowed upon approval`
- If country restrictions or preferences are stated, append detail:
  - `GE / Germany`
  - `Trench / Any country except China`

# K-1
# Surge Arrester
# Type
# "Specifies the required type of surge arrester for high-voltage switchgear. This includes the arrester's configuration (e.g., free-standing, GIS-housed) and its internal design characteristics (e.g., metal-oxide without gaps). Only explicitly stated types or clearly referenced standards (e.g., IEC 60099-4, IEEE C62.11) shall be considered."
# AI Extraction Logic
- Identify any explicit mention of surge arrester type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted types:
  - `Metal-oxide surge arrester (gapless)`
  - `Free-standing outdoor surge arrester`
  - `GIS-housed surge arrester`
- Normalize variations such as `gap-less`, `gapless`, `without gap` → `gapless`.
- If the specification references standards that define arrester types:
  - IEC 60099-4 → `Metal-oxide surge arrester (gapless) (as per IEC 60099-4)`
  - IEEE C62.11 → `Metal-oxide surge arrester (gapless) (as per IEEE C62.11)`

# K-2
# Surge Arrester
# Designation
# "Specifies the Line Discharge Class of the Surge Arrester applied to high-voltage switchgear (GIS, DTB, HGIS, or SF₆ Gas-Free systems). Line Discharge Class defines the energy absorption capability of the arrester and is typically designated by IEC 60099-4 as SL (low), SM (medium), or SH (high). IEEE-based specifications may instead describe discharge current values or energy ratings without using SL/SM/SH terms."
# AI Extraction Logic
- Identify any explicit mention of Line Discharge Class or equivalent designation in the project-binding specification applicable to the validated SwitchgearType.
- If IEC 60099-4 is referenced, accepted values:
  - `SL` (Station Low)
  - `SM` (Station Medium)
  - `SH` (Station High)
- If IEEE C62.11 is referenced, accepted values:
  - 'Station'
  - 'Intermediate'

# K-3
# Surge Arrester
# Rated Voltage
# "Specifies the rated voltage of the surge arrester, defined as the maximum RMS power frequency voltage the arrester can withstand for a limited time without failure. In IEC 60099-4, this is denoted as 'Rated Voltage' and typically expressed in kV RMS. In IEEE C62.11, it is also referred to as 'Arrester Rated Voltage' and has the same definition. This value is used to coordinate the surge arrester’s operation with the insulation level and system temporary overvoltage performance."
# AI Extraction Logic
- Identify any explicit mention of surge arrester rated voltage in the project-binding specification applicable to the validated SwitchgearType.
- Extract the numerical value and unit (e.g., `96 kV`, `144 kV`), ensuring it refers specifically to the surge arrester.

# K-4
# Surge Arrester
# Continuous Operating Voltage
# "Specifies the maximum RMS voltage that may be continuously applied across the terminals of the surge arrester without degrading its performance. This value is commonly referred to as the Maximum Continuous Operating Voltage (MCOV) in IEEE C62.11 and as Continuous Operating Voltage (Uc) in IEC 60099-4. The value is essential for ensuring coordination with system voltage levels and arresters' temporary overvoltage withstand capability."
# AI Extraction Logic
- Identify any explicit mention of Continuous Operating Voltage (IEC term: `Uc`) or Maximum Continuous Operating Voltage (IEEE term: `MCOV`) in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Continuous Operating Voltage`
  - `Uc`
  - `MCOV`
  - `Maximum Continuous Operating Voltage`
- Extract the numerical value and unit (e.g., `10.2 kV`, `144 kV`), ensuring it refers specifically to the surge arrester.

# K-5
# Surge Arrester
# Nominal Discharge Current
# "Specifies the nominal discharge current (In) of the surge arrester, which is the peak value of lightning impulse current (8/20 μs waveform) used to classify the arrester’s energy-handling capability. According to IEC 60099-4, this value is typically 5 kA or 10 kA for distribution and transmission class arresters, and may reach 20 kA for high-energy designs. IEEE C62.11 also defines nominal discharge current for station- and intermediate-class arresters, commonly 10 kA or 5 kA respectively."
# AI Extraction Logic
- Identify any explicit mention of nominal discharge current (In) in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Nominal discharge current`
  - `In`
  - `Discharge current rating`
- Extract the numerical value and unit (e.g., `5 kA`, `10 kA`, `20 kA`), ensuring it refers to lightning impulse current (8/20 μs waveform).

# K-6
# Surge Arrester
# Isolating Device Requirement
# "Specifies whether the surge arrester requires an isolating device or disconnecting feature to allow safe disconnection during system dielectric tests. This may include plug-in construction, disconnecting link types, or other means that enable mechanical isolation of the arrester from the system. The requirement is typically stated for SF₆ gas-insulated arresters but may also apply to other types depending on installation conditions."
# AI Extraction Logic
- Identify any explicit mention of isolating or disconnecting device for surge arresters in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `plug-in construction`
  - `disconnecting link`
  - `arrester isolation`
  - `removal for dielectric test`
  - `test disconnection device`
- If isolating device is required, output as: `Required`.
- If multiple isolating device options are allowed, output all using slash format:
  - Example: `Plug-in construction / Disconnecting link`
- If one option is preferred and another conditionally allowed, output as:
  - `Plug-in construction (preferred); Disconnecting link allowed upon approval`

# K-7
# Surge Arrester
# Leakage Current Monitor Requirement
# "Specifies whether the surge arrester shall be equipped with a leakage current monitor (LCM) for condition monitoring purposes. The LCM typically measures the resistive or total leakage current to ground and may be specified with a particular measurement range (e.g., 0–50 mA), sensitivity, or display format (e.g., analog, digital)."
# AI Extraction Logic
- Identify any explicit mention of leakage current monitor (LCM) or leakage current measurement for surge arresters in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `leakage current monitor`
  - `LCM`
  - `leakage current measurement`
  - `leakage detection device`
- If required, output as: `Required`.
- If optional or conditionally required, output as:
  - `Optional; required under specific conditions`

# K-8
# Surge Arrester
# Manufacturer / Country of Origin
# "Specifies the required manufacturer and/or country of origin for Surge Arresters used in high-voltage switchgear. Such requirements may reflect customer preferences, quality assurance policies, supply chain localization goals, or trade restrictions."
# AI Extraction Logic
- Identify any explicit mention of surge arrester manufacturer and/or country of origin in the project-binding specification applicable to the validated SwitchgearType.
- Extract manufacturer name (e.g., `Siemens`, `Hitachi`) and country of origin (e.g., `Germany`, `Japan`).
- If multiple manufacturers are listed, output as:
  - `Siemens / Hitachi`
- If one manufacturer is preferred and others are conditionally allowed, output as:
  - `Siemens (preferred); Hitachi allowed upon approval`
- If country restrictions or preferences are stated, append detail:
  - `Siemens / Germany`
  - `Hitachi / Any country except China`

# L-1
# Air Bushing
# Insulating Bushing – Type
# "Specifies the required external insulation type of the gas-to-air bushing used in high-voltage switchgear. Extract only if explicitly stated for the validated SwitchgearType. Common types include Porcelain, Composite (e.g., silicone rubber), Polymer, and RIP (Resin-Impregnated Paper). If exclusions are mentioned (e.g., 'Porcelain not acceptable'), capture them as well."
# AI Extraction Logic
- Identify any explicit mention of air bushing insulation type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Porcelain`
  - `Composite`
  - `Polymer`
  - `Silicone rubber`
  - `Resin-Impregnated Paper (RIP)`
- Output rules:
  - If multiple types are allowed → "Porcelain / Composite"
  - If one option is preferred and another conditionally allowed → "Composite (preferred); Porcelain allowed upon approval"
  - If specific type is not allowed → "Prohibited: Porcelain"
  - If location-specific (e.g., transformer-side, cable-side), append context:
    - Example: "Composite (transformer-side)"

# L-2
# Air Bushing
# Insulating Bushing – Creepage
# "Specifies the required minimum creepage distance of the gas-to-air insulating bushing used in high-voltage switchgear. Creepage distance is the shortest path along the surface of the insulating material between two conductive parts, typically between live conductor and ground. It is often specified in mm or mm/kV and may reflect environmental conditions such as pollution level, altitude, or installation type. Extract only if explicitly stated for the validated SwitchgearType or calculable from provided pollution severity or insulation class."
# AI Extraction Logic
- Identify any explicit mention of creepage distance in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Creepage distance`
  - `Minimum creepage`
  - `Surface distance`
  - `mm/kV`
  - `mm per kV`
- Output rules:
  - If specific value given → "2750 mm" or "31 mm/kV"
  - If linked to pollution severity → "Based on Heavy Pollution – 31 mm/kV"
  - If exclusion or special condition → "Not less than 31 mm/kV"
  - If location-specific (e.g., transformer-side, cable-side), append context:
    - Example: "2750 mm (transformer-side)"

# L-3
# Air Bushing
# Insulating Bushing - Color
# # "Specifies the required external color of the gas-to-air insulating bushing used in high-voltage switchgear. The color is typically specified for standardization, visual inspection, UV resistance, or coordination with other substation equipment. Common options include brown, light gray, and dark gray. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of bushing color in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Bushing color`
  - `Insulator color`
  - `Porcelain color`
  - `Silicone rubber color`
  - `Housing color`
  - Color codes (e.g., `RAL 7035`)
- Output rules:
  - If specific color stated → "Brown", "Light gray", "Dark gray", or "RAL 7035"
  - If multiple colors allowed → "Brown / Light gray"
  - If one color preferred and another conditionally allowed → "Light gray (preferred); Brown allowed upon approval"
  - If exclusion → "Prohibited: Black"
  - If location-specific → append context:
    - Example: "Light gray (transformer-side)"

# L-4
# Air Bushing
# Insulating Bushing – Terminal Hole Requirement
# "Specifies any customer-specified requirements for the terminal hole or terminal pad of the gas-to-air insulating bushing in high-voltage switchgear. These may include standard flange specifications such as NEMA 2-hole or 4-hole pad, or specific bolt dimensions and drilling patterns. Such specifications are intended to ensure mechanical compatibility with site equipment. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of terminal hole or pad requirements in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Terminal hole`
  - `Bolt hole`
  - `Pad type`
  - `Drilling pattern`
  - `NEMA 2-hole`
  - `NEMA 4-hole`
  - `IEC flange`
  - `M16 bolt spacing`
- Output rules:
  - If standard flange type stated → "NEMA 2-hole" or "NEMA 4-hole"
  - If described by dimensions → "4-hole terminal pad" or "2 x M12 bolt hole"
  - If multiple options allowed → "NEMA 2-hole / NEMA 4-hole"
  - If exclusion → "Prohibited: NEMA 2-hole"
  - If location-specific → append context:
    - Example: "NEMA 4-hole (transformer-side)"

# M-1
# Cable Connection
# Cable Termination – Type
# "Specifies the required type of cable termination for high-voltage switchgear, typically classified by its insulation and sealing structure. Common types include dry type, fluid-filled (oil or compound), and plug-in or separable connector types. The requirement is often based on installation conditions such as indoor/outdoor use, fire protection, or compatibility with specific cable designs. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of cable termination type in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Cable termination type`
  - `Termination method`
  - `Terminating system`
  - `Dry type`
  - `Fluid-filled`
  - `Oil-filled`
  - `Plug-in termination`
  - `Separable connector`
  - `Stress cone`
  - `Heat-shrink termination`
- Output rules:
  - If only one type stated → "Dry type" or "Plug-in type"
  - If multiple types allowed → "Dry type / Plug-in type"
  - If one type preferred and another conditionally allowed → "Dry type (preferred); Plug-in allowed upon approval"
  - If exclusion → "Prohibited: Fluid-filled"
  - If location-specific → append context:
    - Example: "Dry type (indoor)"

# M-2
# Cable Connection
# Cable Termination – DC Withstand Test Requirement
# "Specifies whether the customer requires a DC dielectric (withstand) test to be performed on the cable termination or cable interface of the high-voltage switchgear. This test is typically required for verifying insulation integrity after installation or as part of routine field commissioning. Extract only if explicitly stated for the validated SwitchgearType."
# AI Extraction Logic
Identify any explicit mention of DC dielectric (withstand) test requirement in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `DC test`
  - `DC withstand test`
  - `Direct current voltage test`
- Output rules:
  - If required → "DC test required for cable termination"
  - If specific details provided → append context:
    - Example: "DC test required for cable termination (80 kV for 15 min)"

# M-3
# Cable Connection
# Cable Termination – Socket Supply Responsibility
# "Specifies whether the customer requires the high-voltage switchgear supplier to provide insular collar (socket-type) components for cable termination. This includes plug-in or separable connector collars that form part of the cable interface system. Extract if the specification explicitly assigns supply responsibility to the high-voltage switchgear manufacturer or if FAT conditions imply high-voltage switchgear must deliver with collar assembled."
# AI Extraction Logic
- Identify any explicit mention that insular collar (socket-type termination components) shall be supplied by the high-voltage switchgear supplier in the project-binding specification applicable to the validated SwitchgearType.
- Also check for indirect requirements via FAT conditions (e.g., cable head conductor assembled during GIS routine test).
- Accepted keywords:
  - Insular collar
  - Socket to be supplied
  - Socket-type termination to be provided
  - GIS supplier shall supply socket
  - Socket included in GIS scope
  - Cable head assembled during GIS factory test
- Output rules:
  - If high-voltage switchgear supplier required to supply → "Supplier to supply insular collar"
  - If optional or conditionally required → "Optional; Supplier to supply upon request"
  - If implied by FAT → "Implied: Collar assembled for FAT"`

# M-4
# Cable Connection
# Cable Termination – Dummy Plug Supply Responsibility
# "Specifies whether the customer explicitly requires unused cable termination ports or bushings on the high-voltage switchgear to be sealed using dummy plugs, blind flanges, or equivalent specific sealing methods, rather than simply requiring sealing for gas containment. Extract only if the use of dummy plugs or similar hardware is clearly stated for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of sealing unused or spare cable terminations using specific hardware in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - `Dummy plug`
  - `Dummy bushing`
  - `Blind plug`
  - `Blanking flange`
  - `Spare termination shall be plugged`
- Output rules:
  - If dummy plug required → "Dummy plug required"
  - If blind flange required → "Blind flange required"
  - If multiple options allowed → "Dummy plug / Blind flange"

# M-5
# Cable Connection
# Cable Termination – Non-Linear Resistor Supply Responsibility
# "Specifies whether the customer requires the high-voltage switchgear supplier to provide non-linear resistors (NLR) as part of the cable termination assembly. NLRs are typically used for grading or controlling transient voltages at the cable-GIS interface. Extract if the specification explicitly assigns supply responsibility to the high-voltage switchgear manufacturer or if interface compliance (IEC 62271-209) implies inclusion in high-voltage switchgear scope."
# AI Extraction Logic
- Identify any explicit mention that non-linear resistors shall be supplied by the high-voltage switchgear supplier in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Non-linear resistor
  - NLR to be supplied
  - GIS supplier shall supply NLR
  - NLR included in GIS scope
  - Grading resistor
  - Voltage grading device
- Output rules:
  - If high-voltage switchgear supplier required → "Supplier to supply NLR"
  - If optional or conditionally required → "Optional; Supplier to supply upon request"

# N-1
# Direct Connection
# Insulated Junction Requirement
# "Specifies whether the customer requires an insulated junction (also called insulated joint or insulated spacer) when making a direct cable-to-GIS connection without a conventional cable termination. This component ensures electrical insulation and mechanical integrity at the interface. Extract only if the specification explicitly mandates the use of an insulated junction for the validated SwitchgearType."
# AI Extraction Logic
- Identify any explicit mention of insulated junction/joint requirement in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Insulated junction
  - Insulated joint
  - Direct connection with insulated joint
  - GIS direct cable connection requires insulated junction
- Output rules:
  - If required → "Insulated junction required for direct connection"
  - If optional or conditionally required → "Optional; insulated junction upon request"

# N-2
# Direct Connection
# Non-Linear Resistor Requirement
# "Specifies whether the customer requires non-linear resistors (NLR) to be installed when making a direct cable-to-GIS connection without a conventional cable termination. NLRs are typically used for voltage grading and transient control at the direct interface. Extract only if the specification explicitly mandates NLR for direct connection or if compliance with IEC 62271-209 implies its inclusion."
# AI Extraction Logic
- Identify any explicit mention of non-linear resistor requirement for direct cable-to-GIS connection in the project-binding specification applicable to the validated SwitchgearType.
- Accepted keywords:
  - Non-linear resistor
  - NLR required for direct connection
  - Voltage grading resistor
  - Grading device for direct interface
  - GIS direct cable connection requires NLR
- Output rules:
  - If required → "NLR required for direct connection"
  - If optional or conditionally required → "Optional; NLR upon request"
  - If implied by standard → "Implied: NLR included for interface compliance"

# O-1
# Installation Requirements
# Embedded Materials for Foundations — Supply Responsibility & Scope
# "Specifies whether the high-voltage switchgear supplier is responsible for supplying embedded materials required for foundations and defines the scope of such materials. Embedded materials typically include items such as embedded plates, anchor bolts, inserts, and other foundation-related components necessary for installation of GIS, GIB, or associated equipment."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of supply responsibility for embedded materials in sections related to site installation, foundation preparation, or civil interface requirements.
- Accepted keywords:
  - embedded plate
  - anchor bolt
  - foundation insert
  - embedded materials
  - civil interface
  - supply responsibility
- Output Rules
  - If high-voltage switchgear supplier must supply all embedded materials → "Supplier responsible for embedded plates and anchor bolts"
  - If high-voltage switchgear supplier supplies only specific items → List items explicitly, e.g., "Supplier responsible for anchor bolts only"
  - If responsibility lies with customer or civil contractor → "Not supplier's scope"

# O-2
# Installation Requirements
# Cable Duct Type Requirement
# "Specifies whether the customer requires or prefers a specific cable duct type integrated with GIS or GIB bodies for routing outgoing control cables (e.g., CT, VT, LCC cables). Common descriptions include 'perforated duct', 'solid duct', or 'mesh duct'."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of cable duct type in sections related to GIS body accessories, cable routing, or installation details.
- Accepted keywords:
  - cable duct
  - duct type
  - perforated duct
  - solid duct
  - mesh duct
- Output rules:
  - If only one type allowed → "Perforated Duct" (or the exact type mentioned).
  - If multiple types acceptable → List all allowed types, e.g., "Perforated / Solid / Mesh Duct".
  - If preferred type and acceptable types are mentioned → "Perforated Duct (Solid acceptable)".

# O-3
# Installation Requirements
# Gas Piping Allowance for Gas Monitoring
# "Specifies whether the customer requires or prohibits external gas piping connections for GIS or GIB."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of gas piping allowance or prohibition for monitoring systems in sections related to gas monitoring, accessories, or interface provisions.
- Accepted keywords:
  - gas piping
  - external piping
  - monitoring pipe
  - no external piping
  - eye level, remote readable, accessible for operator
- Output rules:
  - If explicitly allowed → "Allowed".
  - If explicitly required (e.g., “pressure gauge/density monitor shall be at eye level” or “remote readable”) → "Required".
  - If prohibited (including phrases like “direct mounting required” or “no external piping”) → "Not Allowed".

# O-4
# Installation Requirements
# Platform Type Requirement
# "Specifies whether the customer requires or prefers a specific type of access platform for maintenance or operation of GIS or GIB equipment. Access platforms may include fixed galleries, stairways, walkways, or movable platforms. Extract only if explicitly mentioned as required, preferred, or allowed in the specification."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of access platform type in sections related to GIS maintenance, operation, or installation.
- Accepted keywords:
  - fixed platform
  - movable platform
  - gallery
  - stairway
  - walkway
  - removable platform
  - transportable platform
- Output Rules
  - If only fixed type allowed → "Fixed type only"
  - If only movable type allowed → "Movable type only"
  - If both acceptable → "Fixed or movable type acceptable"

# O-5
# Installation Requirements
# Sun Shade Requirement
# "Specifies whether the customer requires a sun shade (protective roof or cover) to be installed over outdoor high-voltage switchgear or gas-insulated bus ducts (GIB). Sun shades are typically intended to reduce thermal stress, protect personnel from heat, and extend equipment life. The requirement may apply to GIS, DTB, HGIS, GIB, or outdoor-mounted control panels. Extract only if explicitly required, conditionally specified, or optional in the specification."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of sun shade or protective cover in sections related to outdoor GIS, GIB, DTB, HGIS, or local control panels.
- Accepted keywords:
  - Sun shade
  - Protective cover
  - Roof cover
  - UV shielding
  - Solar protection
  - Canopy over GIS
  - Heat shield over switchgear
- Output Rules
  - If explicitly required → "Required"
  - If requirement is conditional (e.g., based on ambient temperature or solar exposure) → "Conditionally required"
  - If mentioned but not mandatory → "Optional"

# P-1
# Surface Treatment & Painting
# Enclosure Painting Policy
# "Specifies whether the specification prohibits painting of the internal or external surface of the enclosure or metallic compartment of high-voltage switchgear. This typically applies to gas compartments or metallic inner walls (internal) and external surfaces where painting may be restricted by project requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of painting prohibition for internal or external surfaces in sections related to surface treatment, painting, or gas compartment requirements.
- Accepted keywords:
  - internal surface shall not be painted
  - no painting on the inside of the enclosure
  - painting inside gas compartments is not allowed
  - do not paint internal metallic surfaces
  - prohibit internal painting
  - painting to be limited to external surfaces only
  - external surface shall not be painted
  - no painting on the outside of the enclosure
- Output Rules:
  - If internal painting is prohibited → "Internal painting not allowed"
  - If external painting is prohibited → "External painting not allowed"
  - If both prohibited → "Internal and external painting not allowed"

# P-2
# Surface Treatment & Painting
# Enclosure Color
# "Specifies the required color for the exterior surface of the high-voltage switchgear enclosure. The color may be defined by a specific code (e.g., RAL 7032, ANSI 70 Gray), customer-defined naming (e.g., “light gray”, “desert tan”), or referred to project standard specifications."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of enclosure color in sections related to painting, finishing, or surface treatment of HV switchgear.
- Accepted keywords:
  - color of enclosure
  - paint color shall be
  - RAL
  - ANSI color code
  - finish: [color]
  - shade: light gray / grey / desert tan / beige / olive green
- Output Rules
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7032").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light gray").

# P-3
# Surface Treatment & Painting
# Color of Partition
# "Specifies whether the customer requires or prefers a specific paint color for partition components used in high-voltage switchgear, such as gas-tight support insulators or gas barrier insulators. These components are part of internal compartments in GIS or hybrid switchgear and are often referred to as gas barrier insulators, partition insulators, or sectionalizing bushings."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of color requirement for partition components in sections related to internal GIS compartments or partitioning insulators.
- Accepted keywords:
  - partition color
  - gas barrier insulator color
  - sectionalizing bushing color
  - RAL
  - light grey
  - neutral tone
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light grey").

# P-4
# Surface Treatment & Painting
# Mechanism Color
# "Specifies the required color for the exterior surface of the high-voltage circuit breaker mechanism housing (e.g., operating mechanism box). The color may be defined by a standard code (e.g., RAL, ANSI), a descriptive name (e.g., “dark gray”, “silver gray”)."
# AI Extraction Logic:
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of mechanism housing color in sections related to painting, finishing, or surface treatment of operating mechanism boxes.
- Accepted keywords:
  - mechanism box color
  - operating mechanism enclosure finish/color
  - paint color for mechanism
  - mechanism housing shall be painted
  - CB mechanism color
  - RAL
  - ANSI color code
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Silver gray").

# P-5
# Surface Treatment & Painting
# Paint Thickness of Enclosure
# "Specifies whether the customer defines the required total paint thickness for the main enclosure of high-voltage switchgear. The enclosure refers to the external housing of GIS, DTB, or HGIS. The paint system may be described as a total dry film thickness (DFT) or as individual layers (e.g., primer + top coat). The total thickness is often specified in micrometers (μm) and may vary depending on installation area (e.g., outdoor, coastal)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness for the main enclosure in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - paint thickness
  - DFT
  - dry film thickness
  - primer + top coat
  - minimum microns
  - μm
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "100 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 40 μm + Top coat 60 μm" → "100 μm").
  - If a range is specified → extract as range (e.g., "80–120 μm").

# P-6
# Surface Treatment & Painting
# Paint Thickness of Mechanism Box
# "Specifies whether the customer defines the required paint thickness or alternative surface treatment for the mechanism box of high-voltage switchgear. The mechanism box houses operating components such as spring or hydraulic mechanisms and is typically mounted outside the main GIS, DTB, or HGIS enclosure. The specification may define a total dry film thickness (DFT), a layer-wise coating system, or an alternative treatment (e.g., zinc plating, galvanization) instead of painting."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness or surface treatment for the mechanism box (or operating mechanism housing) in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - mechanism box
  - operating mechanism housing
  - control box (if contextually linked to mechanical parts)
  - paint thickness
  - DFT
  - dry film thickness
  - primer + top coat
  - minimum microns
  - μm
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "80 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 30 μm + Top coat 50 μm" → "80 μm").
  - If a range is specified → extract as range (e.g., "70–100 μm").

# P-7
# Surface Treatment & Painting
# Surface Preparation Standard
# "Specifies whether the customer defines the required surface preparation standard before painting (e.g., SSPC-SP 5, SSPC-SP 10, SSPC-SP 8). This refers to the cleaning and preparation of steel surfaces prior to coating, which affects corrosion protection and paint adhesion."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of surface preparation standards in sections related to painting, finishing, or corrosion protection.
- Accepted keywords:
  - SSPC-SP 5
  - SSPC-SP 10
  - SSPC-SP 8
  - White Metal Blast Cleaning
  - Near White Blast Cleaning
  - Pickling
- Output Rules:
  - If a single standard is specified → extract as text (e.g., "SSPC-SP 5 White Metal Blast Cleaning").
  - If multiple standards are mentioned → list all in order (e.g., "SSPC-SP 5, SSPC-SP 10").

# P-8
# Surface Treatment & Painting
# Surface Treatment and Thickness of Support Structure
# "Specifies whether the customer defines the required surface treatment and/or paint thickness for the support structure of high-voltage switchgear. The support structure refers to steel frames or stands that hold GIS, DTB, or HGIS equipment. The specification may require hot-dip galvanizing, painting with a defined dry film thickness (DFT), or a duplex system (galvanizing + painting) for corrosion protection."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of:
  - Hot-dip galvanizing (including zinc coating thickness)
  - Painting system and DFT
  - Duplex system (galvanizing + painting)
- Accepted keywords:
  - support structure
  - steel frame
  - hot-dip galvanizing
  - zinc coating
  - duplex system
  - paint thickness
  - DFT
  - μm
- Output Rules:
  - If unconditional → extract treatment and thickness (e.g., "Hot-dip galvanizing + 100 μm painting").
  - If conditional → extract treatment and thickness with condition in parentheses:
    - Example: Hot-dip galvanizing (Duplex system required if within 25 km from coast: + 80 μm painting)

# P-9
# Surface Treatment & Painting
# Duplex System Requirement
# "Specifies whether the customer requires a duplex system (hot-dip galvanizing + painting) for corrosion protection and under what conditions. Duplex systems are typically applied in highly corrosive environments such as coastal areas, industrial zones, or as per ISO 12944 corrosion category (e.g., C5-M)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the specification.
- Identify any condition or environment that triggers duplex system requirement:
  - Coastal installation
  - High salinity or marine environment
  - ISO 12944 category (C4, C5-M)
- Accepted keywords:
  - duplex system
  - galvanizing + painting
  - coastal
  - marine
  - ISO 12944
  - corrosion category
- Output Rules:
  - If duplex system is required → extract as text (e.g., "Duplex system required").
  - If condition is specified → include condition (e.g., "Required for coastal installation (within 25 km)").

# Q-1
# Local Control Cubicle
# Installation Area
# "Specifies where the Local Control Cubicle (LCC) for high-voltage switchgear shall be installed according to the customer specification. This includes whether the LCC is installed outdoors, inside an outdoor container, in a GIS room, or near outdoor GIS equipment. The requirement may also define environmental conditions or physical constraints for installation."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Installation location type:
    - Outdoor (e.g., “LCC shall be installed outdoors”)
    - Outdoor container (e.g., “LCC inside container”)
    - GIS room (e.g., “LCC in GIS building”)
    - Near outdoor GIS (e.g., “LCC adjacent to GIS yard”)
- Accepted keywords:
  - outdoor installation
  - container
  - GIS room
  - GIS building
  - near GIS
- Output Rules:
  - If location clearly specified → 
    - "Installation area: Outdoor",
    - "Installation area: GIS room",
    - "Installation area: Outdoor container"
  - If multiple options allowed → list all:
    - "Installation area: Outdoor or GIS room"

# Q-2
# Local Control Cubicle
# Installation Type
# "Specifies whether the customer defines the installation type of the Local Control Cubicle (LCC) for high-voltage switchgear. The LCC may be freestanding (separate from the GIS) or integrated (attached or built into the GIS structure). The specification may state one option explicitly or allow both. This requirement affects panel design, wiring layout, and enclosure mounting."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of installation type for LCC in sections related to control panels or cubicles.
- Accepted keywords:
  - Local Control Cubicle
  - LCC
  - Local Control Panel
  - freestanding
  - integrated
  - attached to GIS
  - separate panel
- Output Rules:
  - If only freestanding is mentioned → extract: Freestanding
  - If only integrated is mentioned → extract: Integrated
  - If both are allowed → extract: Freestanding or Integrated

# Q-3
# Local Control Cubicle
# Protection Degree
# "Specifies whether the customer defines the required protection degree (e.g., IP rating) for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. The protection degree may be defined based on IEC 60529 (e.g., IP41, IP54) and may vary depending on the switchgear type or installation location, such as indoor GIS or outdoor HGIS. The specification may assign fixed ratings or define minimum values per installation condition."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of protection degree for LCC in sections related to enclosure protection or environmental requirements.
- Accepted keywords:
  - IP rating
  - protection degree
  - IEC 60529
  - IP41, IP54, IP55, etc.
- Output Rules:
  - If one rating applies to all conditions → extract as single value (e.g., "IP54").
  - If different ratings apply for GIS vs HGIS → extract both with conditions:
    - Example: IP41 (GIS), IP54 (HGIS)
  - If minimum rating is specified → extract as "Minimum IP54".

# Q-4
# Local Control Cubicle
# Enclosure Material
# "Specifies whether the customer defines the required enclosure material for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. The enclosure material may be mild steel, stainless steel, or aluminum, and the choice often depends on environmental conditions (e.g., indoor vs outdoor, corrosive areas) and protection requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of enclosure material for LCC in sections related to cabinet construction or corrosion protection.
- Accepted keywords:
  - enclosure material
  - mild steel
  - stainless steel
  - aluminum
  - galvanized steel
  - non-ferrous material
- Output Rules:
  - If one material is specified → extract as text (e.g., "Stainless steel").
  - If multiple options are allowed → extract all (e.g., "Mild steel or stainless steel").

# Q-5
# Local Control Cubicle
# Duplex System Requirement
# "Specifies whether the customer requires a duplex system (hot-dip galvanizing + painting) for the Local Control Cubicle (LCC) and under what conditions. Duplex systems are typically applied in highly corrosive environments such as coastal areas, industrial zones, or as per ISO 12944 corrosion category (e.g., C5-M)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of duplex system requirement for LCC and any associated condition:
  - Coastal installation
  - High salinity or marine environment
  - ISO 12944 category (C4, C5-M)
- Accepted keywords:
  - duplex system
  - galvanizing + painting
  - coastal
  - marine
  - ISO 12944
  - corrosion category
- Output Rules:
  - If duplex system is required unconditionally → extract as text (e.g., "Duplex system required").
  - If condition is specified → include condition in parentheses:
    - Example: Duplex system required (for coastal installation within 25 km)

# Q-6
# Local Control Cubicle
# External Color
# "Specifies the required external color of the Local Control Cubicle (LCC) used with high-voltage switchgear, including any RAL code, ANSI designation, or general color description such as light grey or beige. Only requirements clearly associated with LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of external color for Local Control Cubicle (LCC), Local Control Cabinet, Local Control Panel, or High-voltage Control Cabinet.
- Accepted keywords:
  - Local Control Cubicle color
  - Local Control Cabinet color
  - Local Control Panel color
  - High-voltage Control Cabinet color
  - RAL
  - ANSI color code
  - light grey, beige
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "Light grey").

# Q-7
# Local Control Cubicle
# Internal Color
# "Specifies the required internal color of the Local Control Cubicle (LCC) used with high-voltage switchgear, including any RAL code, ANSI designation, or general color description such as white or light grey. Only requirements clearly associated with the interior of LCCs for high-voltage applications shall be extracted."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of internal color for Local Control Cubicle (LCC), Local Control Cabinet, Local Control Panel, or High-voltage Control Cabinet.
- Accepted keywords:
  - internal color of LCC
  - inside of local control cabinet
  - interior shall be painted
  - RAL
  - white, light grey
- Output Rules:
  - If a standard color code is mentioned → extract the code (e.g., "RAL 7035").
  - If only a color name is mentioned → extract the descriptive term (e.g., "White").

# Q-8
# Local Control Cubicle
# Paint Thickness
# "Specifies whether the customer defines the required paint thickness for the Local Control Cubicle (LCC) used in high-voltage switchgear installations. LCC refers to the external panel or cabinet that contains protection and control devices. The paint system may be defined by a total dry film thickness (DFT), a layer-wise structure, or a specific coating method (e.g., powder coating, epoxy paint). In some cases, the specification may refer to manufacturer’s standard or require matching the main switchgear enclosure."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of paint thickness or coating method for LCC in sections related to painting or finishing.
- Accepted keywords:
  - paint thickness
  - DFT
  - dry film thickness
  - powder coating
  - epoxy paint
  - μm
  - same as enclosure
- Output Rules:
  - If a single total thickness is specified → extract as numeric value with unit (e.g., "80 μm").
  - If layer-wise thickness is given → sum and extract total (e.g., "Primer 30 μm + Top coat 50 μm" → "80 μm").
  - If only coating method is specified → extract method (e.g., "Powder coating").
  - If reference to another item (e.g., "same as enclosure") → extract as "Same as enclosure".

# Q-9
# Local Control Cubicle
# Inclusion of BCU / IED in LCC
# "Specifies whether the customer requires the Bay Control Unit (BCU) or Intelligent Electronic Device (IED) to be installed inside the Local Control Cubicle (LCC) of high-voltage switchgear. The requirement may be stated directly (e.g., BCU shall be mounted in LCC) or indirectly (e.g., LCC to include IEC 61850-compliant IEDs). Inclusion of BCU or IED affects wiring, communication interface, and functional integration of the switchgear control system."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of BCU or IED inclusion in LCC in sections related to control system or cubicle configuration.
- Accepted keywords:
  - BCU
  - Bay Control Unit
  - IED
  - Intelligent Electronic Device
  - IEC 61850
  - installed in LCC
  - housed inside control cabinet
- Output Rules:
  - If explicitly included → extract: Included
  - If explicitly excluded (e.g., “by others” or “to be installed elsewhere”) → extract: Not included
  - If ambiguous but implied (e.g., “BCU to be connected to LCC terminal blocks”) → extract: Possibly included

# Q-10
# Local Control Cubicle
# Protection Devices in LCC
# "Specifies whether the customer requires protection-related devices to be installed inside the Local Control Cubicle (LCC) of high-voltage switchgear. These devices may include Pole Discrepancy Switch, Trip Coil Supervision Relay, Lockout Relay, Protection Unit, or other auxiliary protection components. Inclusion of these devices affects wiring, space allocation, and functional integration of the control system."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of protection devices in LCC in sections related to protection, control, or auxiliary equipment.
- Accepted keywords:
  - Pole Discrepancy Switch
  - Trip Coil Supervision Relay
  - Lockout Relay
  - Bay Protection Unit / BPU
  - Fuse Failure Relay
  - Auxiliary protection device
- Output Rules:
  - If one or more devices are specified → extract all as a comma-separated list (e.g., "Pole Discrepancy Switch, Trip Coil Supervision Relay").

# Q-11
# Local Control Cubicle
# Control Cable Connection Type
# "Specifies whether the customer defines the required method for connecting control cables to the Local Control Cubicle (LCC) of high-voltage switchgear. The requirement may include the cable entry direction (bottom, top, side) and the connection method (gland type or plug-in type). This affects cubicle design, gland plate arrangement, sealing, and cable routing."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of:
  - Cable entry direction (bottom, top, side)
  - Connection method (gland type, plug-in type, jack type)
- Accepted keywords:
  - bottom entry
  - top entry
  - side entry
  - cable gland
  - gland plate
  - plug-in connection
  - pre-fabricated cable
  - connector type
  - jack type (rare, but include for completeness)
- Output Rules:
  - If only entry type is specified → extract as text (e.g., "Bottom entry").
  - If only connection method is specified → extract as text (e.g., "Gland type" or "Plug-in type").
  - If both are specified → extract combined (e.g., "Bottom entry, Gland type").

# Q-12
# Local Control Cubicle
# Voltage Detector Requirement
# "Specifies whether the customer requires a voltage detector to be installed in the Local Control Cubicle (LCC) of high-voltage switchgear. A voltage detector provides indication of live status for safety and maintenance purposes. The specification may require a specific type (e.g., capacitive voltage indicator, neon lamp type) or compliance with standards (e.g., IEC 61243)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify any explicit mention of voltage detector requirement in sections related to safety devices or LCC equipment.
- Accepted keywords:
  - voltage detector
  - voltage indicator
  - capacitive voltage indicator
  - neon lamp type
  - IEC 61243
- Output Rules:
  - If voltage detector is required → extract as "Voltage detector required" (include type if specified, e.g., "Capacitive voltage indicator").
  - If explicitly excluded → extract "Not required".

# Q-13
# Local Control Cubicle
# Key Interlock System Requirement
# "Specifies whether the customer requires a key interlock system to be provided in the Local Control Cubicle (LCC) for high-voltage switchgear, and identifies the interlock logic and associated devices. A key interlock system ensures operational safety by mechanically or electrically preventing incorrect switching sequences. The specification may define the number of keys, interlock logic, and the devices involved (e.g., Line Earthing Switch, Disconnector, Circuit Breaker, Capacitor Bank)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for key interlock system
  - Devices involved in interlock (e.g., ES, DS, CB, Capacitor Bank)
  - Logic or sequence (e.g., key released only when ES closed)
- Accepted keywords:
  - key interlock
  - mechanical interlock
  - electrical interlock
  - interlock between ES and DS
  - interlock between CB and Capacitor Bank
  - interlock logic
  - sequence
- Output Rules:
  - If interlock required → extract "Key interlock system required"
  - If devices and logic specified → extract full detail:
    - Key interlock system required (between Line ES and DS; logic: key released when ES closed)
  - If multiple interlocks → list all in parentheses:
    - Key interlock system required (ES–DS; CB–Capacitor Bank)

# Q-14
# Local Control Cubicle
# Manufacturer / Country of Origin
# "Specifies whether the customer requires a specific manufacturer or country of origin for the Local Control Cubicle (LCC) used in high-voltage switchgear. This may include explicit naming of a vendor, restriction to domestic or international suppliers, or requirements for factory origin or sourcing. The specification may also indicate whether the LCC shall be supplied by the switchgear manufacturer or by a designated third-party vendor."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer name (e.g., ABB, Siemens, Hyundai Electric)
  - Country of origin (e.g., Korea, Japan, EU)
  - Nationality requirement (e.g., local vendor only, domestic manufacturing)
  - Supply condition (e.g., LCC shall be supplied by switchgear manufacturer)
- Accepted keywords:
  - manufacturer
  - vendor
  - supplier
  - country of origin
  - domestic
  - local vendor
  - approved manufacturer
  - supplied by switchgear manufacturer
- Output Rules:
  - If manufacturer specified → extract as:
    - "Manufacturer: [Name]"
    - (e.g., Manufacturer: Hyundai Electric)
  - If country of origin specified → extract as:
    - "Country of origin: [Country]"
    - (e.g., Country of origin: Korea)
  - If requirement is local/domestic only → extract as:
    - "Local vendor only"

# R-1
# Control Cable
# Insulation Type
# "Specifies whether the customer requires a specific insulation material for control or power cables used in high-voltage switchgear. Valid insulation materials include PVC (polyvinyl chloride), XLPE (cross-linked polyethylene), and EPR (ethylene propylene rubber), or may be referenced via standard cable codes (e.g., CVV, CCV)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Insulation material (e.g., PVC, XLPE, EPR)
  - Cable type codes (e.g., CVV, CCV, CVVS) and map to insulation type if possible
- Accepted keywords:
  - PVC
  - XLPE
  - EPR
  - CVV
  - CCV
  - CVVS
- Output Rules:
  - If one or more insulation materials specified → extract as list:
    - "Insulation type: PVC, XLPE"
  - If only cable codes provided → extract codes and map if possible:
    - "Insulation type: CVV (PVC), CCV (XLPE)"

# R-2
# Control Cable
# Steel Armored Shield Requirement
# "Specifies whether the customer requires control cables used in high-voltage switchgear to include a steel armored shield. This may be specified to enhance mechanical protection, improve EMI shielding, or comply with installation conditions such as underground routing or outdoor exposure. The requirement may apply to all control cables, or only to specific portions (e.g., external, underground, or field-installed cables)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for steel armor (e.g., steel wire armored, SWA)
  - Conditional requirement (e.g., only for underground or outdoor cables)
- Accepted keywords:
  - steel armored
  - steel wire armored (SWA)
  - armored control cable
  - metallic sheath
  - mechanical protection
- Output Rules:
  - If steel armor required → "Steel armored shield: Required"
  - If conditionally required → "Steel armored shield: Conditionally required (e.g., underground cables)"

# R-3
# Control Cable
# Jacket Material
# "Specifies whether the customer requires a specific jacket (outer sheath) material for control or power cables used in high-voltage switchgear systems. The jacket provides mechanical protection, environmental resistance, and may contribute to flame retardancy or chemical resistance. Common jacket materials include PVC, PE, neoprene, LSZH, and hypalon."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Jacket material (e.g., PVC, PE, neoprene, LSZH, hypalon)
  - Multiple options if listed
- Accepted keywords:
  - PVC
  - PE
  - neoprene
  - LSZH
  - hypalon
  - outer sheath
  - cable jacket
- Output Rules:
  - If one or more jacket materials specified → extract as list:
    - "Jacket material: PVC, LSZH"
  - If multiple options allowed → list all:
    - "Jacket material: PVC or neoprene"

# R-4
# Control Cable
# Halogen-Free Requirement
# "Specifies whether the customer requires control or power cables used in high-voltage switchgear to be halogen-free (also referred to as LSZH: Low Smoke Zero Halogen). This requirement is often applied to reduce toxic gas emission in case of fire and to comply with environmental or safety standards. It may apply to all cable types or only to specific installations (e.g., indoor, tunnel, control room)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for halogen-free or LSZH cables
  - Conditional requirement (e.g., only for indoor or tunnel installations)
- Accepted keywords:
  - halogen-free
  - LSZH
  - low smoke zero halogen
  - zero halogen material
  - non-toxic under fire
- Output Rules:
  - If halogen-free required → "Halogen-free: Required"
  - If conditionally required → "Halogen-free: Conditionally required (e.g., indoor cables only)"

# R-5
# Control Cable
# Anti-Termite Requirement
# "Specifies whether the customer requires control or power cables used in high-voltage switchgear systems to be resistant to termites or other insects. Anti-termite protection is typically applied through the use of treated PVC or special sheath materials and is often required for underground or outdoor installations in regions with termite risk."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement for anti-termite protection
  - Conditional requirement (e.g., only for underground or outdoor cables)
- Accepted keywords:
  - anti-termite
  - termite-proof cable
  - termite resistant
  - PVC treated against insects
- Output Rules:
  - If anti-termite required → "Anti-termite: Required"
  - If conditionally required → "Anti-termite: Conditionally required (e.g., underground cables)"

# S-1
# Monitoring System
# Manufacturer-Specified Monitoring System
# "Specifies whether the customer requires or permits a specific manufacturer’s monitoring system to be applied to the high-voltage switchgear. It includes cases where the name of a vendor, brand, or model of the monitoring system is explicitly stated or referenced. The requirement may appear in the main technical specification, appendix, or in approved vendor list (AVL)-related documents. Only extract the manufacturer name and system type if clearly mentioned."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer or brand name (e.g., Qualitrol, Omicron, GE, ABB, Siemens)
  - Monitoring system type (e.g., PDMS, gas monitoring, CB monitoring)
- Accepted keywords:
  - monitoring system
  - vendor name
  - approved vendor
  - preferred monitoring system
  - brand-specific monitoring
- Output Rules:
  - If clearly required → "Required: [Manufacturer Name] ([System Type])"
    (e.g., Required: Qualitrol (Gas Monitoring))
  - If only permitted or preferred → "Preferred: [Manufacturer Name] ([System Type])"

# S-2
# Monitoring System
# Partial Discharge (PD) Sensor Provision
# "Specifies whether the customer requires PD sensors to be installed on the high-voltage switchgear or prepared for future connection. The requirement may refer to UHF sensors, couplers, or test ports for PD monitoring. The condition may be stated as mandatory, optional, or not required."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Quantity or scope:
    - Every compartment
    - Each bay
    - Specific number per bay (e.g., 1 per bay)
    - Specific section (e.g., cable termination)
- Accepted keywords:
    - every compartment
    - each gas section
    - each bay
    - PD sensor per [unit]
    - minimum [X] sensors per bay
- Output Rules:
    - If requirement is general → "PD sensor placement: Every compartment"
    - If per bay → "PD sensor placement: Each bay"
    - If numeric → "PD sensor placement: [X] sensors per bay"
    - If specific section → "PD sensor placement: Cable termination section"

# S-3
# Monitoring System
# PD Sensor Quantity / Placement
# "Specifies whether the customer defines the required quantity or installation scope of PD sensors for high-voltage switchgear. This includes requirements such as sensors in every gas compartment, each bay, or specific sections like cable terminations."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Quantity or scope:
    - Every compartment
    - Each bay
    - Specific number per bay (e.g., 1 per bay)
    - Specific section (e.g., cable termination)
- Accepted keywords:
  - every compartment
  - each gas section
  - each bay
  - PD sensor per [unit]
  - minimum [X] sensors per bay
- Output Rules:
  - If requirement is general → **"PD sensor placement: Every compartment"**
  - If per bay → **"PD sensor placement: Each bay"**
  - If numeric → **"PD sensor placement: [X] sensors per bay"**
  - If specific section → **"PD sensor placement: Cable termination section"**

# S-4
# Monitoring System
# Online PD Monitoring System
# "Specifies whether the customer requires an online Partial Discharge (PD) monitoring system to be installed on the high-voltage switchgear. This includes continuous monitoring solutions integrated into GIS or AIS equipment, typically involving UHF sensors, couplers, and real-time diagnostic software. The requirement may be mandatory, optional, or not required."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “online PD monitoring system shall be provided”)
    - Optional / provision only (e.g., “interface for online PD monitoring”)
    - Not required (e.g., “online PD monitoring not required”)
- Accepted keywords:
  - online PD monitoring
  - continuous PD monitoring
  - PDMS (Partial Discharge Monitoring System)
  - UHF monitoring system
  - real-time PD monitoring
- Output Rules:
  - If required → "Online PD monitoring: Required"
  - If provision only → "Online PD monitoring: Provision only (interface or preparation)"
  - If explicitly excluded → "Online PD monitoring: Not required"

# S-5
# Monitoring System
# Portable PD Monitoring Device
# "Specifies whether the customer requires a portable Partial Discharge (PD) monitoring device to be supplied for use with high-voltage switchgear. This includes handheld or portable PD analyzers for offline or on-site diagnostics. The requirement may be mandatory, optional, or not required, and may also specify the type or brand of the device."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “portable PD device shall be supplied”)
    - Optional (e.g., “may be provided upon request”)
  - Device details if specified (e.g., brand, model)
- Accepted keywords:
  - portable PD monitoring
  - handheld PD analyzer
  - PD test kit
  - offline PD measurement device
- Output Rules:
  - If required → "Portable PD device: Required"
  - If optional → "Portable PD device: Optional"
  - If explicitly excluded → "Portable PD device: Not required"
  - If brand/model mentioned → append in parentheses (e.g., "Portable PD device: Required (Omicron MPD600)")

# S-6
# Monitoring System
# VFTO Capacitive Sensor Provision
# "Specifies whether the customer requires VFTO (Very Fast Transient Overvoltage) capacitive sensors to be installed or provisioned on the high-voltage switchgear for monitoring purposes. VFTO sensors are typically used in GIS applications to detect transient overvoltages during switching operations. The requirement may be mandatory, optional, or not required, and may also specify the installation scope (e.g., per bay, per compartment)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “VFTO sensor shall be provided”)
    - Provision only (e.g., “interface for VFTO sensor”)
  - Scope if specified (e.g., per bay, per compartment)
- Accepted keywords:
  - VFTO sensor
  - VFTO capacitive sensor
  - VFTO monitoring
  - VFTO detection
  - transient overvoltage sensor
- Output Rules:
  - If required → "VFTO sensor: Required"
  - If provision only → "VFTO sensor: Provision only (interface or preparation)"
  - If scope mentioned → append in parentheses (e.g., "VFTO sensor: Required (per bay)")

# S-7
# Monitoring System
# Circuit Breaker Condition Monitoring System
# "Specifies whether the customer requires a circuit breaker (CB) condition monitoring system for high-voltage switchgear. The system continuously monitors breaker performance indicators such as operation count, operating time, contact wear, travel curve profile, stored energy mechanism status, spring charging motor behavior, or insulation condition. It is typically used for predictive maintenance and lifetime assessment."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “CB condition monitoring system shall be provided”)
    - Provision only (e.g., “interface for CB diagnostics shall be included”)
Ignore:
  - Basic breaker operations (open/close) without diagnostics
  - Protection relays or trip signals unless explicitly linked to condition monitoring
- Accepted keywords:
  - CB condition monitoring
  - breaker diagnostics
  - CB health monitoring
  - lifetime monitoring
  - mechanical wear monitoring
  - travel time analysis
  - contact wear detection
- Output Rules:
  - If required → "CB condition monitoring: Required"
  - If provision only → "CB condition monitoring: Provision only"

# S-8
# Monitoring System
# Arc Detection System
# "Specifies whether the customer requires an arc detection system to be installed in the high-voltage switchgear. This includes optical arc detection using light sensors or other technologies capable of detecting internal arc faults and initiating fast breaker operation or protection signaling. Valid implementations may include fiber-optic sensors, light-based detectors, or any system explicitly linked to fault trip logic."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Arc detection system shall be provided”)
    - Provision only (e.g., “Provision for future arc detection system shall be included”)
- Accepted keywords:
  - arc detection system
  - internal arc detection
  - optical arc sensor
  - light-based arc detection
  - fiber optic arc sensor
  - arc flash sensor
  - light-triggered trip
- Output Rules:
  - If required → "Arc detection system: Required"
  - If provision only → "Arc detection system: Provision only"

# S-9
# Monitoring System
# Camera-Based Position Verification System for DS/ES
# "Specifies whether the customer requires a camera-based system or other advanced visual verification method to monitor the open/close position of disconnector (DS) or earthing switch (ES). Standard auxiliary switches or mechanical indicator windows shall NOT be considered valid monitoring systems for this item."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (camera-based or sensor-based)
    - Provision only (interface or preparation)
  - Monitoring method:
    - Camera-based
    - Sensor-based (e.g., proximity sensor, optical sensor)
- Ignore:
  - Any mention of auxiliary contacts or mechanical indicator windows (basic functionality)
- Accepted keywords:
  - camera monitoring for DS/ES
  - optical position sensor
  - proximity sensor for DS/ES
  - position verification system
  - DS/ES position feedback to SCADA (only if camera/sensor-based)
- Output Rules:
  - If required → "DS/ES position monitoring: Required (camera-based)" or "Required (sensor-based)"
  - If provision only → "DS/ES position monitoring: Provision only"

# S-10
# Monitoring System
# Surge Arrester Leakage Current Monitoring System
# "Specifies whether the customer requires a monitoring system for the leakage current of surge arresters (lightning arresters) in high-voltage switchgear. Acceptable systems may include analog gauges, local indicators, or digital sensors. Only extract a requirement if it is explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (continuous monitoring with SCADA or trending analysis)
    - Required (periodic/manual monitoring via analog gauge or local indicator)
    - Provision only (interface or preparation for future monitoring)
- Accepted keywords:
  - leakage current monitoring
  - surge arrester monitoring
  - lightning arrester current detection
  - LA leakage current sensor
  - online leakage current monitor
  - resistive current component monitoring
- Output Rules:
  - If continuous monitoring required → "Surge arrester monitoring: Required (continuous)"
  - If periodic/manual monitoring required → "Surge arrester monitoring: Required (manual)"
  - If provision only → "Surge arrester monitoring: Provision only"

# S-11
# Monitoring System
# Online Gas Monitoring System
# "Specifies whether the customer requires an online gas monitoring system for high-voltage switchgear. Online systems continuously monitor and transmit gas-related parameters such as SF₆ pressure, temperature, density, humidity, or decomposition byproducts in real-time using digital communication protocols. Mechanical devices like local gauges or two-stage relays are NOT considered online monitoring unless explicitly stated as remote or SCADA-integrated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (real-time or SCADA-integrated system)
    - Provision only (interface or preparation for future system)
  - Parameters if specified (e.g., pressure, temperature, density, humidity, dew point, decomposition gases)
- Ignore:
  - Local-only gauges or mechanical relays without remote access
  - Mentions of “continuous monitoring” without SCADA or remote communication
- Accepted keywords:
  - online gas monitoring
  - real-time SF₆ monitoring
  - SCADA-integrated gas monitoring
  - gas analyzer
  - dew point sensor
  - decomposition gas detection
- Output Rules:
  - If required → "Online gas monitoring: Required - [parameters]"
    - (e.g., Online gas monitoring: Required - pressure, temperature, humidity)
  - If provision only → "Online gas monitoring: Provision only - [parameters]"

# S-12
# Monitoring System
# Gas Density Transmitter
# "Specifies whether the customer requires a gas density transmitter (digital type) for high-voltage switchgear. Unlike mechanical gas density monitors or relays, transmitters provide real-time measurement and remote communication of SF₆ gas parameters (pressure, temperature, density) to SCADA or monitoring systems. Only extract if explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Gas density transmitter shall be provided”)
    - Provision only (e.g., “Interface for transmitter shall be included”)
  - Parameters if specified (e.g., pressure, temperature, density)
- Ignore:
  - References to mechanical gas density monitors or relays without remote communication
- Accepted keywords:
  - gas density transmitter
  - SF₆ transmitter
  - digital gas monitoring
  - remote gas monitoring
  - SCADA-integrated gas transmitter
- Output Rules:
  - If required → "Gas density transmitter: Required"
    - (append parameters if mentioned, e.g., "Gas density transmitter: Required (pressure, temperature)")
  - If provision only → "Gas density transmitter: Provision only"

# S-13
# Monitoring System
# Gas Density Monitor - Manufacturer
# "Specifies whether the customer requires a specific manufacturer or brand for the Gas Density Monitor used in high-voltage switchgear. Commonly specified brands include Trafag, WIKA, or other well-known pressure switch providers."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Manufacturer name (e.g., Trafag, WIKA)
  - Multiple acceptable brands if listed
- Ignore:
  - Mentions like “any equivalent,” “supplier standard,” or “no preference”
- Accepted keywords:
  - gas density monitor
  - SF₆ density relay
  - pressure switch
  - Trafag
  - WIKA
  - specified manufacturer
- Output Rules:
  - If manufacturer specified → "Gas density monitor manufacturer: [Brand]"
    - (e.g., Gas density monitor manufacturer: WIKA)
  - If multiple brands → list all (e.g., Gas density monitor manufacturer: Trafag, WIKA)

# S-14
# Monitoring System
# Gas Density Monitor – Number of Alarm Stages
# "Specifies whether the customer defines the number of alarm stages for the gas density monitor in high-voltage switchgear. Alarm stages typically correspond to different SF₆ gas pressure or density thresholds, such as ‘Alarm’ and ‘Lockout.’ Common configurations include two-stage (Alarm + Lockout) or three-stage (Alarm 1, Alarm 2, Lockout). Requirements may vary by circuit breaker, bay, or section. Extract all distinct requirements with their applicable scope."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Number of alarm stages (e.g., 2-stage, 3-stage)
  - Names or functions of stages if specified (e.g., Alarm, Lockout)
  - Scope: For circuit breaker, for bay, for bus section, etc.
- Ignore:
  - General mentions of gas density monitoring without stage details
- Accepted keywords:
  - alarm stage
  - two-stage alarm
  - three-stage alarm
  - lockout stage
  - SF₆ alarm levels
  - per breaker / per bay / per section
- Output Rules:
  - If specified → list all requirements with scope:
    - Gas density monitor alarm stages:
      - [Number] ([Stage details]) – [Scope]
  - If multiple options allowed → include all
- Examples:
  - Gas density monitor alarm stages:
    - 2 (Alarm, Lockout) – For Circuit Breaker
    - 3 (Alarm 1, Alarm 2, Lockout) – For Bus Section

# S-15
# Monitoring System
# Gas Density Monitor - Contact Types by Equipment
# "Specifies whether the customer defines the type and number of electrical contacts (e.g., NO, NC) for gas density monitors used in high-voltage switchgear, and whether these requirements vary by equipment (such as circuit breaker, bus section, disconnector). Contacts are typically used for alarm and lockout signaling. Extract all distinct requirements with their applicable scope."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Contact type(s): Normally Open (NO), Normally Closed (NC)
  - Number of contacts per function (e.g., Alarm: 2 NO + 1 NC)
  - Scope: For circuit breaker, for bay, for bus section, etc.
- Ignore:
  - General mentions without specifying contact type or quantity
- Accepted keywords:
  - contact type
  - NO / NC contacts
  - alarm contact
  - lockout contact
  - per breaker / per bay / per section
- Output Rules
  - If specified → list all requirements with scope:
    - Gas density monitor contacts:
      - [Function]: [Quantity and Type] – [Scope]
  - If multiple options allowed → include all
- Examples:
  - Gas density monitor contacts:
    - Alarm: 2 NO + 1 NC – For Circuit Breaker
    - Lockout: 1 NO – For Bus Section

# S-16
# Monitoring System
# Gas Density Monitor - High Alarm Setting
# "Specifies whether the customer explicitly requires a high-pressure alarm setting (high alarm) for gas density monitoring, particularly for compartments where higher internal pressure may exist (e.g., circuit breaker compartments). While most specifications focus on gas leakage (low alarm), high alarm may be required in CB-related compartments to detect abnormal overpressure. Extract the requirement and the specified threshold if stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether a high alarm is required (e.g., “High alarm shall be provided”)
  - Target compartment(s) (e.g., “Circuit breaker compartment,” “CB left/right side”)
  - Alarm threshold value if specified (e.g., “set at 0.70 MPa”)
- Ignore:
  - General mentions of low alarm or leakage alarm without high alarm
- Accepted keywords:
  - high alarm
  - overpressure alarm
  - excessive gas pressure
  - high-pressure detection
  - high-pressure setting
  - CB compartment high alarm
- Output Rules:
  - If specified →
    - Gas Density Monitor - High Alarm Setting: Required – [Scope] (Threshold: [Value])
  - If multiple compartments → list all
- Examples:
  - Gas Density Monitor - High Alarm Setting: Required – For Circuit Breaker Compartment (Threshold: 0.70 MPa)

# S-17
# Monitoring System
# Gas Density Monitor – System Configuration
# "Specifies the configuration and quantity of SF₆ gas density monitors required for the high-voltage switchgear gas compartments or systems. Extract only if the specification explicitly states the requirement for independent or common gas systems."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Independent gas system → density monitor per compartment
    - Common gas system → shared density monitor
  - Quantity if specified
  - Temperature compensation requirement if mentioned
- Ignore:
  - General mentions of “gas monitoring” without specifying density monitor configuration.
- Accepted keywords:
  - density monitor
  - gas density monitor
  - temperature compensated density monitor
  - independent gas system
  - common gas system
  - SF₆ monitoring device
- Output Rules
  - If specified →
    - Gas Density Monitor – System Configuration: [Independent/Common] – [Quantity if specified] – [Temperature compensation if specified]
- Examples
  - Gas Density Monitor – System Configuration: Independent – 1 per gas compartment – Temperature compensated
  - Gas Density Monitor – System Configuration: Common gas system – 1 monitor for 3-pole breaker – Temperature compensated
  - Gas Density Monitor – System Configuration: Independent – Each SF₆ pressure system shall have a density monitor

# T-1
# Service Continuity Requirement
# MRE Requirement
# "Specifies whether the customer provides an MRE code (Maintenance, Repair, Extension) according to IEC 62271-203. The MRE code defines the number of busbar sections and feeders that can remain in service during maintenance, repair, extension, or on-site dielectric tests. Extract the code if explicitly stated."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - MRE code (e.g., “MRE 1.2”)
  - If multiple codes for different bays → list all
- Accepted keywords:
  - MRE
  - Maintenance, Repair, Extension code
- Output Rules:
  - If specified →  
    - MRE Requirement: [Code]
- Example:
  - MRE Requirement: MRE 1.2

# T-2
# Service Continuity Requirement
# Additional Buffer Bus Requirement
# "Specifies whether the customer requires an additional buffer bus to maintain service continuity during maintenance or extension, and identifies the scope (e.g., main bus, specific bay, section). Extract only if explicitly stated."
# AI Extraction Logic 
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (Required / Not required)
  - Scope: Where the buffer bus is required (e.g., main bus, CB bay, section)
- Accepted keywords:
  - buffer bus
  - additional bus for continuity
  - auxiliary bus for maintenance
- Output Rules
  - If specified →
    - Additional buffer bus: Required – [Scope]
- Examples:
  - Additional buffer bus: Required – For Main Bus Section
  - Additional buffer bus: Required – Between CB and Main bus DS

# T-3
# Service Continuity Requirement
# Service Continuity During Bus DS/ES Maintenance
# "Specifies whether the customer requires service continuity to be maintained during maintenance of the bus disconnector switch (DS) or earthing switch (ES). This includes planned maintenance activities that may affect the busbar section or connected feeders. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during DS/ES maintenance. General statements about equipment-level isolation or mechanical independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
  - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - DS maintenance
  - ES maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during Bus DS/ES maintenance: [Requirement] – [Scope]
- Examples:
  - Service continuity during Bus DS/ES maintenance: All other feeders must remain in service
  - Service continuity during Bus DS/ES maintenance: Minimum one line and one transformer feeder to remain in service
  - Service continuity during Bus DS/ES maintenance: Partial outage permitted during DS/ES maintenance

# T-4
# Service Continuity Requirement
# Service Continuity During CB Maintenance
# "Specifies whether the customer requires service continuity to be maintained during circuit breaker (CB) maintenance. This includes planned maintenance activities that may affect feeders or transformers connected to the CB. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during CB maintenance. General statements about mechanical isolation or equipment-level independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
  - Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
  - Identify:
    - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
    - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - CB maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during CB maintenance: [Requirement] – [Scope]
- Examples:
  - Service continuity during CB maintenance: All other feeders must remain in service
  - Service continuity during CB maintenance: Minimum one line and one transformer feeder to remain in service
  - Service continuity during CB maintenance: Partial outage permitted during CB maintenance

# T-5
# Service Continuity Requirement
# Service Continuity During Main Bus Maintenance
# "Specifies whether the customer requires service continuity to be maintained during main bus maintenance. This includes planned maintenance activities that may affect the entire busbar section or connected feeders. Requirements must clearly refer to operational continuity—such as keeping feeders or transformers energized—during main bus maintenance. General statements about mechanical isolation or equipment-level independence shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type (e.g., “All feeders must remain in service,” “Minimum one line and one transformer feeder to remain energized,” “Partial outage permitted”)
  - Scope: Which feeders or sections are affected
- Ignore:
  - Statements about mechanical isolation or independence without operational continuity
  - Design/layout constraints not linked to keeping feeders energized
- Accepted keywords:
  - main bus maintenance
  - service continuity during maintenance
  - feeders remain in service
- Output Rules
  - If specified →
    - Service continuity during Main Bus maintenance: [Requirement] – [Scope]
- Examples:
  - Service continuity during Main Bus maintenance: All other feeders must remain in service
  - Service continuity during Main Bus maintenance: Minimum one line and one transformer feeder to remain in service
  - Service continuity during Main Bus maintenance: Partial outage permitted during Main Bus maintenance

# T-6
# Service Continuity Requirement
# Service Continuity During Extension
# "Specifies whether the customer requires service continuity to be maintained during extension of the switchgear. This includes installation of new feeders, bays, or sections added to an existing switchgear lineup. Requirements must explicitly state that the existing system shall remain energized or that no feeder outage is allowed during extension work. General statements about future extensibility or modularity shall not be interpreted as service continuity requirements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Full service continuity required during extension
    - Partial outage allowed during extension
    - Extension requires full shutdown
  - Scope: Which feeders or sections are affected
- Ignore:
  - General references to “future extension,” “modular layout,” or “expandability” without stating continuity during work
- Accepted keywords:
  - switchgear extension
  - expansion
  - addition of feeders/bays
  - no outage allowed during extension
  - feeders remain energized during extension
- Output Rules
  - If specified →
    - Service continuity during extension: [Requirement] – [Scope]
- Examples:
  - Service continuity during extension: Full service continuity required during extension
  - Service continuity during extension: Partial outage allowed during extension
  - Service continuity during extension: Extension requires full shutdown

# U-1
# Customer-Specific Requirements
# Manufacturing Country Restriction
# "Specifies whether the customer requires certain equipment (e.g., switchgear, local control panels) to be manufactured in a specific country or prohibits manufacturing in certain countries. Such restrictions typically aim to ensure quality control, compliance with national standards, or strategic sourcing preferences. These requirements may apply to main equipment or subcomponents, such as control cabinets, and are usually enforced through expressions like 'shall be manufactured in [Country]' or 'not permitted to be manufactured in [Country]'."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Country restriction type:
    - Mandatory manufacturing in a specific country
    - Prohibition of manufacturing in certain countries
    - Conditional approval for external manufacturing
  - Scope: Which equipment or component is affected (e.g., switchgear, control cabinet)
- Ignore:
  - Any inference based on supplier location or origin unless explicitly stated
- Accepted keywords:
  - shall be manufactured in [Country]
  - must not be manufactured in [Country]
  - manufacturing outside [Country] not permitted without approval
- Output Rules
  - If specified →
    - Manufacturing Country Requirement: [Country] ([mandatory / preferred / conditional]) – [Scope]
- Examples:
  - Manufacturing Country Requirement: Spain (preferred) – For Local Control Cabinet
  - Manufacturing Country Requirement: Korea (mandatory) – For Switchgear
  - Manufacturing Country Requirement: China (prohibited) – For Switchgear, LCC
  - Manufacturing Country Requirement: External manufacturing requires RE approval – For Control Panels

# U-2
# Customer-Specific Requirements
# Customer Audit / Surveillance during Manufacturing
# "Specifies whether the customer explicitly requires the right to perform audits, surveillance inspections, or quality reviews during the manufacturing process of high-voltage switchgear. This includes any access to workshops, in-process inspections, welding inspections, or review of quality records prior to Factory Acceptance Test (FAT). This requirement is distinct from routine FAT attendance and implies additional oversight during fabrication or assembly. Any such manufacturing-stage audit requirement must be identified and cited."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Required (e.g., “Buyer may audit manufacturing process”)
    - Conditionally Required (e.g., “may audit upon notice”)
    - Not Required (explicitly restricted to FAT only)
  - Scope: Which activities or areas are included (e.g., welding inspection, workshop access)
- Ignore:
  - General mentions of FAT attendance or inspection before shipment
  - Any inference based on common practice or QA statements unless explicitly stated
- Accepted keywords:
  - audit during manufacturing
  - surveillance inspection during fabrication
  - access to production area
  - in-process inspection
- Output Rules
  - If specified →
    - Customer audit during manufacturing: [Required / Conditionally Required / Not Required] – [Scope]
- Examples:
  - Customer audit during manufacturing: Required – Access to workshop and welding inspection
  - Customer audit during manufacturing: Conditionally Required – Upon prior notice
  - Customer audit during manufacturing: Not Required – Limited to FAT attendance only

# U-3
# Customer-Specific Requirements
# Nondestructive Test Requirement
# "Specifies whether the project specification explicitly requires any nondestructive testing (NDT) methods to be applied to high-voltage switchgear components such as welded parts, structural frames, pressure vessels, or bushings. NDT methods may include but are not limited to Radiographic Testing (RT), Ultrasonic Testing (UT), Magnetic Particle Testing (MT), or Dye Penetrant Testing (PT). This item ensures compliance with applicable manufacturing standards and quality assurance criteria."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - NDT method(s) required (e.g., RT, UT, MT, PT)
  - If multiple methods are listed → extract all in the order stated
  - If requirement is conditional → mark as “Conditionally Required” and include source phrase
  - If requirement is based on a referenced standard (e.g., ISO 17636 → RT) → extract method and cite standard
- Ignore:
  - Any inference based on typical QA practice or product type unless explicitly stated
- Accepted keywords:
  - Radiographic Testing (RT)
  - Ultrasonic Testing (UT)
  - Magnetic Particle Testing (MT)
  - Dye Penetrant Testing (PT)
  - NDT as per [Standard]
- Output Rules
  - If specified →
    - Nondestructive Test Requirement: [Method(s)] – [Scope or Standard]
- Examples:
  - Nondestructive Test Requirement: RT – For all weld joints
  - Nondestructive Test Requirement: RT / UT / PT – As per ISO 17636
  - Nondestructive Test Requirement: RT (Conditionally Required) – As per ASME Sec. VIII

# U-4
# Customer-Specific Requirements
# Impact Recorder Requirement during Transportation
# "Specifies whether the customer requires an Impact Recorder for high-voltage switchgear. The recorder must be a digital or mechanical device that logs acceleration, shock, or vibration during transportation, handling, or installation. Do not extract if the requirement applies only to transformers, medium-voltage equipment, or passive indicators such as shock/tilt labels."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Impact Recorder required
    - Optional or conditional requirement (e.g., “may be required upon request”)
- Ignore:
  - Requirements for transformers, MV equipment, or passive devices (shock/tilt labels, shock indicators)
- Accepted keywords:
  - impact recorder
  - shock recorder
  - vibration logger
  - acceleration recorder
- Output Rules
  - If specified →
    - Impact Recorder Requirement: [Required / Conditional] – [Scope]
- Examples:
  - Impact Recorder Requirement: Required – For high-voltage switchgear during transportation
  - Impact Recorder Requirement: Conditional – May be required upon request

# U-5
# Customer-Specific Requirements
# Special Conditions for FAT / SAT
# "Specifies whether the customer specifies special conditions or additional requirements for FAT (Factory Acceptance Test) or SAT (Site Acceptance Test) beyond standard procedures. These may include dedicated time for Employer testing, SCADA signal verification, GOOSE simulation, full switchgear control access, or integrated FAT (iFAT), which involves multi-vendor system-level testing. Only extract if the condition is explicitly stated as a requirement."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Special conditions for FAT or SAT such as:
    - Dedicated time for Employer testing (e.g., “minimum 5 working days”)
    - Control access requirements (e.g., “Employer must have full control access during FAT”)
    - Communication tests (e.g., SCADA signal verification, IEC 61850 GOOSE simulation)
    - Integrated FAT (iFAT) or multi-vendor testing requirements
    - Restrictions on supplier activities during Employer testing
- Ignore:
  - General mentions of FAT or SAT without additional conditions
- Accepted keywords:
  - FAT special conditions
  - SAT special conditions
  - iFAT
  - SCADA verification
  - GOOSE simulation
- Output Rules
  - If specified →
    - Special Conditions for FAT / SAT: [Summary of conditions]
- Examples:
  - Special Conditions for FAT / SAT: Minimum 5 working days for Employer testing; Full control access during FAT; SCADA and GOOSE signal verification required
  - Special Conditions for FAT / SAT: Integrated FAT (iFAT) including multi-vendor system-level testing

# U-6
# Customer-Specific Requirements
# Seismic Qualification Requirement
# "Specifies whether the customer requires seismic qualification for high-voltage switchgear and whether compliance must be demonstrated through a certified test report, analysis, or both. This includes identifying if the requirement applies to the entire switchgear assembly, specific components (e.g., circuit breaker, LCC), or auxiliary equipment. Extract the method of qualification (test vs. analysis), applicable standard (e.g., IEEE 693, IEC 62271-207), and any performance level (e.g., High, Moderate)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether seismic qualification is required
  - Method of compliance:
    - Test report required
    - Analysis accepted
    - Both required
  - Applicable standard (e.g., IEEE 693, IEC 62271-207)
  - Performance level (e.g., High, Moderate)
  - Scope: Entire switchgear or specific components
- Ignore:
  - General mentions of “robust design” or “earthquake resistance” without specifying qualification method or standard
- Accepted keywords:
  - seismic qualification
  - earthquake test
  - seismic analysis
  - IEEE 693
  - IEC 62271-207
- Output Rules
  - If specified →
    - Seismic Qualification Requirement: [Required] – [Method: Test / Analysis / Both] – [Standard] – [Performance Level] – [Scope]
- Examples:
  - Seismic Qualification Requirement: Required – Method: Test – Standard: IEEE 693 – Level: High – Scope: Entire Switchgear
  - Seismic Qualification Requirement: Required – Method: Analysis accepted – Standard: IEC 62271-207 – Level: Moderate – Scope: Circuit Breaker only

# U-7
# Customer-Specific Requirements
# Indoor Installation Clearances (Front)
# "Specifies the minimum clearance required at the front of the high-voltage switchgear for installation and maintenance purposes. This typically refers to the working space between the front of the equipment and any wall or obstruction."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Ignore:
  - Requirements for MV equipment or unrelated components
- Accepted keywords:
  - front clearance
  - working space front
  - installation clearance front
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Front): [Value + Unit] – [Condition if any]
- Examples:
  - Indoor Installation Clearance (Front): 1500 mm – Minimum working space required
  - Indoor Installation Clearance (Front): 1800 mm – For maintenance access

# U-8
# Customer-Specific Requirements
# Indoor Installation Clearances (Rear)
# "Specifies the minimum clearance required at the rear of the high-voltage switchgear for installation and maintenance purposes."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Ignore:
  - Requirements for MV equipment or unrelated components
- Accepted keywords:
  - rear clearance
  - working space rear
  - installation clearance rear
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Rear): [Value + Unit] – [Condition if any]
- Examples:
  - Indoor Installation Clearance (Rear): 1200 mm – For maintenance access
  - Indoor Installation Clearance (Rear): 1000 mm – Minimum clearance behind GIS

# U-9
# Customer-Specific Requirements
# Indoor Installation Clearances (Side)
# "Specifies the minimum clearance required between the side of the high-voltage switchgear and adjacent walls, doors, or structural elements."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Any conditions (e.g., “for maintenance access”)
- Accepted keywords:
  - side clearance
  - lateral clearance
  - clearance to wall/door
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Side): [Value + Unit] – [Condition if any]
- Examples:
  - Indoor Installation Clearance (Side): 1000 mm – For safe passage
  - Indoor Installation Clearance (Side): 800 mm – Minimum clearance to wall

# U-10
# Customer-Specific Requirements
# Indoor Installation Clearances (Between Equipment)
# "Specifies the minimum clearance required between adjacent equipment, including: Between GIS bays, Between GIS and operating mechanisms, Between GIS and GIB (Gas-Insulated Bus) sections or other associated components."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Clearance value and unit (e.g., mm)
  - Scope: Bay-to-bay, GIS-to-mechanism, GIS-to-GIB
  - Any conditions (e.g., “for maintenance access”)
- Accepted keywords:
  - clearance between bays
  - clearance between equipment
  - spacing between GIS and mechanism
  - GIB clearance
  - bus duct spacing
- Output Rules
  - If specified →
    - Indoor Installation Clearance (Between Equipment): [Value + Unit] – [Scope]
- Examples:
  - Indoor Installation Clearance (Between Equipment): 2000 mm – Between adjacent GIS bays
  - Indoor Installation Clearance (Between Equipment): 1500 mm – Between GIS and operating mechanism
  - Indoor Installation Clearance (Between Equipment): 1000 mm – Between GIS and GIB section

# U-11
# Customer-Specific Requirements
# Tank Heater Requirement
# "Specifies whether the customer requires a tank heater for high-voltage switchgear. The tank heater is typically used to prevent condensation or maintain temperature inside the equipment tank during cold conditions."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Requirement type:
    - Tank heater required
    - Optional or conditional requirement (e.g., “may be required upon request”)
- Ignore:
  - Requirements for transformers, MV equipment, or unrelated auxiliary heaters
- Accepted keywords:
  - tank heater
  - anti-condensation heater
  - heater for GIS tank
  - heating device for circuit breaker tank
- Output Rules
  - If specified →
    - Tank Heater Requirement: [Required / Conditional] – [Scope]
- Examples:
  - Tank Heater Requirement: Required – For GIS tank to prevent condensation
  - Tank Heater Requirement: Conditional – May be required in cold climate conditions

# U-12
# Customer-Specific Requirements
# Outdoor Equipment - Supply Responsibility
# " Specifies whether the customer requires the high-voltage switchgear supplier to supply outdoor free-standing equipment such as:
  - Current Transformers (CT)
  - Capacitive Voltage Transformers (CVT)
  - Surge Arresters
  - Other outdoor auxiliary components associated with the switchyard
This requirement determines the scope of supply responsibility for outdoor equipment beyond the GIS or switchgear itself."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Whether outdoor equipment supply is required by the switchgear supplier
  - Scope: Which outdoor items (CT, CVT, surge arrester, etc.)
  - Any conditions (e.g., “only if specified by Employer”)
- Ignore:
  - General mentions of outdoor installation without specifying supply responsibility
- Accepted keywords:
  - outdoor equipment supply
  - free-standing CT/CVT
  - surge arrester supply
  - outdoor auxiliary equipment responsibility
- Output Rules
  - If specified →
    - Outdoor Equipment Supply Responsibility: [Required / Conditional] – [Scope]
- Examples:
  - Outdoor Equipment Supply Responsibility: Required – Supplier to provide outdoor CT, CVT, and surge arresters
  - Outdoor Equipment Supply Responsibility: Conditional – May be required upon Employer request

# U-13
# Customer-Specific Requirements
# Customer-Specified Spare Part Requirement
# "Specifies whether the customer requires specific spare parts for high-voltage switchgear beyond the standard manufacturer recommendation. Only extract if the specification explicitly lists the spare parts to be supplied (e.g., interrupters, trip coils, CTs, VTs, gaskets)."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Exact spare parts requested by the customer (e.g., interrupters, trip coils, CTs, VTs, gaskets)
  - Quantity if provided
- Accepted keywords:
  - spare parts
  - mandatory spares
  - spare interrupter
  - spare trip coil
  - spare CT / VT
  - gasket spares
- Output Rules
  - If specified →
    - Customer-Specified Spare Part Requirement: [List of items with quantities if available]
  - If only general mention → Spare Parts Requirement: Yes – Manufacturer recommended
- Examples:
  - Customer-Specified Spare Part Requirement: 3 interrupters; 2 trip coils; 2 CTs; 2 pressure gauges
  - Customer-Specified Spare Part Requirement: 1 set of gaskets; 1 spare CVT

# U-14
# Customer-Specific Requirements
# Customer-Specified Special Tool Requirement
# "Specifies whether the customer explicitly requires particular special tools to be supplied for installation, operation, or maintenance of high-voltage switchgear. Only extract tools that are clearly listed or mandated by the customer. Do not extract if the specification only refers to tools to be used by the supplier or states that tools shall be proposed by the manufacturer."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to the validated SwitchgearType.
- Identify:
  - Exact special tools requested by the customer (e.g., disassembly jig, SF₆ filling adapter, operating mechanism tool kit)
  - Quantity if provided
- Accepted keywords:
  - special tools
  - mandatory tools
  - disassembly jig
  - SF₆ filling adapter
  - operating mechanism tool kit
  - lifting device
  - alignment tool
- Output Rules
  - If specified →
    - Customer-Specified Special Tool Requirement: [List of tools with quantities if available]
  - If only general mention → Special Tool Requirement: Yes – Manufacturer recommended
- Examples:
  - Customer-Specified Special Tool Requirement: 1 disassembly jig; 1 SF₆ filling adapter
  - Customer-Specified Special Tool Requirement: 1 operating mechanism tool kit; 2 alignment tools

# U-15
# Customer-Specific Requirements
# 3D / BIM Model Deliverable Requirement
# "Specifies whether the customer requires 3D models or BIM (Building Information Modeling) deliverables for high-voltage switchgear. These models are typically used for integration into the overall substation design, clash detection, and construction planning."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification applicable to high-voltage switchgear.
- Identify:
  - Requirement type:
    - 3D model required
    - BIM model required
  - Specific format (e.g., IFC, Revit, STEP)
  - LOD (Level of Development) if specified (e.g., LOD 100, 200, 300, 350, 400, 500)
  - Scope (e.g., entire GIS assembly, LCC only)
- Ignore:
  - General mentions of drawings or 2D layouts without specifying 3D/BIM deliverables
- Accepted keywords:
  - 3D model
  - BIM model
  - IFC format
  - Revit model
  - STEP file
  - LOD 300 / LOD 400 / LOD 500
  - CAD 3D deliverable
- Output Rules
  - If specified →
    - 3D / BIM Model Deliverable Requirement: [Required / Conditional] – [Format if specified] – [LOD Level if specified] – [Scope]
- Examples:
  - 3D / BIM Model Deliverable Requirement: Required – Format: Revit – LOD 400 – Scope: Entire GIS assembly
  - 3D / BIM Model Deliverable Requirement: Conditional – May be required upon Employer request – Format: STEP – Scope: Entire GIS assembly

# U-16
# Customer-Specific Requirements
# Offshore Platform Structural Design & Deflection
# "Specifies whether the customer requires compliance with offshore structural standards (e.g., DNV-OS-C101) and any deflection criteria for GIS installation on offshore platforms. Also check if the specification includes conditions about cost responsibility for structural modifications or deviation requests."
# AI Extraction Logic
- Extract only if explicitly mentioned in the project-binding specification.
- Identify:
  - Applicable standard (e.g., DNV-OS-C101)
  - Deflection criteria (e.g., L/200, L/2000)
  - Whether GIS supplier must accept platform design without deviation
  - Any cost responsibility clauses (e.g., “supplier bears cost if deviation requested”)
- Accepted keywords:
  - DNV standard
  - deflection criteria
  - beam deflection
  - offshore platform structural design
  - cost responsibility for redesign
- Output Rules:
  - If specified →
    - Offshore Platform Structural Requirement: [Standard] – [Deflection Criteria] – [Cost Responsibility if mentioned]
- Examples:
  - Offshore Platform Structural Requirement: Standard: DNV-OS-C101 – Deflection: L/2000 for GIS – Cost Responsibility: Supplier bears redesign cost if deviation requested
  - Offshore Platform Structural Requirement: Standard: DNV – Deflection: L/200 – No cost responsibility clause
""", "version": 251127})

True

In [ ]:
# 업데이트(switchgeartype_prompt)
manager.update_entry("switchgeartype_prompt", {"text": '''
## Variable Definition:
- SwitchgearType: The standard field name for the combination of Nominal System Voltage and Switchgear Type (e.g., "380 kV GIS").
- Always use "SwitchgearType" (case-sensitive) as the field name in all outputs and subsequent prompts.

## Purpose:
Extract and bind the High-Voltage Switchgear Type and Nominal System Voltage from project-binding documents required for the project scope.

## Extraction Logic:
Step 0: Voltage Terminology Correction
Normalize voltage terms based on context:
- If both nominal and operating voltage are mentioned in the same scoped section (sentence, paragraph, or table row), treat operating voltage as the valid Nominal System Voltage.
- If only nominal voltage is mentioned, verify whether it refers to actual system operation or just product rating. If it refers to product rating (e.g., 145 kV GIS used in a 132 kV system), exclude it.
- Exclude voltage terms like “Rated voltage,” “Maximum voltage,” or “Nominal voltage” if used in the context of equipment specification only.

Step 1: Switchgear Type Identification
Extract switchgear types and it's operating voltage from project-binding documents only (e.g., Project Specification, SOW, BOQ, datasheets, drawings).
Use General Specification only if:
- Project-binding documents do not specify voltage for a required switchgear type.
- The document structure makes a single switchgear family unambiguous.
Do not use General Specification to create new voltage-switchgear combinations not explicitly required in the project scope.
Do not limit extraction to summary tables or introductory sections.
Include types mentioned in:
- Equipment lists
- Technical descriptions
- Interface descriptions
A distinction should be made between Hybrid Gas-Insulated Switchgear (HGIS) and Gas-Insulated Switchgear (GIS).
While identifying switchgear types, immediately pair them with operating voltage only if:
- The voltage and switchgear type appear together in the same scoped section (sentence, paragraph, or table row).
- The voltage is confirmed to be part of the actual project scope (not just general specification).
Exclude voltage-switchgear combinations that are only mentioned in general specifications unless:
- The project-binding documents do not specify voltage but clearly require a specific switchgear type.
- The document structure makes a single switchgear family unambiguous.
Exclude any switchgear type paried with operating voltage below 66 kV.

Step 2: Switchgear Type Normalization
Normalize extracted types to the following canonical labels:
- GIS (Gas-Insulated Switchgear)
- HGIS (Hybrid Gas-Insulated Switchgear)
- DTB (Dead Tank Breakers)
- SF₆-free GIS (Gas Insulated Switchgear using alternative gases / Eco-friendly GIS)
Exclude individual components (e.g., DS, ES, CT, VT, SA).

Step 3: Override Handling
- If a switchgear type is **explicitly described as SF₆-free** (e.g., "GIS without SF₆", "SF₆-free GIS"), reclassify it as:
  - **SwitchgearType: "{Voltage} SF₆-free GIS"**
- If a switchgear type is **explicitly excluded** (e.g., "GIS shall not be used") or **explicitly stated to be replaced by another** (e.g., "GIS shall be replaced by HGIS"), then discard the excluded type.
- **Do not discard a switchgear type** if multiple types (e.g., GIS and HGIS) are **both required for different purposes or locations** within the project. In such cases, retain all valid types.

Step 4: Output Formatting
Output the result as a list of strings in the format:
SwitchgearType: "{Operating Voltage} {High-Voltage Switchgear Type}" (e.g., SwitchgearType: "380 kV HGIS")
Exclude the pair which is paired with rated voltage.
output format(list) : 
[Switchgear Type1, Switchgear Type2, Switchgear Type3...]
'''
, "version": 251209})

True

In [10]:
# 업데이트(instruction)
manager.update_entry("instruction_prompt", {"text": '''
## Variable Definition
- SwitchgearType: The standard field name for the combination of Nominal System Voltage and Switchgear Type (e.g., "132 kV GIS").
- Always use "SwitchgearType" (case-sensitive) as the field name in all outputs and subsequent prompts.

## Scope & Source
- Analyze the provided PDF document and extract information based on the categories and specification items defined in ## Given Text ##.
- Use information explicitly stated in project-binding contexts (Project Spec / SOW / BOQ / project datasheets / drawings).
- General Standards inside the PDF are auxiliary only and may be used only if the PDF explicitly makes them binding for this project.

## Context Binding
- Set SwitchgearType = "{switchgear_type}"
- Assume SwitchgearType is already validated and uniquely defined in the previous step.
- Do not revalidate voltage/type pairing unless ambiguity is detected.
- Proceed to extract item specifications directly within the context relevant to the given SwitchgearType.

## Extraction Rules
- For each of the specification items (see note in conflicts), extract the value as it applies to the validated SwitchgearType context.
- Follow the provided Description and AI Extraction Logic for each item strictly.
- If the value is not explicitly stated in the validated context, set:
        "Value": "None",
        "Ref": "Not found"
- Do not infer values or cross-reference unrelated contexts.

## Conflict Resolution
- If multiple thresholds exist, always choose the more conservative requirement.

## Output Format
- Output must follow the JSON array format defined in format_prompt.
- Ensure completeness: provide values (or "None") for all items corresponding to the validated SwitchgearType.

## Field-Level Truncation
- For any single string field exceeding 300 characters, truncate to the first 290 ASCII characters and append '.....'
- Maintain valid JSON formatting after truncation.
'''
, "version": 251106})

True

In [11]:
# 업데이트(format_prompt)
manager.update_entry("format_prompt", {"text": '''
[
  {'SerialNumber':'Serial Number, The numeric part of SerialNumber is written in three-digit format. ex) G-1 → G-001'},
  {'SwitchgearType':'SwitchgearType string as extracted, e.g. "380 kV GIS"'},
  {'Category':'Category Name'},
  {'Item':'Specification Item Name'},
  {'Value':'Extracted Value'},
  {'Unit':'Unit of the value, if applicable'},
  {'Ref':'Verbatim source sentence from the PDF used to extract the value'},
  {'Title':'File name containing the Ref text. See "Filename" at the bottom right of the page.'},
  {'Page':'Page number containing the Ref text. See "Page" at the bottom right of the page.'}
]
'''
, "version": 251106})

ValueError: 업데이트하려는 text 값이 기존 엔트리와 동일합니다.